# 分布鲁棒优化

**作者：**  
Daniel Kuhn  
风险分析与优化主席，  
洛桑联邦理工大学，瑞士洛桑  
电子邮件：[daniel.kuhn@epfl.ch](mailto:daniel.kuhn@epfl.ch)

Soroosh Shafiee  
康奈尔大学，运营研究与信息工程学院，  
美国纽约伊萨卡  
电子邮件：[shafiee@cornell.edu](mailto:shafiee@cornell.edu)

Wolfram Wiesemann  
帝国理工学院商学院，  
英国伦敦帝国理工大学  
电子邮件：[ww@imperial.ac.uk](mailto:ww@imperial.ac.uk)

---

## 摘要

分布鲁棒优化（DRO）研究不确定条件下的决策问题，其中控制不确定参数的概率分布也不确定。DRO模型的一个关键组成部分是其模糊集，即满足已知结构信息或统计信息的概率分布族。DRO寻找在模糊集中的最坏分布下表现最好的决策。这一最坏情况(悲观)标准得到了心理学和神经科学研究的支持，研究表明许多决策者对分布不确定性有较低的容忍度。DRO根植于统计学、运筹学和控制理论，近期研究揭示了其与机器学习中的正则化技术和对抗性训练之间的深刻联系。本综述以统一且独立的方式呈现了该领域的关键发现。

---

## 目录

1. [引言](#引言)
2. [模糊集](#模糊集)
3. [模糊集的拓扑属性](#模糊集的拓扑属性)
4. [最坏期望问题的对偶理论](#最坏期望问题的对偶理论)
5. [最坏风险问题的对偶理论](#最坏风险问题的对偶理论)
6. [自然子问题的解析解](#自然子问题的解析解)
7. [自然子问题的有限凸重构](#自然子问题的有限凸重构)
8. [通过鲁棒化进行正则化](#通过鲁棒化进行正则化)
9. [DRO问题的数值求解方法](#dro问题的数值求解方法)
10. [统计保证](#统计保证)
11. [参考文献](#参考文献)


# 1. 引言 (Introduction)
数学优化研究以下形式的问题：

$$
\inf _{x \in \mathcal{X}} \ell(x),
$$

其中，决策变量 $x$ 从可行解组成的集合 $\mathcal{X} \subseteq \mathbb{R}^n$ 中选择，目的是最小化损失函数 $\ell: \mathbb{R}^n \rightarrow \overline{\mathbb{R}}$。数学优化起源于17世纪末艾萨克·牛顿、戈特弗里德·威廉·莱布尼茨、皮埃尔·德·费马等人对微积分的研究，至今已有丰富的历史，涉及许多数学家、经济学家、工程师和科学家的贡献。现代数学优化的诞生通常归功于乔治·丹茨格，他在1947年提出的单纯形法可以求解线性优化问题，其中损失函数 $\ell$ 是仿射的，且可行集 $\mathcal{X}$ 是一个多面体（Dantzig 1956）。随后的重要进展包括：凸分析的丰富理论（Rockafellar 1970），以及线性优化问题的多项式时间求解方法（Khachiyan 1979，Karmarkar 1984）以及许多类别的非线性凸优化问题（Nesterov 和 Nemirovskii 1994）。

经典的优化问题是确定性的，即假设所有问题数据都已知且确定。然而，实践中遇到的大多数决策问题都依赖于那些受到测量误差影响的参数，或者仅在决策必须做出并且实施之后才能揭示的参数。将受到不确定性影响的决策问题简单建模为确定性优化问题的一种方法是用期望值或适当的点预测值替代所有不确定的参数。然而，早在很久以前就有人发现并记录了这样一个事实：那些将不确定的参数替换为其均值的决策者往往会陷入“平均误差”的陷阱（Savage, Scholtes 和 Zweidler 2006，Savage 2012）。为了考虑那些偏离均值的不确定性实现，Beale（1955）和Dantzig（1955）独立提出了以下形式的随机规划：

$$
\inf _{x \in \mathcal{X}} \mathbb{E}_{\mathbb{P}}[\ell(x, Z)],
$$

这些方法明确地将不确定的问题参数 $Z$ 建模为一个服从概率分布 $\mathbb{P}$ 的随机向量，并且在此基础上寻找在期望下表现最好的决策（或根据某些风险度量进行优化）。此后，随机规划已发展成为一个成熟的领域（Birge 和 Louveaux 2011，Shapiro, Dentcheva 和 Ruszczyński 2009），并为机器学习中的经验风险最小化原则提供了理论基础（Bishop 2006，Hastie, Tibshirani 和 Friedman 2009）。

尽管随机规划在理论和实践中取得了成功，但它至少存在两个缺点。首先，其假设概率分布 $\mathbb{P}$ 是完全已知的，在许多实际情况下是不现实的， 且随机规划对分布$\mathbb{P}$ 的误设非常敏感。不同领域对这一效应的描述有：优化者的诅咒（Smith 和 Winkler 2006），优化的误差最大化效应（Michaud 1989，DeMiguel 和 Nogales 2009），优化偏差（Shapiro 2003）或过拟合（Bishop 2006，Hastie 等 2009）。其次，评估固定决策的期望损失需要计算多维积分，而这对于非常简单的损失函数和分布来说也非常复杂。因此，随机规划受到维度灾难的挑战，即其计算复杂度通常随着随机向量 $Z$ 的维度呈指数增长。为了缓解这两个缺点，Soyster（1973）提出将不确定性影响的决策问题建模为鲁棒优化问题，形式为：

$$
\inf _{x \in \mathcal{X}} \sup _{z \in \mathcal{Z}} \ell(x, z).
$$

鲁棒优化用基于集合的描述代替了不确定问题参数的概率描述，寻找在参数$z$最坏情况下的实现上表现最好的决策。经过长时间的忽视，Soyster（1973）的思想在90年代末得到了重新审视并大大扩展，Kouvelis 和 Yu（1997），El Ghaoui、Oustry 和 Lebret（1998），El Ghaoui 和 Lebret（1998a,b），BenTal 和 Nemirovski（1999b, 1998, 1999a），Bertsimas 和 Sim（2004）等人进一步推进了该领域。有关鲁棒优化文献的综述，请参见 Ben-Tal、El Ghaoui 和 Nemirovski（2009），Rustem 和 Howe（2009）和 Bertsimas 和 den Hertog（2022）。我们指出，类似的思想在鲁棒稳定性（Horn 和 Johnson 1985，Doyle、Glover、Khargonekar 和 Francis 1989，Green 和 Limebeer 1995）和鲁棒控制（Zames 1966，Khalil 1996，Zhou、Doyle 和 Glover 1996）领域中也得到了独立发展，这些领域研究系统在参数变化下是否能够保持稳定，或者在参数变化下设计出具有良好性能的系统。有关鲁棒稳定性和控制的教材介绍，请参见 Zhou 和 Doyle（1999）和 Dullerud 和 Paganini（2001）。Hansen 和 Sargent（2008）将鲁棒控制技术应用于受模型不确定性影响的经济问题，设计在多种可能的模型误设下都能表现良好的政策。

尽管鲁棒优化减少了随机规划中的信息和计算负担，但它对模糊集内所有参数实现的平等对待且聚焦于最坏情境，可能导致其在实际应用中过于保守。这些问题促使研究人员研究分布鲁棒优化问题，形式为：

$$
\inf _{x \in \mathcal{X}} \sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(x, Z)],
$$

该模型将不确定的参数$Z$建模为一个随机向量，其由一个模糊集$\mathcal{P}$中的某个分布$\mathbb{P}$支配，寻求在最坏分布$\mathbb{P} \in \mathcal{P}$下，其期望值表现最好的决策。分布鲁棒优化（DRO）因此将随机规划的分布视角与鲁棒优化的最坏情况聚焦结合在一起。Herbert E. Scarf通常被认为是这一方法的先驱，他在关于新闻供应商问题的研究中提出，未知的需求分布仅通过其均值和方差来表征（Scarf 1958）。随后，Dupačová（1966，1987，1994）和Shapiro与Kleywegt（2002）研究了那些模糊集规定了未知概率分布的支持集、某些低阶矩、独立模式或其他结构特性的问题。Ermoliev、Gaivoronski 和 Nedeva（1985）及 Gaivoronski（1991）为基于矩模糊集的DRO问题开发了早期的求解方法。现代DRO的出现通常归功于Bertsimas和Popescu（2002，2005）的工作，他们在部分分布信息下推导了概率不等式，并将其技术应用于期权定价问题；El Ghaoui、Oks和Oustry（2003）以及Calafiore和El Ghaoui（2006）的研究，他们研究了DRO问题最小化目标函数的某个分位数，或一组受不确定性影响的约束应当以高概率满足，跨所有具有已知矩约束的概率分布；以及Delage和Ye（2010）的研究，其研究了类似的DRO问题，目标为最小化最坏情况下的期望目标值。

早期关于DRO的研究主要集中在矩模糊集上，这些模糊集包含所有满足有限多个矩约束的指定支持集$\mathcal{Z}$上的分布。与随机规划不同，具有矩模糊集的DRO问题有时会在随机向量$Z$的维度上展现出有利的扩展性。然而，截然不同的分布可能具有相同的矩。因此，矩模糊集总是包含广泛的分布，其中包括一些不太可能的分布，当有充足的历史数据时，这些不太可能的分布可以安全地被排除。这促使Ben-Tal、den Hertog、De Waegenaere、Melenberg和Rennen（2013）以及Wang、Glynn和Ye（2016）引入了模糊集，这些模糊集包含所有在某个参考分布的邻域内的分布（通常是从历史数据形成的经验分布）。这些邻域可以通过概率分布之间的距离函数来定义，例如$\phi$-散度（Csiszar 1963）或Wasserstein距离（Villani 2008）。与矩模糊集不同，基于统计距离的模糊集具有可调的大小参数（例如，半径），因此可以缩小为仅包含参考分布的单例。如果参考分布收敛到未知的真实分布，并且随着历史数据的增加模糊集大小参数趋近于0，则DRO问题最终会简化为在真实分布下的经典随机规划。早期关于基于差异的模糊集的研究假定$Z$是一个离散随机向量，具有有限的支持集$\mathcal{Z}$。基于差异的DRO问题扩展到一般的（可能是连续）随机向量的工作由Mohajerin Esfahani 和 Kuhn（2018），Zhao 和 Guan（2018），Blanchet 和 Murthy（2019），Zhang、Yang 和 Gao（2024b）以及 Gao 和 Kleywegt（2023）完成，他们使用最优传输差异构建了模糊集。我们请参阅Kuhn、Mohajerin Esfahani、Nguyen 和 Shafieezadeh-Abadeh（2019）以及Rahimian 和 Mehrotra（2022）以获取DRO文献的先前综述。

“分布鲁棒性”一词最早源于鲁棒统计。Huber（1981）首次提出该术语，用以描述在数据中存在异常值时做出鲁棒决策的方法。这个概念扩展了Box（1953，1979）的早期工作，其研究了在基础分布偏离正态分布时的鲁棒性，正态分布是许多统计模型中的常见假设。为了应对异常值带来的挑战，统计学家们发展了几种污染模型，每个模型提供了不同的解决数据异常的途径。Huber污染模型由Huber（1964，1968）提出，并由Hampel（1968，1971）进一步发展，假设观察到的数据是从真实分布和任意污染分布的混合中抽取的。邻域污染模型通过统计距离定义偏离真实分布的情况，例如总变差（Donoho 和 Liu 1988）或Wasserstein距离（Zhu、Jiao 和 Steinhardt 2022a，Liu 和 Loh 2023）。最近，数据依赖的自适应污染模型允许将部分观察到的数据点替换为从任意分布中抽取的点（Diakonikolas、Kamath、Kane、Li、Moitra 和 Stewart 2019，Zhu 等 2022a）。有趣的是，DRO模型的乐观对偶模型，优化的是模糊集中的最佳分布（而非最坏分布），恢复了许多来自鲁棒统计的估计量（Blanchet、Li、Lin 和 Zhang 2024b，Jiang 和 Xie 2024）。关于算法鲁棒统计的最新进展，请参阅Diakonikolas 和 Kane（2023）。

鲁棒优化和分布鲁棒优化在机器学习中找到了广泛的应用。例如，机器学习文献中的流行正则化器具有鲁棒性解释，这为正则化在实践中的强大表现提供了理论洞察（Xu，Caramanis 和 Mannor 2009，Shafieezadeh-Abadeh、Kuhn 和 Mohajerin Esfahani 2019，Li、Lin、Blanchet 和 Nguyen 2022，Gao、Chen 和 Kleywegt 2024）。同样，DRO模型的乐观对偶模型，优化的是模糊集中的最佳（而非最坏）分布，产生了上置信区间算法，这些算法在赌博和强化学习文献中非常普遍（Blanchet 等 2024b，Jiang 和 Xie 2024）。DRO也与对抗训练有关，对抗训练旨在通过在对抗性样本的视角下训练模型，从而提高机器学习模型的泛化性能（Goodfellow、Shlens 和 Szegedy 2015）。对抗样本是现有数据点的扰动，旨在误导模型做出不正确的预测。


DRO 与扩展的随机（动态）规划之间也存在深刻的联系，这些扩展通过用一致的风险度量代替期望值。与期望值相似，风险度量将随机变量映射到扩展实数上。与期望值不同，期望值是风险中立的，因为它对正负结果赋予相同的权重，而风险度量通常赋予负面结果更大的权重，从而考虑到决策者经常表现出的风险厌恶。Artzner、Delbaen、Eber 和 Heath (1999) 以及 Delbaen (2002) 表明，满足一致性公理以及 Fatou 性质的风险度量可以等效地表示为在特定分布集上的最坏情况期望值。换句话说，优化最坏情况期望值（如DRO中所做的）和优化一致风险度量之间有直接的联系。在金融数学文献中，针对一类非线性期望——所谓的 $G$-期望，已经发展了类似的表示定理，这些期望基于反向随机微分方程的解（Peng 1997, 2007a,b, 2019）。Peng (2023) 显示，次线性 $G$-期望等价于在特定分布族上的最坏情况期望值，从而在 $G$-期望理论和DRO之间架起了桥梁。

从哲学上讲，DRO 与模糊厌恶原则相关，即个体更喜欢已知风险而不是未知风险，即使未知风险可能带来更高的回报。在经济学文献中，风险性结果的概率已知与模糊性结果的概率（部分）未知之间的区别可以追溯到至少凯恩斯（1921）和奈特（1921）。模糊厌恶的概念通过Ellsberg悖论（Ellsberg 1961）广泛传播，这是一种思想实验，在该实验中，人们被要求在一个已知分布的彩色球（例如50个红球和50个蓝球）的盒子和一个具有未知分布的相同彩色球的盒之间做出选择（即红蓝球的比例未知）。尽管可能有相等或更好的赔率，许多人仍然更愿意在已知分布的盒上下注，即他们表现出模糊厌恶。Ellsberg悖论挑战了经典的期望效用理论，它导致了如最大最小期望效用理论（Gilboa 和 Schmeidler 1989）等扩展，作为DRO的理论基础。之后，模糊厌恶在金融市场（Epstein 和 Miao 2003，Bossaerts、Ghirardato、Guarnaschelli 和 Zame 2010）、保险市场（Cabantous 2007）、个人决策（Dimmock、Kouwenberg 和 Wakker 2016）、宏观经济政策（Hansen 和 Sargent 2010）、拍卖（Salo 和 Weber 1995）以及信任博弈（Li、Turmunkh 和 Wakker 2019b）中被发现。

同时，也有大量医学和神经科学证据支持模糊厌恶的存在。Hsu、Bhatt、Adolphs、Tranel 和 Camerer（2005）发现，当个体面临模糊性时，大脑中的关键情绪处理中心——杏仁体的活动比面对已知概率的情境时更为活跃，这表明杏仁体在驱动模糊厌恶中的作用。Krain、Wilson、Arbuckle、Castellanos 和 Milham（2006）的荟萃分析突出了前额叶皮层的作用，该区域负责高级认知控制、理性决策和情绪调节，用于处理模糊性。此外，Wu、Sun、Camilleri、Eickhoff 和 Yu（2021）进行的荟萃分析表明，风险和模糊的处理都依赖于前岛叶。风险处理还会激活背内侧前额叶皮层和腹侧纹状体，而模糊处理则特定地涉及背外侧前额叶皮层、下顶叶和右前岛叶。这支持了这样一个观点：当个体面对模糊决策与风险决策时，涉及的是不同的神经机制。遗传因素可能影响个体对模糊厌恶的倾向。He、Xue、Chen、Lu、Dong、Lei、Ding、Li、Li、Chen、Li、Moyzis 和 Bechara（2010）将某些遗传多态性与个体在风险和模糊决策下的表现联系起来。在另一项研究中，Buckert、Schwieren、Kudielka 和 Fiebach（2014）研究了荷尔蒙变化（例如，与压力和焦虑相关的皮质醇水平升高）如何影响风险和模糊决策。这些发现共同表明，风险和模糊的感知不仅仅是认知现象，还受到大脑结构、遗传和荷尔蒙因素的影响，这些因素塑造了个体在模糊性决策中的差异。最后，我们提到Hartley 和 Somerville（2015）以及 Blankenstein、Crone、van den Bos 和 van Duijvenvoorde（2016），他们研究了模糊厌恶如何在儿童、青少年和成人之间有所不同，Hayden、Heilbronner 和 Platt（2010）观察到，当恒河猕猴在选择风险和模糊游戏时，表现出模糊厌恶。

我们的分析的一个重要部分是研究最坏期望值 $\sup _{\mathrm{P} \in \mathcal{P}} \mathbb{E}_{\mathrm{P}}[\ell(x, Z)]$，这是DRO问题（1.2）的目标函数。评估这个表达式通常需要求解一个半无穷优化问题，这个问题有无限多个变量来表征概率分布 $P$，并且受到模糊集 $\mathcal{P}$ 施加的有限个约束。我们称这个问题为“自然的子问题”，它是区分DRO问题（1.2）与确定性、随机和鲁棒优化问题的关键特征。本综述的其余部分结构如下：第2和第3节回顾了常见的模糊集 $\mathcal{P}$ 及其拓扑性质，特别关注自然子问题在什么条件下能够达到最优值。第4和第5节为自然子问题开发了对偶性理论，使我们能够等价地重新表述最坏期望值或其上界为一个半无限优化问题：其具有有限多个对偶决策变量，无限多个约束。这个对偶性框架为第6节自然子问题的解析解奠定了基础，其依赖于构建原始和对偶可行解，二者具有强对偶性从而产生相同的目标值。第7和第8节利用相同的对偶性理论，开发了自然子问题以及整个DRO问题（1.2）的等价重构和保守逼近。第9节展示了对偶性理论如何为自然子问题和整个DRO问题提供数值求解技术。最后，第10节回顾了不同模糊集所享有的统计保障。

由于篇幅限制，本综述在选择涵盖的主题时面临艰难的取舍。我们决定聚焦于最常用的模糊集，并简要回顾其他可能的选择，例如边际模糊集、具有结构约束的模糊集（包括对称性和单峰性）、Sinkhorn模糊集或条件相对熵模糊集。同样，我们没有涵盖分布优先优化和决策随机化等重要但略显复杂的话题。最后，我们专注于单阶段问题，其中不确定性在作出此时此刻的决策 $x \in \mathcal{X}$ 后完全解决；至于不确定性随时间实现并且可以进行响应决策的两阶段和多阶段DRO问题，已由 Delage 和 Iancu（2015）以及 Yanikoglu、Gorissen 和 den Hertog（2019）进行了综述。





## 1.1 符号

本文中考虑的所有向量空间都定义在实数域上。为了简洁起见，我们简单地称它们为“向量空间”而不是“实向量空间”。我们使用 $\overline{\mathbb{R}} = \mathbb{R} \cup \{-\infty, \infty\}$ 来表示扩展实数。函数 $f: \mathbb{R}^d \rightarrow \overline{\mathbb{R}}$ 的有效域定义为 $\operatorname{dom}(f) = \{z \in \mathbb{R}^d : f(z) < \infty\}$，而 $f$ 的上图定义为 $\operatorname{epi}(f) = \{(z, \alpha) \in \mathbb{R}^d \times \mathbb{R}: f(z) \leq \alpha\}$。当且仅当 $\operatorname{dom}(f) \neq \emptyset$ 且对于所有 $z \in \mathbb{R}^d$，$f(z) > -\infty$时，我们称 $f$ 为适当的。$f$ 的凸共轭是函数 $f^*: \mathbb{R}^d \rightarrow \overline{\mathbb{R}}$，通过 $f^*(y) = \sup_{z \in \mathbb{R}^d} y^{\top} z - f(z)$ 定义。一个凸函数 $f$ 被称为闭合的，如果它是适当的并且下半连续，或者它恒等于 $+\infty$ 或 $-\infty$。可以证明，$f$ 是闭合的当且仅当它与其双共轭 $f^{**}$ 一致，即与 $f^*$ 的共轭一致。如果 $f$ 是适当的、凸的且下半连续的，那么它的退化函数 $f^{\infty}: \mathbb{R}^d \rightarrow \mathbb{R}$ 通过以下公式定义：$f^{\infty}(z) = \lim_{\alpha \to \infty} \alpha^{-1}(f(z_0 + \alpha z) - f(z_0))$，其中 $z_0$ 是 $\operatorname{dom}(f)$ 中的任意一点（Rockafellar 1970，第8.5定理）。$f$ 的透视图是函数 $f^\pi: \mathbb{R}^d \times \mathbb{R} \rightarrow \overline{\mathbb{R}}$，通过以下公式定义：$f^\pi(z, t) = t f(z / t)$（当 $t > 0$ 时），$f^\pi(z, t) = f^{\infty}(z)$（当 $t = 0$ 时），$f^\pi(z, t) = \infty$（当 $t < 0$ 时）。可以证明，$f^\pi$ 是适当的、凸的且下半连续的（Rockafellar 1970，第67页）。在没有混淆的风险时，我们偶尔使用 $t f(z / t)$ 来表示 $f^\pi(z, t)$，即使 $t = 0$。集合 $\mathcal{Z} \subseteq \mathbb{R}^d$ 的指示函数 $\delta_{\mathcal{Z}}: \mathbb{R}^d \rightarrow \mathbb{R}$ 定义为：如果 $z \in \mathcal{Z}$，则 $\delta_{\mathcal{Z}}(z) = 0$；如果 $z \notin \mathcal{Z}$，则 $\delta_{\mathcal{Z}}(z) = \infty$。$\delta_{\mathcal{Z}}$ 的共轭 $\delta_{\mathcal{Z}}^*$ 被称为 $\mathcal{Z}$ 的支持函数。它满足 $\delta_{\mathcal{Z}}^*(y) = \sup_{z \in \mathcal{Z}} y^{\top} z$。随机对象用大写字母表示（例如，$Z$），其实现用小写字母表示（例如，$z$）。对于任意闭集 $\mathcal{Z} \subseteq \mathbb{R}^d$，我们使用 $\mathcal{M}(\mathcal{Z})$ 来表示所有有限签名的Borel测度空间，而 $\mathcal{M}_+(\mathcal{Z})$ 表示 $\mathcal{M}(\mathcal{Z})$ 中所有（非负的）Borel测度的凸锥，$\mathcal{P}(\mathcal{Z})$ 表示 $\mathcal{M}_+(\mathcal{Z})$ 中所有概率分布的凸集。关于 $P \in \mathcal{P}(\mathcal{Z})$ 的期望算子定义为：$\mathbb{E}_{P}[f(Z)] = \int_{\mathcal{Z}} f(z) \mathbb{dP}(z)$，对于任何 Borel 函数 $f: \mathcal{Z} \rightarrow \overline{\mathbb{R}}$。如果 $f$ 的正负部分的积分都为 $\infty$，则我们“敌对”地定义 $\mathbb{E}_{P}[f(Z)]$。即，如果积分出现在最小化（最大化）问题的目标函数中，则我们设定 $\mathbb{E}_{P}[f(Z)] = \infty$（或 $-\infty$）。Dirac 概率分布将单位概率赋给 $z \in \mathcal{Z}$，记作 $\delta_z$。Dirac 分布 $\delta_z$ 不应与单点集合 $\{z\}$ 的指示函数 $\delta_{\{z\}}$ 混淆。对于任何 $\mathcal{P} \in \mathcal{P}(\mathcal{Z})$ 和任何 Borel 可测变换 $f: \mathcal{Z} \rightarrow \mathcal{Z}^{\prime}$，其中 $\mathcal{Z} \subseteq \mathbb{R}^d$ 和 $\mathcal{Z}^{\prime} \subseteq \mathbb{R}^{d^{\prime}}$，我们用 $\mathbb{P} \circ f^{-1}$ 表示在 $f$ 下的推前分布。因此，如果 $Z$ 是一个由 $P$ 支配的在 $\mathcal{Z}$ 上的随机向量，那么 $f(Z)$ 就是一个由 $\mathbb{P} \circ f^{-1}$ 支配的在 $\mathcal{Z}^{\prime}$ 上的随机向量。一个集合 $\mathcal{Z} \subseteq \mathbb{R}^d$ 的闭包、内点和相对内点分别用 $\mathrm{cl}(\mathcal{Z})$、$\operatorname{int}(\mathcal{Z})$ 和 $\operatorname{rint}(\mathcal{Z})$ 表示。我们使用 $\mathbb{R}_+^d$ 和 $\mathbb{R}_{++}^d$ 来表示 $\mathbb{R}^d$ 中的非负正交体及其内点。此外，我们使用 $\mathbb{S}^d$ 来表示所有对称矩阵的空间，$\mathbb{R}^{d \times d}$ 中的对称矩阵。正半定矩阵的锥体用 $\mathbb{S}_+^d$ 表示，而 $\mathbb{S}_{++}^d$ 表示其内点，即所有正定矩阵的集合。逻辑命题 $\mathbb{1}_{\mathcal{E}}$ 的真值为1，当且仅当 $\mathcal{E}$ 为真，否则为0。所有自然数集合 $\{1, 2, 3, \ldots\}$ 用 $\mathbb{N}$ 表示，$[n] = \{1, \ldots, n\}$ 表示所有小于等于 $n$ 的整数的集合。

# 2. 模糊集合

模糊集合 $\mathcal{P}$ 是一个概率分布族，定义在一个公共的可测空间上。本文中，我们假设 $\mathcal{P} \subseteq \mathcal{P}(\mathcal{Z})$，其中 $\mathcal{P}(\mathcal{Z})$ 表示所有 Borel 概率分布的全集，这些分布定义在闭集 $\mathcal{Z} \subseteq \mathbb{R}^d$ 上。本节回顾了几种常见的模糊集合类别。对于每个类别，我们首先给出正式定义并提供历史背景信息。随后，我们举出重要的模糊集合实例，并突出它们的应用。




### 2.1. 矩量模糊集合

矩量模糊集合是一个满足有限个（广义的）矩量条件的概率分布族。形式上，它可以表示为：

$$
\mathcal{P}=\left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{E}_{\mathrm{P}}[f(\mathrm{Z})] \in \mathcal{F}\right\},
$$

其中 $f: \mathcal{Z} \rightarrow \mathbb{R}^m$ 是一个 Borel 可测矩量函数，$\mathcal{F} \subseteq \mathbb{R}^m$ 是一个不确定性集合。根据定义，矩量模糊集合（2.1）包含所有支持在 $\mathcal{Z}$ 上的概率分布 $P$，其广义矩量 $\mathbb{E}_{\mathrm{P}}[f(Z)]$ 是定义良好的，并且属于不确定性集合 $\mathcal{F}$。类型为（2.1）的模糊集合首次由 Isii $(1960, 1962)$ 和 Karlin 与 Studden (1966) 研究，用于建立广义切比雪夫不等式的精确度。以下小节回顾了矩量模糊集合的常见实例。


#### 2.1.1. 仅支持模糊集合

仅支持模糊集合包含所有支持在 $\mathcal{Z} \subseteq \mathbb{R}^d$ 上的概率分布，即 $\mathcal{P} = \mathcal{P}(\mathcal{Z})$。它可以视为（2.1）的一种实例，设 $f(z) = 1$ 且 $\mathcal{F} = \{1\}$。任何具有模糊集合 $\mathcal{P}(\mathcal{Z})$ 的 DRO 问题，本质上都等同于一个经典的鲁棒优化问题，其不确定性集合为 $\mathcal{Z}$，即：

$$
\inf _{x \in \mathcal{X}} \sup _{P \in \mathcal{P}(\mathcal{Z})} \mathrm{B}_{\mathrm{P}}[\ell(x, Z)] = \inf _{x \in \mathcal{X}} \sup _{z \in \mathcal{Z}} \ell(x, z) .
$$

有关鲁棒优化的理论和应用的综合回顾，请参见 (Ben-Tal 和 Nemirovski 1998, 1999a, 2000, 2002, Bertsimas 和 Sim 2004, Ben-Tal 等人 2009, Bertsimas, Brown 和 Caramanis 2011, Ben-Tal, den Hertog 和 Vial 2015a, Bertsimas 和 den Hertog 2022)。

如果不确定集合 $\mathcal{Z}$ 覆盖了某个分布 $\mathbb{P}$ 的总概率质量的 $1-\varepsilon$ 的一部分，那么最坏情况损失 $\sup _{z \in \mathcal{Z}} \ell(x, z)$ 保证超过在 $\mathbb{P}$ 下 $\ell(x, Z)$ 的 $(1-\varepsilon)$-分位数。这可以通过利用 $\mathbb{P}$ 的先验结构信息或统计数据来实现。例如，如果 $\mathbb{P}(Z \in \mathcal{Z}) \geq 1-\varepsilon$（确定成立），则可以通过构造一个合适大小的半空间和椭球体的交集来满足这一条件，且如果 $Z$ 在 $\mathbb{P}$ 下具有独立的、对称的、单峰的和/或次高斯分量（Bertsimas and Sim 2004, Janak, Lin and Floudas 2007, BenTal et al. 2009, Li, Ding and Floudas 2011, Bertsimas, den Hertog and Pauphilet 2021）。或者，如果 $\mathcal{Z}$ 是通过使用统计假设检验（Postek, den Hertog and Melenberg 2016, Bertsimas, Gupta and Kallus 2018b,a）、分位数估计（Hong, Huang and Lam 2021）或基于学习的方法（Han, Shang and Huang 2021, Goerigk and Kurtz 2023, Wang, Becker, Van Parys and Stellato 2023）从 $\mathbb{P}$ 的独立样本中构建的，则可能高置信度成立。

### 2.1.2. Markov 模糊集合

Markov 不等式为一个非负单变量随机变量 $Z$ 提供了一个上界，该随机变量的均值为 $\mu \geq 0$，且其超过一个正阈值 $\tau>0$ 的概率。正式地，它表明对于模糊集合 $\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}\left(\mathbb{R}_{+}\right): \mathbb{E}_{\mathbb{P}}[Z]=\mu\right\}$ 中的每个可能的 $Z$ 的概率分布，$\mathbb{P}(Z \geq \tau) \leq \mu / \tau$。如果 $\mu \leq \tau$，那么 Markov 不等式是紧的，即存在一个概率分布 $\mathbb{P}^{\star} \in \mathcal{P}$，使得该不等式成立为等式。实际上，分布 $\mathbb{P}^{\star}=(1-\mu / \tau) \delta_0 + \mu / \tau \delta_\tau$，其中 $\delta_z$ 是 Dirac 分布，它将点质量放置在 $z \in \mathbb{R}$ 上，属于 $\mathcal{P}$ 并满足 $\mathbb{P}(Z \geq \tau) = \mu / \tau$。这些洞察表明 $\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{P}(Z \geq \tau) = \mu / \tau$，并且当 $\mu \leq \tau$ 时，最大值由 $\mathbb{P}^{\star}$ 达到。因此，Markov 上界可以被解释为一个 DRO 问题的最优值。因此，通常将 $\mathcal{P}$ 称为 Markov 模糊集合。更一般地，我们将与闭合支持集 $\mathcal{Z} \subseteq \mathbb{R}^d$ 和均值向量 $\mu \in \mathbb{R}^d$ 对应的 Markov 模糊集合定义为形式为

$$
\mathcal{P}=\left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_{\mathrm{P}}[Z]=\mu\right\} .
$$

因此，Markov 模糊集合（2.2）包含所有支持在 $\mathcal{Z}$ 上的分布，这些分布具有相同的均值向量 $\mu$。然而，这些分布可能具有截然不同的形状和高阶矩。Markov 模糊集合上的最坏情况期望有时被用作在随机规划中对期望成本函数的有效计算上界。如果成本函数在不确定问题参数中是凹的，则这些最坏情况期望与 Jensen 不等式紧密相关（Jensen 1906）；详见第 6.1 节。另一方面，如果成本函数是凸的，并且 $\mathcal{Z}$ 是一个多面体，则这些最坏情况期望与 Edmundson-Madansky 不等式相关（Edmundson 1956，Madansky 1959）；详见第 6.2 节。



### 2.1.3. Chebyshev 模糊集合

Chebyshev 不等式提供了一个上界，说明一个具有有限均值 $\mu \in \mathbb{R}$ 和方差 $\sigma^2>0$ 的单变量随机变量 $Z$ 偏离其均值超过 $k>0$ 个标准差的概率。正式地，它表明对于模糊集合 $\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathbb{R}): \mathbb{E}_{\mathbb{P}}[Z]=\mu, \mathbb{E}_{\mathbb{P}}\left[Z^2\right]=\sigma^2+\mu^2\right\}$ 中的每个可能的 $Z$ 的概率分布，$\mathbb{P}(|Z-\mu| \geq k \sigma) \leq 1 / k^2$。如果 $k \geq 1$，则 Chebyshev 不等式是紧的。实际上，可以很容易验证分布

$$
\mathbb{P}^{\star}=\frac{1}{2 k^2} \delta_{\mu-k \sigma}+\left(1-\frac{1}{k^2}\right) \delta_\mu+\frac{1}{2 k^2} \delta_{\mu+k \sigma}
$$

是 $\mathcal{P}$ 的一个元素，并且满足 $\mathbb{P}(|Z-\mu| \geq k \sigma)=1 / k^2$。这些洞察表明，Chebyshev 上界可以被解释为一个 DRO 问题的最优值。因此，通常将 $\mathcal{P}$ 称为 Chebyshev 模糊集合。更一般地，我们将与闭合支持集 $\mathcal{Z} \subseteq \mathbb{R}^d$、均值向量 $\mu \in \mathbb{R}^d$ 和二阶矩阵 $M \in \mathbb{S}_{+}^d, M \geq \mu \mu^{\top}$ 对应的 Chebyshev 模糊集合定义为

$$
\mathcal{P}=\left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{E}_{\mathrm{P}}[Z]=\mu, \mathbb{E}_{\mathrm{P}}\left[Z Z^{\top}\right]=M\right\}
$$

因此，Chebyshev 模糊集合（2.3）包含所有支持在 $\mathcal{Z}$ 上的分布，这些分布具有相同的均值向量 $\mu$ 和二阶矩阵 $M$（因此也具有相同的协方差矩阵 $\Sigma=M-\mu \mu^{\top} \in \mathbb{S}_{+}^d$）。然而，这些分布可能具有截然不同的形状和高阶矩。

Chebyshev 模糊集合（2.3）捕捉了与多变量 Chebyshev 不等式相关的分布信息（Lal 1955，Marshall 和 Olkin 1960，Tong 1980，Rujeerapaiboon, Kuhn 和 Wiesemann 2018）。在运筹学中，Chebyshev 模糊集合自 Scarf（1958）关于分布鲁棒新闻供应商的开创性工作以来被广泛使用，该工作通常被认为是 DRO 的第一篇论文。从那时起，许多与 Chebyshev 模糊集合相关的 DRO 模型在新闻供应商和投资组合选择问题的背景下涌现。这些模型涉及各种不同的决策标准，如期望值（Gallego 和 Moon 1993，Natarajan 和 Linyi 2007，Popescu 2007），风险价值（El Ghaoui 等 2003，Xu, Caramanis 和 Mannor 2012b，Zymler, Kuhn 和 Rustem 2013a,b，Rujeerapaiboon, Kuhn 和 Wiesemann 2016，Yang 和 Xu 2016，Zhang, Jiang 和 Shen 2018），条件风险价值（Natarajan, Sim 和 Uichanco 2010，Chen, He 和 Zhang 2011，Zymler 等 2013b，Hanasusanto, Kuhn, Wallace 和 Zymler 2015a），谱风险度量（Li 2018）和失真风险度量（Cai, Li 和 Mao 2023，Pesenti, Wang 和 Wang 2024），以及最大最小遗憾标准（Yue, Chen 和 Wang 2006，Perakis 和 Roels 2008）。2008)。除此之外，切比雪夫模糊集在期权和股票定价（Bertsimas 和 Popescu 2002）、统计学和机器学习（Lanckriet, El Ghaoui, Bhattacharyya 和 Jordan 2001, 2002, Strohmann 和 Grudic 2002, Huang, Yang, King, Lyu 和 Chan 2004, Bhattacharyya 2004, Farnia 和 Tse 2016, Nguyen, Shafieezadeh-Abadeh, Yue, Kuhn 和 Wiesemann 2019, Rontsis, Osborne 和 Goulart 2020）、随机规划（Birge 和 Wets 1986, Dulá 和 Murthy 1992, Dokov 和 Morton 2005, Bertsimas, Doan, Natarajan 和 Teo 2010, Natarajan, Teo 和 Zheng 2011）、控制（Van Parys, Kuhn, Goulart 和 Morari 2015, Yang 2018, Xin 和 Goldberg 2021, 2022）、电力系统运行（Xie 和 Ahmed 2017, Zhao 和 Jiang 2017）、复杂网络分析（Van Leeuwaarden 和 Stegehuis 2021, Brugman, Van Leeuwaarden 和 Stegehuis 2022）、排队系统（van Eekelen, Hanasusanto, Hasenbein 和 van Leeuwaarden 2023）、医疗保健（Mak, Rong 和 Zhang 2015, Shehadeh, Cohn 和 Jiang 2020）、极端事件分析（Lam 和 Mottet 2017）等方面都得到了广泛应用。


2.1.4. 切比雪夫模糊集与不确定的矩

在已知 $\mathbb{P}$ 的一阶和二阶矩时，使用切比雪夫模糊集是合适的，但所有更高阶的矩是未知的。然而，在实际中，即使是已知的一阶和二阶矩也从来不能绝对确定。相反，它们必须从统计数据中估计，因此会受到估计误差的影响。这促使 El Ghaoui 等人（2003）引入了具有不确定矩的切比雪夫模糊集，可以表示为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}):\left(\mathrm{E}_{\mathrm{P}}[Z], \mathrm{B}_{\mathrm{P}}\left[Z Z^{\top}\right]\right) \in \mathcal{F}\right\} .
$$

这里，$\mathcal{F} \subseteq \mathbb{R}^d \times \mathbb{S}_{+}^d$ 是一个凸集，用于捕捉矩的不确定性。显然，$\mathcal{P}$ 可以表示为一系列切比雪夫模糊集的并集，即：

$$
\mathcal{P}=\bigcup_{(\mu, M) \in \mathcal{F}}\left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{E}_{\mathrm{P}}[Z]=\mu, \mathbb{E}_{\mathrm{P}}\left[Z Z^{\top}\right]=M\right\} .
$$

注意，具有不确定矩的切比雪夫模糊集涵盖了支持集模糊集、马尔可夫模糊集和切比雪夫模糊集作为特例。它们通过设置 $\mathcal{F}=\mathbb{R}^d \times \mathbb{S}_{+}^d, \mathcal{F}=\{\mu\} \times \mathbb{S}_{+}^d$ 和 $\mathcal{F}=\{\mu\} \times\{M\}$ 分别恢复。

El Ghaoui 等人（2003）通过以下盒子捕捉矩的不确定性：

$$
\mathcal{F}=\left\{(\mu, M) \in \mathbb{R}^d \times \mathbb{S}_{+}^d: \underline{\mu} \leq \mu \leq \bar{\mu}, \underline{M} \leq M \leq \bar{M}\right\}
$$

该框由矩的界限参数化，其中 $\underline{\mu}, \bar{\mu} \in \mathbb{R}^d$ 和 $\underline{M}, \bar{M} \in \mathbb{S}_{+}^d$。

给定对未知均值向量和协方差矩阵的噪声估计 $\hat{\mu}$ 和 $\hat{\Sigma}$，Delage 和 Ye（2010）提出了以下模糊集：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \begin{array}{l}
\left(\mathbb{E}_{\mathrm{P}}[Z]-\hat{\mu}\right)^{\top} \hat{\Sigma}^{-1}\left(\mathbb{E}_{\mathrm{P}}[Z]-\hat{\mu}\right) \leq \gamma_1 \\
\mathbb{E}_{\mathrm{P}}\left[(Z-\hat{\mu})(Z-\hat{\mu})^{\top}\right] \leq \gamma_2 \hat{\Sigma}
\end{array}\right\} .
$$

通过构造，$\mathcal{P}$ 包含所有分布在 $\mathcal{Z}$ 上，其一阶矩位于以 $\hat{\mu}$ 为中心的椭球内，二阶矩（相对于 $\hat{\mu}$）位于以 $\gamma_2 \hat{\Sigma}$ 为顶点的半正定锥内。一个基本的计算揭示了：

$$
\mathbb{E}_{\mathrm{P}}\left[(Z-\hat{\mu})(Z-\hat{\mu})^{\top}\right]=\mathrm{E}_{\mathrm{P}}\left[Z Z^{\top}\right]-\mathbb{B}_{\mathrm{P}}[Z] \hat{\mu}^{\top}-\hat{\mu} \mathrm{E}_{\mathrm{P}}[Z]^{\top}+\hat{\mu} \hat{\mu}^{\top}
$$
因此，$\mathbb{P}$ 可以视为一个具有不确定矩的切比雪夫模糊集合。实际上，如果我们将矩不确定集定义为

$$\mathcal{F}=\left\{(\mu, M) \in \mathbb{R}^d \times \mathbb{S}_{+}^d: \begin{array}{l}(\mu-\hat{\mu})^{\top} \hat{\Sigma}(\mu-\hat{\mu}) \leq \gamma_1 \\ M-\mu \hat{\mu}^{\top}-\hat{\mu} \mu^{\top}+\hat{\mu} \hat{\mu}^{\top} \leq \gamma_2 \hat{\Sigma}\end{array}\right\}.
$$

Delage 和 Ye（2010）表明，如果 $\rho$ 和 $\sum$ 被设置为样本均值和从有限数量的独立样本中提取的协方差矩阵，那么可以调整集参数 $\gamma_1 \geq 0$ 和 $\gamma_2 \geq 1$，以确保 $\mathbb{P}$ 属于 $\mathcal{P}$，并具有任何期望的置信度。

具有不确定矩的切比雪夫和马尔可夫模糊集合在多个领域找到了应用，涉及的领域包括控制（Naka, Jiang 和 Shen 2021）、积分随机规划（Bertamas, Natarajan 和 Teo 2004）、事件分析（Bai, Lam 和 Doung 2023）、机制设计和定价（Bertsimas 2024）等众多领域。

不确定集$\mathcal{F}$通常对应于某个名义均值-协方差对$(\hat{\mu}, \hat{\Sigma})$的邻域，依据某种偏差度量。例如，矩阵范数，如Frobenius范数、谱范数或核范数（Bernstein 2009, § 9），提供了自然的度量方法来量化协方差矩阵的相似性。两个均值-协方差对$(\mu, \Sigma)$和$(\hat{\mu}, \hat{\Sigma})$之间的偏差也可以定义为正态分布$\mathcal{N}(\mu, \Sigma)$和$\mathcal{N}(\hat{\mu}, \hat{\Sigma})$之间的偏差，依据某种概率度量或信息论上的散度，如Kullback-Leibler散度（Kullback 1959）、Fisher-Rao距离（Atkinson 和 Mitchell 1981）或其他谱散度（Zorzi 2014）。

正如我们将在第2.3节中更详细地讨论的那样，两个正态分布$\mathcal{N}(\mu, \Sigma)$和$\mathcal{N}(\hat{\mu}, \hat{\Sigma})$之间的2-Wasserstein距离与其底层均值-协方差对$(\mu, \Sigma)$和$(\hat{\mu}, \hat{\Sigma})$之间的Gelbrich距离一致。接下来，我们首先给出Gelbrich距离的正式定义，然后举例说明如何用它来定义一个矩 moment 不确定集$\mathcal{F}$。

定义 2.1 (Gelbrich 距离)。在 $\mathbb{R}^d \times \mathbb{S}_{+}^d$ 中，两个均值协方差对 $(\mu, \Sigma)$ 和 $(\hat{\mu}, \hat{\Sigma})$ 之间的 Gelbrich 距离定义为

$$
\mathrm{G}((\mu, \Sigma),(\hat{\mu}, \hat{\Sigma}))=\sqrt{\|\mu-\hat{\mu}\|_2^2+\operatorname{Tr}\left(\Sigma+\hat{\Sigma}-2\left(\hat{\Sigma}^{\frac{1}{2}} \Sigma \hat{\Sigma}^{\frac{1}{2}}\right)^{\frac{1}{2}}\right)} .
$$

Gelbrich 距离是非负的、对称的和次可加的，且当且仅当 $(\mu, \Sigma) = (\hat{\mu}, \hat{\Sigma})$ 时它为零。因此，它表示了 $\mathbb{R}^d \times \mathbb{S}_{+}^d$ 上的一个度量（Givens 和 Short 1984, 第239页）。当 $\mu = \hat{\mu}$ 时，Gelbrich 距离退化为 $\Sigma$ 和 $\hat{\Sigma}$ 之间的 Bures 距离，后者作为量子信息理论中密度矩阵之间相异性的度量。Bures 距离已知能在正半定矩阵空间上诱导一个黎曼度量（Bhatia, Jain 和 Lim 2018, 2019）。当 $\Sigma$ 和 $\hat{\Sigma}$ 可以同时对角化时，它们的 Bures 距离与它们谱之间的 Hellinger 距离一致。Hellinger 距离与信息理论中广泛使用的 Fisher-Rao 度量密切相关（Liese 和 Vajda 1987）。尽管 Gelbrich 距离是非凸的，但平方的 Gelbrich 距离在其两个参数中是联合凸的。这是以下命题的直接结果，该命题可以在（Olkin 和 Pukelsheim 1982, Dowson 和 Landau 1982, Givens 和 Shortt 1984, Panaretos 和 Zemel 2020）中找到。

命题 2.2 (Gelbrich 距离的 SDP 表示)。对于任意均值协方差对 $(\mu, \Sigma)$ 和 $(\hat{\mu}, \hat{\Sigma})$ 在 $\mathbb{R}^d \times \mathbb{S}_{+}^d$ 中，我们有

$$
\mathrm{G}^2((\mu, \Sigma),(\hat{\mu}, \hat{\Sigma}))=\left\{\begin{array}{cl}
\min _{C \in \mathbb{R}^{d \times d}} & \|\mu-\hat{\mu}\|_2^2+\operatorname{Tr}(\Sigma+\hat{\Sigma}-2 C) \\
\text { s.t. } & {\left[\begin{array}{cc}
\Sigma & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0 .}
\end{array}\right.
$$

证明：在证明过程中，我们保持 $\mu, \hat{\mu}$ 和 $\Sigma$ 不变，并将 $\hat{\Sigma}$ 视为参数。我们还使用 $f(\hat{\Sigma})$ 作为 (2.5) 左侧的简写，$g(\hat{\Sigma})$ 作为 (2.5) 右侧的简写。基础操作显示

$$
g(\hat{\Sigma})=\|\mu-\hat{\mu}\|_2^2+\operatorname{Tr}(\Sigma+\hat{\Sigma})-\left\{\begin{array}{cl}
\max _{C \in \mathbb{R}^{d \times d}} & \operatorname{Tr}(2 C) \\
\text { s.t. } & {\left[\begin{array}{cc}
\Sigma & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0 .}
\end{array}\right.
$$

(2.6) 中的最大化问题是以下最小化问题的对偶问题。

$$
\begin{array}{cl}
\inf _{A_{11}, A_{22} \in \mathbb{S}^d} & \operatorname{Tr}\left(A_{11} \Sigma\right)+\operatorname{Tr}\left(A_{22} \hat{\Sigma}\right) \\
\text { s.t. } & {\left[\begin{array}{cc}
A_{11} & I_d \\
I_d & A_{22}
\end{array}\right] \geq 0}
\end{array}
$$

由于 $A_{11}=A_{22}=2 I_d$ 构成了对偶问题的 Slater 点（Ben-Tal 和 Nemirovski 2001, Theorem 2.4.1），因此强对偶性成立。Slater 点的存在进一步意味着 (2.6) 中的原始最大化问题以及 (2.5) 中的最小化问题是可解的。根据 (Bernstein 2009, Corollary 8.2.2)，为了保证对偶可行性，$A_{11}$ 和 $A_{22}$ 必须是正定的。因此，它们是可逆的。我们可以使用 Schur 互补论证（Ben-Tal 和 Nemirovski 2001, Lemma 4.2.1）将对偶问题简化为

$$
\inf _{A_{11} \geq A_{22}^{-1}>0} \operatorname{Tr}\left(A_{11} \Sigma\right)+\operatorname{Tr}\left(A_{22} \hat{\Sigma}\right)=\inf _{A_{22}>0} \operatorname{Tr}\left(A_{22}^{-1} \Sigma\right)+\operatorname{Tr}\left(A_{22} \hat{\Sigma}\right),
$$
其中等式成立是因为 $\Sigma \geq 0$。得到的最小化问题的最优值在 $\hat{\Sigma}$ 中是凹的，并且是上半连续的，因为它构成了 $\hat{\Sigma}$ 的仿射函数的逐点下确界。因此，$g(\hat{\Sigma})$ 是凸的并且下半连续的。现在我们展示，如果 $\hat{\Sigma} > 0$，那么 (2.7) 中对 $A_{22}$ 的凸最小化问题可以解析求解。为此，我们构造一个满足问题一阶最优性条件的正定矩阵 $A_{22}^{\star}$：

$$
\hat{\Sigma}-A_{22}^{-1} \Sigma A_{22}^{-1}=0 \quad \Longleftrightarrow \quad A_{22} \hat{\Sigma} A_{22}-\Sigma=0 .
$$

事实上，右侧的二次方程两边同时与 $\hat{\Sigma}^{\frac{1}{2}}$ 相乘，得到等价方程 $\left(\hat{\Sigma}^{\frac{1}{2}} A_{22} \hat{\Sigma}^{\frac{1}{2}}\right)^2=\hat{\Sigma}^{\frac{1}{2}} \Sigma^{\frac{1}{2}}$。由于 $\hat{\Sigma} > 0$，该方程唯一解为 $A_{22}^{\star}=\hat{\Sigma}^{-\frac{1}{2}}\left(\hat{\Sigma}^{\frac{1}{2}} \Sigma \hat{\Sigma}^{\frac{1}{2}}\right)^{\frac{1}{2}} \hat{\Sigma}^{-\frac{1}{2}}$。将 $A_{22}^{\star}$ 代入 (2.7) 中，结果显示对偶最小化问题的最优值为 $2 \operatorname{Tr}\left(\left(\hat{\Sigma}^{\frac{1}{2}} \Sigma \hat{\Sigma}^{\frac{1}{2}}\right)^{\frac{1}{2}}\right)$。然后将该值代入 (2.6) 中，显示当 $\hat{\Sigma} > 0$ 时，$g(\hat{\Sigma})=f(\hat{\Sigma})$。

接下来我们展示，如果 $\hat{\Sigma}$ 是奇异的，那么 $g(\hat{\Sigma}) = f(\hat{\Sigma})$。为此，我们回顾 (Nguyen, Shafieezadeh-Abadeh, Kuhn 和 Mohajerin Esfahani 2023, Lemma A.2) 中的结论，即矩阵平方根在 $\mathbf{S}_{+}^d$ 上是连续的，这意味着 $f(\hat{\Sigma})$ 在 $\mathbb{S}_{+}^d$ 上是连续的。对于任何奇异的 $\hat{\Sigma} \geq 0$，我们有

$$
f(\hat{\mathbf{\Sigma}})=\liminf _{\hat{\mathbf{\Sigma}}^{\prime} \rightarrow \hat{\mathbf{\Sigma}}, \hat{\mathbf{\Sigma}}^{\prime}>0} f\left(\hat{\mathbf{\Sigma}}^{\prime}\right)=\liminf _{\hat{\mathbf{\Sigma}}^{\prime} \rightarrow \hat{\mathbf{\Sigma}}, \hat{\Sigma}^{\prime}>0} g\left(\hat{\Sigma}^{\prime}\right)=g(\hat{\mathbf{\Sigma}}) .
$$

这里，第一个等式利用了 $f$ 的连续性，第二个等式成立是因为对于每个 $\hat{\Sigma}^{\prime} > 0$，$f\left(\hat{\Sigma}^{\prime}\right)=g\left(\hat{\Sigma}^{\prime}\right)$。第三个等式来自于 $g$ 的凸性和下半连续性，这意味着极限下确界不能小于或大于 $g(\hat{\Sigma})$，分别地。由此，证明完毕。

命题 2.2 显示，平方的 Gelbrich 距离与一个可解的半正定程序的最优值相同。这使得 Gelbrich 距离在计算中具有吸引力。作为副产品，命题 2.2 的证明揭示了平方 Gelbrich 距离在其定义域上是凸的并且连续的。

根据 Nguyen、Shafiee、Filipovic 和 Kuhn (2021) 的方法，我们现在可以将 Gelbrich 模糊集合作为不确定矩的 Chebyshev 模糊集合 (2.4) 的一个实例。相应的矩不确定集合如下所示：

$$
\mathcal{F}=\left\{(\mu, M) \in \mathbb{R}^d \times \mathbb{S}_{+}^d: \begin{array}{l}
\exists \Sigma \in \mathbb{S}_{+}^d \text { with } M=\Sigma+\mu \mu^{\top}, \\
\mathrm{G}((\mu, \Sigma),(\hat{\mu}, \hat{\Sigma})) \leq r
\end{array}\right\},
$$

其中 $(\hat{\mu}, \hat{\Sigma})$ 是一个名义均值-协方差对，半径 $r \geq 0$ 是一个可调节的大小参数。下面我们将 $\mathcal{F}$ 称为 Gelbrich 不确定集合。下一个命题建立了 $\mathcal{F}$ 的基本拓扑和计算性质。

命题 2.3 (Gelbrich 不确定集合)。不确定集合 $\mathcal{F}$ 如 (2.8) 所定义，是凸的并且是紧的。此外，它具有以下半正定表示：

$$
\mathcal{F}=\left\{\begin{array}{ll}
(\mu, M) \in \mathbb{R}^d \times \mathbb{S}_{+}^d: & \exists C \in \mathbb{R}^{d \times d}, U \in \mathbb{S}_{+}^d \text { with } \\
\|\hat{\mu}\|_2^2-2 \mu^{\top} \hat{\mu}+\operatorname{Tr}(M+\hat{\Sigma}-2 C) \leq r^2, \\
{\left[\begin{array}{cc}
M-U & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0,\left[\begin{array}{cc}
U & \mu \\
\mu^{\top} & 1
\end{array}\right] \geq 0}
\end{array}\right\} .
$$

证明。该证明利用了命题 2.2 中建立的平方 Gelbrich 距离的半正定表示。首先注意到，如果 $M = \Sigma + \mu \mu^{\top}$，那么

$$
\|\mu - \hat{\mu}\|_2^2 + \operatorname{Tr}(\Sigma + \hat{\Sigma} - 2 C) = \|\hat{\mu}\|_2^2 - 2 \mu^{\top} \hat{\mu} + \operatorname{Tr}(M + \hat{\Sigma} - 2 C) .
$$

根据命题 2.2，Gelbrich 不确定集合 $\mathcal{F}$ 可以表示为

$$
\mathcal{F} = \left\{ \begin{array}{ll}
& \exists C \in \mathbb{R}^{d \times d} \text{ with } \\
& \|\hat{\mu}\|_2^2 - 2 \mu^{\top} \hat{\mu} + \operatorname{Tr}(M + \hat{\Sigma} - 2 C) \leq r^2, \\
{\left[\begin{array}{cc}
M - \mu \mu^{\top} & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0}
\end{array} \right\} .
$$

一个标准的 Schur 余补推论进一步揭示

$$
\left[\begin{array}{cc}
M - \mu \mu^{\top} & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0 \Longleftrightarrow \exists U \in \mathbb{S}_{+}^d \text{ with } \left[\begin{array}{cc}
M - U & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0, \left[\begin{array}{cc}
U & \mu \\
\mu^{\top} & 1
\end{array}\right] \geq 0 .
$$

因此，Gelbrich 不确定集合 $\mathcal{F}$ 具有命题中给出的半正定表示。由于该表示通过锥不等式定义的集合的投影表达了 $\mathcal{F}$，因此其凸性显而易见。

接下来需要证明的是，$\mathcal{F}$ 是紧的。为此，我们定义

$$
\mathcal{V} = \left\{ (\mu, \Sigma) \in \mathbb{R}^d \times \mathbb{S}_{+}^d: \mathrm{G}((\mu, \Sigma), (\hat{\mu}, \hat{\Sigma})) \leq r \right\}
$$

作为关于 $(\hat{\mu}, \hat{\Sigma})$ 的半径为 $r$ 的球，使用 Gelbrich 距离来度量。注意到 $\mathcal{F} = f(\mathcal{V})$，其中变换 $f: \mathbb{R}^d \times \mathbb{S}_{+}^d \rightarrow \mathbb{R}^d \times \mathbb{S}_{+}^d$ 定义为 $f(\mu, \Sigma) = \left(\mu, \Sigma + \mu \mu^{\top}\right)$。现在我们将证明 $\mathcal{V}$ 是紧的。由于 $f$ 是连续的，并且紧性在连续变换下保持，因此这将直接推导出 $\mathcal{F}$ 是紧的。显然，$\mathcal{V}$ 是闭的，因为 Gelbrich 距离是连续的。为了证明 $\mathcal{V}$ 也是有界的，固定任意 $(\mu, \Sigma) \in \mathcal{V}$。根据 Gelbrich 距离的定义，我们有 $\|\mu - \hat{\mu}\| \leq r^2$。此外，我们可以得到

$$
\begin{aligned}
\operatorname{Tr}\left(\left(\hat{\Sigma}^{\frac{1}{2}} \Sigma \hat{\Sigma}^{\frac{1}{2}}\right)^{\frac{1}{2}}\right) &= \max_{C \in \mathbb{R}^{d \times d}} \left\{ \operatorname{Tr}(C): \left[\begin{array}{cc}
\Sigma & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0 \right\} \\
&\leq \max_{C \in \mathbb{R}^{d \times d}} \left\{ \operatorname{Tr}(C): C_{ij}^2 \leq \Sigma_{ii} \hat{\Sigma}_{jj} \ \forall i,j \in[d] \right\} \leq \sqrt{\operatorname{Tr}(\Sigma) \operatorname{Tr}(\hat{\Sigma})},
\end{aligned}
$$

其中，等式已在命题 2.2 的证明中建立。两个不等式来自于线性矩阵不等式的放松，它利用了一个观察结果，即一个正半定矩阵的所有二阶主子式是非负的，并且使用了 Cauchy-Schwarz 不等式。因此，$\Sigma$ 满足

$$
\left.\left(\operatorname{Tr}(\Sigma)^{\frac{1}{2}} - \operatorname{Tr}(\hat{\Sigma})^{\frac{1}{2}}\right)^2 \leq \operatorname{Tr}\left(\Sigma + \hat{\Sigma} - 2\left(\hat{\Sigma}^{\frac{1}{2}} \Sigma \hat{\Sigma}^{\frac{1}{2}}\right)^{\frac{1}{2}}\right)\right) \leq r^2,
$$

其中第二个不等式成立是因为 $(\mu, \Sigma) \in \mathcal{V}$。因此，我们可以得出结论：$\operatorname{Tr}(\Sigma) \leq \left(r + \operatorname{Tr}(\hat{\Sigma})^{\frac{1}{2}}\right)^2$，进而得到 $0 \leq \Sigma \leq \left(r + \left(\operatorname{Tr}(\hat{\Sigma})\right)^{\frac{1}{2}}\right)^2 I_d$。总之，我们已经证明了 $\mu$ 和 $\Sigma$ 都属于有界集合。由于选择的 $(\mu, \Sigma) \in \mathcal{V}$ 是任意的，这证明了 $\mathcal{V}$ 确实是有界的，因此是紧的。得证。

命题 2.2 显示了不确定性集合 $\mathcal{F}$ 是凸的。这一点令人惊讶，因为 $\mathcal{F} = f(\mathcal{V})$，其中 Gelbrich 球 $\mathcal{V}$ 在均值-协方差对的空间中是凸的，得益于命题 2.2，而 $f$ 是一个二次双射。事实上，通常只有在仿射变换下，凸性才会被保留。

Gelbrich 不确定性集是由 Nguyen 等人（2021）在稳健投资组合优化的背景下引入的。它们还在机器学习（Bui, Nguyen 和 Nguyen 2022, Vu, Tran, Yue 和 Nguyen 2021, Nguyen, Bui 和 Nguyen 2022a）、估计（Nguyen 等人 2023）、滤波（Shafieezadeh-Abadeh, Nguyen, Kuhn 和 Mohajerin Esfahani 2018, Kargin, Hajar, Malik 和 Hassibi 2024b）以及控制（McAllister 和 Mohajerin Esfahani 2023, Al Taha, Yan 和 Bitar 2023, Hajar, Kargin 和 Hassibi 2023, Hakobyan 和 Yang 2024, Taşkesen, Iancu, Koçyiğit 和 Kuhn 2024, Kargin, Hajar, Malik 和 Hassibi 2024a,c,d）中找到了应用。



### 2.1.5. 均值-离散不确定性集

如果 $\mathcal{K} \subseteq \mathbb{R}^k$ 是一个适当的凸锥，且 $v_1, v_2 \in \mathbb{R}^k$，则不等式 $v_1 \leq \mathcal{K} v_2$ 表示 $v_2 - v_1 \in \mathcal{K}$。此外，函数 $G: \mathbb{R}^m \rightarrow \mathbb{R}^k$ 被称为 $\mathcal{K}$-凸的，如果满足

$$
G\left(\theta v_1 + (1 - \theta) v_2\right) \leq \mathcal{K} \theta G\left(v_1\right) + (1 - \theta) G\left(v_2\right) \quad \forall v_1, v_2 \in \mathbb{R}^m, \forall \theta \in [0,1] .
$$

对应于一个凸闭支持集 $\mathcal{Z} \subseteq \mathbb{R}^d$，均值向量 $\mu \in \mathbb{R}^d$，一个 $\mathcal{K}$-凸的离散函数 $G: \mathbb{R}^m \rightarrow \mathbb{R}^k$ 和一个离散界限 $g \in \mathbb{R}^k$，均值-离散不确定性集定义为

$$
\mathcal{P} = \left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{B}_{\mathrm{P}}[Z] = \mu, \mathbb{E}_{\mathrm{P}}[G(Z)] \leq \mathcal{K} g\right\}
$$

均值-离散不确定性集具有很强的表达能力，即它能够建模未知概率分布的各种典型特征。例如，如果 $\|\cdot\|$ 是 $\mathbb{R}^d$ 上的一个范数，$G(z) = \|z - \mu\|$ 是通常意义上的凸函数，且 $g = \sigma \in \mathbb{R}_{+}$，则所有的分布 $\mathbb{P} \in \mathcal{P}$ 的均值绝对偏差都被 $\sigma$ 有界。或者，如果 $G(z) = (z - \mu)(z - \mu)^{\top}$ 是 $\mathbb{S}_{+}^d$-凸的，且 $g = \Sigma \in \mathbb{S}_{+}^d$，则 $\mathcal{P}$ 退化为具有时刻不确定性的 Chebyshev 不确定性集。具体来说，任何 $\mathbb{P} \in \mathcal{P}$ 的协方差矩阵都被 $\Sigma$ 限制在 Loewner 顺序中。Wiesemann, Kuhn 和 Sim (2014) 表明，包含 $d$ 维随机向量 $Z$ 分布的不确定性集 $\mathcal{P}$ 与提升的不确定性集紧密相关：

$$
\mathcal{Q} = \left\{\mathbb{Q} \in \mathcal{P}(\mathcal{C}): \mathbb{B}_{\mathrm{Q}}[Z] = \mu, \mathbb{E}_{\mathrm{Q}}[U] = g\right\}
$$

其中支持集 $\mathcal{C} = \left\{(z, u) \in \mathcal{Z} \times \mathbb{R}^k: G(z) \leq_{\mathcal{K}} u\right\}$，包含 $Z$ 和一个辅助的 $m$ 维随机向量 $U$ 的联合分布。实际上，可以证明 $\mathcal{P} = \left\{Q_z: Q \in \mathcal{Q}\right\}$，其中 $Q_z$ 表示 $Q$ 下的 $Z$ 的边际分布。由于损失函数仅依赖于 $Z$ 而不依赖于 $U$，这一推理意味着（1.2）中的内层最坏情况期望问题满足

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{B}_{\mathrm{P}}[\ell(x, Z)] = \sup _{\mathrm{Q} \in \mathcal{Q}} \mathbb{E}_{\mathrm{Q}}[\ell(x, Z)] .
$$

因此，可以用提升的不确定性集 $\mathcal{Q}$ 替代原始的不确定性集 $\mathcal{P}$。这是有用的，因为 $\mathcal{Q}$ 是一个简单的 Markov 不确定性集，只指定联合随机向量 $(Z, U)$ 的支持集 $\mathcal{C}$ 和均值 $(\mu, g)$。此外，可以证明 $\mathcal{Z}$ 是凸的，因为 $\mathcal{Z}$ 本身是凸的，且 $G$ 是 $\mathcal{K}$-凸的。总之，具有均值-离散不确定性集（形式为（2.9））的 DRO 问题可以系统地转化为具有 Markov 不确定性集的 DRO 问题。

一种更一般的均值-离散不确定性集可以用来塑造 $Z$ 在 $\mathbb{P}$ 下的矩生成函数。具体来说，Chen、Sim 和 Xu（2019）引入了熵支配不确定性集：

$$
\mathcal{P} = \left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_{\mathbb{P}}[Z] = \mu, \log \left(\mathbb{E}_{\mathbb{P}}\left[\exp \left(\theta^{\top}(Z-\mu)\right)\right]\right) \leq g(\theta) \forall \theta \in \mathbb{R}^d \right\},
$$

其中 $g: \mathbb{R}^d \rightarrow \mathbb{R}$ 是一个凸且连续二次可微的函数，满足 $g(0) = 0$ 且 $\nabla g(0) = 0$。由 $\theta$ 参数化的约束对聚合生成函数（即对中心化随机变量 $Z-\mu$ 的对数矩生成函数）施加了连续的上界。$g$ 的选择决定了不确定性集中包含的具体分布类别。例如，如果 $g(\theta) = \sigma^2 \theta^{\top} \theta / 2$（其中 $\sigma > 0$），则该不确定性集只包含方差代理为 $\sigma^2$ 的亚高斯分布。亚高斯分布是尾部受高斯分布尾部限制的概率分布。它们在概率论和统计学中扮演着重要角色，特别是在浓度不等式和高维现象的研究中（Vershynin 2018，Wainwright 2019）。

熵支配不确定性集对 $\mathbb{P}$ 强加了无限多的约束。Chen 等人（2019）表明，在该不确定性集上的最坏情况期望问题可以重新表述为半无限锥程序。他们提出了一种切平面算法，以高效地求解这些锥程序。熵支配不确定性集还在非线性和 PDE 约束的 DRO 问题研究中得到了应用（Milz 和 Ulbrich 2020，2022）。Chen、Fu、Si、Sim 和 Xiong（2023）考虑了广义的熵支配不确定性集。


### 2.1.6. Higher-order Moment Ambiguity Sets

Markov 和 Chebyshev 不确定性集仅对 $\mathbb{P}$ 的一阶和/或二阶矩施加条件。具有这种不确定性集的 DRO 问题通常是可处理的。在本节中，我们简要讨论对 $\mathbb{P}$ 的高阶（多项式）矩施加条件的矩不确定性集，这通常会导致 NP-hard 的 DRO 问题（Popescu 2005，第 4.5 和 4.6 命题）。

现在假设 $\mathcal{Z}$ 是一个闭半代数集，可以表示为有限多个多项式不等式的可行集。此外，定义在 $z \in \mathbb{R}^d$ 中的阶数为 $\alpha \in \mathbb{Z}_{+}^d$ 的单项式为函数 $\prod_{i=1}^d z_i^{\alpha_i}$，我们简洁地表示为 $z^\alpha$。由有限索引集 $\mathcal{A} \subseteq \mathbb{Z}_{+}^d$ 和矩界限 $m_\alpha \in \mathbb{R}, \alpha \in \mathcal{A}$ 诱导的高阶矩不确定性集为

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_{\mathbb{P}}\left[Z^\alpha\right] \leq m_\alpha \forall \alpha \in \mathcal{A}\right\}
$$

因此，评估多项式函数（或半代数集的特征函数）在所有分布 $\mathcal{P}$ 上的最坏情况期望就等同于求解一个广义的矩问题。这个矩问题及其对偶问题构成了半无限线性规划，可以被重新构造为某些矩锥和相应的非负多项式对偶锥上的有限维锥优化问题（Karlin 和 Studden 1966，Zuluaga 和 Pena 2005）。尽管一般情况下是 NP-hard，这些锥优化问题可以通过多项式优化工具（Parrilo 2000, 2003，Lasserre 2001, 2009）逼近为越来越紧的可处理半定规划序列。这种通用技术为不确定性集 $\mathcal{P}$ 提供了最坏情况期望界限和广义 Chebyshev 不等式（Bertsimas 和 Sethuraman 2000，Lasserre 2002，Popescu 2005，Lasserre 2008）。此外，它还为最坏情况风险度量提供了紧的界限（Natarajan，Pachamanova 和 Sim 2009a）。


## 2.2. $\phi$-Divergence Ambiguity Sets

概率分布之间的差异通常通过 $\phi$-散度来量化，这由一个熵函数 $\phi$ 唯一决定。

**定义 2.4** (熵函数)。熵函数 $\phi: \mathbb{R} \rightarrow \overline{\mathbb{R}}$ 是一个下半连续凸函数，满足 $\phi(1)=0$ 且 $\phi(s)=+\infty$ 对于所有 $s<0$。

注意，任何熵函数 $\phi$ 相对于其定义域是连续的。事实上，这对于任何一元的凸下半连续函数都是成立的。然而，我们强调，多元凸下半连续函数可能在其定义域内有不连续点（Rockafellar 和 Wets 2009, 示例 2.38）。$\phi$-散度的概念依赖于熵函数 $\phi$ 的视角 $\phi^\pi$。

**定义 2.5** ($\phi$-散度 (Csiszár 1963, 1967, Ali 和 Silvey 1966))。$\mathbb{P} \in \mathcal{P}(\mathcal{Z})$ 相对于 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 的（广义）$\phi$-散度定义为

$$
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})=\int_{\mathcal{Z}} \phi^\pi\left(\frac{\mathrm{d} \mathbb{P}}{\mathrm{~d} \rho}(z), \frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z)\right) \mathrm{d} \rho(z),
$$

其中 $\phi$ 是熵函数，$\rho \in \mathcal{M}_{+}(\mathcal{Z})$ 是任何支配测度。这意味着 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 相对于 $\rho$ 是绝对连续的，即 $\mathbb{P}, \hat{\mathbb{P}} \ll \rho$。

根据 $\phi^\pi$ 的定义以及我们的约定 $0 \phi(s / 0)$ 应该解释为 recession function $\phi^{\infty}(s)$，$\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$ 可以重新表达为

$$
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})=\int_{\mathcal{Z}} \frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z) \cdot \phi\left(\frac{\frac{\mathrm{d}}{\mathrm{P}}(z)}{\frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z)}\right) \mathrm{d} \rho(z) .
$$

一个支配测度 $\rho$ 总是存在的，但它必须依赖于 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$。例如，可以设置 $\rho = \mathbb{P} + \hat{\mathbb{P}}$。绝对连续性条件 $\mathbb{P} \ll \rho$ 和 $\hat{\mathbb{P}} \ll \rho$ 确保了 Radon-Nikodym 导数 $\frac{\mathrm{d}}{\mathrm{d} \rho}$ 和 $\frac{\mathrm{d}}{\mathrm{d} \rho}$ 分别是良定义的。以下命题推导了一个通用的 $\phi$-散度的对偶表示，揭示了 $\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$ 实际上与选择 $\rho$ 无关。

命题 2.6 （$\phi$-散度的对偶表示）。我们有

$$
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})=\sup _{f \in \mathcal{F}} \int_{\mathcal{Z}} f(z) \, \mathrm{d}\mathbb{P}(z) - \int_{\mathcal{Z}} \phi^*(f(z)) \, \mathrm{d} \hat{\mathbb{P}}(z),
$$

其中 $\mathcal{F}$ 表示所有有界的 Borel 函数 $f: \mathcal{Z} \rightarrow \operatorname{dom}\left(\phi^*\right)$ 的集合。

Proof. As the entropy function $\phi(s)$ is proper, convex and lower semicontinuous on $\mathbb{R}$ and as $0 \phi(s / 0)$ is interpreted as the recession function $\phi^{\infty}(s)$, the perspective function $\phi^\pi(s, t)=t \phi(s / t)$ is proper, convex and lower semicontinuous on $\mathbb{R} \times \mathbb{R}_{+}$. By (Rockafellar 1970, Theorem 12.2), $\phi^\pi(s, t)$ can therefore be expressed as the conjugate of its conjugate. Note that the conjugate of $\phi^\pi(s, t)$ satisfies

$$
\begin{aligned}
\left(\phi^\pi\right)^*(f, g) & =\sup _{s \in \mathbb{R}, t \in \mathbb{R}_{+}} f s+g t-t \phi(s / t) \\
& =\sup _{t \in \mathbb{R}_{+}} g t+t \phi^*(f)=\left\{\begin{array}{cl}
0 & \text { if } f \in \operatorname{dom}\left(\phi^*\right) \text { and } g+\phi^*(f) \leq 0, \\
+\infty & \text { otherwise, }
\end{array}\right.
\end{aligned}
$$

for all $f, g \in \mathbb{R}$. The second equality in the above expression follows from (Rockafellar 1970, Theorem 16.1). As $\phi^\pi(s, t)=\sup _{f, g \in \mathbb{R}} s f+t g-\left(\phi^\pi\right)^*(f, g)$ by virtue of (Rockafellar 1970, Theorem 12.2), the $\phi$-divergence is thus given by

$$
\begin{aligned}
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) & =\int_{\mathcal{Z}} \sup _{f, g \in \mathbb{R}}\left\{\frac{\mathrm{~d} \mathbb{P}}{\mathrm{~d} \rho}(z) \cdot f+\frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z) \cdot g-\left(\phi^\pi\right)^*(f, g)\right\} \mathrm{d} \rho(z) \\
& =\int_{\mathcal{Z}} \sup _{f \in \operatorname{dom}\left(\phi^*\right)}\left\{\frac{\mathrm{d} \mathbb{P}}{\mathrm{~d} \rho}(z) \cdot f-\frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z) \cdot \phi^*(f)\right\} \mathrm{d} \rho(z) \\
& =\sup _{f \in \mathcal{F}} \int_{\mathcal{Z}}\left\{\frac{\mathrm{d} \mathbb{P}}{\mathrm{~d} \rho}(z) \cdot f(z)-\frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z) \cdot \phi^*(f(z))\right\} \mathrm{d} \rho(z)
\end{aligned}
$$

第二个等式利用了我们上面推导出的$\left(\phi^\pi\right)^*$的显式公式，而第三个等式则根据（Rockafellar 和 Wets 2009, 定理 14.60）得出。这个定理之所以适用，是因为函数 $h: \operatorname{dom}\left(\phi^*\right) \times \mathcal{Z} \rightarrow \mathbb{R}$ 定义为

$$
h(f, z)=\frac{\mathrm{d} \mathbb{P}}{\mathrm{d} \rho}(z) \cdot f-\frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{d} \rho}(z) \cdot \phi^*(f)
$$

在 $f$ 中是连续的，并且在 $z$ 中是 Borel 可测的，因此根据（Rockafellar 和 Wets 2009, 示例 14.29），它构成了一个 Carathéodory 积分器。然后，由于 Radon-Nikodym 导数的定义，该声明立即得出。

命题 2.6 显示了$\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$在$\mathbb{P}$和$\hat{\mathbb{P}}$中是联合凸的。如果$\phi(s)$随着$s$的增长速度超过线性，即如果其渐近增长率$\phi^{\infty}(1)$是无穷大的，那么当且仅当$\frac{\mathrm{d}}{\mathrm{d} \rho}(z)=0$对于$\rho$几乎所有的$z \in \mathcal{Z}$，且$\frac{\mathrm{d} \hat{\mathrm{P}}}{\mathrm{d} \rho}(z)=0$时，$\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$是有限的。换句话说，$\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$只有在$\mathbb{P} \ll \hat{\mathbb{P}}$时才是有限的。在这种特殊情况下，Radon-Nikodym 导数的链式法则意味着$\frac{\mathrm{dP}}{\mathrm{d} \rho} / \frac{\mathrm{d} \mathrm{P}}{\mathrm{d} \rho}=\frac{\mathrm{dP}}{\mathrm{dP}}$。如果$\phi^{\infty}(1)=\infty$，则$\phi$-散度有更常见（但较不一般）的表示形式：

$$
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})= \begin{cases}\int_{\mathcal{Z}} \phi\left(\frac{\mathrm{dP}}{\mathrm{~d} \hat{\mathbb{P}}}(z)\right) d \hat{\mathbb{P}}(z) & \text { if } \mathbb{P} \ll \hat{\mathbb{P}} \\ +\infty & \text { otherwise. }\end{cases}
$$

我们现在可以定义$\phi$-散度模糊集为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\} .
$$

该集合包含所有支持在$\mathcal{Z}$上的概率分布$\mathbb{P}$，其相对于某个指定参考分布$\hat{\mathbb{P}}$的$\phi$-散度最大为$r \geq 0$。


**Remark 2.7 (Csiszár 对偶性)**. 广义的 $\phi$-散度（即使 $\mathrm{P} \nless \hat{\mathrm{P}}$，也可以取有限值）在 P 和 $\hat{\mathrm{P}}$ 交换时是不变的。形式上，我们有 $\mathrm{D}_\phi(\mathbb{P}, \hat{P}) = \mathrm{D}_\phi(\hat{\mathbb{P}}, \mathbb{P})$，其中 $\psi$ 表示 $\phi$ 的 Csiszár 对偶，通过 $\psi(s) = \phi^\pi(1, s) = t \phi(1 / t)$ 定义（Ben-Tal, Ben-Israel 和 Teboulle 1991, 引理 2.3）。可以验证，如果 $\phi$ 是定义 2.4 中所定义的有效熵函数，那么 $\psi$ 也是一个有效熵函数。这一关系表明，尽管 $\phi$-散度通常是不对称的，但通过聚焦于形式为 (2.10) 的散度模糊集，其中名义分布 P 作为散度的第二个参数，我们并没有牺牲一般性。从 Proposition 2.6 之后的讨论中可以看出，如果 $\phi^{\infty}(1) = \infty$，那么所有在 $\phi$-散度模糊集 (2.10) 中的分布 $P$ 满足 $\mathbb{P} \ll \hat{\mathbb{P}}$。类似地，如果 $\phi$ 的 Csiszár 对偶满足 $\psi^{\infty}(1) = \infty$，那么所有在 $\phi$-散度模糊集中的分布 P 满足 $\hat{P} \ll \mathbb{P}$。

表 2.1 列出了常见的熵函数及其 Csiszár 对偶。我们强调，Cressie-Read 散度家族包括 Pearson $\chi^2$-散度（当 $\beta = 2$）、Kullback-Leibler 散度（当 $\beta \rightarrow 1$）以及似然散度（当 $\beta \rightarrow 0$）作为特例。

DRO 文献通常关注于受限的 $\phi$-散度模糊集

$$
\mathcal{P} = \left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{P} \ll \hat{\mathrm{P}}, \mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\}
$$

该模糊集由 Ben-Tal 等人（2013）引入。与标准的 $\phi$-散度模糊集（2.10）不同，它仅包含相对于参考分布 $\hat{\mathbb{P}}$ 完全连续的分布，即使 $\phi^{\infty}(1) < \infty$。Ben-Tal 等人（2013）研究了在受限的 $\phi$-散度模糊集上的 DRO 问题，假设参考分布 $\hat{\mathbb{P}}$ 是离散的。在这种情况下，绝对连续性约束 $\mathbb{P} \ll \hat{\mathbb{P}}$ 确保模糊集仅包含支持在 $\hat{\mathbb{P}}$ 原子上的离散分布，因此自然的最坏情况期望问题转化为一个有限的凸规划问题。Ben-Tal 等人（2013）进一步发展了这一问题类的对偶性理论。Shapiro（2017）将这一对偶性理论扩展到一般的参考分布 $\hat{\mathbb{P}}$，这些参考分布不一定是离散的。Hu, Hong 和 So（2013）以及 Jiang 和 Guan（2016）表明，任何受限的 $\phi$-散度模糊集上的分布鲁棒单个机会约束都等价于参考分布 $\hat{\mathbb{P}}$ 下的经典机会约束，只是置信度水平进行了缩放。Bayraksan 和 Love（2015）提供了对各种 $\phi$-散度的分类，并分析了对应的 $\phi$-散度模糊集的结构性质，假设 $\mathcal{Z}$ 是有限的。下面我们回顾标准和受限的 $\phi$-散度模糊集的常见实例。



| Divergence                        | $\phi(s)~(s \geq 0)$                                  | $\psi(s)~(s \geq 0)$                                  | $\phi^\infty(1)$ | $\psi^\infty(1)$   |
|------------------------------------|-------------------------------------------------------|-------------------------------------------------------|------------------|---------------------|
| **Kullback-Leibler**               | $s \log(s) - s + 1$                                   | $- \log(s) + s - 1$                                   | $\infty$         | 1                   |
| **Likelihood**                     | $-\log(s) + s - 1$                                   | $s \log(s) - s + 1$                                   | 1                | $\infty$       |
| **Total variation**                | $\frac{1}{2}\|s - 1\|$  | $\frac{1}{2}\|s - 1\|$ | $\frac{1}{2}          $     | $\frac{1}{2}          $   |
| **Pearson $\chi^2$**               | $(s - 1)^2$                                           | $\frac{1}{s} (s - 1)^2$                               | $\infty$         | 1                   |
| **Neyman $\chi^2$**                | $\frac{1}{s} (s - 1)^2$                               | $(s - 1)^2$                                           | 1                | $\infty$           |
| **Cressie-Read for $\beta \in (0, 1)$** | $\frac{s^\beta - \beta s + \beta - 1}{\beta (\beta - 1)}$ | $s^\beta - \beta s + \beta - 1$                       | $\frac{1}{1 - \beta}$ | $\frac{1}{\beta}$ |
| **Cressie-Read for $\beta > 1$**   | $\frac{s^\beta - \beta s + \beta - 1}{\beta (\beta - 1)}$ | $s^\beta - \beta s + \beta - 1$                       | $\infty$         | $\frac{1}{\beta - 1}$ |


表 2.1. 熵函数及其 Csiszár 对偶的示例

### 2.2.1. Kullback-Leibler 模糊集

Kullback-Leibler 散度是对应于熵函数的 $\phi$-散度，该熵函数满足 $\phi(s) = s \log(s) - s + 1$，适用于所有 $s \geq 0$；详见表 2.1。由于 $\phi^{\infty}(1) = +\infty$，因此它具有以下等价定义。

**定义 2.8 (Kullback-Leibler 散度)**. 对于 $\mathbb{P} \in \mathcal{P}(\mathcal{Z})$ 和 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$，Kullback-Leibler 散度定义为

$$
\mathrm{KL}(\mathbb{P}, \hat{\mathbb{P}}) = \begin{cases} 
\int_{\mathcal{Z}} \log \left(\frac{\mathrm{d} \mathbb{P}}{d \hat{\mathbb{P}}}(z)\right) \mathrm{d} \mathbb{P}(z) & \text{如果 } \mathbb{P} \ll \hat{\mathbb{P}}, \\
+\infty & \text{否则.}
\end{cases}
$$

我们现在回顾一下 Kullback-Leibler 散度的著名变分公式。

**命题 2.9 (Donsker 和 Varadhan (1983))**. 对于 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$，Kullback-Leibler 散度满足

$$
\operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) = \sup_{f \in \mathcal{F}} \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z) - \log \left(\int_{\mathcal{Z}} e^{f(z)} \mathrm{d} \hat{\mathbb{P}}(z)\right),
$$

其中 $\mathcal{F}$ 表示所有有界的 Borel 函数 $f: \mathcal{Z} \rightarrow \mathbb{R}^d$ 的集合。

**证明**. 诱导 Kullback-Leibler 散度的熵函数 $\phi$ 的凸对偶满足 $\phi^*(t) = \exp(t) - 1$，其定义域为 $\mathbb{R}$。因此，命题 2.6 中建立的泛 $\phi$-散度的对偶表示式意味着

$$
\mathrm{KL}(\mathbb{P}, \hat{\mathbb{P}}) = \sup_{f \in \mathcal{F}} \int_{\mathcal{Z}} f(z) \mathrm{dP}(z) - \int_{\mathcal{Z}}\left(\mathrm{e}^{f(z)} - 1\right) \mathrm{d} \hat{\mathbb{P}}(z),
$$

其中 $\mathcal{F}$ 表示所有有界的 Borel 函数 $f: \mathcal{Z} \rightarrow \mathbb{R}$ 的集合。注意，$\mathcal{F}$ 在常数平移下是不变的。也就是说，如果 $f(z)$ 是有界的 Borel 函数，那么对于任意常数 $c \in \mathbb{R}$，$f(z) + c$ 也是有界的 Borel 函数。因此，在上述最大化问题中，我们可以在 $f \in \mathcal{F}$ 和 $c \in \mathbb{R}$ 上同时优化，从而得到

$$
\operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) = \sup_{f \in \mathcal{F}} \sup_{c \in \mathbb{R}} \int_{\mathcal{Z}}(f(z)+c) \mathrm{d\mathbb{P}}(z) - \int_{\mathcal{Z}}\left(\mathrm{e}^{f(z)+c}-1\right) \mathrm{d} \hat{\mathbb{P}}(z).
$$

对于任何固定的 $f \in \mathcal{F}$，关于 $c$ 的内部最大化问题的最优解是

$$
c^{\star} = -\log \left(\int_{\mathcal{Z}} e^{f(z)} \mathrm{d} \hat{\mathbb{P}}(z)\right).
$$

将这个表达式代入目标函数得到（2.12）。命题 2.9 建立了 Kullback-Leibler 散度与熵风险度量之间的联系。这个联系将在第 4.3 节中变得有用。

Kullback-Leibler 模糊集的半径为 $r \geq 0$，围绕 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 给出为

$$
\mathcal{P} = \{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\}.
$$

由于 $\phi^{\infty}(1) = +\infty$，因此所有的分布 $\mathbb{P} \in \mathcal{P}$ 都是相对于 $\hat{\mathbb{P}}$ 完全连续的。因此，$\mathcal{P}$ 与受限的 Kullback-Leibler 模糊集是相同的。El Ghaoui 等人（2003）推导了当 $\hat{\mathbb{P}}$ 是高斯分布时，线性损失函数的最坏情况value-at-risk 的封闭式表达式。Hu 和 Hong（2013）使用类似的方法表明，任何受限的 Kullback-Leibler 模糊集下的分布鲁棒个体机会约束都等价于参考分布 $\hat{\mathbb{P}}$ 下的经典机会约束，只是置信度水平被缩放了。Calafiore（2007）研究了当 $\hat{\mathbb{P}}$ 是离散分布时，最坏情况的均值-风险投资组合选择问题。Kullback-Leibler 模糊集还在最小二乘估计（Levy 和 Nikoukhah 2004）、假设检验（Levy 2008，Gül 和 Zoubir 2017）、滤波（Levy 和 Nikoukhah 2012，Zorzi 2016，2017a,b）、风险度量理论（Ahmadi-Javid 2012，Postek 等 2016）和极值分析（Blanchet, He 和 Murthy 2020）等方面找到了应用。


### 2.2.2. 似然模糊集

由于 Kullback-Leibler 散度并非对称，因此它产生了两个严格不同的模糊集。第二节中的 Kullback-Leibler 模糊集是通过将 Kullback-Leibler 散度的第二个参数固定为参考分布 $\hat{\mathbb{P}}$，并考虑所有满足 $\operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) \leq r$ 的分布 $\mathbb{P}$ 获得的。另一种模糊集则是通过使用 $\hat{\mathbb{P}}$ 作为第一个参数并设置

$$
\mathcal{P} = \{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \operatorname{KL}(\hat{\mathbb{P}}, \mathbb{P}) \leq r\}.
$$

我们将 $\mathcal{P}$ 称为以 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 为中心的似然模糊集。事实上，$\mathbb{P} \in \mathcal{P}(\mathcal{Z})$ 相对于 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 的似然或 Burg-熵散度通常定义为反向的 Kullback-Leibler 散度 $\operatorname{KL}(\hat{\mathbb{P}}, \mathbb{P})$。这种术语的基础是以下直觉。如果 $\mathcal{Z}$ 是一个离散集合，并且 $\hat{\mathbb{P}} = \frac{1}{N} \sum_{i=1}^N \delta_{\hat{z}_i}$ 是与来自未知分布的 $N$ 个独立样本 $\left\{\hat{z}_i\right\}_{i=1}^N$ 相对应的经验分布，那么构造所有分布族，使得观察到的数据在规定的似然水平下能够实现，便是自然的。这些分布族对应于似然函数 $\mathcal{L}(\mathbb{P})=\prod_{i=1}^N \mathbb{P}\left(Z=\hat{z}_i\right)$ 在 $\mathcal{P}(\mathcal{Z})$ 上的超水平集。可以证明，任何这样的超水平集与似然散度 $\operatorname{KL}(\hat{\mathbb{P}}, \mathbb{P})$ 的子水平集一致。因此，它构成了形式为 (2.14) 的似然模糊集。我们强调，这种对应关系不容易延伸到 $\mathcal{Z}$ 不是离散的情况。

似然模糊集最初由 Wang 等人（2016）在静态 DRO 的背景下提出，后来被 Wiesemann、Kuhn 和 Rustem（2013）用于稳健的马尔可夫决策过程。Bertsimas 等人（2018a,b）表明，似然模糊集包含所有在规定显著性水平下通过 G 检验的分布。

即使 $\mathcal{Z}$ 是不可数的，似然模糊集也显示出几个统计最优性性质。为了说明这些性质，我们考虑以下任务：在给定来自 $\mathbb{P}$ 的 $N$ 个独立样本的情况下，评估某个损失函数在未知分布 $\mathbb{P}$ 下的 $(1-\eta)$ 上置信区间。利用 Owen（1988）提出的经验似然定理，Lam（2019）展示了以经验分布 $\hat{\mathbb{P}}$ 为中心的似然模糊集的一个理想性质：与之关联的最坏情况期望损失提供了对常数显著性水平 $\eta$ 的最不保守的置信区间，当半径 $r$ 以 $1 / N$ 的速度衰减时，渐近地得出类似保证。Duchi、Glynn 和 Namkoong（2021）报告了对更广泛类别的 $\phi$-散度的类似保证。此外，Van Parys、Mohajerin Esfahani 和 Kuhn（2021）利用 Sanov 的大偏差原理（Cover 和 Thomas 2006，第 11.4.1 定理）证明，针对以常数半径 $r$ 为中心的似然模糊集，最坏情况的期望损失为渐近下显著性水平 $\eta \propto e^{-r N}$ 的最不保守置信区间。Gupta（2019）进一步表明，围绕 $\hat{\mathbb{P}}$ 的半径 $r \propto N^{-1/2}$ 的似然模糊集代表了满足贝叶斯鲁棒性保证的最小凸模糊集。


### 2.2.3. 总变差模糊集

两个分布 $\mathbb{P}, \hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 的总变差距离是 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 在任何事件上所分配的概率之间的最大绝对差异。

**定义 2.10（总变差距离）** 总变差距离是定义为

$$
\operatorname{TV}(\mathbb{P}, \hat{\mathbb{P}}) = \sup \{|\mathbb{P}(\mathcal{B}) - \hat{\mathbb{P}}(\mathcal{B})| : \mathcal{B} \subseteq \mathcal{Z} \text{ 是一个 Borels 集合} \}
$$

总变差距离显然是对称的，满足不可区分的恒等式和三角不等式。因此，它构成了 $\mathcal{P}(\mathcal{Z})$ 上的一个度量。此外，总变差距离是一个 $\phi$-散度的实例。

** 命题 2.11.** 总变差距离与由熵函数 $\phi(s) = \frac{1}{2}|s - 1|$ 对所有 $s \geq 0$ 引起的 $\phi$-散度一致。

**证明：** 诱发 Kullback-Leibler 散度的熵函数的共轭熵函数满足 $\phi^*(t) = \max\{t, -\frac{1}{2}\}$，当 $t \leq \frac{1}{2}$ 时，$\phi^*(t) = +\infty$ 当 $t > \frac{1}{2}$ 时。根据命题 2.6，与给定熵函数对应的 $\phi$-散度因此具有如下的对偶表示：

$$
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) = \sup _{f \in \mathcal{F}} \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z) - \int_{\mathcal{Z}} \max\{f(z), -\frac{1}{2}\} \mathrm{d} \hat{\mathbb{P}}(z),
$$

其中 $\mathcal{F}$ 表示所有有界的 Borel 函数 $f: \mathcal{Z} \rightarrow \left(-\infty, \frac{1}{2}\right]$ 的族。注意，$\mathcal{F}$ 对常数平移不变。也就是说，如果 $f(z)$ 是一个有界的 Borel 函数，那么 $f(z) + c$ 对于任何常数 $c \in \mathbb{R}$ 也是有界的 Borel 函数。因此，我们可以在上述最大化问题中同时对 $f \in \mathcal{F}$ 和 $c \in \mathbb{R}$ 进行优化，从而得到：

$$
\operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) = \sup _{f \in \mathcal{F}} \sup _{c \in \mathbb{R}} \int_{\mathcal{Z}} (f(z) + c) \mathrm{d\mathbb{P}}(z) - \int_{\mathcal{Z}} \left(\mathrm{e}^{f(z) + c} - 1\right) \mathrm{d} \hat{\mathbb{P}}(z).
$$

对于任何固定的 $f \in \mathcal{F}$，关于 $c$ 的内层最大化问题可以通过以下方式唯一求解：

$$
c^{\star} = -\log \left(\int_{\mathcal{Z}} \mathrm{e}^{f(z)} \mathrm{d} \hat{\mathbb{P}}(z)\right).
$$

将此表达式代回目标函数中，得到（2.12）。命题 2.9 建立了 Kullback-Leibler 散度与熵风险度量之间的联系。这个联系将在第 4.3 节中变得有用。

Kullback-Leibler 模糊集的半径为 $r \geq 0$ 的模糊集定义为：

$$
\mathcal{P} = \{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\}.
$$

由于 $\phi^{\infty}(1) = +\infty$，因此所有分布 $\mathbb{P} \in \mathcal{P}$ 都是相对于 $\hat{\mathbb{P}}$ 的绝对连续的。因此，$\mathcal{P}$ 与限制 Kullback-Leibler 模糊集一致。El Ghaoui 等人（2003）推导出了当 $\hat{\mathbb{P}}$ 是高斯分布时，线性损失函数的最坏情况价值风险的封闭形式表达式。Hu 和 Hong（2013）使用类似的技术表明，任何相对于 Kullback-Leibler 模糊集的分布鲁棒性个体机会约束等价于参考分布 $\hat{\mathbb{P}}$ 下的经典机会约束，但具有重新调整的置信水平。Calafiore（2007）研究了当 $\hat{\mathbb{P}}$ 是离散分布时，最坏情况下的均值风险投资组合选择问题。Kullback-Leibler 模糊集还在最小二乘估计（Levy 和 Nikoukhah 2004）、假设检验（Levy 2008, Gül 和 Zoubir 2017）、滤波（Levy 和 Nikoukhah 2012, Zorzi 2016, 2017a,b）、风险度量理论（Ahmadi-Javid 2012, Postek 等人 2016）和极值分析（Blanchet, He 和 Murthy 2020）等领域找到了应用。

半径为 $r \geq 0$ 的总变差模糊集围绕 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 定义为：

$$
\mathcal{P}=\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \operatorname{TV}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\}
$$

现有文献大多集中在限制性总变差模糊集上，该模糊集包含所有满足 $\mathbb{P} \ll \hat{\mathbb{P}}$ 的分布 $\mathbb{P} \in \mathcal{P}$。Jiang 和 Guan (2018, Theorem 1) 以及 Shapiro (2017, Example 3.7) 证明，相对于限制性总变差模糊集的最坏期望损失与条件价值-at-risk 和相对于 $\hat{\mathbb{P}}$ 的损失的本质上确界的组合一致，详见第6.10节。Rahimian、Bayraksan 和 Homem-deMello (2019a, $b$, 2022) 研究了在 $\mathcal{Z}$ 是有限集的情况下，未限制总变差模糊集上的 DRO 问题的最坏分布。总变差模糊集与鲁棒统计中的 Huber 污染模型相关（Huber 1981），该模型假设统计数据集中的一部分样本（比例为 $r \in(0,1)$）来自于一个任意的污染分布。因此，目标分布和污染数据生成分布之间的总变差距离至多为 $r$。因此，围绕某个估计分布的半径 $r$ 的总变差模糊集作为目标分布的搜索空间是自然的选择（Nishimura 和 Ozaki 2004, 2006；Bose 和 Daripa 2009；Duchi, Hashimoto 和 Namkoong 2023；Tsanga 和 Shehadeha 2024）。


### 2.2.4. $\chi^2$-Divergence 模糊集

$\chi^2$-散度是对应于熵函数的 $\phi$-散度，该熵函数满足对于所有 $s \geq 0$，有 $\phi(s)=(s-1)^2$；另见表 2.1。由于 $\phi^{\infty}(1)=+\infty$，因此它有以下等价定义。

定义 2.12 ( $\chi^2$-散度)。对于 $\mathbb{P} \in \mathcal{P}(\mathcal{Z})$，相对于 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 的 $\chi^2$-散度定义为

$$
\chi^2(\mathbb{P}, \hat{\mathbb{P}})= \begin{cases}\int_{\mathcal{Z}}\left(\frac{\mathrm{d} \mathbb{P}}{\mathrm{~d} \hat{\mathbb{P}}}(z)-1\right)^2 \mathrm{~d} \hat{\mathbb{P}}(z) & \text { 如果 } \mathbb{P} \ll \hat{\mathbb{P}} \\ +\infty & \text { 否则 }\end{cases}
$$

$\chi^2$-散度有以下对偶表示。

**命题 2.13**.对于 $\mathbb{P}$，相对于 $\hat{\mathbb{P}}$ 的 $\chi^2$-散度满足

$$
\chi^2(\mathbb{P}, \hat{\mathbb{P}})=\sup _{f \in \mathcal{F}} \frac{\left(\mathbb{E}_{\mathbb{P}}[f(Z)]-\mathbb{E}_{\hat{\mathbb{p}}}[f(Z)]\right)^2}{\mathbb{V}_{\hat{\mathbb{P}}}[f(Z)]},
$$

其中 $\mathcal{F}$ 表示所有有界 Borel 函数的集合 $f: \mathcal{Z} \rightarrow \mathbb{R}$，$\mathbb{V}_{\hat{\mathbb{P}}}[f(Z)]$ 表示 $f(Z)$ 在 $\hat{\mathbb{P}}$ 下的方差。如果 $\mathbb{V}_{\hat{\mathbb{P}}}[f(Z)] = 0$，则上述分数被解释为 0（如果 $\mathbb{E}_{\mathbb{P}}[f(Z)] = \mathbb{E}_{\hat{\mathbb{P}}}[f(Z)]$）或 $+\infty$（否则）。

证明。引起 $\chi^2$-散度的熵函数的凸共轭满足 $\phi^*(t)=\frac{t^2}{4}+t$ 如果 $t \geq-2$，并且如果 $t<-2$，则 $\phi^*(t)=-1$，其定义域为 $\operatorname{dom}\left(\phi^*\right)=\mathbb{R}$。因此，命题 2.6 表明

$$
\chi^2(\mathbb{P}, \hat{\mathbb{P}})=\sup _{f \in \mathcal{F}} \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z)-\int_{\mathcal{Z}}\left(\frac{f(z)^2}{4}+f(z)\right) \mathrm{d} \hat{\mathbb{P}}(z),
$$

其中 $\mathcal{F}$ 表示所有有界 Borel 函数 $f: \mathcal{Z} \rightarrow \mathbb{R}$。请注意，我们在第二个积分中将 $\phi^*(f(z))$ 替换为 $f(z)^2 / 4 + f(z)$。这样做没有损失一般性。事实上，如果函数 $f(z)$ 取值小于 -2，那么它（弱）被函数 $f^{\prime}(z)=\max \{f(z),-2\}$ 所主导。还要注意，$\mathcal{F}$ 对常数偏移不变。也就是说，如果 $f(z)$ 是有界的 Borel 函数，那么 $f(z)+c$ 对于任何常数 $c \in \mathbb{R}$ 也是有界的。一个简单的计算表明，对于任何固定的 $f \in \mathcal{F}$，最优的偏移量是 $c^{\star}=-\mathbb{E}_{\hat{\mathbb{p}}}[f(Z)]$。因此，我们可以将 $f(z)$ 替换为 $f(z)-\mathbb{E}_{\hat{\mathbb{p}}}[f(Z)]$，这将得到

$$
\chi^2(\mathbb{P}, \hat{\mathbb{P}})=\sup _{f \in \mathcal{F}} \mathbb{E}_{\mathbb{P}}[f(Z)]-\mathbb{E}_{\hat{\mathbb{P}}}[f(Z)]-\frac{\mathbb{V}_{\hat{\mathbb{P}}}[f(Z)]}{4} .
$$

注意，集合 $\mathcal{F}$ 对缩放也不变。也就是说，如果 $f(z)$ 是有界的 Borel 函数，那么 $c f(z)$ 对于任何常数 $c \in \mathbb{R}$ 也是有界的。因此，我们可以分别在上述最大化问题中对 $f \in \mathcal{F}$ 和 $c \in \mathbb{R}$ 进行优化，从而得到

$$
\begin{aligned}
\chi^2(\mathbb{P}, \hat{\mathbb{P}}) & =\sup _{f \in \mathcal{F}} \sup _{c \in \mathbb{R}}\left(\mathbb{E}_{\mathbb{P}}[f(Z)]-\mathbb{E}_{\hat{\mathbb{P}}}[f(Z)]\right) c-\frac{\mathbb{V}_{\hat{\mathbb{P}}}[f(Z)]}{4} c^2 \\
& =\sup _{f \in \mathcal{F}} \frac{\left(\mathbb{E}_{\mathbb{P}}[f(Z)]-\mathbb{E}_{\hat{\mathbb{P}}}[f(Z)]\right)^2}{\mathbb{V}_{\hat{\mathbb{P}}}[f(Z)]} .
\end{aligned}
$$

注意，内层的最大化问题对 $c$ 进行求解时，实际上是在 $\mathbb{V}_{\hat{\mathbb{P}}}[f(Z)] c^2 / 4$ 上计算共轭函数，这可以用闭式形式得出。因此，结论得以成立。

由于 $\chi^2$-散度不是对称的，它产生了两个互补的模糊集，这取决于参考分布 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 是作为 $\chi^2$-散度的第一个还是第二个参数来使用。Lam (2018) 定义了围绕 $\hat{\mathbb{P}}$ 的 Pearson $\chi^2$-模糊集，半径为 $r \geq 0$：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \chi^2(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\}
$$

用于分析具有依赖数据的操作和服务系统。Philpott、de Matos 和 Kapelevich (2018) 发展了一种随机对偶动态规划算法，用于求解具有 Pearson 模糊集的分布鲁棒多阶段随机程序。在静态 DRO 的背景下，Duchi 和 Namkoong (2019) 证明了，相对于 Pearson 模糊集的鲁棒化与方差正则化密切相关。请注意，由于 $\phi^{\infty}(1)=+\infty$，Pearson 模糊集与其限制版本相同，后者仅包含满足 $\mathbb{P} \ll \hat{\mathbb{P}}$ 的分布。

Klabjan、Simchi-Levi 和 Song (2013) 定义了 Neyman $\chi^2$-模糊集：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \chi^2(\hat{\mathbb{P}}, \mathbb{P}) \leq r\right\}
$$

用于表述鲁棒批量选择问题。Hanasusanto 和 Kuhn (2013) 在有限的 $\mathcal{Z}$ 上使用了 Neyman 模糊集，背景是鲁棒数据驱动的动态规划。最后，Hanasusanto 等人 (2015a) 使用相同的模糊集来建模多模态需求分布的混合权重的不确定性。


2.3. 最优运输模糊集

最优运输理论为量化概率分布之间的差异提供了一种自然的方式，并产生了一类丰富的模糊集。为了说明这一点，我们首先引入运输成本函数的概念。

定义 2.14 (运输成本函数)。一个下半连续函数 $c: \mathcal{Z} \times \mathcal{Z} \rightarrow[0,+\infty]$，满足对于所有 $z \in \mathcal{Z}$，有 $c(z, z)=0$，称为运输成本函数。

每个运输成本函数都引发一个最优运输差异。

定义 2.15 (最优运输差异)。与给定的运输成本函数 $c$ 相关的最优运输差异 $\mathrm{OT}_c: \mathcal{P}(\mathcal{Z}) \times \mathcal{P}(\mathcal{Z}) \rightarrow[0,+\infty]$ 定义为：

$$
\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})=\inf _{\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})} \mathbb{E}_\gamma[c(Z, \hat{Z})],
$$

其中 $\Gamma(\mathbb{P}, \hat{\mathbb{P}})$ 表示 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 的所有耦合 $\gamma$ 的集合，也就是所有具有边际 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 的 $Z$ 和 $\hat{Z}$ 的联合概率分布。

根据定义，当且仅当 $\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})$ 时，我们有 $\gamma((Z, \hat{Z}) \in \mathcal{B} \times \mathcal{Z})=\mathbb{P}(Z \in \mathcal{B})$ 和 $\gamma((Z, \hat{Z}) \in \mathcal{Z} \times \hat{\mathcal{B}})=\hat{\mathbb{P}}(\hat{Z} \in \mathcal{B})$ 对所有 Borel 集 $\mathcal{B}$，$\mathcal{B} \subseteq \mathcal{Z}$。若把概率分布 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 看作两堆沙子，那么任何耦合 $\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})$ 可以被解释为一个运输计划，也就是通过在 $\mathcal{Z}$ 中移动沙子来将 $\hat{\mathbb{P}}$ 变形为 $\mathbb{P}$。事实上，对于任何固定的源点 $\hat{z} \in \mathcal{Z}$，条件概率 $\gamma(z \leq Z \leq z+\mathrm{d} z \mid \hat{Z}=\hat{z})$ 确定了从 $\hat{z}$ 移动到目标点 $z$ 的沙子的比例。如果将一个单位的概率质量从 $\hat{z}$ 移动到 $z$ 的成本为 $c(z, \hat{z})$，那么 $\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})$ 就是将 $\hat{\mathbb{P}}$ 变形为 $\mathbb{P}$ 所需的最小费用。接下来我们为一般的最优运输差异提供一个对偶表示。

**命题 2.16** (Kantorovich 对偶性 I)。我们有：

$$
\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})=\left\{\begin{array}{cl}
\sup _{f \in \mathcal{L}^1(\mathbb{P}), g \in \mathcal{L}^1(\hat{\mathbb{P}})} & \int_{\mathcal{Z}} f(z) \mathrm{dP}(z)-\int_{\mathcal{Z}} g(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z}) \\
\text { s.t. } & f(z)-g(\hat{z}) \leq c(z, \hat{z}) \quad \forall z, \hat{z} \in \mathcal{Z},
\end{array}\right.
$$

其中 $\mathcal{L}^1(\mathbb{P})$ 和 $\mathcal{L}^1(\hat{\mathbb{P}})$ 表示从 $\mathcal{Z}$ 到 $\mathbb{R}$ 的所有有界 Borel 函数集合，这些函数分别是相对于 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 可积的。

对偶问题 (2.19) 代表了第三方的利润最大化问题，第三方代表问题所有者将沙子从 $\hat{\mathbb{P}}$ 分配到 $\mathbb{P}$，通过在源点 $\hat{z}$ 以单位价格 $g(\hat{z})$ 买入沙子，并在目标点 $z$ 以单位价格 $f(z)$ 卖出沙子。约束条件确保，对于每个源点-目标点对 $(\hat{z}, z)$，对问题所有者来说，使用第三方服务比在运输成本 $c(z, \hat{z})$ 下自己移动沙子要便宜。最优价格函数 $f^{\star}$ 和 $g^{\star}$（如果存在的话）被称为 Kantorovich 潜力。

命题 2.16 证明。关于一般证明，请参见（Villani 2008，定理 5.10（i））。我们在简化假设 $\mathcal{Z}$ 是紧集的情况下证明这一主张。在这种情况下，所有连续（因此有界）的函数 $f: \mathcal{Z} \times \mathcal{Z} \rightarrow \mathbb{R}$ 所组成的家族 $\mathcal{C}(\mathcal{Z} \times \mathcal{Z})$，配备上极大值范数，构成了一个 Banach 空间。它的拓扑对偶空间是所有有限有符号 Borel 测度的空间 $\mathcal{M}(\mathcal{Z} \times \mathcal{Z})$，配备总变差范数（Folland 1999，推论 7.18）。这意味着，对于每一个连续线性泛函 $\varphi: \mathcal{C}(\mathcal{Z} \times \mathcal{Z}) \rightarrow \mathbb{R}$，都存在 $\gamma \in \mathcal{M}(\mathcal{Z} \times \mathcal{Z})$，使得对于所有 $f \in \mathcal{C}(\mathcal{Z} \times \mathcal{Z})$，有 $\varphi(f)=\int_{\mathcal{Z} \times \mathcal{Z}} f(z, \hat{z}) \mathrm{d} \gamma(z, \hat{z})$。

我们首先使用 Fenchel - Rockafellar 对偶定理来证明：

$$
\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})=\left\{\begin{array}{cl}
\sup _{f, g \in \mathcal{C}(\mathcal{Z})} & \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z)-\int_{\mathcal{Z}} g(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z}) \\
\text { s.t. } & f(z)-g(\hat{z}) \leq c(z, \hat{z}) \quad \forall z, \hat{z} \in \mathcal{Z},
\end{array}\right.
$$

即我们证明了如果对偶问题中的价格函数 $f$ 和 $g$ 被限制在 $\mathcal{C}(\mathcal{Z})$ 空间中，强对偶性成立。为此，我们将（2.20）中的最大化问题重新表达为更加简洁的形式：

$$
\sup _{h \in \mathcal{C}(\mathcal{Z} \times \mathcal{Z})}-\phi(h)-\psi(h),
$$

其中凸函数 $\phi, \psi: \mathcal{C}(\mathcal{Z} \times \mathcal{Z}) \rightarrow(-\infty,+\infty]$ 通过以下方式定义：

$$
\phi(h)= \begin{cases}0 & \text { 如果 }-h(z, \hat{z}) \leq c(z, \hat{z}) \forall z, \hat{z} \in \mathcal{Z}, \\ +\infty & \text { 否则, }\end{cases}
$$

和

$$
\psi(h)= \begin{cases}\int_{\mathcal{Z}} \int_{\mathcal{Z}} h(z, \hat{z}) \mathrm{d} \mathbb{P}(z) \mathrm{d} \hat{\mathbb{P}}(\hat{z}) & \left\{\begin{array}{l}
\text { 如果 } \exists f, g \in \mathcal{C}(\mathcal{Z}) \text { 且 } \\ 
\\
h(z, \hat{z})=g(\hat{z})-f(z) \forall z, \hat{z} \in \mathcal{Z}, \\
+\infty
\end{array}\right. \\
\text { 否则. }\end{cases}
$$

注意，(2.21) 可以看作是相对于 $\mathcal{C}(\mathcal{Z} \times \mathcal{Z})$ 和 $\mathcal{M}(\mathcal{Z} \times \mathcal{Z})$ 的配对在零测度处的对偶。注意到，由于运输成本函数 $c$ 是非负的，$\phi$ 在常数函数 $h_0 \equiv 1$ 处是连续的。此外，$h_0$ 属于 $\psi$ 的定义域。因此，Fenchel-Rockafellar 对偶定理（Brezis 2011，定理 1.12）确保了凸函数 $\phi$ 和 $\psi$ 的和的对偶与它们的对偶函数 $\phi^*$ 和 $\psi^*$ 的极小卷积一致。因此，(2.21) 等于：

$$
(\phi+\psi)^*(0)=\inf _{\gamma \in \mathcal{M}(\mathcal{Z} \times \mathcal{Z})} \phi^*(-\gamma)+\psi^*(\gamma) .
$$

接下来，我们需要评估 $\phi$ 和 $\psi$ 的对偶。对于任意 $\gamma \in \mathcal{M}(\mathcal{Z} \times \mathcal{Z})$，我们有：

$$
\begin{aligned}
\phi^*(-\gamma) & =\sup _{h \in \mathcal{C}(\mathcal{Z} \times \mathcal{Z})}\left\{-\int_{\mathcal{Z} \times \mathcal{Z}} h(z, \hat{z}) \mathrm{d} \gamma(z, \hat{z}): -h(z, \hat{z}) \leq c(z, \hat{z}) \forall z, \hat{z} \in \mathcal{Z}\right\} \\
& = \begin{cases}\int_{\mathcal{Z} \times \mathcal{Z}} c(z, \hat{z}) \mathrm{d} \gamma(z, \hat{z}) & \text { 如果 } \gamma \in \mathcal{M}_{+}(\mathcal{Z} \times \mathcal{Z}) \\
+\infty & \text { 否则 }\end{cases}
\end{aligned}
$$

其中 $\mathcal{M}_{+}(\mathcal{Z} \times \mathcal{Z})$ 代表 $\mathcal{Z} \times \mathcal{Z}$ 上的有限 Borel 测度的锥体。实际上，如果 $\gamma \in \mathcal{M}_{+}(\mathcal{Z} \times \mathcal{Z})$，那么第二个等式来自单调收敛定理，因其应用于 $c$ 是下半连续的，因此可以写成一列非递减的连续函数的逐点极限（参见下面的引理 3.1）。另一方面，如果 $\gamma \notin \mathcal{M}_{+}(\mathcal{Z} \times \mathcal{Z})$，那么第二个等式成立，因为每个 $\gamma \in \mathcal{M}(\mathcal{Z} \times \mathcal{Z})$ 都是 Radon 测度，这保证了任何 Borel 集合的测度可以通过连续函数的积分来逼近。类似地，对于任何 $\gamma \in \mathcal{M}(\mathcal{Z} \times \mathcal{Z})$，我们可以立即验证：

$$
\psi^*(\gamma)= \begin{cases}0 & \text { 如果 } \gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}}) \\ +\infty & \text { 否则. }\end{cases}
$$

将上述 $\phi^*$ 和 $\psi^*$ 的公式代入（2.22）中即可得到（2.20）。
放宽对偶问题右侧对 $f, g \in \mathcal{C}(\mathcal{Z})$ 的要求为 $f \in \mathcal{L}^1(\mathbb{P})$ 和 $g \in \mathcal{L}^1(\hat{\mathbb{P}})$，立即得到上界：

$$
\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}}) \leq\left\{\begin{array}{cl}
\sup _{f \in \mathcal{L}^1(\mathrm{P}), g \in \mathcal{L}^1(\hat{\mathbb{P}})} & \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z)-\int_{\mathcal{Z}} g(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z}) \\
\text { s.t. } & f(z)-g(\hat{z}) \leq c(z, \hat{z}) \quad \forall z, \hat{z} \in \mathcal{Z}
\end{array}\right.
$$

另一方面，很明显：

$$
\begin{aligned}
\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})= & \inf _{\gamma \in \mathcal{M}_{+}(\mathcal{Z} \times \mathcal{Z})} \sup _{f \in \mathcal{L}^1(\mathbb{P}), g \in \mathcal{L}^1(\hat{\mathbb{P}})} \int_{\mathcal{Z} \times \mathcal{Z}}(c(z, \hat{z})-f(z)+g(\hat{z})) \mathrm{d} \gamma(z, \hat{z}) \\
& +\int_{\mathcal{Z}} f(z) \mathrm{dP}(z)-\int_{\mathcal{Z}} g(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z}) .
\end{aligned}
$$

交换上述表达式中的最小化和最大化顺序，然后闭式计算内部的最小值，得到：

$$
\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}}) \geq\left\{\begin{array}{cl}
\sup _{f \in \mathcal{L}^1(\mathbb{P}), g \in \mathcal{L}^1(\hat{\mathbb{P}})} & \int_{\mathcal{Z}} f(z) \mathrm{dP}(z)-\int_{\mathcal{Z}} g(\hat{z}) \mathrm{d}(\hat{\mathbb{P}}(\hat{z}) \\
\text { s.t. } & f(z)-g(\hat{z}) \leq c(z, \hat{z}) \quad \forall z, \hat{z} \in \mathcal{Z} .
\end{array}\right.
$$

结合（2.23）和（2.24）证明了（2.19），因此结论成立。


## 2.3. 最优运输模糊集

最优运输问题（2.19）构成了一个关于价格函数 $f \in \mathcal{L}^1(\mathbb{P})$ 和 $g \in \mathcal{L}^1(\hat{\mathbb{P}})$ 的线性规划，其目标函数在 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 上是线性的。由于线性函数的逐点上确界是凸的，因此 $\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})$ 在 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 上是联合凸的。问题（2.19）可以通过引入价格函数 $f$ 的 $c$-变换 $f^c: \mathcal{Z} \rightarrow(-\infty,+\infty]$ 进一步简化，$f^c$ 定义为：

$$
f^c(\hat{z})=\sup _{z \in \mathcal{Z}} f(z)-c(z, \hat{z}) .
$$

现在，双对偶问题（2.19）的约束条件可以重新表示为：

$$
g(\hat{z}) \geq f(z)-c(z, \hat{z}) \quad \forall z, \hat{z} \in \mathcal{Z} \quad \Longleftrightarrow \quad g(\hat{z}) \geq f^c(\hat{z}) \quad \forall \hat{z} \in \mathcal{Z} .
$$

请注意，问题（2.19）寻求一个尽可能小的价格函数 $g$。由于 $g$ 被 $f^c$ 下界，因此这表明在最优解时 $g=f^c$。反之，通过定义价格函数 $g$ 的 $c$-变换 $g^c: \mathcal{Z} \rightarrow[-\infty,+\infty)$，通过以下方式定义：

$$
g^c(z)=\inf _{\hat{z} \in \mathcal{Z}} g(\hat{z})+c(z, \hat{z})
$$

问题（2.19）的约束条件可以重新表示为：

$$
f(z) \leq g(\hat{z})+c(z, \hat{z}) \quad \forall z, \hat{z} \in \mathcal{Z} \quad \Longleftrightarrow \quad f(z) \leq g^c(z) \quad \forall z \in \mathcal{Z} .
$$

这表明在最优解时 $f=g^c$。请注意，$f^c$ 和 $g^c$ 是上半连续函数的逐点上确界，因此它们也是上半连续的。此外，注意到 $f^c$ 和 $g^c$ 可能无法相对于 $\hat{\mathbb{P}}$ 和 $\mathbb{P}$ 分别可积。然而，如果 $f \in \mathcal{L}^1(\mathbb{P})$ 且 $g \in \mathcal{L}^1(\hat{\mathbb{P}})$，则可以验证积分 $\int_{\mathcal{Z}} f^c(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z})<+\infty$ 和 $\int_{\mathcal{Z}} g^c(z) \mathrm{dP}(z)>-\infty$ 存在为扩展实数。上述洞察最终得出以下推论，我们不提供证明。有关详细信息，请参见（Villani 2008，定理 5.10（i））。

**推论 2.17** (Kantorovich 对偶性 II)。我们有：

$$
\begin{aligned}
\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}}) & =\sup _{f \in \mathcal{L}^1(\mathbb{P})} \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z)-\int_{\mathcal{Z}} f^c(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z}) \\
& =\sup _{g \in \mathcal{L}^1(\hat{\mathbb{P}})} \int_{\mathcal{Z}} g^c(z) \mathrm{d} \mathbb{P}(z)-\int_{\mathcal{Z}} g(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z}),
\end{aligned}
$$

其中 $f^c$ 和 $g^c$ 的 $c$-变换分别在（2.25）和（2.26）中定义。此外，如果我们要求对于某个函数 $g: \mathcal{Z} \rightarrow(-\infty,+\infty](f: \mathcal{Z} \rightarrow[-\infty,+\infty))$，则第一个（第二个）上确界不变，要求 $f=g^c$ $\left(g=f^c\right)$。

给定任何运输成本函数 $c$，参考分布 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 和运输预算 $r \geq 0$，最优运输模糊集定义为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\} .
$$

通过构造，$\mathcal{P}$ 包含所有能够通过在最大成本 $r \geq 0$ 内变形参考分布 $\hat{\mathbb{P}}$ 获得的概率分布 $\mathbb{P}$。最优运输模糊集最早由 Pflug 和 Wozabal（2007）研究，他们提出了一种连续线性规划算法，用于解决当 $\mathcal{Z}$ 有限时的稳健均值风险投资组合选择问题。Postek 等人（2016）利用对偶理论工具，开发了一种精确解法来解决相同问题类。Wozabal（2012）和 Pflug 和 Pichler（2014，§ 7.1）将具有最优运输模糊集的 DRO 问题重新表述为在 $\mathcal{Z} \subseteq \mathbb{R}^d$ 上的无穷支持集的有限维非凸问题，并采用全局优化方法解决它们。Mohajerin Esfahani 和 Kuhn（2018）以及 Zhao 和 Guan（2018）使用专门的对偶性结果，表明这些 DRO 问题实际上等价于广义的矩问题，并且可以准确地重新表述为有限维凸程序。Blanchet 和 Murthy（2019），Gao 和 Kleywegt（2023），以及 Zhang 等人（2024b）表明，即使 $\mathcal{Z}$ 是一个 Polish 空间，基础对偶性结果仍然有效。有关具有最优运输模糊集的 DRO 理论和应用的最新综述，请参阅 Kuhn 等人（2019）和 Blanchet、Murthy 和 Nguyen（2021）。


### 2.3.1. $p$-Wasserstein模糊集

在定义2.15中，通常将运输成本函数 $c$ 设置为某个度量 $d(\cdot, \cdot)$ 的 $p$ 次方。在这种情况下，最优运输差异的 $p$ 次方根被称为 $p$-Wasserstein 距离。

定义 2.18 ($p$-Wasserstein 距离)。假设 $d(\cdot, \cdot)$ 是 $\mathcal{Z}$ 上的度量，$p \in[1,+\infty)$ 是给定的指数。那么，$p$-Wasserstein 距离 $\mathrm{W}_p: \mathcal{P}(\mathcal{Z}) \times \mathcal{P}(\mathcal{Z}) \rightarrow[0,+\infty]$ 对应于 $d$ 和 $p$ 通过以下方式定义：

$$
\mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}})=\inf _{\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})}\left(\mathbb{E}_\gamma\left[d(Z, \hat{Z})^p\right]\right)^{\frac{1}{p}} .
$$

**定义 2.18** 意味着如果 $c(z, \hat{z})=d(z, \hat{z})^p$，则有 $W_p^p(\mathbb{P}, \hat{\mathbb{P}})=\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})$。在以下内容中，我们用 $\mathcal{P}_p(\mathcal{Z})=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_{\mathbb{P}}\left[d\left(Z, \hat{z}_0\right)^p\right]<\infty\right\}$ 来表示所有在 $\mathcal{Z}$ 上具有有限 $p$-阶矩的分布的集合。由于 $d$ 是度量，$\mathcal{P}_p(\mathcal{Z})$ 与参考点 $\hat{z}_0 \in \mathcal{Z}$ 的选择无关。$p$-Wasserstein 距离构成了 $\mathcal{P}_p(\mathcal{Z})$ 上的度量。事实上，很明显，$W_p(\mathbb{P}, \hat{\mathbb{P}})$ 是对称的，并且当且仅当 $\mathbb{P}=\hat{\mathbb{P}}$ 时为零。证明 $W_p(\mathbb{P}, \hat{\mathbb{P}})$ 满足三角不等式需要一个关于运输计划的粘合引理，因此更为复杂；参见（Villani 2008，第1章）。$p$-Wasserstein 距离进一步使得分布的弱收敛和它们的 $p$-阶矩的收敛成为度量。这意味着 $W_p\left(\mathbb{P}, \hat{\mathbb{P}}_N\right)$ 当且仅当 $\hat{\mathbb{P}}_N$ 弱收敛到 $\mathbb{P}$ 并且 $\mathbb{E}_{\hat{P}_N}\left[d\left(Z, \hat{z}_0\right)^p\right]$ 收敛到 $\mathbb{E}_{\mathbb{P}}\left[d\left(Z, \hat{z}_0\right)^p\right]$ 时收敛到 0（Villani 2008，第6.9定理）。此外，$p$-Wasserstein 距离具有吸引力的测度集中性质。具体来说，如果 $\hat{\mathbb{P}}_N$ 表示从 $\mathbb{P}$ 中独立抽取的 $N$ 个样本所得到的经验分布，则 $\hat{\mathbb{P}}_N$ 收敛到 $\mathbb{P}$ 的 $p$-Wasserstein 距离的速率存在严格的渐近和有限样本界限（Fournier 和 Guillin 2015，Weed 和 Bach 2019）。

由于 $p$-Wasserstein 距离是最优运输差异的 $p$ 次方根，命题 2.16 和推论 2.17 自然而然地意味着它也有对偶表示。对于 $p=1$，这个对偶表示特别简单。事实上，可以证明，1-Wasserstein 距离与由所有相对于度量 $d$ 是 Lipschitz 连续且 Lipschitz 模数至多为 1 的测试函数生成的积分概率度量相一致。

推论 2.19 (Kantorovich-Rubinstein 对偶性)。我们有：

$$
\mathrm{W}_1(\mathbb{P}, \hat{\mathbb{P}})=\sup _{f \in \mathcal{L}^1(\mathbb{P}), \operatorname{lip}(f) \leq 1} \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z)-\int_{\mathcal{Z}} f(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z}) .
$$

证明。推论 2.17 意味着

$$
\mathrm{W}_1(\mathbb{P}, \hat{\mathbb{P}})=\sup _{f \in \mathcal{L}^1(\mathbb{P})} \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z)-\int_{\mathcal{Z}} f^c(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z})
$$

此外，它确保，如果我们将搜索空间限制为可表示为 $f=g^c$ 的函数，其中 $g: \mathcal{Z} \rightarrow(-\infty,+\infty]$，则上确界不变。根据（2.26），我们有 $f(z)=\inf _{\hat{z} \in \mathcal{Z}} g(\hat{z})-d(z, \hat{z})$。对于任何固定的 $\hat{z} \in \mathcal{Z}$，辅助函数 $f_{\hat{z}}(z)=g(\hat{z})-d(z, \hat{z})$ 显然是相对于度量 $d$ 的 1-Lipschitz 函数。由于 1-Lipschitz 函数的下确界仍然是 1-Lipschitz 函数，因此我们得到 $\operatorname{lip}(f) \leq 1$。总之，我们已经证明了，限制注意力集中在 1-Lipschitz 函数上不会减少对偶最优运输问题的上确界。接下来，我们证明 $\operatorname{lip}(f) \leq 1$ 蕴含 $f^c=f$。事实上，对于任何 $\hat{z} \in \mathcal{Z}$，我们有：

$$
f(\hat{z}) \leq \sup _{z \in \mathcal{Z}} f(z)-d(z, \hat{z}) \leq \sup _{z \in \mathcal{Z}} f(\hat{z})+d(z, \hat{z})-d(z, \hat{z})=f(\hat{z}),
$$

由于两个不等式分别成立，因为 $d(\hat{z}, \hat{z})=0$ 和 $\operatorname{lip}(f) \leq 1$。这通过（2.25）意味着对于所有 $\hat{z} \in \mathcal{Z}$，$f(\hat{z})=\sup _{z \in \mathcal{Z}} f(z)-d(z, \hat{z})=f^c(\hat{z})$。因此，当 $\operatorname{lip}(f) \leq 1$ 时，$f^c$ 与 $f$ 一致，因此该命题成立。

$p$-Wasserstein 模糊集的半径为 $r \geq 0$，围绕参考分布 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 定义为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\} .
$$

Pflug、Pichler 和 Wozabal（2012）研究了稳健投资组合选择问题，其中资产收益分布的不确定性由 $p$-Wasserstein 球体表示。他们证明了，当 $r$ 趋近于无穷大时，最优策略是将资本均等分配给所有可用资产。因此，这一结果揭示了在极端模糊下，实践中广泛使用的 $1 / N$ 投资策略（DeMiguel, Garlappi 和 Uppal 2009）是最优的。Pflug 等人（2012）、Pichler（2013）和 Wozabal（2014）进一步证明，对于广泛的凸风险度量，在 $p$-Wasserstein 球体内所有分布的最坏情况投资组合风险等于参考分布下的名义风险，加上一个随着 Wasserstein 半径 $r$ 扩展的正则化项；参见第8.3节。


### 2.3.1. $p$-Wasserstein模糊集

$p$-Wasserstein 距离在DRO中具有特别的重要性。Kantorovich-Rubinstein 对偶性可用于构造1-Wasserstein球体内所有分布的Lipschitz连续损失函数的最坏期望的简单上界。这个上界是由名义分布 $\hat{\mathbb{P}}$ 下的期望损失加上一个正则化项组成，正则化项由损失函数的Lipschitz模数和模糊集的半径 $r$ 加权而成。Shafieezadeh-Abadeh、Mohajerin Esfahani 和 Kuhn (2015) 证明了这个上界在分布稳健的逻辑回归问题中是准确的。然而，实际上这一准确性结果扩展到了许多线性预测模型，且损失函数是凸的（Chen 和 Paschalidis 2018, 2019, Blanchet, Kang 和 Murthy 2019b, Shafieezadeh-Abadeh 等 2019, Wu, Li 和 Mao 2022），甚至是非凸的损失函数（Gao 等 2024, Ho-Nguyen 和 Wright 2023）。更一般地，1-Wasserstein 模糊集在不同领域中得到了广泛应用，如二阶段和多阶段随机规划（Zhao 和 Guan 2018, Hanasusanto 和 Kuhn 2018, Duque 和 Morton 2020, Bertsimas, Shtern 和 Sturt 2023）、机会约束规划（Chen, Kuhn 和 Wiesemann 2024b, Xie 2021, Ho-Nguyen, Kılınç-Karzan, Küçükyavuz 和 Lee 2022, Shen 和 Jiang 2023）、逆优化（Mohajerin Esfahani, Shafieezadeh-Abadeh, Hanasusanto 和 Kuhn 2018）、统计学习（Blanchet, Glynn, Yan 和 Zhou 2019a, Zhu, Xie, Zhang, Gao 和 Xie 2022b）、假设检验（Gao, Xie, Xie 和 Xu 2018）、上下文随机优化（Zhang, Yang 和 Gao 2024a）、运输（Sun, Xie 和 Witten 2023）、控制（Cherukuri 和 Cortés 2019, Yang 2020, Boskos, Cortés 和 Martínez 2020, Li 和 Martínez 2020, Coulson, Lygeros 和 Dörfler 2021, Aolaritei, Lanzetti, Chen 和 Dörfler 2022a, Terpin, Lanzetti, Yardim, Dörfler 和 Ramponi 2022, Terpin, Lanzetti 和 Dörfler 2024）、电力系统分析（Wang, Gao, Wei, Shafie-khah, Bi 和 Catalao 2018, Ordoudis, Nguyen, Kuhn 和 Pinson 2021）等。

$p$-Wasserstein 距离（特别是 $p=2$）也有着广泛的应用。在回顾其各种用途之前，我们强调2-Wasserstein 距离与在2.1.4节中介绍的Gelbrich距离之间的有趣联系（参见定义2.1）。正如Gelbrich（1990，第2.1定理）指出的那样，两个概率分布之间的2-Wasserstein 距离提供了它们均值协方差对之间Gelbrich 距离的上界。

**定理 2.20** (Gelbrich界限)。假设$\mathcal{Z}$ 配备了欧几里得度量 $d(z, \hat{z})=\|z-\hat{z}\|_2$。对于任意具有有限均值向量 $\mu, \hat{\mu} \in \mathbb{R}^d$ 和协方差矩阵 $\Sigma, \hat{\Sigma} \in \mathbb{S}_{+}^d$ 的分布 $\mathbb{P}, \hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$，我们有：

$$
\mathrm{W}_2(\mathbb{P}, \hat{\mathbb{P}}) \geq \mathrm{G}((\mu, \Sigma),(\hat{\mu}, \hat{\Sigma})) .
$$

证明。根据定义，平方的2-Wasserstein 距离满足：

$$
\begin{aligned}
& \mathrm{W}_2^2(\mathbb{P}, \hat{\mathbb{P}})=\inf _{\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})} \int_{\mathcal{Z} \times \mathcal{Z}}\|z-\hat{z}\|_2^2 \mathrm{~d} \gamma(z, \hat{z}) \\
& = \begin{cases}\text { inf } & \|\mu-\hat{\mu}\|_2^2+\operatorname{Tr}[\Sigma+\hat{\Sigma}-2 C] \\
\text { s.t. } & \gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}}), C \in \mathbb{R}^{d \times d} \\
& \int_{\mathcal{Z} \times \mathcal{Z}}\left[\begin{array}{l}
z-\mu \\
\hat{z}-\hat{\mu}
\end{array}\right]\left[\begin{array}{c}
z-\mu \\
\hat{z}-\hat{\mu}
\end{array}\right]^{\top} \mathrm{d} \gamma(z, \hat{z})=\left[\begin{array}{cc}
\Sigma & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right], \quad\left[\begin{array}{cc}
\Sigma & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0 .\end{cases}
\end{aligned}
$$

注意，新的决策变量 $C$ 是由运输计划 $\gamma$ 唯一确定的，也就是它表示 $Z$ 和 $\hat{Z}$ 在 $\gamma$ 下的交叉协方差矩阵。因此，它的存在并不会扩大可行集。还要注意，最后表达式中的线性矩阵不等式是多余的，因为 $\gamma$ 的二阶矩阵必然是半正定的。因此，它的存在不会减少可行集。最后，注意到二次函数的积分：

$$
\begin{aligned}
\|z-\hat{z}\|_2^2= & \|\mu-\hat{\mu}\|_2^2+\|z-\mu\|_2^2+\|\hat{z}-\hat{\mu}\|_2^2-2(z-\mu)^{\top}(\hat{z}-\hat{\mu}) \\
& +2(\mu-\hat{\mu})^{\top}(z-\mu)-2(\mu-\hat{\mu})^{\top}(\hat{z}-\hat{\mu})
\end{aligned}
$$

相对于 $\gamma$ 的积分是唯一由 $\gamma$ 的一阶和二阶矩确定的，并且它等于 $\|\mu-\hat{\mu}\|_2^2+\operatorname{Tr}[\Sigma+\hat{\Sigma}-2 C]$。通过放宽最后的优化问题，移除所有涉及 $\gamma$ 的约束，从而得到：

$$
\mathrm{W}_2^2(\mathbb{P}, \hat{\mathbb{P}}) \geq\left\{\begin{array}{cl}
\min _{C \in \mathbb{R}^{d \times d}} & \|\mu-\hat{\mu}\|_2^2+\operatorname{Tr}[\Sigma+\hat{\Sigma}-2 C] \\
\text { s.t. } & {\left[\begin{array}{cc}
\Sigma & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0}
\end{array}\right.
$$

根据命题 2.2，得到的半正定规划的最优值即为 $\mathrm{G}^2((\mu, \Sigma),(\hat{\mu}, \hat{\Sigma}))$。通过对两边开方，得出结论。

定理 2.20 的证明揭示了平方的Gelbrich 距离与一个松弛的最优运输问题的最小值一致，这个松弛问题只要求运输计划 $\gamma$ 的边际具有与 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 分别相同的一阶和二阶矩。Gelbrich 不等式在无法精确计算2-Wasserstein 距离时可能会很有用。事实上，在离散分布只有两个原子时，计算离散分布和连续分布之间的2-Wasserstein 距离就已经是 #P-hard（Taşkesen, Shafieezadeh-Abadeh 和 Kuhn 2023a）。当两个分布都是离散时，计算2-Wasserstein 距离甚至可能是 #P-hard（Taşkesen, Shafieezadeh-Abadeh, Kuhn 和 Natarajan 2023b）。如果 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 都是高斯分布，那么Gelbrich 不等式就变为等式。因此，两个高斯分布之间的2-Wasserstein 距离与它们的均值向量和协方差矩阵之间的Gelbrich 距离相等（Givens 和 Shortt 1984，第7条命题）。这个经典结果实际上早于Gelbrich 不等式，如今被认为是Brenier（1991）提出的最优运输问题最优性的直接结果。利用Brenier的最优性条件，可以更一般地证明，如果 $\hat{\mathbb{P}}$ 是 $\mathbb{P}$ 的正半定仿射推前，即存在一个仿射函数 $f(z)=A z+b$，其中 $A \in \mathbb{S}_{+}^d$ 和 $b \in \mathbb{R}^d$ 使得 $\hat{\mathbb{P}}=\mathbb{P} \circ f^{-1}$，那么 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 之间的2-Wasserstein 距离再次等于它们的均值向量和协方差矩阵之间的Gelbrich 距离（Nguyen 等 2021，第2条定理）。2-Wasserstein 模糊集已广泛应用于机器学习（Sinha, Namkoong 和 Duchi 2018, Blanchet 等 2019b, Blanchet, Murthy 和 Si 2022b, Blanchet, Murthy 和 Zhang 2022c）、逆优化（Mohajerin Esfahani 等 2018）、二阶段随机规划（Hanasusanto 和 Kuhn 2018）、估计和滤波（Shafieezadeh-Abadeh 等 2018, Nguyen 等 2023, Kargin 等 2024b）、投资组合优化（Blanchet, Chen 和 Zhou 2022a, Nguyen 等 2021）以及控制理论（Al Taha 等 2023, Hajar 等 2023, Hakobyan 和 Yang 2024, Taşkesen 等 2024, Kargin 等 2024a,c,d）。


### 2.3.2. Lévy-Prokhorov模糊集

Lévy-Prokhorov距离是最广泛使用的概率度量之一，因为它对$\mathcal{P}(\mathcal{Z})$上的弱收敛拓扑进行了度量化。下面我们假设$d(\cdot, \cdot)$是$\mathcal{Z}$上的一个连续度量。对于任何集合$\mathcal{B} \subseteq \mathcal{Z}$和$r \geq 0$，我们使用

$$
\mathcal{B}_r=\left\{z \in \mathcal{Z}: \exists z^{\prime} \in \mathcal{B} \text { with } d\left(z, z^{\prime}\right) \leq r\right\}
$$

来表示集合$\mathcal{B}$的$r$-邻域。由于度量$d$通常是上下文中显而易见的，所以在符号上省略了$\mathcal{B}_r$对度量$d$的依赖。通过这些准备，我们现在可以定义Lévy-Prokhorov距离。

**定义2.21**（Lévy-Prokhorov距离）。对于$\mathcal{Z}$上的任意度量$d(\cdot, \cdot)$，由$d$引出的Lévy-Prokhorov距离$\mathrm{LP}: \mathcal{P}(\mathcal{Z}) \times \mathcal{P}(\mathcal{Z}) \rightarrow[0,1]$定义为：

$$
\operatorname{LP}(\mathbb{P}, \hat{\mathbb{P}})=\inf \left\{r \geq 0: \mathbb{P}(\mathcal{B}) \leq \hat{\mathbb{P}}\left(\mathcal{B}_r\right)+r \text { for all Borel sets } \mathcal{B} \subseteq \mathcal{Z}\right\},
$$

其中$\mathcal{B}_r$在(2.29)中定义。
Lévy-Prokhorov距离的范围是[0,1]，并且当且仅当其参数相等时，它的值为0。此外，可以轻松证明它满足三角不等式。然而，它似乎是非对称的。下面的命题揭示了Lévy-Prokhorov距离与最优运输理论的紧密联系。

**命题2.22**（Strassen 1965）。如果对于$r \geq 0$，传输成本函数$c_r$通过$c_r(z, \hat{z})=\mathbb{1}_{d(z, \hat{z})>r}$定义，那么有：

$$
\operatorname{LP}(\mathbb{P}, \hat{\mathbb{P}})=\inf \left\{r \geq 0: \mathrm{OT}_{c_r}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\} .
$$

证明。注意到$c_r$是下半连续的，因为假设度量$d$是连续的。根据命题2.16，$\mathrm{OT}_{c_r}(\mathbb{P}, \hat{\mathbb{P}})$因此具有如下对偶表示：

$$
\begin{array}{cl}
\sup _{f \in \mathcal{L}^1(\mathbb{P}), g \in \mathcal{L}^1(\hat{\mathbb{P}})} & \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z)-\int_{\mathcal{Z}} g(\hat{z}) \mathrm{d} \hat{\mathbb{P}}(\hat{z}) \\
\text { s.t. } & f(z)-g(\hat{z}) \leq \mathbb{1}_{d(z, \hat{z})>r} \quad \forall z, \hat{z} \in \mathcal{Z}
\end{array}
$$

这里，对于任何固定的$g$，最优地推高$f$，使得对于所有$z \in \mathcal{Z}$，我们有：

$$
f(z)=\inf _{\hat{z} \in \mathcal{Z}} g(\hat{z})+\mathbb{1}_{d(z, \hat{z})>r} \Longrightarrow \inf _{\hat{z} \in \mathcal{Z}} g(\hat{z}) \leq f(z) \leq 1+\inf _{\hat{z} \in \mathcal{Z}} g(\hat{z}) .
$$

同样，对于任何固定的$f$，最优地推低$g$，使得对于所有$\hat{z} \in \mathcal{Z}$，我们有：

$$
g(\hat{z})=\sup _{z \in \mathcal{Z}} f(z)-\mathbb{1}_{d(z, \hat{z})>r} \Longrightarrow \sup _{z \in \mathcal{Z}} f(z)-1 \leq g(\hat{z}) \leq \sup _{z \in \mathcal{Z}} f(z) .
$$
结合(2.31b)中$g(\hat{z})$的上界与(2.31a)中$f(z)$的上界，进一步推导出$g(\hat{z}) \leq \sup _{z \in \mathcal{Z}} f(z) \leq 1+\inf _{z^{\prime} \in \mathcal{Z}} g\left(z^{\prime}\right)$。最优时，(2.31a)和(2.31b)必须同时成立，因此我们得到：

$$
\inf _{z^{\prime} \in \mathcal{Z}} g\left(z^{\prime}\right) \leq f(z) \leq 1+\inf _{z^{\prime} \in \mathcal{Z}} g\left(z^{\prime}\right) \quad \text { and } \quad \inf _{z^{\prime} \in \mathcal{Z}} g\left(z^{\prime}\right) \leq g(\hat{z}) \leq 1+\inf _{z^{\prime} \in \mathcal{Z}} g\left(z^{\prime}\right)
$$

对于所有$z, \hat{z} \in \mathcal{Z}$。注意到，由于$\mathbb{P}$和$\hat{\mathbb{P}}$都是概率分布，对偶最优运输问题(2.30)的目标函数在替换$f(z) \leftarrow f(z)-\inf _{z^{\prime} \in \mathcal{Z}} g\left(z^{\prime}\right)$和$g(\hat{z}) \leftarrow g(\hat{z})-\inf _{z^{\prime} \in \mathcal{Z}} g\left(z^{\prime}\right)$时保持不变。因此，以下我们可以在不失一般性的情况下假设，对于所有$z \in \mathcal{Z}$，有$0 \leq f(z) \leq 1$，对于所有$\hat{z} \in \mathcal{Z}$，有$0 \leq g(\hat{z}) \leq 1$。

由于$f$和$g$现在被归一化到$[0,1]$，它们具有以下积分表示：

$$
f(z)=\int_0^1 \mathbb{1}_{f(z) \geq \tau} \mathrm{d} \tau \quad \forall z \in \mathcal{Z} \quad \text { and } \quad g(\hat{z})=\int_0^1 \mathbb{1}_{g(\hat{z}) \geq \tau} \mathrm{d} \tau \quad \forall \hat{z} \in \mathcal{Z} .
$$

接下来，可以证明，$f$和$g$当且仅当满足以下约束时，在(2.30)中是可行的：

$$
\mathbb{1}_{f(z) \geq \tau}-\mathbb{1}_{g(\hat{z}) \geq \tau} \leq \mathbb{1}_{d(z, \hat{z})>r} \quad \forall z, \hat{z} \in \mathcal{Z}, \forall \tau \in[0,1]
$$

首先注意，(2.32)在其左侧计算为1且右侧计算为0时是平凡满足的。这仅在某些$\tau \in[0,1]$和$z, \hat{z} \in \mathcal{Z}$满足$f(z) \geq \tau$且$g(\hat{z}) < \tau$，且$d(z, \hat{z}) \leq r$时才会发生。然而，由于这意味着对于某些$z, \hat{z}$，$f(z)-g(\hat{z}) > 0$，这与(2.30)中的约束相矛盾。因此，(2.30)中的约束意味着(2.32)。反之，由于$f$和$g$的积分表示，(2.32)也立即成立。

最后，注意到$\mathbb{1}_{f(z) \geq \tau}$和$\mathbb{1}_{g(\hat{z}) \geq \tau}$分别是Borel集$\mathcal{B}=\{z \in \mathcal{Z}: f(z) \geq \tau\}$和$\mathcal{C}=\{\hat{z} \in \mathcal{Z}: g(\hat{z}) \geq \tau\}$的特征函数。同样，(2.32)当且仅当$\mathcal{C} \supseteq \mathcal{B}_r$时成立。回顾它们的积分表示，我们可以得出结论，只有当$f$和$g$表示为（无限多个）特征函数的凸组合时，$f$和$g$才是(2.30)的可行解，这些特征函数形如$\mathbb{1}_{z \in \mathcal{B}}$和$\mathbb{1}_{\hat{z} \in \mathcal{C}}$，其中$\mathcal{B}$和$\mathcal{C}$是Borel集，且满足$\mathcal{C} \supseteq \mathcal{B}_r$。由于(2.30)的目标函数是线性的，因此如果我们将可行集限制为这些特征函数，其上确界不会改变。因此，(2.30)简化为：

$$
\mathrm{OT}_{c_r}(\mathbb{P}, \hat{\mathbb{P}})=\sup \left\{\mathbb{P}(\mathcal{B})-\hat{\mathbb{P}}(\mathcal{C}): \mathcal{B}, \mathcal{C} \subseteq \mathcal{Z} \text { are Borel sets with } \mathcal{C} \supseteq \mathcal{B}_r\right\}
$$

显然，始终最优地设置$\mathcal{C}=\mathcal{B}_r$，因此该结论得以证明。

虽然命题2.22来源于(Strassen 1965, Theorem 11)，但这里的证明与(Villani 2003, Theorem 1.27)的证明类似。作为副产品，命题2.22揭示了Lévy-Prokhorov距离是对称的，这在其定义中并不显而易见。因此，它确实是一个度量。

Lévy-Prokhorov模糊集的半径$r \geq 0$围绕着$\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$定义为：

$$
\mathcal{P}=\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \operatorname{LP}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\}
$$

对于我们的目的，命题2.22的最重要含义是$\mathcal{P}$可以看作是一个最优运输模糊集的特例，即对于任何半径$r \geq 0$，我们有：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{OT}_{c_r}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\}
$$

Lévy-Prokhorov模糊集最早是在机会约束规划的背景下引入的（Erdoğan 和 Iyengar 2006）。它们也自然出现在数据驱动决策和训练鲁棒机器学习模型中（Pydi 和 Jog 2021，Bennouna 和 Van Parys 2023，Bennouna，Lucas 和 Van Parys 2023）。我们在本节的结尾给出一个有用的推论，它直接来自命题2.22证明的最后部分。

推论2.23。如果对应于$r \geq 0$的传输成本函数$c_r$通过$c_r(z, \hat{z})=\mathbb{1}_{d(z, \hat{z})>r}$定义，那么我们有：

$$
\mathrm{OT}_{c_r}(\mathbb{P}, \hat{\mathbb{P}})=\sup \left\{\mathbb{P}(\mathcal{B})-\hat{\mathbb{P}}\left(\mathcal{B}_r\right): \mathcal{B} \subseteq \mathcal{Z} \text { is a Borel set }\right\},
$$

其中$r$-邻域$\mathcal{B}_r$在(2.29)中定义。


### 2.2.3. 总变差模糊集

两个分布 $\mathbb{P}, \hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 的总变差距离是 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 在任何事件上所分配的概率之间的最大绝对差异。

**定义 2.10（总变差距离）** 总变差距离是定义为

$$
\operatorname{TV}(\mathbb{P}, \hat{\mathbb{P}}) = \sup \{|\mathbb{P}(\mathcal{B}) - \hat{\mathbb{P}}(\mathcal{B})| : \mathcal{B} \subseteq \mathcal{Z} \text{ 是一个 Borels 集合} \}
$$

总变差距离显然是对称的，满足不可区分的恒等式和三角不等式。因此，它构成了 $\mathcal{P}(\mathcal{Z})$ 上的一个度量。此外，总变差距离是一个 $\phi$-散度的实例。

** 命题 2.11.** 总变差距离与由熵函数 $\phi(s) = \frac{1}{2}|s - 1|$ 对所有 $s \geq 0$ 引起的 $\phi$-散度一致。

**证明：** 诱发 Kullback-Leibler 散度的熵函数的共轭熵函数满足 $\phi^*(t) = \max\{t, -\frac{1}{2}\}$，当 $t \leq \frac{1}{2}$ 时，$\phi^*(t) = +\infty$ 当 $t > \frac{1}{2}$ 时。根据命题 2.6，与给定熵函数对应的 $\phi$-散度因此具有如下的对偶表示：

$$
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) = \sup _{f \in \mathcal{F}} \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z) - \int_{\mathcal{Z}} \max\{f(z), -\frac{1}{2}\} \mathrm{d} \hat{\mathbb{P}}(z),
$$

其中 $\mathcal{F}$ 表示所有有界的 Borel 函数 $f: \mathcal{Z} \rightarrow \left(-\infty, \frac{1}{2}\right]$ 的族。注意，$\mathcal{F}$ 对常数平移不变。也就是说，如果 $f(z)$ 是一个有界的 Borel 函数，那么 $f(z) + c$ 对于任何常数 $c \in \mathbb{R}$ 也是有界的 Borel 函数。因此，我们可以在上述最大化问题中同时对 $f \in \mathcal{F}$ 和 $c \in \mathbb{R}$ 进行优化，从而得到：

$$
\operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) = \sup _{f \in \mathcal{F}} \sup _{c \in \mathbb{R}} \int_{\mathcal{Z}} (f(z) + c) \mathrm{d\mathbb{P}}(z) - \int_{\mathcal{Z}} \left(\mathrm{e}^{f(z) + c} - 1\right) \mathrm{d} \hat{\mathbb{P}}(z).
$$

对于任何固定的 $f \in \mathcal{F}$，关于 $c$ 的内层最大化问题可以通过以下方式唯一求解：

$$
c^{\star} = -\log \left(\int_{\mathcal{Z}} \mathrm{e}^{f(z)} \mathrm{d} \hat{\mathbb{P}}(z)\right).
$$

将此表达式代回目标函数中，得到（2.12）。命题 2.9 建立了 Kullback-Leibler 散度与熵风险度量之间的联系。这个联系将在第 4.3 节中变得有用。

Kullback-Leibler 模糊集的半径为 $r \geq 0$ 的模糊集定义为：

$$
\mathcal{P} = \{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\}.
$$

由于 $\phi^{\infty}(1) = +\infty$，因此所有分布 $\mathbb{P} \in \mathcal{P}$ 都是相对于 $\hat{\mathbb{P}}$ 的绝对连续的。因此，$\mathcal{P}$ 与限制 Kullback-Leibler 模糊集一致。El Ghaoui 等人（2003）推导出了当 $\hat{\mathbb{P}}$ 是高斯分布时，线性损失函数的最坏情况价值风险的封闭形式表达式。Hu 和 Hong（2013）使用类似的技术表明，任何相对于 Kullback-Leibler 模糊集的分布鲁棒性个体机会约束等价于参考分布 $\hat{\mathbb{P}}$ 下的经典机会约束，但具有重新调整的置信水平。Calafiore（2007）研究了当 $\hat{\mathbb{P}}$ 是离散分布时，最坏情况下的均值风险投资组合选择问题。Kullback-Leibler 模糊集还在最小二乘估计（Levy 和 Nikoukhah 2004）、假设检验（Levy 2008, Gül 和 Zoubir 2017）、滤波（Levy 和 Nikoukhah 2012, Zorzi 2016, 2017a,b）、风险度量理论（Ahmadi-Javid 2012, Postek 等人 2016）和极值分析（Blanchet, He 和 Murthy 2020）等领域找到了应用。

半径为 $r \geq 0$ 的总变差模糊集围绕 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 定义为：

$$
\mathcal{P}=\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \operatorname{TV}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\}
$$

现有文献大多集中在限制性总变差模糊集上，该模糊集包含所有满足 $\mathbb{P} \ll \hat{\mathbb{P}}$ 的分布 $\mathbb{P} \in \mathcal{P}$。Jiang 和 Guan (2018, Theorem 1) 以及 Shapiro (2017, Example 3.7) 证明，相对于限制性总变差模糊集的最坏期望损失与条件价值-at-risk 和相对于 $\hat{\mathbb{P}}$ 的损失的本质上确界的组合一致，详见第6.10节。Rahimian、Bayraksan 和 Homem-deMello (2019a, $b$, 2022) 研究了在 $\mathcal{Z}$ 是有限集的情况下，未限制总变差模糊集上的 DRO 问题的最坏分布。总变差模糊集与鲁棒统计中的 Huber 污染模型相关（Huber 1981），该模型假设统计数据集中的一部分样本（比例为 $r \in(0,1)$）来自于一个任意的污染分布。因此，目标分布和污染数据生成分布之间的总变差距离至多为 $r$。因此，围绕某个估计分布的半径 $r$ 的总变差模糊集作为目标分布的搜索空间是自然的选择（Nishimura 和 Ozaki 2004, 2006；Bose 和 Daripa 2009；Duchi, Hashimoto 和 Namkoong 2023；Tsanga 和 Shehadeha 2024）。


### 2.3.4. $\infty$-Wasserstein模糊集

第2.3.1节专门讨论了对应于有限指数$p \in[1, \infty)$的$p$-Wasserstein距离。$\infty$-Wasserstein距离需要特别的处理。

**定义2.25**（$\infty$-Wasserstein距离）。对于$\mathcal{Z}$上的一个连续度量$d(\cdot, \cdot)$，对应的$\infty$-Wasserstein距离$\mathrm{W}_{\infty}$ : $\mathcal{P}(\mathcal{Z}) \times \mathcal{P}(\mathcal{Z}) \rightarrow[0, \infty]$定义为：

$$
\mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})=\inf _{\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})} \operatorname{ess} \sup _\gamma[d(Z, \hat{Z})],
$$

其中，$d(Z, \hat{Z})$在$\gamma$下的本质上确界由以下给出：

$$
\operatorname{ess} \sup _\gamma[d(Z, \hat{Z})]=\inf _{\tau \in \mathbb{R}}\{\tau: \gamma(d(Z, \hat{Z})>\tau)=0\}
$$

定义2.25是合理的，因为$\infty$-Wasserstein距离可以通过$p$-Wasserstein距离在$p$趋向无穷大时的极限来获得。

**命题2.26**（Givens和Shortt 1984）。对于任何$\mathbb{P}, \hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$，有：

$$
\mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})=\lim _{p \rightarrow \infty} \mathrm{~W}_p(\mathbb{P}, \hat{\mathbb{P}})=\sup _{p \geq 1} \mathrm{~W}_p(\mathbb{P}, \hat{\mathbb{P}}) .
$$

证明。如果$p \geq q \geq 1$，则$f(t)=t^{q / p}$在$\mathbb{R}_{+}$上是凹的。这意味着：

$$
\mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}})=\inf _{\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})}\left(\mathbb{E}_\gamma\left[d(Z, \hat{Z})^p\right]^{\frac{q}{p}}\right)^{\frac{1}{q}} \geq \inf _{\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})}\left(\mathbb{E}_\gamma\left[d(Z, \hat{Z})^q\right]\right)^{\frac{1}{q}}=\mathrm{W}_q(\mathbb{P}, \hat{\mathbb{P}})
$$

根据Jensen不等式。因此，$\mathbf{W}_p(\mathbb{P}, \hat{\mathbb{P}})$在指数$p$上是单调递增的，只要$p \in[1, \infty)$。此外，对于任何交通计划$\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})$和指数$p \in[1, \infty)$，本质上确界的定义直接意味着：

$$
\left(\mathbb{E}_\gamma\left[d(Z, \hat{Z})^p\right]\right)^{\frac{1}{p}} \leq \operatorname{ess}^{-1} \sup _\gamma\left[d(Z, \hat{Z})^p\right]^{\frac{1}{p}}=\operatorname{ess} \sup _\gamma[d(Z, \hat{Z})] .
$$

对上述不等式两边在所有$\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})$上进行最小化，进一步推导出$\mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}}) \leq \mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})$对于所有$p \in[1, \infty)$成立。因此，我们可以得出结论：

$$
\lim _{p \rightarrow \infty} \mathrm{~W}_p(\mathbb{P}, \hat{\mathbb{P}})=\sup _{p \geq 1} \mathrm{~W}_p(\mathbb{P}, \hat{\mathbb{P}}) \leq \mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}}) .
$$

剩下的任务是证明这个不等式实际上成立为等式。为此，固定一些容忍度$\varepsilon>0$。对于任意$p \in \mathbb{N}$，令$\gamma_p \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})$是一个配对，使得$\mathbb{E}_{\gamma_p}\left[d(Z, \hat{Z})^p\right]^{1 / p}=\mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}})$。注意到$\gamma_p$是存在的，因为正如我们将在推论3.16和命题3.3中看到的那样，$\Gamma(\mathbb{P}, \hat{\mathbb{P}})$是弱紧的，并且$\mathbb{E}_\gamma\left[d(Z, \hat{Z})^p\right]$在$\gamma$中的弱下半连续性。接下来，令$\left\{\gamma_{p(j)}\right\}_{j \in \mathbb{N}}$是一个子序列，它弱收敛到某个配对$\gamma_{\infty} \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})$，它再次存在，因为$\Gamma(\mathbb{P}, \hat{\mathbb{P}})$是弱紧的。我们通过分情况讨论。

**情况1**. 如果$\operatorname{ess} \sup _{\gamma_{\infty}}[d(Z, \hat{Z})]$是有限的，定义开集：

$$
\mathcal{B}=\left\{(z, \hat{z}) \in \mathcal{Z} \times \mathcal{Z}: d(z, \hat{z})>\operatorname{ess} \sup _{\gamma_{\infty}}[d(Z, \hat{Z})]-\varepsilon\right\},
$$

并注意到根据本质上确界的定义，$\gamma_{\infty}(\mathcal{B})>0$。我们得到：

$$
\begin{aligned}
\mathrm{W}_{p(j)}(\mathbb{P}, \hat{\mathbb{P}}) & \geq\left(\int_{\mathcal{B}} d(z, \hat{z})^{p(j)} \mathrm{d} \gamma_{p(j)}(z, \hat{z})\right)^{\frac{1}{p(j)}} \\
& \geq \gamma_{p(j)}(\mathcal{B})^{\frac{1}{p(j)}}\left(\operatorname{ess} \sup _{\gamma_{\infty}}[d(Z, \hat{Z})]-\varepsilon\right) \\
& \geq \gamma_{p(j)}(\mathcal{B})^{\frac{1}{p(j)}}\left(\mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})-\varepsilon\right) .
\end{aligned}
$$

由于$\mathcal{B}$是开放的，并且$\gamma_{p(j)}$弱收敛到$\gamma_{\infty}$，随着$j$增大，Portmanteau定理（Billingsley 2013, Theorem 2.1 (iiv)）意味着$\lim \inf _{j \rightarrow \infty} \gamma_{p(j)}(\mathcal{B}) \geq \gamma_{\infty}(\mathcal{B})>0$。因此，$\gamma_{p(j)}(\mathcal{B})^{1 / p(j)}$随着$j$的增大收敛到1，我们得到：

$$
\lim _{p \rightarrow \infty} \mathrm{~W}_p(\mathbb{P}, \hat{\mathbb{P}}) \geq \mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})-\varepsilon
$$

由于这个不等式对任意容忍度$\varepsilon>0$都成立，因此上述推理最终意味着$\mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}})$确实收敛到$\mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})$，当$p$足够大时。

**情况2**. 如果$\operatorname{ess} \sup _{\gamma_{\infty}}[d(Z, \hat{Z})]=\infty$，则我们将定义中开集$\mathcal{B}$的$\operatorname{ess}_{\sup _{\gamma_{\infty}}}[d(Z, \hat{Z})]$替换为一个任意大的常数。按照情况1的推理，最终揭示$\lim _{p \rightarrow \infty} \mathrm{~W}_p(\mathbb{P}, \hat{\mathbb{P}})=\mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})=\infty$。

为了理解命题2.26的直觉，考虑在$\mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}})$定义中的最优运输问题。如果$p>1$，则运输一单位概率质量从$\hat{z}$到$z$的费用$c(z, \hat{z})=d(z, \hat{z})^p$随着距离$d(z, \hat{z})$的增大呈超线性增长。因此，在最优运输计划中，$\hat{\mathbb{P}}$的部分分布在更远的地方被运输，对$\mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}})$的贡献更大。此外，随着$p$趋向无穷大，最终只有在最远处运输的分布部分才对$\mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})$有影响。更确切地说，只有最大的运输距离才重要，而运输的概率质量数量则不重要。

尽管有命题2.26的存在，$p<\infty$和$p=\infty$阶Wasserstein距离定义中的最优运输问题在根本上是不同的。事实上，如果$p<\infty$，则最优运输问题的目标函数$\mathbb{E}_\gamma\left[d(Z, \hat{Z})^p\right]$是关于运输计划$\gamma$的线性的。而当$p=\infty$时，目标函数$\operatorname{ess} \sup _\gamma[d(Z, \hat{Z})]$在$\gamma$中的函数形式甚至不是凸的，而是准凸的（Jylhä 2015, Lemma 2.2）；参见（Champion, De Pascale和Juutinen 2008）。因此，$\infty$-Wasserstein距离需要更为细致的处理。

接下来的命题将$\infty$-Wasserstein距离与标准的最优运输问题联系起来，因此具有计算上的相关性。

命题2.27。如果对应于$r \geq 0$的交通成本函数$c_r$通过$c_r(z, \hat{z})=\mathbb{1}_{d(z, \hat{z})>r}$定义，那么我们有：

$$
\mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})=\inf \left\{r \geq 0: \mathrm{OT}_{c_r}(\mathbb{P}, \hat{\mathbb{P}}) \leq 0\right\}
$$

证明。回顾$\mathrm{OT}_{c_r}(\mathbb{P}, \hat{\mathbb{P}})=\inf \left\{\mathbb{E}_\gamma\left[c_r(Z, \hat{Z})\right]: \gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})\right\}$。注意到基础的最优运输问题是可解的，因为$\Gamma(\mathbb{P}, \hat{\mathbb{P}})$是弱紧的，并且$\mathbb{E}_\gamma\left[d(Z, \hat{Z})^p\right]$在$\gamma$中的弱下半连续性得益于推论3.16和命题3.3。由于此，得出：

$$
\mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})=\inf \left\{r \geq 0: \mathrm{OT}_{c_r}(\mathbb{P}, \hat{\mathbb{P}}) \leq 0\right\}
$$

命题2.27结合推论2.23立即得出$\infty$-Wasserstein距离的以下等效表述。

推论2.28（Givens和Shortt 1984）。$\infty$-Wasserstein距离满足：

$$
\mathbf{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}})=\inf \left\{r \geq 0: \mathbb{P}(\mathcal{B}) \leq \hat{\mathbb{P}}\left(\mathcal{B}_r\right) \text { for all Borel sets } \mathcal{B} \subseteq \mathcal{Z}\right\},
$$

其中$r$-邻域$\mathcal{B}_r$在(2.29)中定义。

$\infty$-Wasserstein模糊集的半径$r \geq 0$围绕着$\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$定义为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\}
$$

命题2.27表明，$\mathcal{P}$与交通成本函数$c_r(z, \hat{z})=\mathbb{1}_{d(z, \hat{z})>r}$的最优运输模糊集重合，即我们有：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{OT}_{c_r}(\mathbb{P}, \hat{\mathbb{P}}) \leq 0\right\}
$$

带有$\infty$-Wasserstein模糊集的DRO与对抗性机器学习（Gao, Chen和Kleywegt 2017, García Trillos和García Trillos 2022, García Trillos和Murray 2022, García Trillos和Jacobs 2023, Bungert, García Trillos和Murray 2023, Bungert, Laux和Stinson 2024, Gao等 2024, Pydi和Jog 2024, Frank和Niles-Weed 2024a,b）以及核密度估计（Xu, Caramanis和Mannor 2012a）有着密切的联系。此外，$\infty$-Wasserstein模糊集还被用于二阶段和多阶段随机规划（Xie 2020, Bertsimas, Shtern和Sturt 2022, Bertsimas等 2023）、投资组合优化（Nguyen, Zhang, Wang, Blanchet, Delage和Ye 2024）和鲁棒学习（Nguyen, Zhang, Blanchet, Delage和Ye 2020, Wang, Nguyen和Hanasusanto 2024c）等多个领域。


## 2.4. 其他模糊集

存在一些无法归类为矩 moment、$\phi$-散度或最优运输模糊集的模糊集。以下是对这些模糊集的简要概述，未提供详细的数学细节。


### 2.4.1. 边缘模糊集

边缘模糊集指定多个子向量的边缘分布，而不详细描述它们的联合分布。边缘模糊集的最简单例子是Fréchet模糊集，它指定了所有个体组成部分$Z$的边缘分布，但未提供它们的联合分布信息。因此，Fréchet模糊集通过$d$个边缘累计分布函数$F_i: \mathbb{R} \rightarrow[0,1], i \in[d]$来参数化，并可以表示为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}\left(\mathbb{R}^d\right): \mathbb{P}\left(Z_i \leq z_i\right)=F_i\left(z_i\right) \forall z_i \in \mathbb{R}, \forall i \in[d]\right\} .
$$

这里，$F_i$是任意的累计分布函数，即一个右连续、单调递增的函数，满足$\lim _{z_i \rightarrow-\infty} F_i\left(z_i\right)=0$和$\lim _{z_i \rightarrow+\infty} F_i\left(z_i\right)=1$。Fréchet模糊集对于概率逻辑具有重要意义。假设每个$Z_i$表示一个二值变量，如果某个事件发生，则评估为1，否则为0，并且假设每个事件的概率是已知的，而所有事件的联合分布未知。在这种情境下，Boole（1854）曾感兴趣的是计算由$Z_i, i \in[d]$的布尔函数编码的复合事件的概率界限。近一个世纪后，Fréchet（1935）推导出了此类复合事件的概率的显式不等式，现在称为Fréchet不等式。注意，这些Fréchet不等式可以通过在具有伯努利边缘的Fréchet模糊集中的所有分布上最小化或最大化复合事件的概率来获得。近年来，广义Fréchet不等式受到了越来越多的关注，这些不等式界定了在Fréchet模糊集内所有分布下，$Z$的广义（不一定是布尔）函数的风险。例如，在金融和风险管理中，已经出现了大量针对随机变量和其和的Fréchet不等式（Rüschendorf 1983, 1991, Embrechts和Puccetti 2006, Wang和Wang 2011, Wang, Peng和Yang 2013, Puccetti和Rüschendorf 2013, Van Parys, Goulart和Embrechts 2016a, Blanchet, Lam, Liu和Wang 2024a）。此外，Natarajan, Song和Teo（2009b）推导出了Fréchet模糊集上的分段仿射函数的最坏期望的精确界限。我们还要强调，Fréchet模糊集还很重要，因为它们与多边缘最优运输问题的可行集一致，而这些问题有时可以在多项式时间内求解（Pass 2015, Altschuler和Boix-Adsera 2023, Natarajan, Padmanabhan和Ramachandra 2023）。

一般边缘模糊集指定了多个（可能重叠）子集$\left\{Z_i: i \in[d]\right\}$的边缘分布。然而，即使每个$Z_i$是一个伯努利随机变量，并且每个子集仅包含两个元素，检查此类模糊集是否非空也是NP完全的（Honeyman, Ladner和Yannakakis 1980, Georgakopoulos, Kavvadias和Papadimitriou 1988）。因此，计算边缘模糊集上的最坏期望是不可行的，除非已知边缘的随机变量子集是互不相交的（Doan和Natarajan 2012），或者相应的重叠图具有运行交集性质（Doan, Li和Natarajan 2015）。边缘模糊集具有吸引力，因为在有限的统计数据下，估计低维边缘比其全局依赖结构容易得多。然而，即使是单变量边缘，也无法精确估计。因此，许多研究者研究了只提供有限信息（例如边缘矩或边缘离散度量界限）的边缘模糊集（Bertsimas等 2004, Bertsimas, Natarajan和Teo 2006a,b, Chen, Sim, Sun和Teo 2010, Mishra, Natarajan, Tao和Teo 2012, Natarajan, Sim和Uichanco 2018）。

相关文献流关注的是随机变量$Z_i, i \in[d]$是独立的且服从模糊边缘分布的模糊集。例如，Hoeffding模糊集包含所有在一个盒子中的联合分布，具有独立的（且完全未知的）边缘，而Bernstein模糊集包含所有在Hoeffding模糊集中的分布，且服从边缘矩的界限（Nemirovski和Shapiro 2007, Hanasusanto等 2015a）。对每个边缘同时施加均值和均值绝对偏差界限的Bernstein模糊集被用来为基于模糊分布的机会约束程序（Postek, Ben-Tal, den Hertog和Melenberg 2018）、二阶段整数规划（Postek等 2018, Postek, Romeijnders, den Hertog和van der Vlerk 2019）以及排队系统（Wang, Prasad, Hanasusanto和Hasenbein 2024d）推导出安全的可处理逼近。

基于边缘模糊集的DRO与风险管理中的子模性理论和共单调性理论密切相关（Tchen 1980, Rüschendorf 2013, Bach 2013, 2019, Natarajan等 2023, Long, Qi和Zhang 2024）。它在从离散选择建模（Natarajan等 2009b, Mishra, Natarajan, Padmanabhan, Teo和Li 2014, Chen, Ma, Natarajan, Simchi-Levi和Yan 2022, Ruan, Li, Murthy和Natarajan 2022）、排队理论（van Eekelen, den Hertog和van Leeuwaarden 2022）、交通运输（Wang, Chen和Liu 2020, Shehadeh 2023）、机会约束规划（Xie, Ahmed和Jiang 2022）、调度（Mak等 2015）、库存管理（Liu, Chen, Wang和Wang 2024a）、复杂网络分析（Chen, Padmanabhan, Lim和Natarajan 2020, Van Leeuwaarden和Stegehuis 2021, Brugman等 2022）到机制设计（Carroll 2017, Gravin和Lu 2018, Chen等 2024a, Wang, Liu和Zhang 2024b, Wang 2024）等众多应用中具有广泛的应用。有关更多详细信息，请参阅Natarajan（2021）的全面专著。

### 2.4.2. 混合模糊集与结构模糊集

设$\Theta \subseteq \mathbb{R}^m$为一个Borel集合，$\mathbb{P}_\theta \in \mathcal{P}(\mathcal{Z})$为一个由$\theta \in \Theta$唯一确定的参数化分布。假设对于每个固定的Borel集合$\mathcal{B} \subseteq \mathcal{Z}$，$\mathbb{P}_\theta(Z \in \mathcal{B})$是$\theta$的Borel可测函数。然后，参数化的分布族$\left\{\mathbb{P}_\theta: \theta \in \Theta\right\}$可以作为一个混合分布族，进而引入混合模糊集：

$$
\mathcal{P}=\left\{\int_{\Theta} \mathbb{P}_\theta \mathrm{d} \mathbb{Q}(\theta): \mathbb{Q} \in \mathcal{P}(\Theta)\right\} .
$$

因此，$\mathcal{P}$包含所有可以表示为$\mathbb{P}_\theta, \theta \in \Theta$的混合分布。换句话说，对于每个$\mathbb{P} \in \mathcal{P}$，存在一个混合分布$\mathbb{Q} \in \mathcal{P}(\Theta)$，使得对于所有Borel集合$\mathcal{B} \subseteq \mathcal{Z}$，$\mathbb{P}(Z \in \mathcal{B})=\int_{\Theta} \mathbb{P}_\theta(Z \in \mathcal{B}) \mathrm{d} \mathbb{Q}(\theta)$。这一构造确保了$\mathcal{P} \subseteq \mathcal{P}(\mathcal{Z})$是凸的。例如，如果$\mathbb{P}_{\boldsymbol{\theta}}$是一个高斯分布，其均值和协方差矩阵由$\theta$编码，则$\mathcal{P}$包含（可能是连续的）高斯分布的混合。Lasserre和Weisser（2021）研究了与紧参数集$\Theta$对应的混合模糊集，提出了基于半正定规划的逐步逼近方法，用于解决分布鲁棒机会约束的问题。

注意，$\mathcal{P}$可以视为参数化分布族$\left\{\mathbb{P}_\theta: \theta \in \Theta\right\}$的凸包。Minkowski的经典凸分析结果表明，任何Euclidean向量空间中的紧凸子集都与其极点的凸包一致。Choquet理论（Phelps 1965）寻求类似的极点表示，用于拓扑向量空间中的紧凸子集。例如，如果$\left\{\mathbb{P}_\theta: \theta \in \Theta\right\}$是弱紧凸模糊集$\mathcal{P}$的所有极分布集合，则（2.36）构成了$\mathcal{P}$的Choquet表示。

具有某些结构性质的分布族有时承认形式为（2.36）的Choquet表示。例如，设$\mathcal{P}$为所有关于原点对称的分布$\mathbb{P} \in \mathcal{P}\left(\mathbb{R}^d\right)$的族。这意味着，对于每个Borel集合$\mathcal{B} \subseteq \mathbb{R}^d$，$\mathbb{P}(Z \in \mathcal{B})=\mathbb{P}(-Z \in \mathcal{B})$。然后，可以证明，$\mathcal{P}$的所有极分布都可以表示为$\mathbb{P}_\theta=\frac{1}{2} \delta_{+\theta}+\frac{1}{2} \delta_{-\theta}$，其中$\theta \in \mathbb{R}^d$。因此，$\mathcal{P}$承认形式为（2.36）的Choquet表示。作为另一个例子，设$\mathcal{P}$为所有关于原点$\alpha$-单峰的分布$\mathbb{P} \in \mathcal{P}\left(\mathbb{R}^d\right)$的族，某些$\alpha>0$。这意味着，对于每个Borel集合$\mathcal{B} \subseteq \mathcal{R}^d$，$t^\alpha \mathbb{P}(Z \in \mathcal{B} / t)$在$t>0$时是单调递增的。然后，可以证明，$\mathcal{P}$的每个极分布都是一种分布$\mathbb{P}_\theta$，其支持在从0到$\theta \in \mathbb{R}^d$的线段上，具有$\mathbb{P}_\theta\left(\|Z\|_2 \leq t\|\theta\|_2\right)=t^\alpha$，对于所有$t \in[0,1]$。因此，$\mathcal{P}$再次承认形式为（2.36）的Choquet表示。我们还注意到，$\mathbb{R}^d$上的$d$-单峰分布也被称为星型单峰分布。人们可以轻松验证，如果一个具有连续概率密度函数的分布是星型单峰的，当且仅当其密度函数沿每条从原点发出的射线单调递减。此外，人们可以证明，所有$\alpha$-单峰分布的族以精确的方式收敛到所有可能分布的族，随着$\alpha$趋于无穷大。有关结构分布族及其Choquet表示的更多信息，请参见（Dharmadhikari和Joag-Dev 1988）。

第2.1节中的矩模糊集已知包含仅具有少数原子的离散分布；参见第7节。然而，真实的物理、技术或经济系统中遇到的不确定性不太可能遵循这些离散分布。相反，它们通常被期望是单峰的。因此，消除矩模糊集中的病态离散分布的有效方法是将其与某个$\alpha>0$的单峰分布的结构模糊集交集。Popescu（2005）结合了Choquet理论和求和平方多项式优化的思想，通过逐步逼近的方法，为求解基于此交集模糊集的最坏期望问题提供了一个层次化的可处理上界，每个上界都是通过求解一个可解的半正定程序得到的。Van Parys, Goulart和Kuhn（2016b）以及Van Parys, Goulart和Morari（2019）扩展了这一方法，并为基于所有$\alpha$-单峰分布的切比雪夫模糊集上的多面体的最坏情况概率和分段线性凸损失函数的最坏情况条件价值-at-risk提供了准确的半正定规划重构；参见（Hanasusanto, Roitch, Kuhn和Wiesemann 2015b）。Li, Jiang和Mathieu（2019a）证明，这些半正定规划重构有时可以简化为高效的二阶锥规划。Li, Jiang和Mathieu（2016）在电力系统应用的背景下证明，补充结构信息通常会导致较少保守的DRO模型。Lam, Liu和Zhang（2021）考虑了分布形状的另一个基本概念——正交单峰性，并建立了相应的Choquet表示，用于解决多变量极端事件估计。最近，Lam, Liu和Singham（2024）将Choquet理论与重要性采样和似然比技术相结合，用于建模分布形状。


### 2.4.3. 非标准$\phi$-散度和最优传输模糊集

已提出大量非标准$\phi$-散度和最优传输差异度量，用于衡量概率分布之间的不相似性。这些方法在设计具有互补计算和统计特性的模糊集时提供了极大的灵活性。非标准距离度量特别包括平滑$\phi$-散度（Zeitouni和Gutman 1991，Yang和Chen 2018，Liu，Van Parys和Lam 2023），以及$\phi$-散度和最优传输差异的组合（Reid和Williamson 2011，Dupuis和Mao 2022，Van Parys 2024）。此外，它们还包括一致性Wasserstein距离（Li和Mao 2022）和Sinkhorn散度（Wang，Gao和Xie 2021），以及基于因果最优传输的散度（Analui和Pflug 2014，Pflug和Pichler 2014，Yang，Zhang，Chen，Gao和Hu 2022，Arora和Gao 2022，Jiang和Obloj 2024），抗离群值最优传输（Nietert，Goldfeld和Shafiee 2024a,b），混合特征最优传输（Selvi，Belbasi，Haugh和Wiesemann 2022，Belbasi，Selvi和Wiesemann 2023），基于聚类的最优传输（Wang，Becker，Van Parys和Stellato 2022），部分最优传输（Esteban-Pérez和Morales 2022），切片最优传输（Olea，Rush，Velez和Wiesel 2022），多边际最优传输（Lau和Liu 2022，García Trillos，Jacobs和Kim 2023，Rychener，Esteban-Pérez，Morales和Kuhn 2024），以及约束条件矩最优传输（Li等 2022，Blanchet，Kuhn，Li和Taşkesen 2023，Sauldubois和Touzi 2024）。


### 2.4.4. 基于积分概率度量的模糊集

设$\mathcal{F}$为一族Borel可测的测试函数$f: \mathcal{Z} \rightarrow \mathbb{R}$，使得当且仅当$f \in \mathcal{F}$时，$-f \in \mathcal{F}$。由$\mathcal{F}$生成的积分概率度量定义为

$$
\mathrm{D}_{\mathcal{F}}(\mathbb{P}, \hat{\mathbb{P}})=\sup _{f \in \mathcal{F}} \int_{\mathcal{Z}} f(z) \mathrm{dP}(z)-\int_{\mathcal{Z}} f(\hat{z}) \mathrm{d} \mathbb{P}(\hat{z})
$$

对于所有分布$\mathbb{P}, \hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$，在这些分布下，所有测试函数$f \in \mathcal{F}$均可积。上述最大化问题探讨了测试函数如何区分$\mathbb{P}$与$\hat{\mathbb{P}}$。根据构造，$\mathrm{D}_{\mathcal{F}}$是一个伪度量，即它是非负对称的（因为$\mathcal{F}=-\mathcal{F}$），当其参数相同时为零，并且满足三角不等式。此外，当$\mathcal{F}$能够区分分布时，$\mathrm{D}_{\mathcal{F}}$成为一个适当的度量，此时$\mathrm{D}_{\mathcal{F}}(\mathbb{P}, \hat{\mathbb{P}})$仅在$\mathbb{P}=\hat{\mathbb{P}}$时为零。关于$\mathrm{D}_{\mathcal{F}}$的模糊集的半径$r \geq 0$，围绕$\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$定义为

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{D}_{\mathcal{F}}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\}
$$

Proposition 2.11的证明揭示了全变差距离是由所有Borel函数$f: \mathcal{Z} \rightarrow[-1 / 2,1 / 2]$生成的积分概率度量；见(2.16)。在Corollary 2.19中建立的Kantorovich-Rubinstein对偶性进一步表明，1-Wasserstein距离是由所有Lipschitz连续函数$f: \mathcal{Z} \rightarrow \mathbb{R}$且满足$\operatorname{lip}(f) \leq 1$生成的积分概率度量。此外，如果$\mathcal{H}$是一个再生核希尔伯特空间，包含了Borel函数$f: \mathcal{Z} \rightarrow \mathbb{R}$且具有希尔伯特范数$\|\cdot\|_{\mathcal{H}}$，那么与$\mathcal{H}$对应的最大均值差异距离是由标准单位球$\mathcal{F}=\left\{f \in \mathcal{H}:\|f\|_{\mathcal{H}} \leq 1\right\}$在$\mathcal{H}$中生成的积分概率度量。最大均值差异模糊集在(Staib and Jegelka 2019, Zhu, Jitkrittum, Diehl and Schölkopf 2020, 2021, Zeng and Lam 2022, Iyengar, Lam and Wang 2022)中有研究。Husain (2020)揭示了DRO问题与正则化经验风险最小化问题之间的深刻联系，当模糊集通过积分概率度量定义时，此联系成立。


# 3. 模糊集的拓扑性质

理论上和实践中一个重要的问题是，是否自然的子问题（1.2）是可解的，换句话说，是否（1.2）中的内极大值能够被达成。在本节中，我们将探讨在什么条件下，Weierstrass极值定理适用于自然的子问题。也就是说，我们将发展一些容易检查的条件，在这些条件下，模糊集$\mathcal{P}$是弱紧的，并且期望损失$\mathbb{E}_{\mathbb{P}}[\ell(x, Z)]$在$\mathbb{P}$中是弱下半连续的。在整个讨论中，我们假设$\mathcal{Z}$是$\mathbb{R}^d$的一个闭子集。

Baire的经典结果表明，实数线上一个函数是下半连续的当且仅当它可以表示为一系列非递减连续函数的逐点上确界（Baire 1905）。下面我们将使用这一结果的多维推广。

**引理 3.1 (Stromberg (2015, p. 132))**. 一个函数$f: \mathcal{Z} \rightarrow(-\infty,+\infty]$是下半连续的当且仅当存在一个非递减的连续函数序列$f_i: \mathcal{Z} \rightarrow \mathbb{R}, i \in \mathbb{N}$，使得对于所有$z \in \mathcal{Z}$，有$f(z)=\sup _{i \in \mathbb{N}} f_i(z)$。

如果$f$是下界有界的，那么可以假设连续函数$f_i$是均匀有界的。事实上，如果$f(z) \geq 0$，则可以将连续函数$f_i(z)$替换为有界连续函数$f_i^{\prime}(z)=\min \{\max \{f(z), 0\}, i\}$。序列$f_i^{\prime}, i \in \mathbb{N}$仍然是非递减的，并且逐点收敛于$f$。

**定义 3.2 (概率分布的弱收敛)**. 一列概率分布$\mathbb{P}_j \in \mathcal{P}(\mathcal{Z}), j \in \mathbb{N}$，若对于每个有界连续函数$f: \mathcal{Z} \rightarrow \mathbb{R}$，我们有

$$
\lim _{j \in \mathbb{N}} \mathbb{E}_{\mathbb{P}_j}[f(Z)]=\mathbb{E}_{\mathbb{P}}[f(Z)]，
$$

则称$\mathbb{P}_j$弱收敛于$\mathbb{P}$。

期望值$\mathbb{E}_{\mathbb{P}}[f(Z)]$对于分布$\mathbb{P}$的连续性与$f$的连续性特性之间有密切联系。回顾一下，如果一个函数$F: \mathcal{P}(\mathcal{Z}) \rightarrow \overline{\mathbb{R}}$是弱连续的，那么对于每一个收敛到$\mathbb{P}$的概率分布序列$\mathbb{P}_i \in \mathcal{P}(\mathcal{Z}), i \in \mathbb{N}$，都有$\lim _{i \rightarrow \infty} F\left(\mathbb{P}_i\right)=F(\mathbb{P})$。弱下半连续和弱上半连续也可以用类似的方式定义。

**命题 3.3 (期望值的连续性)**. 如果$f: \mathcal{Z} \rightarrow[-\infty,+\infty]$是下半连续且下界有界的，那么$\mathbb{E}_{\mathbb{P}}[f(Z)]$在$\mathbb{P} \in \mathcal{P}(\mathcal{Z})$中是弱下半连续的。反之，如果$f$是上半连续且上界有界的，那么$\mathbb{E}_{\mathbb{P}}[f(Z)]$在$\mathbb{P} \in \mathcal{P}(\mathcal{Z})$中是弱上半连续的。最后，如果$f$是连续且有界的，那么$\mathbb{E}_{\mathbb{P}}[f(Z)]$在$\mathbb{P} \in \mathcal{P}(\mathcal{Z})$中是弱连续的。

证明：假设首先$f$是下半连续且下界有界的。以下我们假设在不失一般性的情况下，$f$实际上是非负的。然后，根据引理3.1，存在一个非递增的有界、连续且非负的函数序列$f_i, i \in \mathbb{N}$，使得$f(z)=\sup _{i \in \mathbb{N}} f_i(z)$。如果$\mathbb{P}_j \in \mathcal{P}(\mathcal{Z}), j \in \mathbb{N}$是任何收敛到$\mathbb{P}$的分布序列，那么我们得到

$$
\begin{aligned}
\liminf _{j \in \mathbb{N}} \mathbb{E}_{\mathbb{P}_j}[f(Z)] & =\sup _{k \in \mathbb{N}} \inf _{j \geq k} \mathbb{E}_{\mathbb{P}_j}\left[\sup _{i \in \mathbb{N}} f_i(Z)\right] \\
& =\sup _{k \in \mathbb{N}} \inf _{j \geq k} \sup _{i \in \mathbb{N}} \mathbb{E}_{\mathbb{P}_j}\left[f_i(Z)\right] \\
& \geq \sup _{i \in \mathbb{N}} \sup _{k \in \mathbb{N}} \inf _{j \geq k} \mathbb{E}_{\mathbb{P}_j}\left[f_i(Z)\right] \\
& =\sup _{i \in \mathbb{N}} \mathbb{E}_{\mathbb{P}}\left[f_i(\xi)\right]=\mathbb{E}_{\mathbb{P}}[f(Z)]
\end{aligned}
$$

这里，第二个等式和最后一个等式都来自于单调收敛定理，该定理适用是因为每个$f_i$是有界的，因此对于任何概率分布都是可积的，并且$f_i, i \in \mathbb{N}$形成一个非递增的非负函数序列。这个不等式是通过交换对$i$的上确界和对$j$的下确界得到的，第三个等式成立是因为$\mathbb{P}_j$收敛到$\mathbb{P}$并且$f_i$是连续且有界的。这表明$\mathbb{E}_{\mathbb{P}}[f(Z)]$在$\mathbb{P}$中是弱下半连续的。

关于弱上半连续性和弱连续性的证明是类似的，因此省略。

接下来，我们为$\mathcal{P}(\mathcal{Z})$上的所有概率分布族配备弱拓扑，这个拓扑是由以下开集生成的：

$$
U_{f, \delta}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}):\left|\mathbb{E}_{\mathbb{P}}[f(Z)]\right|<\delta\right\}
$$

由任何连续有界函数$f: \mathcal{Z} \rightarrow \mathbb{R}$和容忍度$\delta>0$编码。$\mathcal{P}(\mathcal{Z})$上的弱拓扑是由Prokhorov度量度量的（Billingsley 2013, 定理6.8），因此在$\mathcal{P}(\mathcal{Z})$上的序列紧性和紧性是等价的；见例如（Munkres 2000，定理28.2）。

**定义 3.4 (紧性)**. 一个分布族$\mathcal{P} \subseteq \mathcal{P}(\mathcal{Z})$是紧的，当且仅当对于任何容忍度$\varepsilon>0$，存在一个紧集$\mathcal{C} \subseteq \mathcal{Z}$，使得对所有$\mathbb{P} \in \mathcal{P}$有$\mathbb{P}(Z \notin \mathcal{C}) \leq \varepsilon$。

Prokhorov的经典结果表明，分布族是弱紧的当且仅当它是紧的并且弱闭的。Prokhorov定理是证明模糊集是弱紧的关键工具。我们在这里不提供证明。

**定理 3.5 (Billingsley (2013, 定理5.1))**. 一个分布族$\mathcal{P} \subseteq \mathcal{P}(\mathcal{Z})$是弱紧的当且仅当它是紧的并且弱闭的。


## 3.1. 矩模糊集

支持集模糊集无疑是最简单的矩模糊集类，因为它们根本不施加任何矩条件。事实上，本文中考虑的所有其他模糊集都是支持集模糊集的子集。

**命题 3.6 (支持集模糊集)**. 如果$\mathcal{Z} \subseteq \mathbb{R}^d$，则所有支持在$\mathcal{Z}$上的分布的集合$\mathcal{P}(\mathcal{Z})$当且仅当$\mathcal{Z}$是紧的时，弱紧。

**证明**. 首先注意到，$\mathcal{P}(\mathcal{Z})$当且仅当$\mathcal{Z}$是有界的时是紧的。实际上，如果$\mathcal{Z}$是有界的，则它是紧的，因为根据我们的假设$\mathcal{Z}$是闭的。给定任意$\varepsilon>0$，我们可以设置$\mathcal{C}=\mathcal{Z}$，这确保对于所有$\mathbb{P} \in \mathcal{P}(\mathcal{Z})$有$\mathbb{P}(Z \notin \mathcal{C})=0 \leq \varepsilon$。因此，$\mathcal{P}(\mathcal{Z})$是紧的。另一方面，如果$\mathcal{Z}$是无界的，则$\mathcal{P}(\mathcal{Z})$显然不是紧的。实际上，对于任何紧集$\mathcal{C} \subseteq \mathcal{Z}$，补集$\mathcal{Z} \backslash \mathcal{C}$是非空的，因为$\mathcal{C}$是有界的而$\mathcal{Z}$不是。因此，存在一个分布$\mathbb{P} \in \mathcal{P}(\mathcal{Z})$，它的支持在$\mathcal{Z} \backslash \mathcal{C}$上，且$\mathbb{P}(Z \notin \mathcal{C})=1$。

接下来，注意到，$\mathcal{P}(\mathcal{Z})$当且仅当$\mathcal{Z}$是闭的时是弱闭的。首先假设$\mathcal{Z}$是闭的，并且注意到通过定义的指示函数$\delta_{\mathcal{Z}}$，当$z \in \mathcal{Z}$时$\delta_{\mathcal{Z}}(z)=0$，否则$\delta_{\mathcal{Z}}(z)=+\infty$，是下半连续且下界有界的。根据命题3.3，$\mathbb{E}_{\mathbb{P}}\left[\delta_{\mathcal{Z}}(Z)\right]$因此在$\mathbb{P}$中是弱下半连续的。如果$\mathbb{P}_j \in \mathcal{P}(\mathcal{Z}), j \in \mathbb{N}$，收敛到$\mathbb{P}$，那么我们有

$$
0=\liminf _{j \in \mathbb{N}} \mathbb{E}_{\mathbb{P}_j}\left[\delta_{\mathcal{Z}}(Z)\right] \geq \mathbb{E}_{\mathbb{P}}\left[\delta_{\mathcal{Z}}(Z)\right] \geq 0
$$

其中，等式成立是因为对于每个$j \in \mathbb{N}$，$\mathbb{P}_j$支持在$\mathcal{Z}$上，第一个不等式来自弱下半连续性。这表明$\mathbb{P} \in \mathcal{P}(\mathcal{Z})$，因此$\mathcal{P}(\mathcal{Z})$是弱闭的。反之，假设$\mathcal{P}(\mathcal{Z})$是弱闭的，考虑一个序列$z_j \in \mathcal{Z}, j \in \mathbb{N}$，收敛到$z$。然后，狄拉克分布序列$\delta_{z_j}, j \in \mathbb{N}$收敛到$\delta_z$，因此我们得到

$$
0=\liminf _{j \in \mathbb{N}} \mathbb{E}_{\delta_{z j}}\left[\delta_{\mathcal{Z}}(Z)\right] \geq \mathbb{E}_{\delta_z}\left[\delta_{\mathcal{Z}}(Z)\right] \geq 0
$$

这里，第一个不等式再次成立，因为$\mathbb{E}_{\mathbb{P}}\left[\delta_{\mathcal{Z}}(Z)\right]$在$\mathbb{P}$中是弱下半连续的。这表明$\mathbb{E}_{\delta_z}\left[\delta_{\mathcal{Z}}(Z)\right]=0$，当且仅当$z \in \mathcal{Z}$时成立。因此，$\mathcal{Z}$是闭的。根据这些见解，结论可以通过定理3.5得出。
通过使用命题3.6，我们现在可以证明，如果基础支撑集 $\mathcal{Z}$ 是紧的，矩量函数 $f$ 是连续的，并且不确定集 $\mathcal{F}$ 是闭的，那么形式为(2.1)的矩量模糊集是弱紧的。

**命题3.7（矩量模糊集）**  
如果 $\mathcal{Z} \subseteq \mathbb{R}^d$ 是紧的支撑集，$f: \mathcal{Z} \rightarrow \mathbb{R}^m$ 是连续的矩量函数，且 $\mathcal{F} \subseteq \mathbb{R}^m$ 是闭的不确定集，则按(2.1)定义的矩量模糊集 $\mathcal{P}$ 是弱紧的。

**证明：**  
由于 $\mathcal{Z}$ 是紧的，仅支撑集的模糊集 $\mathcal{P}(\mathcal{Z})$ 根据命题3.6是弱紧的。因此，$\mathcal{P}(\mathcal{Z})$ 是紧的并且弱闭的。这很容易推导出，$\mathcal{P}$ 作为紧集的子集仍然是紧的。命题3.3进一步表明，$\mathbb{E}_{\mathbb{P}}[f(Z)]$ 在 $\mathbb{P}$ 上是弱连续的。由于 $\mathcal{F}$ 是闭的，并且任何闭集在连续变换下的原像也是闭的，我们可以得出结论，$\mathcal{P}_f=\left\{\mathbb{P} \in \mathcal{P}(\mathbb{R}^d): \mathbb{E}_{\mathbb{P}}[f(Z)] \in \mathcal{F}\right\}$ 是弱闭的。因此，$\mathcal{P}=\mathcal{P}(\mathcal{Z}) \cap \mathcal{P}_f$ 是弱闭的，因为它是两个弱闭集的交集。根据这些见解，结论从定理3.5中可以直接推导出来。

命题3.7的条件仅是弱紧的充分条件，而非必要条件。以下示例表明，即使支撑集 $\mathcal{Z}$ 或矩量函数 $f$ 是无界的，矩量模糊集仍然可以是紧的或弱紧的。

**例3.8（马尔科夫模糊集）**  
如果 $\mathcal{Z}=\mathbb{R}^d$，则马尔科夫模糊集(2.2)可能不紧。例如，如果 $\mathcal{Z}=\mathbb{R}$ 且 $\mu=0$，那么对于每个紧集 $\mathcal{C} \subseteq \mathbb{R}$，都有一个常数 $R>0$，使得两点分布 $\mathbb{P}=\frac{1}{2} \delta_{-R}+\frac{1}{2} \delta_R$ 完全支持于 $\mathcal{C}$ 的补集上。然而，如果 $\mathcal{Z}=\mathbb{R}_{+}$ 且 $\mu=1$，则马尔科夫不等式意味着，对于每个 $\mathbb{P} \in \mathcal{P}$ 和任意 $\varepsilon>0$，都可以定义紧集 $\mathcal{C}$ 为区间 $[0, 1/\varepsilon]$，使得 $\mathbb{P}(Z \notin \mathcal{C}) \leq \varepsilon$。即使在这种情况下，$\mathcal{P}$ 也不是弱闭的。实际上，分布 $\mathbb{P}_i=\frac{i}{i+1} \delta_0+\frac{1}{i+1} \delta_{i+1}$ 对于所有 $i \in \mathbb{N}$ 都属于 $\mathcal{P}$，但它们的弱极限 $\mathbb{P}=\delta_0$ 并不属于 $\mathcal{P}$。如果 $\mathcal{Z}$ 是凸的，则可以扩展这一推理，证明只有当 $\mathcal{Z}$ 是紧的时，$\mathcal{P}$ 才是弱紧的。

**例3.9（切比雪夫模糊集）**  
切比雪夫模糊集 $\mathcal{P}$ 定义在(2.3)中，总是紧的。为了说明这一点，假设 $\mu=0$ 且 $M=I_d$，通过仿射坐标变换，可以始终满足这一条件。给定任意的 $\varepsilon>0$，我们可以定义紧集 $\mathcal{C}=\{z \in \mathcal{Z} : \|z\|_2 \leq \sqrt{d/\varepsilon}\}$。然后很容易看出，任何 $\mathbb{P} \in \mathcal{P}$ 都满足

$$
\mathbb{P}(Z \notin \mathcal{C})=\mathbb{P}(\|Z\|_2>\sqrt{d/\varepsilon}) \leq \mathbb{E}_{\mathbb{P}}[\|Z\|_2^2 \cdot \varepsilon / d] = \varepsilon,
$$

其中不等式成立是因为二次函数 $q(z)=\|z\|_2^2 \cdot \varepsilon / d$ 优于 $\mathcal{Z} \backslash \mathcal{C}$ 的特征函数。因此，$\mathcal{P}$ 确实是紧的。然而，$\mathcal{P}$ 不一定是弱闭的。为了看到这一点，假设 $d=1$ 且 $\mathcal{Z}=\mathbb{R}$。在这种情况下，分布 $\mathbb{P}_i=\frac{1}{2i^2} \delta_{-i} + \frac{i^2-1}{i^2} \delta_0 + \frac{1}{2i^2} \delta_i$ 对于所有 $i \in \mathbb{N}$ 都有零均值和单位方差，且它们都属于 $\mathcal{P}$。然而，它们收敛于弱极限 $\mathbb{P}=\delta_0$，后者不属于 $\mathcal{P}$。因此，$\mathcal{P}$ 不是弱紧的。

所有具有有限 $p$-阶矩的 $\mathbb{R}^d$ 上的分布族总是弱紧的，即使那些固定 $p$-阶矩的模糊集（例如切比雪夫模糊集）可能不是弱紧的。

**例3.10（$p$-阶矩模糊集）**  
由任何范数 $\|\cdot\|$ 在 $\mathbb{R}^d$ 上诱导的模糊集

$$
\mathcal{P} = \left\{ \mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_p[\|Z\|^p] \leq R \right\}
$$

对于任意的 $p, R > 0$ 总是弱紧的。使用类似于例3.9中的推理，可以证明，对于任意的 $\varepsilon>0$，存在一个紧集，定义为 $\mathcal{C} = \left\{ z \in \mathcal{Z} : \|z\| \leq (R/\varepsilon)^{1/p} \right\}$，满足 $\mathbb{P}(Z \notin \mathcal{C}) \leq \varepsilon$。因此，$\mathcal{P}$ 是紧的。为了证明 $\mathcal{P}$ 也是弱闭的，注意到 $f(z)=\|z\|^p$ 是连续的并且下界有界。根据命题3.3，期望值 $\mathbb{E}_{\mathbb{P}}[\|Z\|_2^p]$ 在 $\mathbb{P}$ 上是弱下半连续的，且其子水平集是弱闭的。因此，$\mathcal{P}$ 是弱紧的，由定理3.5得出。


## 3.2. $\phi$-散度模糊集

在本节中，我们将证明，当熵函数$\phi$超线性增长时，形式为(2.10)的$\phi$-散度模糊集是弱紧的。否则，如果$\phi$的增长至多是线性的，则相应的$\phi$-散度模糊集通常不会是弱紧的。回想一下，根据定义2.4，当且仅当$\phi^{\infty}(1)=\infty$时，熵函数$\phi$超线性增长；见表2.1。

**引理 3.11 (最坏情况概率映射)**. 设$\mathcal{P}$是由公式(2.10)定义的半径$r>0$的$\phi$-散度模糊集，且假设$\phi$在1处连续且$\phi^{\infty}(1)=\infty$。则存在一个仅依赖于$\phi$和$r$的连续、凹且满射的函数$p:[0,1] \rightarrow[0,1]$，使得对于每个Borel集合$\mathcal{B} \subseteq \mathcal{Z}$，都有

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{P}(Z \in \mathcal{B})=p(\hat{\mathbb{P}}(Z \in \mathcal{B})).
$$

**证明**. 证明是构造性的。即，我们通过以下公式定义函数$p$：

$$
p(t)=\inf _{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}_{+}} \lambda_0+\lambda r+t \cdot\left(\phi^*\right)^\pi\left(1-\lambda_0, \lambda\right)+(1-t) \cdot\left(\phi^*\right)^\pi\left(-\lambda_0, \lambda\right)
$$

对于所有$t \in[0,1]$。接下来，我们证明$p$满足所有期望的属性。根据构造，$p$仅依赖于$\phi$和$r$，并且与$t$的无穷多个线性函数的下包络一致。因此，$p$是凹的，并且是上半连续的。根据$\mathcal{P}$的定义和下面的定理4.15，我们还可以得到：

$$
\begin{aligned}
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{P}(Z \in \mathcal{B}) & =\sup _{\mathbb{P} \in \mathcal{P}(\mathcal{Z})}\left\{\mathbb{E}_{\mathbb{P}}\left[\mathbb{1}_{\mathcal{B}}(Z)\right]: \mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\} \\
& =\inf _{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}_{+}} \lambda_0+\lambda r+\mathbb{E}_{\hat{\mathbb{P}}}\left[\left(\phi^*\right)^\pi\left(\mathbb{1}_{\mathcal{B}}(Z)-\lambda_0, \lambda\right)\right] \\
& =p(\hat{\mathbb{P}}(Z \in \mathcal{B})),
\end{aligned}
$$

对于任何Borel集合$\mathcal{B}$，其中最后一个等式来自于$p$的定义。由于（3.1）式中左侧的最坏情况概率位于$[0,1]$之间，并且$\hat{\mathbb{P}}(Z \in \mathcal{B})$可以取$[0,1]$中的任何值，因此$p$的值域是$[0,1]$的子集。接下来，我们证明$p$是连续的。为此，注意到$p$在$[0,1]$上的凹性和有界性通过（Rockafellar 1970，定理10.1）意味着$p$在$(0,1)$上是连续的。此外，它的上半连续性防止了$p$在0或1处跳跃。因此，$p$在$[0,1]$上确实是连续的。最后，设$\mathcal{B}=\emptyset$或$\mathcal{B}=\mathcal{Z}$，通过（3.1）可以得到$p(0)=0$和$p(1)=1$。因此，我们可以得出结论$p$是满射的。这个观察完成了证明。

由于$\hat{\mathbb{P}} \in \mathcal{P}$，引理3.11中的最坏情况概率映射$p$满足$p(t) \geq t$，对于所有$t \in[0,1]$，即最坏情况概率永远不会小于名义概率。我们注意到，在研究具有$\phi^{\infty}(1)=\infty$的$\phi$-散度模糊集上的分布鲁棒机会约束时，映射$p$也会出现。实际上，任何这样的分布鲁棒机会约束，其违约概率$\varepsilon \in(0,1)$，等价于在参考分布$\hat{\mathbb{P}}$下的经典机会约束，违约概率为$p^{-1}(\varepsilon)$；见（El Ghaoui等，2003；Jiang和Guan，2016；Shapiro，2017）。我们现在可以证明，超线性熵函数对应的散度模糊集是弱紧的。

**命题 3.12 ( $\phi$-散度模糊集)**. 如果$\phi$是一个熵函数，并且$\phi^{\infty}(1)=\infty$，则对于任何闭集$\mathcal{Z} \subseteq \mathbb{R}^d$、分布$\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$和$r \geq 0$，相应的$\phi$-散度模糊集$\mathcal{P}$是弱紧的。

**证明**. 我们首先证明$\mathcal{P}$是紧的。为此，选择任意$\varepsilon \in(0,1)$，并定义$p^{-1}(\varepsilon)$为满足$p(t)=\varepsilon$的唯一$t \in(0,1]$，其中$p$是引理3.11中的最坏情况概率映射。注意，由于$p$是凹的并且是满射的，并且$p(0)=0$且$p(1)=1$，因此$p^{-1}(\varepsilon)$是定义良好的。还注意到$p^{-1}(\varepsilon) \leq \varepsilon$，因为$p(t) \geq t$。接下来，选择一个足够大的$R>0$，使得$\hat{\mathbb{P}}\left(\|Z\|_2>R\right) \leq p^{-1}(\varepsilon)$，并定义一个紧集$\mathcal{C}=\left\{z \in \mathcal{Z}:\|z\|_2 \leq R\right\}$。应用引理3.11，得到

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{P}(Z \notin \mathcal{C})=p(\hat{\mathbb{P}}(Z \notin \mathcal{C})) \leq p\left(p^{-1}(\varepsilon)\right)=\varepsilon,
$$

其中不等式来自$p$的单调性和$R$的选择。我们已经证明，对于所有$\mathbb{P} \in \mathcal{P}$，都有$\mathbb{P}(Z \notin \mathcal{C}) \leq \varepsilon$，因此$\mathcal{P}$是紧的。接下来，我们证明$\mathcal{P}$是弱闭的。为此，首先回忆一下，由于$\mathcal{Z}$是闭的，$\mathcal{P}(\mathcal{Z})$是弱闭的；见命题3.6。接下来，回忆命题2.6中的内容，任何$\phi$-散度都具有以下形式的对偶表示：

$$
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})=\sup _{f \in \mathcal{F}} \int_{\mathcal{Z}} f(z) \mathrm{dP}(z)-\int_{\mathcal{Z}} \phi^*(f(z)) \mathrm{d} \hat{\mathbb{P}}(z),
$$

其中$\mathcal{F}$表示所有有界Borel函数$f: \mathcal{Z} \rightarrow \operatorname{dom}\left(\phi^*\right)$的集合。实际上，$\mathcal{F}$可以限制为所有连续有界函数的空间$\mathcal{F}^c$，而不减少(3.2)中的上确界。这是Lusin定理的直接结果，它确保了对于任意$\delta>0$和$f \in \mathcal{F}$，存在一个紧集$\mathcal{A} \subseteq \mathcal{Z}$，使得$\hat{\mathbb{P}}(Z \notin \mathcal{A}) \leq \delta$，并且存在一个有界连续函数$f_\delta \in \mathcal{F}^c$，它与$f$在$\mathcal{A}$上一致，并且满足$\sup _{z \in \mathcal{Z}}\left|f_\delta(z)\right| \leq \sup _{z \in \mathcal{Z}}|f(z)|=\|f\|_{\infty}$。由于$\phi^*$是一个凸的下半连续函数，它在其定义域上是连续的，因此

$$
\phi_l^*=\inf _{s \in \operatorname{dom}\left(\phi^*\right)}\left\{\phi^*(s):|s| \leq\|f\|_{\infty}\right\} \quad \text { 和 } \quad \phi_u^*=\sup _{s \in \operatorname{dom}\left(\phi^*\right)}\left\{\phi^*(s):|s| \leq\|f\|_{\infty}\right\}
$$

是有界的。因此，我们有

$$
\begin{aligned}
& \int_{\mathcal{Z}} f_\delta(z) \mathrm{d} \mathbb{P}(z)-\int_{\mathcal{Z}} \phi^*\left(f_\delta(z)\right) \mathrm{d} \hat{\mathbb{P}}(z) \\
& \geq \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z)-\int_{\mathcal{Z}} \phi^*(f(z)) \mathrm{d} \hat{\mathbb{P}}(z)-2\|f\|_{\infty} \mathbb{P}(Z \notin \mathcal{A})-\left(\phi_u^*-\phi_l^*\right) \hat{\mathbb{P}}(Z \notin \mathcal{A}) .
\end{aligned}
$$

由于$\phi^{\infty}(1)=\infty$意味着$\mathbb{P} \ll \hat{\mathbb{P}}$，并且$\hat{\mathbb{P}}(Z \notin \mathcal{Z}) \leq \delta$，因此$\mathbb{P}(Z \notin \mathcal{A})$和$\hat{\mathbb{P}}(Z \notin \mathcal{A})$在$\delta$减小时趋近于0。因此，问题(3.2)中$f_\delta$的目标函数值渐进地不低于$f$的目标函数值。这证实了将$\mathcal{H}$限制为$\mathcal{F}^c$不会影响(3.2)中的上确界。现在回忆命题3.3，对于任何有界连续函数$f \in \mathcal{F}^c$，(3.2)中的第一个积分在$\mathbb{P}$中是弱连续的。因此，$\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$作为弱连续函数的点上确界，是弱下半连续的。这意味着$f(\mathbb{P})=\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$的任何子水平集是弱闭的。因此，我们得出结论，散度模糊集是弱闭的。结论然后可以通过定理3.5得出。

命题3.12的证明关键在于假设$\phi^{\infty}(1)=\infty$，这确保了散度模糊集仅包含在$\hat{\mathbb{P}}$下绝对连续的分布。接下来我们将证明，如果熵函数$\phi$至多线性增长（即$\phi^{\infty}(1)<\infty$），并且$\mathcal{Z}$是无界的，那么相应的散度模糊集将不能是弱紧的。作为准备，我们首先确定一个上界，适用于$\mathcal{P}(\mathcal{Z}) \times \mathcal{P}(\mathcal{Z})$上的任何$\phi$-散度。

**引理 3.13 ( $\phi$-散度的上界)**. 如果$\phi$是一个熵函数，且$\mathcal{Z} \subseteq \mathbb{R}^d$是一个闭集，则对所有$\mathbb{P}, \hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$，都有$\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) \leq \phi(0)+\phi^{\infty}(1)$。当$\mathbb{P}$和$\hat{\mathbb{P}}$是互相奇异时，这个上界达到。

**证明**. 证明的第一部分我们推导出所需的上界。为此，假设$\phi(0)<\infty$且$\phi^{\infty}(1)<\infty$，否则上界显然成立。由于熵函数是凸的，我们有

$$
\phi(s) \leq \frac{\Delta}{s+\Delta} \phi(0)+\frac{s}{s+\Delta} \phi(s+\Delta) \Longleftrightarrow \phi(s) \leq \phi(0)+s \frac{\phi(s+\Delta)-\phi(0)}{s+\Delta}
$$

对于每个$s, \Delta \geq 0$。让$\Delta$趋于无穷，这意味着$\phi(s) \leq \phi(0)+s \phi^{\infty}(1)$，对所有$s \geq 0$成立。任何$\mathbb{P}, \hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$之间的$\phi$-散度满足：

$$
\begin{aligned}
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) & =\int_{\mathcal{Z}} \frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z) \phi\left(\frac{\frac{\mathrm{d} \mathbb{P}}{\mathrm{~d} \rho}(z)}{\frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z)}\right) \mathrm{d} \rho(z) \\
& \leq \int_{\mathcal{Z}} \frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z) \phi(0) \mathrm{d} \rho(z)+\int_{\mathcal{Z}} \frac{\mathrm{d} \mathbb{P}}{\mathrm{~d} \rho}(z) \phi^{\infty}(1) \mathrm{d} \rho(z) \\
& =\phi(0)+\phi^{\infty}(1),
\end{aligned}
$$
第一等式成立是因为对于$\hat{\mathcal{B}}$中的几乎所有$z \in \mathcal{Z}$，$\frac{\mathrm{d} \mathrm{P}}{\mathrm{d} \rho}(z)=0$，而对于$\mathcal{B}$中的几乎所有$z \in \mathcal{Z}$，$\frac{\mathrm{d} \hat{\mathrm{P}}}{\mathrm{d} \rho}(z)=0$。第二等式源于视角函数的定义，并利用了$\rho$在$\hat{\mathcal{B}}$上的限制与$\hat{\mathbb{P}}$一致。第三等式最终成立是因为$\rho$在$\mathcal{B}$上的限制与$\mathbb{P}$一致。注意，即使$\phi(0)=\infty$或$\phi^{\infty}(1)=\infty$，上界也能被达到。

以下例子揭示了当$\phi^{\infty}(1)<\infty$且$\mathcal{Z}$去除$\hat{\mathbb{P}}$的原子后是无界时，$\phi$-散度模糊集无法弱紧。

**例子 3.14 ( $\phi$-散度模糊集)**。考虑一个熵函数$\phi$，其中$\phi^{\infty}(1)<\infty$。根据引理3.13，$\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$的上界为$\bar{r}=\phi(0)+\phi^{\infty}(1)$，对于所有$\mathbb{P}, \hat{\mathbb{P}} \in \mathcal{P}$。此外，设$\mathcal{P}$是以中心$\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$和半径$r \in(0, \bar{r})$定义的$\phi$-散度模糊集（见公式(2.10)）。假设对于每个$R>0$，存在$z_0 \in \mathcal{Z}$，使得$\left\|z_0\right\|_2 \geq R$且$\hat{\mathbb{P}}\left(Z=z_0\right)=0$。这个假设成立，例如当$\mathcal{Z}$是无界且凸的时，它意味着$\mathcal{P}$不是紧的。为此，固定一个任意的紧集$\mathcal{C} \subseteq \mathcal{Z}$，选择一个点$z_0 \in \mathcal{Z} \backslash \mathcal{C}$，并且满足$\mathbb{P}\left(Z=z_0\right)=0$。根据假设，存在这样的点。接下来，考虑由参数$\theta \in [0,1]$表示的分布$\mathbb{P}_\theta=(1-\theta) \hat{\mathbb{P}}+\theta \delta_{z_0}$。注意到$\hat{\mathbb{P}}$和$\delta_{z_0}$是互相奇异的，并且$f(\theta)=\mathrm{D}_\phi\left(\mathbb{P}_\theta, \hat{\mathbb{P}}\right)$是从$[0,1]$到$[0, \bar{r}]$的凸连续双射函数。现在设定$\varepsilon=\frac{1}{2} f^{-1}(r)$。当$\theta=f^{-1}(r)$时，分布$\mathbb{P}_\theta$满足$\mathrm{D}_\phi\left(\mathbb{P}_\theta, \hat{\mathbb{P}}\right)=f\left(f^{-1}(r)\right)=r$，因此属于$\mathcal{P}$。此外，$\mathbb{P}_\theta(Z \notin \mathcal{C}) \geq f^{-1}(r)>\varepsilon$，因为$z_0 \notin \mathcal{C}$。注意，$\varepsilon$与$\mathcal{C}$和$z_0$无关，只要$\hat{\mathbb{P}}\left(Z=z_0\right)=0$。由于紧集$\mathcal{C}$是任意选择的，这意味着$\mathcal{P}$不是紧的，因此$\mathcal{P}$也不是弱紧的。


## 3.3. 边际模糊集

作为研究最优传输模糊集的拓扑性质的准备，我们首先研究边际模糊集。以下命题表明，Fréchet模糊集规定了$Z$的所有$d$个分量的边际分布，因此它们总是弱紧的。

**命题 3.15 (Fréchet模糊集)**。对于任何累积分布函数$F_i, i \in[d]$，在(2.35)中定义的Fréchet模糊集$\mathcal{P}$是弱紧的。

**证明**。我们首先证明Fréchet模糊集是紧的。对于任何$\varepsilon>0$和$i \in[d]$，我们可以将$z_i$和$\bar{z}_i$分别设置为分布函数$F_i$的$\varepsilon /(2 d)$-分位数和$(1-\varepsilon /(2 d))$-分位数。设定$\mathcal{C}=\times_{i \in[d]}\left[\underline{z}_i, \bar{z}_i\right]$，得到

$$
\mathbb{P}(Z \notin \mathcal{C}) \leq \sum_{i \in[d]} \mathbb{P}\left(Z_i \notin\left[\underline{z}_i, \bar{z}_i\right]\right)=\sum_{i \in[d]} \varepsilon / d=\varepsilon,
$$

其中不等式是由并集界限得出的。因此，$\mathcal{P}$是紧的。接下来需要证明$\mathcal{P}$是弱闭的。注意，只有当$\mathbb{P}$下的$Z_i$的分布函数与$F_i$一致时，才有

$$
\mathbb{E}_{\mathbb{P}}\left[f\left(Z_i\right)\right]=\int_{-\infty}^{+\infty} f\left(z_i\right) \mathrm{d} F_i\left(z_i\right) .
$$

这是因为每个Borel分布在$\mathbb{R}$上都是Radon测度。对于任何固定的有界连续函数$f$和固定的索引$i \in[d]$，满足上述等式的所有$\mathbb{P} \in \mathcal{P}\left(\mathbb{R}^d\right)$的集合是弱闭的，根据命题3.3。因此，$\mathcal{P}$是弱闭的，因为闭集在交集中保持闭合。

从Fréchet模糊集到一般的边际模糊集的推广是显而易见的，如2.4.1节中讨论的，它们规定了多元边际分布。详细内容省略以简化表达。


## 3.4. 最优传输模糊集

回顾一下，$\Gamma(\mathbb{P}, \hat{\mathbb{P}})$表示所有连接概率分布$\mathbb{P}$和$\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$的运输计划的集合。因此，$\Gamma(\mathbb{P}, \hat{\mathbb{P}})$包含所有$Z$和$\mathcal{Z}$的联合分布，其边际分别为$\mathbb{P}$和$\hat{\mathbb{P}}$。集合$\Gamma(\mathbb{P}, \hat{\mathbb{P}})$出现在最优传输差异$\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})$的定义中；见定义2.15。第3.3节中的推理立即意味着$\Gamma(\mathbb{P}, \mathbb{P})$是弱紧的，因为它构成了一个边际模糊集。这一见解在命题3.15的以下简单推论中得到了形式化。为了简化，省略了证明。

**推论 3.16（运输计划）**。对于任何边际分布$\mathbb{P}, \hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$，所有的运输计划$\Gamma(\mathbb{P}, \hat{\mathbb{P}})$是弱紧的。

**推论 3.16**使我们能够证明（2.18）中的最优传输问题是可解的，因为运输成本函数被假定为下半连续。

**引理 3.17（最优传输问题的可解性）**。在（2.18）中的下确界是达到的。

**证明**。根据推论3.16，集合$\Gamma(\mathbb{P}, \hat{\mathbb{P}})$是弱紧的。此外，运输成本函数$c(z, \hat{z})$是下半连续且有下界。根据命题3.3，期望值$\mathbb{E}_\gamma[c(Z, \hat{Z})]$因此在$\gamma$中是弱下半连续的。因此，根据Weierstrass定理，（2.18）中的最优传输问题是可解的，并且它的下确界是达到的。

**引理 3.17**允许我们证明最优传输差异$\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})$是其输入$\mathbb{P}$和$\hat{\mathbb{P}}$的弱下半连续函数。

**引理 3.18（最优传输差异的弱下半连续性）**。最优传输差异$\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})$是$\mathbb{P}$和$\hat{\mathbb{P}}$的弱下半连续函数。

**证明**。假设$\mathbb{P}_j$和$\hat{\mathbb{P}}_j, j \in \mathbb{N}$，分别弱收敛于$\mathbb{P}$和$\hat{\mathbb{P}}$，并定义可数的模糊集$\mathcal{P}=\left\{\mathbb{P}_j\right\}_{j \in \mathbb{N}}$和$\hat{\mathcal{P}}=\left\{\hat{\mathbb{P}}_j\right\}_{j \in \mathbb{N}}$。根据顺序紧性的定义，$\mathcal{P}$和$\hat{\mathcal{P}}$的弱闭包是弱紧的。因此，Prokhorov定理（见定理3.5）意味着$\mathcal{P}$和$\hat{\mathcal{P}}$都是紧的。因此，对于任何$\varepsilon>0$，存在两个紧集$\mathcal{C}, \hat{\mathcal{C}} \subseteq \mathbb{R}^d$，使得

$$
\mathbb{P}_j(Z \notin \mathcal{C}) \leq \varepsilon / 2 \quad \text { 和 } \quad \hat{\mathbb{P}}_j(\hat{Z} \notin \hat{\mathcal{C}}) \leq \varepsilon / 2 \quad \forall j \in \mathbb{N} .
$$

每当$\gamma \in \Gamma\left(\mathbb{P}_j, \hat{\mathbb{P}}_j\right)$对于某个$j \in \mathbb{N}$时，我们有

$$
\gamma((Z, \hat{Z}) \notin \mathcal{C} \times \hat{\mathcal{C}}) \leq \mathbb{P}_j(Z \notin \mathcal{C})+\hat{\mathbb{P}}_j(Z \notin \hat{\mathcal{C}}) \leq \varepsilon
$$

由于$\mathcal{C} \times \hat{\mathcal{C}}$是紧集且$\varepsilon$是任意选择的，这表明联合

$$
\bigcup_{j \in \mathbb{N}} \Gamma\left(\mathbb{P}_j, \hat{\mathbb{P}}_j\right)
$$

是紧的，这又通过Prokhorov定理意味着它的闭包是弱紧的。现在，设$\gamma_j^{\star}$为$\mathbb{P}_j$和$\hat{\mathbb{P}}_j$的最优耦合，解决问题（2.18），并且由于引理3.17，它是存在的。因为所有这些最优耦合属于某个弱紧集（即（3.3）的弱闭包），我们可以假设不失一般性地，$\gamma_j^{\star}, j \in \mathbb{N}$，弱收敛于某个分布$\gamma$。否则，我们可以对某个子序列进行处理。显然，我们有$\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})$。对于$\gamma^{\star}$，它是$\mathbb{P}$和$\hat{\mathbb{P}}$的最优耦合，我们有

$$
\begin{aligned}
\liminf _{j \rightarrow \infty} \mathrm{OT}_c\left(\mathbb{P}_j, \hat{\mathbb{P}}_j\right) & =\liminf _{j \rightarrow \infty} \mathbb{E}_{\gamma_j^{\star}}[c(Z, \hat{Z})] \\
& \geq \mathbb{E}_\gamma[c(Z, Z)] \geq \mathbb{E}_{\gamma^{\star}}[c(Z, Z)]=\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})
\end{aligned}
$$

其中两个等式分别来自于$\gamma_j^{\star}$和$\gamma^{\star}$的定义。第一个不等式成立，因为$\mathbb{E}_\gamma[c(Z, \hat{Z})]$在$\gamma$中是弱下半连续的，感谢命题3.3，第二个不等式来自于$\gamma$在（2.18）中的次优性。因此，$\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})$是$\mathbb{P}$和$\hat{\mathbb{P}}$的弱下半连续函数。

**引理 3.18**的启发来源于（Clément和Desch 2008, 引理5.2）和（Yue, Kuhn和Wiesemann 2022, 定理1）。接下来，我们证明Wasserstein模糊集是弱紧的。在这整个讨论中，我们假设运输成本函数所依据的度量是由$\mathbb{R}^d$上的某个范数$\|\cdot\|$诱导的。这个假设简化了我们的推导，但可以放宽。回顾一下，$p$-Wasserstein距离$\mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}})$对于$p \geq 1$是$\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}})$的$p$-次根，其中运输成本函数设定为$c(z, \hat{z})=\|z-\hat{z}\|^p$；见定义2.18。

**定理 3.19（$p$-Wasserstein模糊集）**。假设度量$d (\cdot, \cdot)$在$\mathcal{Z}$上是由$\mathbb{R}^d$中的某个范数$\|\cdot\|$诱导的。如果$\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$具有有限的$p$-阶矩（即$\mathbb{E}_{\hat{p}}\left[\|Z\|^p\right]<\infty$），则在（2.28）中定义的$p$-Wasserstein模糊集$\mathcal{P}$是弱紧的。

**证明**。我们首先证明所有的分布$\mathbb{P} \in \mathcal{P}$都有统一有界的$p$-阶矩。为此，设$\hat{r}=\mathbb{E}_{\hat{\mathrm{p}}}\left[\|Z\|^p\right]<\infty$，并注意到任何$\mathbb{P} \in \mathcal{P}$满足

$$
\begin{aligned}
\left(\mathbb{E}_{\mathbb{P}}\left[\|Z\|^p\right]\right)^{\frac{1}{p}}=\mathrm{W}_p\left(\mathbb{P}, \delta_0\right) & \leq \mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}})+\mathrm{W}_p\left(\hat{\mathbb{P}}, \delta_0\right) \\
& =\mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}})+\left(\mathbb{E}_{\hat{\mathrm{P}}}\left[\|Z\|^p\right]\right)^{\frac{1}{p}} \leq r+\hat{r}
\end{aligned}
$$

这里，第一不等式成立是因为$p$-Wasserstein距离是一个度量，因此满足三角不等式，第二不等式成立是因为$\mathbb{P} \in \mathcal{P}$。因此，我们有$\mathbb{E}_{\mathbb{P}}\left[\|Z\|^p\right] \leq(r+\hat{r})^p$，对于每个$\mathbb{P} \in \mathcal{P}$。换句话说，Wasserstein球$\mathcal{P}$是示例3.10中讨论的$p$-阶矩模糊集的一个子集。这意味着$\mathcal{P}$是紧的。进一步注意到，$\mathcal{P}$被定义为函数$f(\mathbb{P})=\mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}})$的子水平集，该函数由于引理3.18而是弱下半连续的。因此，$\mathcal{P}$是弱闭的。

最后，我们证明$\infty$-Wasserstein模糊集总是弱紧的。

**推论 3.20（$\infty$-Wasserstein模糊集）**。假设度量$d(\cdot, \cdot)$在$\mathcal{Z}$上是由$\mathbb{R}^d$中的某个范数$\|\cdot\|$诱导的。那么，对于每个$\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$，在（2.34）中定义的$\infty$-Wasserstein模糊集是弱紧的。

**证明**。我们首先证明$\mathcal{P}$是紧的。为此，选择任意$\varepsilon>0$和任意紧集$\hat{\mathcal{C}} \subseteq \mathcal{Z}$，使得$\hat{\mathbb{P}}(Z \notin \hat{\mathcal{C}}) \leq \varepsilon$。注意，$\hat{\mathcal{C}}$是保证存在的，因为$\hat{\mathbb{P}}$是一个概率分布。接下来，定义$\mathcal{C}$为$\hat{\mathcal{C}}$的$r$-邻域，即设置

$$
\mathcal{C}=\{z \in \mathcal{Z}: \exists \hat{z} \in \hat{\mathcal{C}} \text { 使得 }\|z-\hat{z}\| \leq r\},
$$

参见（2.29）。可以验证$\mathcal{C}$继承了$\hat{\mathcal{C}}$的紧性。任何分布$\mathbb{P} \in \mathcal{P}$满足$\mathrm{W}_{\infty}(\mathbb{P}, \hat{P}) \leq r$。因此，我们得到

$$
\mathbb{P}(Z \notin \mathcal{C})=\mathbb{P}(Z \in \mathcal{Z} \backslash \mathcal{C}) \leq \hat{\mathbb{P}}(Z \in \mathcal{Z} \backslash \hat{\mathcal{C}})=\hat{\mathbb{P}}(Z \notin \hat{\mathcal{C}}) \leq \varepsilon,
$$

其中第一个不等式来自于推论2.28和观察到$\mathcal{Z} \backslash \mathcal{C}$的$r$-邻域与$\mathcal{Z} \backslash \hat{\mathcal{C}}$一致。第二个不等式来自于$\hat{\mathcal{C}}$的定义。由于$\varepsilon$是任意选择的，$\mathcal{P}$是紧的。接下来需要证明$\mathcal{P}$是弱闭的。Proposition 2.26立即表明，如果$\mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}}) \leq r$，则对于所有$p \geq 1$，$\mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}}) \leq r$。因此，我们可以得出结论

$$
\mathcal{P}=\bigcap_{p \geq 1}\left\{\mathbb{P} \in \mathcal{P}\left(\mathbb{R}^d\right): \mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\} .
$$

也就是说，$\infty$-Wasserstein模糊集可以表示为所有$p$-Wasserstein模糊集（对于$p \geq 1$）的交集，所有这些模糊集都是弱闭的，见定理3.19。因此，$\mathcal{P}$确实是弱闭的，命题成立。


# 4. 最坏情况期望问题的对偶理论

DRO问题（1.2）通常被解释为决策者和一个虚拟对手之间的零和博弈。决策者先行动，因此在看到$\mathbb{P}$之前选择了$x$。因此，$x$是针对所有分布$\mathbb{P} \in \mathcal{P}$进行优化的。相比之下，对手后行动，因此在看到$x$之后选择$\mathbb{P}$。因此，$\mathbb{P}$仅针对一个特定的决策$x \in \mathcal{X}$进行优化。换句话说，对手的选择可以适应决策者的选择，但反之则不行。

在本节中，我们为对手的子问题开发了对偶理论，该子问题旨在最大化固定决策$x$在凸不确定性集$\mathcal{P}$中所有分布上的期望损失。为了避免冗长，我们在整个讨论中省略损失函数$\ell$对固定决策$x$的依赖，即我们写作$\ell(z)$而不是$\ell(x, z)$。因此，我们处理如下形式的最坏情况期望问题：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] .
$$

注意，$\mathcal{P}$表示所有有限带符号Borel测度的线性空间中一个凸子集。除非$\mathcal{Z}$是有限的，否则（4.1）构成了一个无限维的凸规划，目标函数是线性的。为了使该问题明确，我们假设$ \ell: \mathcal{Z} \rightarrow \overline{\mathbb{R}} $ 是一个Borel函数。

根据（Rockafellar和Wets 2009, 第14.E节）的定义，如果 $\mathbb{E}_{\mathcal{P}}[\max \{\ell(Z), 0\}] = \infty$ 且 $\mathbb{E}_{\mathcal{P}}[\min \{\ell(Z), 0\}] = -\infty$， 则我们定义$\mathbb{E}_{\mathcal{P}}[\ell(Z)] = -\infty$。这意味着不可行性优先于无界性。更一般地，本文其余部分假设，如果最小化（最大化）问题的目标函数可以表示为两个项的差，并且这两个项的值均为$\infty$，则目标函数的值应解释为$\infty(-\infty)$。这一惯例与（Rockafellar和Wets 2009）中使用的扩展算术规则一致。

在接下来的部分中，我们将展示（4.1）如何通过使用有限维凸分析的基本工具（Fenchel 1953, Rockafellar 1970）对广泛的有限参数化不确定性集进行对偶化，包括所有矩不确定性集（第4.2节）、$\phi$-散度不确定性集（第4.3节）和最优传输不确定性集（第4.4节）。我们广泛采用Shapiro（2001）和Zhang等人（2024b）在矩和最优传输不确定性集上的证明策略，并将其扩展到$\phi$-散度不确定性集。

## 4.1. 一般证明策略

为了概述对偶化（4.1）的高层次思路，我们回顾一下关于参数化最小值的凸性的一条基本结果；参见例如（Rockafellar 1974，第1定理）。

**引理 4.1** （最优值函数的凸性）  
如果$\mathcal{U}$和$\mathcal{V}$是任意的实向量空间，且$H: \mathcal{U} \times \mathcal{V} \rightarrow \overline{\mathbb{R}}$是一个凸函数，那么通过$h(u)=\inf _{v \in \mathcal{V}} H(u, v)$定义的最优值函数$h: \mathcal{U} \rightarrow \overline{\mathbb{R}}$是凸的。

证明：注意，$h$是凸函数当且仅当它的图形epi $(h)$是一个凸集。根据图形和下确界运算的定义，我们有：

$$
\begin{aligned}
\operatorname{epi}(h) & =\{(u, t) \in \mathcal{U} \times \mathbb{R}: h(u) \leq t\} \\
& =\{(u, t) \in \mathcal{U} \times \mathbb{R}: \exists v \in \mathcal{V} \text { with } H(u, v) \leq t+\varepsilon \forall \varepsilon>0\} \\
& =\bigcap_{\varepsilon>0}\{(u, t) \in \mathcal{U} \times \mathbb{R}: \exists v \in \mathcal{V} \text { with } H(u, v)-\varepsilon \leq t\}
\end{aligned}
$$

因此，epi$(h)$可以通过将$\cap_{\varepsilon>0}$epi$(H-\varepsilon)$投影到$\mathcal{U} \times \mathbb{R}$来获得。根据（Rockafellar 1970，第2.1定理和5.7定理），epi$(H-\varepsilon)$是凸的，因此，通过交集和线性变换保持凸性，结论得以证明。

以下结果是凸分析的一个基石。它指出，封闭凸函数$h$的双共轭$h^{**}$（即$h^*$的共轭）与$h$本身相同。这里，我们采用标准惯例，认为$h$是封闭的，如果它是下半连续的，并且$h(u)>-\infty$对于所有$u \in \mathcal{U}$，或者$h(u)=-\infty$对于所有$u \in \mathcal{U}$。我们用$\mathrm{cl}(h)$表示$h$的闭包，即$h$下面的最大封闭函数。

**引理 4.2** （Fenchel-Moreau定理）  
对于任意凸函数$h: \mathbb{R}^d \rightarrow \mathbb{\mathbb { R }}$，我们有$h \geq h^{**}$。该不等式在$\operatorname{rint}(\operatorname{dom}(h))$上成立为等式。

证明：根据（Rockafellar 1970，第12.2定理），我们有$h^{**}=\mathrm{cl}(h) \leq h$。此外，（Rockafellar 1970，第10.1定理）确保凸函数$h$在$\operatorname{rint}(\operatorname{dom}(h))$上是连续的，因此在此处与$\mathrm{cl}(h)$一致。因此，结论得以证明。

对偶化最坏情况期望问题（4.1）的主要思路是将其最优值表示为$-h(u)$，其中$h(u)=\inf _{v \in \mathcal{V}} H(u, v)$，$\mathcal{U}$是一个有限维的参数空间$u$，它编码了不确定性集$\mathcal{P}$（例如规定的矩集合或大小参数集），而$\mathcal{V}$是一个无限维的有限签名测度空间。除此之外，$H(u, v)$表示如果签名测度$v$恰好是$\mathcal{P} \subseteq \mathcal{V}$中的概率测度，则为负期望损失，否则为$\infty$。如果$H(u, v)$在$u$和$v$上是联合凸的，那么根据引理4.1，$h(u)$是凸的。然后，可以通过双共轭$h^{**}(u)$构造出与（4.1）对偶的问题。引理4.2提供了强对偶性的条件。


## 4.2. 矩不确定性集

回顾第2.1节，通用的矩不确定性集（2.1）定义为

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}_f(\mathcal{Z}): \mathbb{E}_{\mathrm{P}}[f(Z)] \in \mathcal{F}\right\},
$$

其中$\mathcal{Z} \subseteq \mathbb{R}^d$是一个封闭的支持集，$f: \mathcal{Z} \rightarrow \mathbb{R}^m$是一个Borel可测的矩函数，$\mathcal{F} \subseteq \mathbb{R}^m$是一个封闭的矩不确定性集，$\mathcal{P}_f(\mathcal{Z})$表示所有满足$\mathbb{E}_{\mathrm{P}}[f(Z)]$有限的分布$\mathbb{P} \in \mathcal{P}(\mathcal{Z})$的族。${ }^1$ 我们可以假设${ }^1$显然，$\mathbb{B}_{\mathrm{P}}[f(\mathcal{Z})]$必须是有限的，才能属于封闭集$\mathcal{F}$。因此，我们可以将$\mathcal{P}(\mathcal{Z})$替换为$\mathcal{P}_f(\mathcal{Z})$，而不失一般性。不过，当我们对最坏情况期望问题（4.1）进行对偶化时，使用$\mathcal{P}_f(\mathcal{Z})$会更为方便。我们假设$\mathcal{F}$被凸集覆盖：

$$
\mathcal{C}=\left\{\mathbb{B}_{\mathrm{P}}[f(Z)]: \mathbb{P} \in \mathcal{P}_f(\mathcal{Z})\right\}
$$

表示$\mathcal{Z}$上任何分布的所有可能的矩。为了排除特殊的边界情况，我们做出普遍假设，即$\mathcal{Z}$和$\mathcal{F}$都非空。

显然，最坏情况期望问题（4.1）在矩不确定性集（2.1）上可以重写为

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathrm{P}}[\ell(Z)]=\sup _{u \in \mathcal{F}} \sup _{P \in \mathcal{P}_f(\mathcal{Z})}\left\{\mathbb{E}_{\mathrm{P}}[\ell(Z)]: \mathbb{B}_{\mathrm{P}}[f(Z)]=u\right\}=\sup _{u \in \mathcal{F}}-h(1, u),
$$

其中，辅助函数$h: \mathbb{R} \times \mathbb{R}^m \rightarrow \mathbb{R}$通过以下方式定义：

$$
h\left(u_0, u\right)=\inf _{v \in \mathcal{M}_{f,+}^{+}(\mathcal{Z})}\left\{-\int_{\mathcal{Z}} \ell(z) \mathrm{d} v(z): \int_{\mathcal{Z}} \mathrm{d} v(z)=u_0, \int_{\mathcal{Z}} f(z) \mathrm{d} v(z)=u\right\} .
$$

这里，$\mathcal{M}_{f,+}(\mathcal{Z})$表示所有满足积分$\int_{\mathcal{Z}} f(z) \mathrm{d} v(z)$有限的Borel测度$v \in \mathcal{M}_{+}(\mathcal{Z})$的族。换句话说，$\mathcal{M}_{f,+}(\mathcal{Z})$表示由$\mathcal{P}_f(\mathcal{Z})$生成的凸锥。由于（4.3）中的最小化问题的目标函数和约束函数在$v, u_0$和$u$上是联合凸的，根据引理4.1，$h$是凸的。在合理的正则性条件下，可以进一步证明$h$的定义域与$\{1\} \times \mathcal{C}$生成的锥相一致。

**引理 4.3**（$h$的定义域）  
如果对于每个$\mathbb{P} \in \mathcal{P}_f(\mathcal{Z})$都有$\mathbb{B}_{\mathrm{P}}[\ell(Z)] > -\infty$，那么我们有：

$$
\operatorname{dom}(h)=\operatorname{cone}(\{1\} \times \mathcal{C}) .
$$

证明：显然，当且仅当$h\left(u_0, u\right) < \infty$时，$\left(u_0, u\right) \in \operatorname{dom}(h)$，也就是说，当且仅当（4.3）中的最小化问题是可行的。于是，剩下需要证明的是（4.3）中的问题在$\left(u_0, u\right)$处是可行的，当且仅当$\left(u_0, u\right) \in \operatorname{cone}(\{1\} \times \mathcal{C})$。首先假设在$\left(u_0, u\right)$处问题（4.3）是可行的。这意味着存在$v \in \mathcal{M}_{f,+}(\mathcal{Z})$，使得$\int_{\mathcal{Z}} \mathrm{d} v(z) = u_0$且$\int_{\mathcal{Z}} f(z) \mathrm{d} v(z) = u$。因此，$u_0 \geq 0$。如果$u_0 = 0$，则必须有$u = 0$。如果$u_0 > 0$，则$v / u_0$必须是$\mathcal{P}_f(\mathcal{Z})$中的概率测度，这意味着$u / u_0 \in \mathcal{C}$。在这两种情况下，$\left(u_0, u\right)$是$\{1\} \times \mathcal{C}$中的某个点的非负倍数，因此属于$\operatorname{cone}(\{1\} \times \mathcal{C})$。接下来，假设$\left(u_0, u\right) \in \operatorname{cone}(\{1\} \times \mathcal{C})$。如果$u_0 = 0$，则$u = 0$，确实，零测度在（4.3）中是可行的。如果$u_0 > 0$，则$u / u_0 \in \mathcal{C}$。根据$\mathcal{C}$的定义，存在$\mathbb{P} \in \mathcal{P}_f(\mathcal{Z})$，使得$\mathbb{B}_{\mathrm{P}}[f(Z)] = u / u_0$。由于$\mathbb{B}_{\mathrm{P}}[\ell(Z)] > -\infty$，这意味着$v = u_0 \mathbb{P}$在（4.3）中是可行的。由此，我们证明了（4.3）当且仅当$\left(u_0, u\right) \in \operatorname{cone}(\{1\} \times \mathcal{C})$时是可行的。这个观察完成了证明。

以下命题描述了$h$的双共轭。

**命题 4.4**（$h$的双共轭）  
在（4.3）中定义的$h$的双共轭满足：

$$
h^{* *}\left(u_0, u\right)=\sup _{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}^m}\left\{-u_0 \lambda_0-u^{\top} \lambda: \lambda_0+f(z)^{\top} \lambda \geq \ell(z) \forall z \in \mathcal{Z}\right\} .
$$

如果对于所有$\mathbb{P} \in \mathcal{P}_f(\mathcal{Z})$都有$\mathbb{B}_{\mathrm{P}}[\ell(Z)] > -\infty$，并且$\mathcal{F}$是一个凸紧集，且对于所有$(\mu, M) \in \operatorname{rint}(\mathcal{F})$都有$M > \mu \mu^{\top}$，那么$h^{* *}$和$h$在由$\{1\} \times \operatorname{rint}(\mathcal{C})$生成的锥上是相等的，除原点外。

**证明**：对于任意固定的$\left(\lambda_0, \lambda\right) \in \mathbb{R} \times \mathbb{R}^m$，$h$的凸共轭满足：

$$
\begin{aligned}
& h^*\left(-\lambda_0,-\lambda\right)=\sup _{u_0 \in \mathbb{R}, u \in \mathbb{R}^m}-u_0 \lambda_0-u^{\top} \lambda-h\left(u_0, u\right) \\
& = \begin{cases}\sup & -u_0 \lambda_0-u^{\top} \lambda+\int_{\mathcal{Z}} \ell(z) \mathrm{d} v(z) \\
\text { s.t. } & u_0 \in \mathbb{R}, u \in \mathbb{R}^m, v \in \mathcal{M}_{f,+}(\mathcal{Z}) \\
& \int_{\mathcal{Z}} \mathrm{d} v(z)=u_0, \int_{\mathcal{Z}} f(z) \mathrm{d} v(z)=u\end{cases} \\
& =\sup _{v \in \mathcal{M}_{f,+}(\mathcal{Z})} \int_{\mathcal{Z}}\left(\ell(z)-\lambda_0-f(z)^{\top} \lambda\right) \mathrm{d} v(z) \\
& = \begin{cases}0 & \text { if } \ell(z)-\lambda_0-f(z)^{\top} \lambda \leq 0 \quad \forall z \in \mathcal{Z}, \\
\infty & \text { otherwise, }\end{cases}
\end{aligned}
$$

这里，最后的等式成立是因为$\mathcal{M}_{f,+}(\mathcal{Z})$包含$\mathcal{Z}$上的所有加权Dirac测度。因此，对于任意固定的$\left(u_0, u\right) \in \mathbb{R} \times \mathbb{R}^m$，$h^*$的共轭满足：

$$
\begin{aligned}
h^{* *}\left(u_0, u\right) & =\sup _{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}^m}-u_0 \lambda_0-u^{\top} \lambda-h^*\left(-\lambda_0,-\lambda\right) \\
& =\sup _{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}^m}\left\{-u_0 \lambda_0-u^{\top} \lambda: \lambda_0+f(z)^{\top} \lambda \geq \ell(z) \forall z \in \mathcal{Z}\right\} .
\end{aligned}
$$

这建立了$h$的双共轭的期望公式。接下来假设，对于所有$\mathbb{P} \in \mathcal{P}_f(\mathcal{Z})$都有$\mathbb{B}_{\mathrm{P}}[\ell(Z)] > -\infty$。接着需要证明在由$\{1\} \times \operatorname{rint}(\mathcal{C})$生成的锥内，对于所有$\left(u_0, u\right) \neq (0,0)$，有$h\left(u_0, u\right) = h^{* *}\left(u_0, u\right)$。然而，这直接来自于引理4.2，以及观察到

$$
\operatorname{rint}(\operatorname{dom}(h)) = \operatorname{rint}(\operatorname{cone}(\{1\} \times \mathcal{C})) = \operatorname{cone}(\{1\} \times \operatorname{rint}(\mathcal{C})) \backslash\{(0,0)\}
$$

其中，两个等式由于引理4.3和（Rockafellar 1970, Corollary 6.8.1）成立。因此，结论得出。

命题4.4表明，对于所有$u \in \operatorname{rint}(\mathcal{C})$，$h(1, u) = h^{* *}(1, u)$。接下来的主要定理利用这一关系将（4.2）右侧的最大化问题转换为等效的对偶最小化问题。

**定理 4.5**（矩不确定性集的对偶理论）  
如果$\mathcal{P}$是矩不确定性集（2.1），则以下弱对偶关系成立：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{B}_{\mathrm{P}}[\ell(Z)] \leq \begin{cases}\text { inf } & \lambda_0 + \delta_{\mathcal{F}}^*(\lambda) \\
\text { s.t. } & \lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}^m \\
& \lambda_0 + f(z)^{\top} \lambda \geq \ell(z) \quad \forall z \in \mathcal{Z}\end{cases}
$$

如果对于所有$\mathbb{P} \in \mathcal{P}_f(\mathcal{Z})$都有$\mathbb{B}_{\mathrm{P}}[\ell(Z)] > -\infty$且$\mathcal{F}$是一个凸紧集，并且$\operatorname{rint}(\mathcal{F}) \subseteq \operatorname{rint}(\mathcal{C})$，那么强对偶性成立，即（4.4）变为等式。
**证明**：为了便于说明，我们引入

$$
\mathcal{L} = \left\{ \left( \lambda_0, \lambda \right) \in \mathbb{R} \times \mathbb{R}^m : \lambda_0 + f(z)^{\top} \lambda \geq \ell(z) \quad \forall z \in \mathcal{Z} \right\}
$$

作为对偶可行集的简写。利用（4.2）中的分解，我们得到

$$
\begin{aligned}
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] = \sup _{u \in \mathcal{F}} -h(1, u) & \leq \sup _{u \in \mathcal{F}, \left( \lambda_0, \lambda \right) \in \mathcal{L}} \inf _{0} + u^{\top} \lambda \\
& \leq \inf _{\left( \lambda_0, \lambda \right) \in \mathcal{L}} \sup _{u \in \mathcal{F}} \lambda_0 + u^{\top} \lambda \\
& = \inf _{\left( \lambda_0, \lambda \right) \in \mathcal{L}} \lambda_0 + \delta_{\mathcal{F}}^*(\lambda) .
\end{aligned}
$$

其中，第一个不等式利用了命题4.4和引理4.2，确保$h \geq h^{**}$，第二个不等式利用了最大-最小不等式，最后的等式则来自支持函数$\delta_{\mathcal{F}}^*$的定义。这建立了弱对偶关系（4.4）。接下来，假设$\mathcal{F}$是一个凸紧集，且$\operatorname{rint}(\mathcal{F}) \subseteq \operatorname{rint}(\mathcal{C})$。在这个额外假设下，我们得到

$$
\begin{aligned}
& \sup _{\mathbb{P} \in \mathcal{P}} \mathbb{B}_{\mathrm{P}}[\ell(Z)] = \sup _{u \in \mathcal{F}} -h(1, u) = \sup _{u \in \operatorname{rint}(\mathcal{F})} -h(1, u) \\
& = \sup _{u \in \operatorname{rint}(\mathcal{F})} \inf _{0, \lambda} \left( u^{\top} \lambda \right) \quad \left( \lambda_0, \lambda \right) \in \mathcal{L} \\
& = \sup _{u \in \mathcal{F}} \inf _{0} + u^{\top} \lambda \quad \left( \lambda_0, \lambda \right) \in \mathcal{L} \\
& = \inf _{\left( \lambda_0, \lambda \right) \in \mathcal{L}} \lambda_0 + \delta_{\mathcal{F}}^*(\lambda) .
\end{aligned}
$$

这里，第一个等式利用了（4.2）。第二个等式来自两个观察：首先，$\operatorname{rint}(\mathcal{F})$是非空的并且凸的（Rockafellar 1970，第6.2定理）；其次，$-h(1, u)$在$u$上是凹的，这确保了$-h(1, u)$在其定义域$\mathcal{C}$的边界上不能跳跃，特别是在$\mathcal{F} \subseteq \mathcal{C}$的边界上。综合这两个观察，我们可以将$\mathcal{F}$限制在$\operatorname{rint}(\mathcal{F})$上，而不降低上确界。第三个等式来自于命题4.4，这允许我们在$\operatorname{rint}(\mathcal{F}) \subseteq \operatorname{rint}(\mathcal{C})$上替换$h$为$h^{**}$。第四个等式成立，因为$-h^{**}(1, u)$在$u$上是凹的，这使得我们可以将$\operatorname{rint}(\mathcal{F})$替换回$\mathcal{F}$。最后，第五个等式来自Sion的最小最大定理（Sion 1958，第4.2定理），适用于因为$\mathcal{F}$是凸紧的，$\mathcal{L}$是凸的，且$\lambda_0 + u^{\top} \lambda$在$u$和$\left( \lambda_0, \lambda \right)$上是双线性的情况。因此，强对偶性成立。

定理4.5表明，最坏情况期望问题（4.1）在矩不确定性集（2.1）上的对偶问题是一个半无限的对偶问题。实际上，右侧的对偶问题有有限个决策变量，但有无限个约束，这些约束由不确定性实现$z \in \mathcal{Z}$来参数化。这个对偶问题也可以被解释为一个具有不确定性集$\mathcal{Z}$的鲁棒优化问题。值得注意的是，我们并没有假设$\mathcal{Z}$是凸的。此外，我们强调，$\mathcal{F}$的紧性不是强对偶性的必要条件。实际上，在Slater类型条件下也可以建立强对偶性（Zhen、Kuhn和Wiesemann 2023）。最后，条件$\operatorname{rint}(\mathcal{F}) \subseteq \operatorname{rint}(\mathcal{C})$等价于似乎较弱的要求，即$\mathcal{F}$与$\operatorname{rint}(\mathcal{C})$相交。确实，如果$\mathcal{F} \cap \operatorname{rint}(\mathcal{C}) \neq \emptyset$，则$\mathcal{F}$并不完全包含在$\mathcal{C}$的相对边界内，这意味着通过（Rockafellar 1970，第6.5.2号推论）我们可以得出$\operatorname{rint}(\mathcal{F}) \subseteq \operatorname{rint}(\mathcal{C})$。

在本节的剩余部分，我们将使用定理4.5对与流行的矩不确定性集类相关的最坏情况期望问题进行对偶化。回顾第2.1.4节，切比雪夫不确定性集（2.4）定义为

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}_2(\mathcal{Z}): \mathbb{B}_{\mathrm{P}}[Z]=\mu, \mathbb{B}_{\mathrm{P}}\left[Z Z^{\top}\right]=M \quad \forall(\mu, M) \in \mathcal{F}\right\},
$$

其中$\mathcal{F} \subseteq \mathbb{R}^d \times S_{+}^d$是一个封闭的矩不确定性集，$\mathcal{P}_2(\mathcal{Z})$表示所有具有有限二阶矩的分布的集合。注意，$\mathcal{P}$是通用矩不确定性集（2.1）的一个实例，其中矩函数$f(z) = (z, zz^{\top})$。

**定理 4.6**（切比雪夫不确定性集的对偶理论）  
如果$\mathcal{P}$是切比雪夫不确定性集（2.4），则以下弱对偶关系成立：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{B}_{\mathrm{P}}[\ell(Z)] \leq \begin{cases}\text { inf } & \lambda_0 + \delta_{\mathcal{F}}^*(\lambda, \Lambda) \\
\text { s.t. } & \lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}^d, \Lambda \in \mathbb{S}^d \\
& \lambda_0 + \lambda^{\top} z + z^{\top} \Lambda z \geq \ell(z) \quad \forall z \in \mathcal{Z} .\end{cases}
$$

如果对于所有$\mathbb{P} \in \mathcal{P}_2(\mathcal{Z})$都有$\mathbb{E}_{\mathbb{P}}[\ell(Z)] > -\infty$，并且$\mathcal{F}$是一个凸紧集，且对于所有$(\mu, M) \in \operatorname{rint}(\mathcal{F})$都有$M > \mu \mu^{\top}$，那么强对偶性成立，即（4.5）变为等式。

定理4.6是定理4.5的直接推论，因此我们省略其证明。回顾第2.1.4节，含有不确定矩的切比雪夫不确定性集（2.4）封装了支持集不确定性集$\mathcal{P}(\mathcal{Z})$、马尔可夫不确定性集（2.2）和具有固定矩的切比雪夫不确定性集（2.3）的特例。通过设置$\mathcal{F} = \mathbb{R}^d \times \mathbb{S}^d, \mathcal{F} = \{\mu\} \times \mathbb{S}^d$和$\mathcal{F} = \{\mu\} \times \{M\}$，可以分别恢复这些特例。以下引理描述了这些矩不确定性集的支持函数的封闭形式。由于证明是基本的，因此我们省略它。

**引理 4.7**（基本集合的支持函数）  
以下关系成立：
(i) 如果$\mathcal{F} = \mathbb{R}^d \times \mathbb{S}^d$，则$\delta_{\mathcal{F}}^*(\lambda, \Lambda) = \delta_{\{(0,0)\}}(\lambda, \Lambda)$。
(ii) 如果$\mathcal{F} = \{\mu\} \times \mathbb{S}^d$，则$\delta_{\mathcal{F}}^*(\lambda, \Lambda) = \lambda^{\top} \mu + \delta_{\{0\}}(\Lambda)$。
(iii) 如果$\mathcal{F} = \{\mu\} \times \{M\}$，则$\delta_{\mathcal{F}}^*(\lambda, \Lambda) = \lambda^{\top} \mu + \operatorname{Tr}(\Lambda M)$。

当与定理4.5结合使用时，引理4.7立即导致支持集不确定性集、马尔可夫不确定性集和切比雪夫不确定性集的对偶理论。为了简洁起见，我们省略了细节。在第2.1.4节中，我们还定义了Gelbrich不确定性集，它是一个切比雪夫不确定性集，其不确定矩形式为（2.4），其中$\mathcal{F}$表示Gelbrich不确定性集（2.8），定义为

$$
\mathcal{F} = \left\{ (\mu, M) \in \mathbb{R}^d \times \mathbb{S}_{+}^d : \begin{array}{l}
\exists \Sigma \in \mathbb{S}_{+}^d \text{ 使得 } M = \Sigma + \mu \mu^{\top}, \\
\mathrm{G}((\mu, \Sigma), (\hat{\mu}, \hat{\Sigma})) \leq r
\end{array} \right\},
$$

其中，$G$是定义2.1中的Gelbrich距离。接下来，我们推导Gelbrich不确定性集$\mathcal{F}$的支持函数$\delta_{\mathcal{F}}^*$。

**引理 4.8**（Gelbrich不确定性集的支持函数）  
设$\mathcal{F}$是一个Gelbrich不确定性集（2.8），其半径为$r \geq 0$，围绕$(\hat{\mu}, \hat{\Sigma}) \in \mathbb{R}^d \times \mathbb{S}_{+}^d$，其中$G$是定义2.1中的Gelbrich距离。对于任意$(\lambda, \Lambda) \in \mathbb{R}^d \times \mathbb{S}^d$，我们有

$$
\delta_{\mathcal{F}}^*(\lambda, \Lambda) = \begin{cases}
\text{inf} & \gamma\left(r^2 - \|\hat{\mu}\|^2 - \operatorname{Tr}(\hat{\Sigma})\right) + \operatorname{Tr}(A) + \alpha \\
\text{s.t.} & \alpha, \gamma \in \mathbb{R}_+, A \in \mathbb{S}_+^d \\
& \left[\begin{array}{cc}
\gamma I_d - \Lambda & \gamma \hat{\Sigma}^{\frac{1}{2}} \\
\gamma \hat{\Sigma}^{\frac{1}{2}} & A
\end{array}\right] \geq 0, \left[\begin{array}{cc}
\gamma I_d - \Lambda & \gamma \hat{\mu} + \frac{\alpha}{2} \\
\left(\gamma \hat{\mu} + \frac{\lambda}{2}\right)^{\top} & \alpha
\end{array}\right] \geq 0
\end{cases}
$$

**证明**：根据命题2.3，Gelbrich不确定性集$\mathcal{F}$的半正定表示，支持函数$\delta_{\mathcal{F}}^*(\lambda, \Lambda)$满足：

$$
\delta_{\mathcal{F}}^*(\lambda, \Lambda) = \begin{cases}
\text{sup} & \mu^{\top} \lambda + \operatorname{Tr}(M \Lambda) \\
\text{s.t.} & \mu \in \mathbb{R}^d, M, U \in \mathbb{S}_+^d, C \in \mathbb{R}^{d \times d} \\
& \operatorname{Tr}\left(M - 2 \mu \hat{\mu}^{\top} - 2 C\right) \leq r^2 - \|\hat{\mu}\|^2 - \operatorname{Tr}(\hat{\Sigma}) \\
& \left[\begin{array}{cc}
M - U & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0, \left[\begin{array}{cc}
U & \mu \\
\mu^{\top} & 1
\end{array}\right] \geq 0
\end{cases}
$$

根据锥对偶性（Ben-Tal 和 Nemirovski 2001，第1.4.2定理），上式中的最大化问题对应于对偶最小化问题：

$$
\begin{array}{ll}
\text{inf} & \gamma\left(r^2 - \|\hat{\mu}\|^2 - \operatorname{Tr}(\hat{\Sigma})\right) + \operatorname{Tr}(\hat{\Sigma} A_{22}) + \alpha \\
\text{s.t.} & \alpha, \gamma \in \mathbb{R}_+, A_{11}, A_{22}, B \in \mathbb{S}_+^d \\
& \left[\begin{array}{cc}
A_{11} & \gamma I_d \\
\gamma I_d & A_{22}
\end{array}\right] \geq 0, \left[\begin{array}{cc}
B & \gamma \hat{\mu} + \frac{\lambda}{2} \\
\left(\gamma \hat{\mu} + \frac{d}{2}\right)^{\top} & \alpha
\end{array}\right] \geq 0, \gamma I_d - \Lambda \geq A_{11} \geq B
\end{array}
$$

通过替换$A \leftarrow \hat{\Sigma}^{\frac{1}{2}} A_{22} \hat{\Sigma}^{\frac{1}{2}}$和等价关系

$$
\left[\begin{array}{cc}
\gamma I_d - \Lambda & \gamma I_d \\
\gamma I_d & A_{22}
\end{array}\right] \geq 0 \Longleftrightarrow \left[\begin{array}{cc}
I_d & 0 \\
0 & \hat{\Sigma}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{cc}
\gamma I_d - \Lambda & \gamma I_d \\
\gamma I_d & A_{22}
\end{array}\right]\left[\begin{array}{cc}
I_d & 0 \\
0 & \hat{\Sigma}^{\frac{1}{2}}
\end{array}\right] \geq 0
$$

最终得到所需的半正定规划。因此，这个半正定规划的最优值确实等于$\delta_{\mathcal{F}}^*(\lambda, \Lambda)$。

结合定理4.6和引理4.8，我们现在可以准备对Gelbrich不确定性集上的最坏情况期望问题进行对偶化。

**定理 4.9**（Gelbrich不确定性集的对偶理论）  
如果$\mathcal{P}$是切比雪夫不确定性集（2.4），其中$\mathcal{F}$表示Gelbrich不确定性集（2.8），则以下弱对偶关系成立：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{B}_{\mathrm{P}}[\ell(Z)] \leq \left\{
\begin{array}{l}
\text{inf} \quad \lambda_0 + \gamma\left(r^2 - \|\hat{\mu}\|^2 - \operatorname{Tr}(\hat{\Sigma})\right) + \operatorname{Tr}(A) + \alpha \\
\text{s.t.} \quad \lambda_0 \in \mathbb{R}, \alpha, \gamma \in \mathbb{R}_+, \lambda \in \mathbb{R}^d, \Lambda \in \mathbb{S}^d, A \in \mathbb{S}_+^d \\
\lambda_0 + \lambda^{\top} z + z^{\top} \Lambda z \geq \ell(z) \quad \forall z \in \mathcal{Z} \\
\left[
\begin{array}{cc}
\gamma I_d - \Lambda & \gamma \hat{\Sigma}^{\frac{1}{2}} \\
\gamma \hat{\Sigma}^{\frac{1}{2}} & A
\end{array}
\right] \geq 0, \left[
\begin{array}{cc}
\gamma I_d - \Lambda & \gamma \hat{\mu} + \frac{\lambda}{2} \\
\left(\gamma \hat{\mu} + \frac{\lambda}{2}\right)^{\top} & \alpha
\end{array}
\right] \geq 0
\end{array}
\right.
$$

如果对于所有$\mathbb{P} \in \mathcal{P}_2(\mathcal{Z})$都有$\mathbb{B}_{\mathbb{P}}[\ell(Z)] > -\infty$，并且$r > 0$，则强对偶性成立，即不等式（4.6）变为等式。

**证明**：弱对偶性直接来自于定理4.6的第一部分和引理4.8。为了证明强对偶性，回顾命题2.3，我们知道Gelbrich不确定性集$\mathcal{F}$是一个凸紧集。此外，回顾命题2.2的证明，Gelbrich距离是连续的。由于$r > 0$，这意味着

$$
\operatorname{rint}(\mathcal{F}) = \left\{ (\mu, M) \in \mathbb{R}^d \times \mathbb{S}_{+}^d : M > \mu \mu^{\top}, G\left( \left( \mu, M - \mu \mu^{\top} \right), (\hat{\mu}, \hat{\Sigma}) \right) < r \right\},
$$

这进一步确保对于所有$(\mu, M) \in \operatorname{rint}(\mathcal{F})$都有$M > \mu \mu^{\top}$。因此，强对偶性成立，来自定理4.6的第二部分。


我们以一些历史性备注结束本节。经典的矩问题问的是，是否存在一个具有给定矩序列的分布在$\mathcal{Z}$上。在本综述的语言中，矩问题即是确定一个形式为（2.1）的矩不确定性集是否非空，其中$f$是一个多项式，$\mathcal{F}$是一个单点集。矩问题的分析在数学中有着悠久且杰出的历史，可以追溯到19世纪。著名的贡献者包括切比雪夫（1874年）、马尔可夫（1884年）、斯蒂尔捷斯（1894年）、汉堡（1920年）和豪斯多夫（1923年）；有关早期综述，参见（Shohat和Tamarkin 1950）。使用数学优化工具——特别是半无限对偶理论——对矩问题的研究由Isii（1960年，1962年）开创。Shapiro（2001年）将具有给定矩的分布族的最坏情况期望问题表述为一个无限维的圆锥线性规划，并建立了强对偶性的条件。


## 4.3. $\phi$-散度不确定性集合

回忆第 2.2 节中定义的 $\phi$-散度不确定性集合（公式 2.10）如下：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\}
$$

这里，$\mathcal{Z}$ 是一个闭的支持集，$r \geq 0$ 是集合的大小参数，$\phi$ 是定义 2.4 中的熵函数，$\mathrm{D}_\phi$ 是定义 2.5 中的相应 $\phi$-散度，$\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 是一个参考分布。为方便起见，我们将 $\mathrm{D}_\phi$ 扩展到任意测度。通过轻微滥用符号，我们将 $v \in \mathcal{M}_{+}(\mathcal{Z})$ 相对于 $\hat{v} \in \mathcal{M}_{+}(\mathcal{Z})$ 的 $\phi$-散度定义为：

$$
\mathrm{D}_\phi(v, \hat{v})=\int_{\mathcal{Z}} \phi^\pi\left(\frac{\mathrm{d} v}{\mathrm{~d} \rho}(z), \frac{\mathrm{d} \hat{v}}{\mathrm{~d} \rho}(z)\right) \mathrm{d} \rho(z),
$$

其中 $\rho \in \mathcal{M}_{+}(\mathcal{Z})$ 是支配测度，并满足 $v, \hat{v} \ll \rho$。由命题 2.6 的显然推广可知，$\mathrm{D}_\phi(v, \hat{v})$ 在 $(v, \hat{v})$ 上是凸的，并且与 $\rho$ 的选择无关。通过将 $\mathrm{D}_\phi$ 扩展到一般测度，不确定性集合 (2.10) 上的最坏情况下期望问题 (4.1) 可重新表述为：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)]=-h(1, r),
$$

其中辅助函数 $h: \mathbb{R}^2 \rightarrow \overline{\mathbb{R}}$ 定义为：

$$
h\left(u_0, u\right)=\inf _{v \in \mathcal{M}_{+}(\mathcal{Z})}\left\{-\int_{\mathcal{Z}} \ell(z) \mathrm{d} v(z): \int_{\mathcal{Z}} \mathrm{d} v(z)=u_0, \mathrm{D}_\phi(v, \hat{\mathbb{P}}) \leq u\right\} .
$$

由于 (4.7) 中的目标函数和约束函数在 $v, u_0$ 和 $u$ 上是联合凸的，故引理 4.1 表明 $h$ 是凸的。显然，我们有 $\operatorname{dom}(h) \subseteq \mathbb{R}_{+}^2$。在温和的正则性条件下，还可以证明 $\{1\} \times \mathbb{R}_{++} \subseteq \operatorname{rint}(\operatorname{dom}(h))$。

**引理 4.10（$h$ 的定义域）** 如果 $\mathbb{E}_{\hat{\mathrm{p}}}[\ell(Z)]>-\infty$ 且 $\phi$ 在 1 处连续，则：

$$
\{1\} \times \mathbb{R}_{++} \subseteq \operatorname{rint}(\operatorname{dom}(h))
$$

**证明**：若 $u_0=1$ 且 $u>0$，则 $v=\hat{\mathbb{P}}$ 在 (4.7) 中是可行的。实际上，$\hat{\mathbb{P}}$ 满足两个约束，其目标函数值满足 $\mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]>-\infty$。如果我们局部扰动 $u_0$ 和 $u$，则 $u_0 \hat{\mathbb{P}}$ 满足等式约束，并且目标函数对于所有 $u_0 \geq 0$ 都不会趋于 $-\infty$。另一方面，对于所有 $u>0$ 和所有足够接近 1 的 $u_0$，不等式约束成立，因为：

$$
\mathrm{D}_\phi\left(u_0 \hat{\mathbb{P}}, \hat{\mathbb{P}}\right)=\phi^\pi\left(u_0, 1\right)=\phi\left(u_0\right)<u .
$$

其中，第一个等号来自于 $\mathrm{D}_\phi$ 在 $\rho=\hat{\mathbb{P}}$ 下的定义，第二个等号来自于透视函数 $\phi^\pi$ 的定义，不等式成立是因为 $\phi(1)=0$，$u>0$ 且 $\phi\left(u_0\right)$ 在 $u_0=1$ 处连续。这确认了 $(1, u) \in \operatorname{rint}(\operatorname{dom}(h))$ 对于每个 $u>0$ 成立，因此结论得证。

以下两个引理对推导 $h$ 的双共轭函数是非常重要的。

**引理 4.11（缩放透视函数的共轭）** 如果 $\phi$ 是定义 2.4 中的熵函数，$t \in \mathbb{R}, \beta \in \mathbb{R}_{+}, \lambda \in \mathbb{R}_{++}$，则有：

$$
\sup _{\alpha \in \mathbb{R}} t \alpha-\lambda \phi^\pi(\alpha, \beta)= \begin{cases}\beta \lambda \phi^*(t / \lambda) & \text { 如果 } \beta>0, \\ \lambda \delta_{\mathrm{cl}\left(\operatorname{dom}\left(\phi^*\right)\right)}(t / \lambda) & \text { 如果 } \beta=0 .\end{cases}
$$

**证明**：若 $\beta>0$，则：

$$
\sup _{\alpha \in \mathbb{R}} t \alpha-\lambda \phi^\pi(\alpha, \beta)=\sup _{\alpha \in \mathbb{R}} t \alpha-\lambda \beta \phi(\alpha / \beta)=\beta \sup _{\alpha \in \mathbb{R}} t \alpha-\lambda \phi(\alpha)=\beta \lambda \phi^*(t / \lambda),
$$

其中三次等号分别来自于透视函数 $\phi^\pi$ 的定义，变量替换 $\alpha \leftarrow \alpha / \beta$ 以及用 $\lambda t / \lambda$ 替代 $t$。注意这些操作是可行的，因为 $\beta, \lambda>0$。若 $\beta=0$，则：

$$
\sup _{\alpha \in \mathbb{R}} t \alpha-\lambda \phi^\pi(\alpha, \beta)=\sup _{\alpha \in \mathbb{R}} t \alpha-\lambda \phi^{\infty}(\alpha)=\sup _{\alpha \in \mathbb{R}} t \alpha-\lambda \delta_{\operatorname{dom}\left(\phi^*\right)}^*(\alpha)=\lambda \delta_{\mathrm{cl}\left(\operatorname{dom}\left(\phi^*\right)\right)}(t / \lambda),
$$

其中第一次等号再次来自于 $\phi^\pi$ 的定义，第二次等号利用了 (Rockafellar 1970, 定理 13.3)。最后一次等号用 $\lambda t / \lambda$ 替代 $t$，并利用凸集支持函数的共轭与该集合闭包的指示函数相等的性质 (Rockafellar 1970, 定理 13.2)。因此，结论得证。

**引理 4.12（共轭熵函数的定义域）** 如果 $\phi$ 是定义 2.4 中的熵函数，则：

$$
\operatorname{cl}\left(\operatorname{dom}\left(\phi^*\right)\right)= \begin{cases}\left(-\infty, \phi^{\infty}(1)\right] & \text { 如果 } \phi^{\infty}(1)<\infty, \\ \mathbb{R} & \text { 如果 } \phi^{\infty}(1)=\infty .\end{cases}
$$

**证明**：由于 $\phi$ 是适定、凸且闭的，(Rockafellar 1970, 定理 8.5) 表明其余函数 $\phi^{\infty}$ 是正齐次的。记 $\phi(s)=\infty$ 对于所有 $s<0$。因此，可以得出 $\phi^{\infty}(t)=t \phi^{\infty}(1)$ 对于 $t>0$ 成立，$\phi^{\infty}(t)=0$ 对于 $t=0$，以及 $\phi^{\infty}(t)=\infty$ 对于 $t<0$。另外，(Rockafellar 1970, 定理 13.3) 表明 $\operatorname{dom}\left(\phi^*\right)$ 的支持函数与 $\phi^{\infty}$ 相同。闭包指示函数 $\operatorname{cl}\left(\operatorname{dom}\left(\phi^*\right)\right)$ 的定义满足：

$$
\delta_{\mathrm{cl}\left(\operatorname{dom}\left(\phi^*\right)\right)}(s)=\sup _{t \in \mathbb{R}_{+}}\left(s-\phi^{\infty}(1)\right) t= \begin{cases}0 & \text { 如果 } s \leq \phi^{\infty}(1), \\ \infty & \text { 否则. }\end{cases}
$$

由此可见，若 $\phi^{\infty}(1)<\infty$，则 $\operatorname{cl}\left(\operatorname{dom}\left(\phi^*\right)\right)=\left(-\infty, \phi^{\infty}(1)\right]$；若 $\phi^{\infty}(1)=\infty$，则 $\operatorname{cl}\left(\operatorname{dom}\left(\phi^*\right)\right)=\mathbb{R}$。因此，结论得证。

**命题 4.13（$h$ 的双共轭）** 假设 $\mathbb{E}_{\hat{p}}[\ell(Z)]>-\infty$。则定义在 (4.7) 中的 $h$ 的双共轭满足：

$$
h^{* *}\left(u_0, u\right)=\left\{\begin{array}{cl}
\sup _{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}_{+}} & -\lambda_0 u_0-\lambda u-\mathbb{E}_{\hat{\mathbb{P}}}\left[\left(\phi^*\right)^\pi\left(\ell(Z)-\lambda_0, \lambda\right)\right] \\
\text { s.t. } & \lambda_0+\lambda \phi^{\infty}(1) \geq \sup _{z \in \mathcal{Z}} \ell(z),
\end{array}\right.
$$

其中，若 $\lambda=0$ 且 $\phi^{\infty}(1)=\infty$，则 $\lambda \phi^{\infty}(1)$ 评估为 $\infty$。若 $\phi$ 在 1 处连续，则 $h^{* *}$ 在 $\{1\} \times \mathbb{R}_{++}$ 上与 $h$ 一致。

由于 $\phi(1)=0$，我们有 $\phi^*(\tau)=\sup _{\alpha \in \mathbb{R}} \tau \alpha-\phi(\alpha) \geq \tau$ 对于所有 $\tau \in \mathbb{R}$。这进一步表明 $\left(\phi^*\right)^\pi(\tau, \lambda) \geq \tau$ 对于所有 $\tau, \lambda \in \mathbb{R}$。因此，$\mathbb{E}_{\hat{\mathbb{P}}}\left[\left(\phi^*\right)^\pi\left(\ell(Z)-\lambda_0, \lambda\right)\right] \geq \mathbb{E}_{\hat{\mathbb{P}}}\left[\ell(Z)-\lambda_0\right]$。此外，由于 $\operatorname{dom}(\phi) \subseteq \mathbb{R}_{+}$，$\phi^*$ 是非减的。常见熵函数及其共轭的例子列在表 4.1 中。


**常见散度函数及其特性**

| 散度类型                  | $\phi(s)$ ($s \geq 0$)                                  | $\phi^{\infty}(1)$ | $\phi^*(t)$                                           |
|---------------------------|-------------------------------------------------------|-------------------|----------------------------------------------------|
| Kullback-Leibler          | $s \log(s) - s + 1$                                   | $\infty$          | $e^t - 1$                                          |
| Likelihood                | $-\log(s) + s - 1$                                   | $1$               | $-\log(1 - t)$                                    |
| Total variation           | $\frac{1}{2}\|s - 1\|$                                 | $\frac{1}{2}$     | $\max\{t, -1/2\} + \delta_{(-\infty, 1/2]}(t)$    |
| Pearson $\chi^2$          | $(s - 1)^2$                                          | $\infty$          | $(t / 2 + 1)^2 - 1$                               |
| Neyman $\chi^2$           | $\frac{1}{s}(s - 1)^2$                               | $1$               | $2 - 2\sqrt{1 - t}$                               |
| Cressie-Read ($\beta \in (0, 1)$) | $\frac{s^\beta - \beta s + \beta - 1}{\beta(\beta - 1)}$    | $1$               | $\frac{[(\beta - 1)t + 1]_+^{\beta/(\beta - 1)}}{\beta}$ |
| Cressie-Read ($\beta > 1$) | $\frac{s^\beta - \beta s + \beta - 1}{\beta(\beta - 1)}$    | $\infty$          | $\frac{[(\beta - 1)t + 1]_+^{\beta/(\beta - 1)}}{\beta}$ |

表 4.1 熵函数的例子、其渐近斜率及共轭函数  
这里，对于任意 $c \in \mathbb{R}$，$[c]_{+}$ 是 $\max \{c, 0\}$ 的简写。


**命题 4.13 的证明**

对于任意固定的 $\left(\lambda_0, \lambda\right) \in \mathbb{R}^2$，函数 $h$ 的共轭满足：

$$
\begin{aligned}
h^*\left(-\lambda_0,-\lambda\right) & =\sup _{\left(u_0, u\right) \in \mathbb{R}^2}-\lambda_0 u_0-\lambda u-h\left(u_0, u\right) \\
& =\sup _{u \in \mathbb{R}_{+}, v \in \mathcal{M}_{+}(\mathcal{Z})}\left\{-\lambda u+\int_{\mathcal{Z}}\left(\ell(z)-\lambda_0\right) \mathrm{d} v(z): \mathrm{D}_\phi(v, \hat{\mathbb{P}}) \leq u\right\},
\end{aligned}
$$

其中，第二个等号成立是因为 $\int_{\mathcal{Z}} \mathrm{d} v(z)=u_0$ 且 $\mathrm{D}_\phi(v, \hat{\mathbb{P}}) \geq 0$。由于 $\mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]>-\infty$，当 $\lambda<0$ 时，$u$ 的最大化问题无界。如果 $\lambda>0$，则我们有：

$$
\begin{aligned}
h^*\left(-\lambda_0,-\lambda\right) & =\sup _{v \in \mathcal{M}_{+}(\mathcal{Z})} \int_{\mathcal{Z}}\left(\ell(z)-\lambda_0\right) \mathrm{d} v(z)-\lambda \mathbf{D}_\phi(v, \hat{\mathbb{P}}) \\
& = \begin{cases}\sup & \int_{\mathcal{Z}}\left(\ell(z)-\lambda_0\right) \frac{\mathrm{d} v}{\mathrm{~d} \rho}(z)-\lambda \phi^\pi\left(\frac{\mathrm{d} v}{\mathrm{~d} \rho}(z), \frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z)\right) \mathrm{d} \rho(z) \\
\text { s.t. } & v, \rho \in \mathcal{M}_{+}(\mathcal{Z}), v \ll \rho, \hat{\mathbb{P}} \ll \rho,\end{cases}
\end{aligned}
$$

第二个等号利用了 $\mathrm{D}_\phi$ 的定义。注意 $\frac{\mathrm{d} v}{\mathrm{d} \rho}(z)$ 和 $\frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{d} \rho}(z)$ 属于 $\mathcal{L}_1(\rho)$ 空间，它包含了所有 $\rho$-可积的 Borel 函数，这些函数可以表示为某些测度对 $\rho$ 的 Radon-Nikodym 导数。引入辅助决策变量 $\alpha, \beta \in \mathcal{L}_1(\rho)$ 来表示 $v$ 和 $\hat{\mathbb{P}}$ 的 Radon-Nikodym 导数，并消去测度 $v$，可得：

$$
h^*\left(-\lambda_0,-\lambda\right)= \begin{cases}\sup & \int_{\mathcal{Z}}\left(\ell(z)-\lambda_0\right) \alpha(z)-\lambda \phi^\pi(\alpha(z), \beta(z)) \mathrm{d} \rho(z) \\ 
\text { s.t. } & \rho \in \mathcal{M}_{+}(\mathcal{Z}), \alpha, \beta \in \mathcal{L}_1(\rho) \\ 
& \frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}=\beta \rho \text {-a.s. }\end{cases}
$$

对于任意 $\rho \in \mathcal{M}_{+}(\mathcal{Z})$ 和 $\beta \in \mathcal{L}_1(\rho)$，满足 $\beta=\frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{d} \rho}$ 几乎处处成立，则我们可以推导出：

$$
\begin{aligned}
& \sup _{\alpha \in \mathcal{L}_1(\rho)} \int_{\mathcal{Z}}\left(\ell(z)-\lambda_0\right) \alpha(z)-\lambda \phi^\pi(\alpha(z), \beta(z)) \mathrm{d} \rho(z) \\
= & \int_{\mathcal{Z}} \sup _{\alpha \in \mathbb{R}}\left\{\left(\ell(z)-\lambda_0\right) \alpha-\lambda \phi^\pi(\alpha, \beta(z))\right\} \mathrm{d} \rho(z),
\end{aligned}
$$

这里的等号来自于 (Rockafellar and Wets 2009, 定理 14.60)，因为第二行的最大化问题目标函数是 (Rockafellar and Wets 2009, 定义 14.27) 中的正则积分函数。这可以通过验证求和、透视和正则积分函数的级联仍然是正则积分函数来确认 (Rockafellar and Wets 2009, 第 14.E 节)。

接下来，将 $\mathcal{Z}$ 分为两部分：$\mathcal{Z}_{+}(\beta)=\{z \in \mathcal{Z}: \beta(z)>0\}$ 和 $\mathcal{Z}_0(\beta)=\{z \in \mathcal{Z}: \beta(z)=0\}$。根据引理 4.11，积分公式等价于：

$$
\int_{\mathcal{Z}_{+}(\beta)} \lambda \phi^*\left(\frac{\ell(z)-\lambda_0}{\lambda}\right) \beta(z) \mathrm{d} \rho(z)+\int_{\mathcal{Z}_0(\beta)} \lambda \delta_{\mathrm{cl}\left(\operatorname{dom}\left(\phi^*\right)\right)}\left(\frac{\ell(z)-\lambda_0}{\lambda}\right) \mathrm{d} \rho(z).
$$

由于 $\beta=\frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{d} \rho}$ 几乎处处成立，且 $\hat{\mathbb{P}}\left(Z \in \mathcal{Z}_{+}(\beta)\right)=1$，第一个积分仅取决于参考分布的期望值，与 $\beta$ 无关。而第二个积分仍然通过积分域 $\mathcal{Z}_0(\beta)$ 依赖于 $\beta$。因此，部分最大化 $\alpha$ 后，我们可以将公式重述为：

$$
\begin{aligned}
h^*\left(-\lambda_0,-\lambda\right)=\mathbb{E}_{\hat{\mathbb{P}}}\left[\lambda \phi^*\left(\frac{\ell(Z)-\lambda_0}{\lambda}\right)\right] \\
\quad+\sup _{\substack{\rho \in \mathcal{M}_{+}(\mathcal{Z}), \beta \in \mathcal{L}_1(\rho)}}\left\{\int_{\mathcal{Z}_0(\beta)} \lambda \delta_{\mathrm{cl}\left(\operatorname{dom}\left(\phi^*\right)\right)}\left(\frac{\ell(z)-\lambda_0}{\lambda}\right) \mathrm{d} \rho(z): \frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}=\beta \rho \text {-a.s. }\right\}.
\end{aligned}
$$

如果存在 $z_0 \in \mathcal{Z}$ 使得 $\left(\ell\left(z_0\right)-\lambda_0\right) / \lambda \notin \operatorname{cl}\left(\operatorname{dom}\left(\phi^*\right)\right)$，那么 $h^*\left(-\lambda_0,-\lambda\right)=\infty$。综上，最终得到双共轭公式：

$$
h^{* *}\left(u_0, u\right)=\left\{\begin{array}{cl}
\sup _{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}_{+}} & -\lambda_0 u_0-\lambda u-\mathbb{E}_{\hat{\mathbb{P}}}\left[\left(\phi^*\right)^\pi\left(\ell(Z)-\lambda_0, \lambda\right)\right] \\
\text { s.t. } & \lambda_0+\lambda \phi^{\infty}(1) \geq \sup _{z \in \mathcal{Z}} \ell(z),
\end{array}\right.
$$

从公式 (4.10) 可以看出，若 $\phi$ 在 1 处连续且 $r>0$，则函数 $h(1, u)$ 和 $h^{**}(1, u)$ 在所有 $u \in \mathbb{R}_{++}$ 上相等。这一结论直接来自引理 4.2 和 4.10。

**定理 4.14**（针对 $\phi$-散度不确定性集合的对偶理论）
假设 $\mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]>-\infty$。如果 $\mathcal{P}$ 是 $\phi$-散度不确定性集合 (2.10)，则以下弱对偶关系成立：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] \leq\left\{\begin{array}{cl}
\inf _{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}_{+}} & \lambda_0+\lambda r+\mathbb{E}_{\hat{\mathbb{P}}}\left[\left(\phi^*\right)^\pi\left(\ell(Z)-\lambda_0, \lambda\right)\right] \\
\text { s.t. } & \lambda_0+\lambda \phi^{\infty}(1) \geq \sup _{z \in \mathcal{Z}} \ell(z),
\end{array}\right.
$$

这里，如果 $\lambda=0$ 且 $\phi^{\infty}(1)=\infty$，则乘积 $\lambda \phi^{\infty}(1)$ 被视为 $\infty$。如果进一步假设 $r>0$ 且 $\phi$ 在 1 处连续，则强对偶成立，即不等式 (4.11) 退化为等式。

**证明**：首先回忆以下等式：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)]=-h(1, r) \leq-h^{**}(1, r),
$$

其中，不等式成立是由于引理 4.2。因此，弱对偶关系直接来自于命题 4.13 的第一部分。如果进一步假设 $\phi$ 在 1 处连续，且 $r>0$，则强对偶关系直接来自于命题 4.13 的第二部分。

现在回忆，受限的 $\phi$-散度不确定性集合 (2.11) 定义为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{P} \ll \hat{\mathbb{P}}, \mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\}.
$$


也就是说，$\mathcal{P}$ 包含所有来自（无限制）$\phi$-散度不确定性集合 $(2.10)$ 且相对于 $\hat{\mathbb{P}}$ 绝对连续的分布。$\mathcal{P}$ 上的最坏情况下的期望损失仍然可以表示为 $-h(1, r)$，其中 $h\left(u_0, u\right)$ 定义为优化问题 (4.7) 的最小值，但带有额外约束 $v \ll \hat{\mathbb{P}}$。可以验证，尽管有此限制，$h$ 仍然是凸的，并且 $\{1\} \times \mathbb{R}_{++}$ 仍包含在 $\operatorname{rint}(\operatorname{dom}(h))$ 中。实际上，引理 4.10 的证明几乎可以逐字适用。

**定理 4.15**（针对受限 $\phi$-散度不确定性集合的对偶理论）

假设 $\mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]>-\infty$。如果 $\mathcal{P}$ 是受限的 $\phi$-散度不确定性集合 (2.11)，则以下弱对偶关系成立：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] \leq \inf _{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}_{+}} \lambda_0+\lambda r+\mathbb{E}_{\hat{\mathbb{P}}}\left[\left(\phi^*\right)^\pi\left(\ell(Z)-\lambda_0, \lambda\right)\right]
$$

如果进一步假设 $r > 0$ 且 $\phi$ 在 1 处连续，则强对偶成立，即不等式 (4.12) 退化为等式。

注意，如果 $\left(\lambda_0, \lambda\right)$ 在 (4.12) 中是可行的，则 $\left(\ell(Z)-\lambda_0, \lambda\right)$ 几乎处处属于 $\operatorname{dom}\left(\left(\phi^*\right)^\pi\right)$（相对于 $\hat{\mathbb{P}}$）。否则，其目标函数值为 $\infty$。根据引理 4.12，这意味着 $\lambda_0+\lambda \phi^{\infty}(1) \geq \operatorname{ess} \sup _{\hat{\mathbb{P}}}[\ell(Z)]$。相比之下，如果 $\left(\lambda_0, \lambda\right)$ 在 (4.11) 中是可行的，则它满足约束 $\lambda_0+\lambda \phi^{\infty}(1) \geq \sup _{z \in \mathcal{Z}} \ell(z)$，除非 $\phi^{\infty}(1)=\infty$，否则这一约束更为严格。因此，公式 (4.12) 的对偶问题具有（弱）更大的可行解集和（弱）更小的下确界。这并不意外，因为公式 (4.12) 对应于受限 $\phi$-散度不确定性集合上的最坏情况下期望问题，而该集合（弱）小于相应的无限制 $\phi$-散度不确定性集合。此外，注意到无限制 $\phi$-散度不确定性集合上最坏情况期望问题的解依赖于 $\mathcal{Z}$，而不仅仅是 $\hat{\mathbb{P}}$ 的支持集。

**定理 4.15 的证明**

如果 $h\left(u_0, u\right)$ 定义为优化问题 (4.7) 的最小值，但带有额外约束 $v \ll \hat{\mathbb{P}}$，则可以证明：

$$
h^{* *}\left(u_0, u\right)=\sup _{\lambda_0, \lambda \in \mathbb{R}}-\lambda_0 u_0-\lambda u-\mathbb{E}_{\hat{\mathbb{P}}}\left[\left(\phi^*\right)^\pi\left(\ell(Z)-\lambda_0, \lambda\right)\right] .
$$

实际上，可以按照命题 4.13 的证明进行。然而，由于额外的约束 $v \ll \hat{\mathbb{P}}$，推导过程显著简化，因为它允许我们将 $\mathrm{D}_\phi$ 定义中的支配测度 $\rho$ 设置为 $\hat{\mathbb{P}}$。因此，Radon-Nikodym 导数 $\beta=\mathrm{d} \hat{\mathbb{P}} / \mathrm{d} \rho$ 几乎处处等于 1（相对于 $\hat{\mathbb{P}}$）。这进一步意味着 $h^*$ 的计算不需要进行情况区分，即集合 $\mathcal{Z}_0(\beta)$ 是空的。

在获得 $h$ 的双共轭之后，弱对偶性和强对偶性可以完全按照定理 4.14 的证明方式建立。为了简洁，细节省略。

Van Parys 等人（2021，命题 5）在第 2.2.2 节介绍的似然不确定性集合上，证明了最坏情况期望的强对偶性结果。定理 4.14 将这一结果扩展到了通用的 $\phi$-散度不确定性集合，并且仅通过凸分析工具给出了显著简化的证明。Ben-Tal 等人（2013）在假设参考分布 $\hat{\mathbb{P}}$ 是离散的情况下，证明了类似于定理 4.15 的强对偶结果。Shapiro（2017）通过无限维分析工具将这一结果扩展到了通用参考分布。相比之下，我们对定理 4.15 的证明使用了有限维凸分析，得出了相同的对偶性结果。



## 4.4 最优传输不确定性集合

回忆第 2.3 节中定义的最优传输不确定性集合 (2.27)：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\} .
$$

这里，$\mathcal{Z}$ 是一个闭的支持集，$r \geq 0$ 是集合的大小参数，$c$ 是定义 2.14 中的传输成本函数，$\mathrm{OT}_c$ 是定义 2.15 中的对应最优传输差异，$\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 是参考分布。类似于第 4.3 节，不确定性集合 (2.27) 上的最坏情况下期望问题 (4.1) 可以重新表述为：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)]=-h(r),
$$

其中辅助函数 $h: \mathbb{R} \rightarrow \overline{\mathbb{R}}$ 定义为：

$$
h(u)=\inf _{\mathbb{P} \in \mathcal{P}(\mathcal{Z})}\left\{-\mathbb{E}_{\mathbb{P}}[\ell(Z)]: \mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}}) \leq u\right\} .
$$

由于 (4.13) 中的目标函数和约束函数在 $\mathbb{P}$ 和 $u$ 上是联合凸的，因此根据引理 4.1，$h$ 是凸的。同时，$c$ 是非负的，并满足 $c(z, z)=0$ 对于所有 $z \in \mathcal{Z}$。如果 $\mathbb{E}_{\mathfrak{p}}[\ell(\hat{Z})]>-\infty$，则可以轻松证明 $\operatorname{dom}(h)=\mathbb{R}_{+}$。

以下引理对于推导 $h$ 的双共轭至关重要。回忆 $\Gamma(\mathbb{P}, \hat{\mathbb{P}})$ 表示 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 的所有耦合的集合；参见定义 2.15。



**引理 4.16**（可交换性原则）

如果 $c$ 是传输成本函数，$\ell$ 是上半连续的，且 $\lambda \geq 0$，则有：

$$
\sup _{\mathbb{P} \in \mathcal{P}(\mathcal{Z})} \sup _{\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})} \mathbb{E}_\gamma[\ell(Z)-\lambda c(Z, \hat{Z})]=\mathbb{E}_{\hat{\mathbb{P}}}\left[\sup _{z \in \mathcal{Z}} \ell(z)-\lambda c(z, \hat{Z})\right] .
$$

可以证明，即使在以下情况下，引理 4.16 仍然成立，例如 $\mathcal{Z}$ 是带有其 Borel $\sigma$-代数的 Polish（可分度量）空间，即使 $c$ 和 $\ell$ 分别未满足下半连续性和上半连续性 (Zhang et al. 2024b, Proposition 1)。



**引理 4.16**  的证明

定义 $L: \mathcal{Z} \rightarrow \overline{\mathbb{R}}$ 为 $L(\hat{z})=\sup _{z \in \mathcal{Z}} \ell(z)-\lambda c(z, \hat{z})$。如果 $\lambda=1$，则 $L$ 简化为 (2.25) 中定义的 $\ell$ 的 $c$-变换。首先，注意到 $L$ 是上半连续函数的逐点上确界，因此 $L$ 也是上半连续的，尤其是 Borel 可测的。

接下来，根据 $L$ 的定义，对于所有 $z, \hat{z} \in \mathcal{Z}$，我们有 $\ell(z)-\lambda c(z, \hat{z}) \leq L(\hat{z})$。如果将两边对任意耦合 $\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})$ 和分布 $\mathbb{P} \in \mathcal{P}(\mathcal{Z})$ 进行积分，则得到：

$$
\sup _{\mathbb{P} \in \mathcal{P}(\mathcal{Z})} \sup _{\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})} \mathbb{E}_\gamma[\ell(Z)-\lambda c(Z, \hat{Z})] \leq \mathbb{E}_{\hat{\mathbb{P}}}[L(\hat{Z})] .
$$

接下来需要证明反向不等式。为此，观察到：

$$
\begin{aligned}
\mathbb{E}_{\hat{\mathbb{P}}}[L(\hat{Z})]=\mathbb{E}_{\hat{\mathbb{P}}}\left[\sup _{z \in \mathcal{Z}} \ell(z)-\lambda c(z, \hat{Z})\right] & =\sup _{f \in \mathcal{F}} \mathbb{E}_{\hat{\mathbb{P}}}[\ell(f(\hat{Z}))-\lambda c(f(\hat{Z}), \hat{Z})] \\
& \leq \sup _{\mathbb{P} \in \mathcal{P}(\mathcal{Z})} \sup _{\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})} \mathbb{E}_\gamma[\ell(Z)-\lambda c(Z, \hat{Z})],
\end{aligned}
$$

其中，$\mathcal{F}$ 表示所有 Borel 函数 $f: \mathcal{Z} \rightarrow \mathcal{Z}$ 的集合。第二个等号来自于 (Rockafellar and Wets 2009, 定理 14.60)，因为 $\ell(z)-\lambda c(z, \hat{z})$ 在 $(z, \hat{z})$ 上是上半连续的，并且根据 (Rockafellar and Wets 2009, 例 14.31) 是正则积分函数。上述结果完成了证明。



**命题 4.17**（$h$ 的双共轭）

假设 $\mathbb{E}_{\hat{\mathbb{P}}}[\ell(\hat{Z})]>-\infty$ 且 $\ell$ 是上半连续的，则 (4.13) 中定义的 $h$ 的双共轭满足：

$$
h^{* *}(u)=\sup _{\lambda \geq 0}-\lambda r-\mathbb{E}_{\hat{\mathbb{P}}}\left[\sup _{z \in \mathcal{Z}} \ell(z)-\lambda c(z, \hat{Z})\right] .
$$

此外，在 $\mathbb{R}_{++}$ 上，$h^{* *}$ 与 $h$ 一致。



**命题 4.17 的证明**

对于任意固定的 $\lambda \in \mathbb{R}$，$h$ 的共轭满足：

$$
\begin{aligned}
h^*(-\lambda) & = \sup _{u \in \mathbb{R}} -\lambda u - h(u) \\
& = \sup _{u \in \mathbb{R}_{+}, \mathbb{P} \in \mathcal{P}(\mathcal{Z})} \left\{ -\lambda u + \mathbb{E}_{\mathbb{P}}[\ell(Z)]: \mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}}) \leq u \right\}.
\end{aligned}
$$

第二个等号成立是因为 $\mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}}) \geq 0$。由于假设 $\mathbb{E}_{\hat{\mathbb{P}}}[\ell(\hat{Z})] > -\infty$，当 $\lambda < 0$ 时，最大化问题无界。如果 $\lambda > 0$，则有：

$$
\begin{aligned}
h^*(-\lambda) & = \sup _{\mathbb{P} \in \mathcal{P}(\mathcal{Z})} \mathbb{E}_{\mathbb{P}}[\ell(Z)] - \lambda \mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}}) \\
& = \sup _{\mathbb{P} \in \mathcal{P}(\mathcal{Z})} \sup _{\gamma \in \Gamma(\mathbb{P}, \hat{\mathbb{P}})} \mathbb{E}_\gamma[\ell(Z)-\lambda c(Z, \hat{Z})] \\
& = \mathbb{E}_{\hat{\mathbb{P}}} \left[ \sup _{z \in \mathcal{Z}} \ell(z) - \lambda c(z, \hat{Z}) \right],
\end{aligned}
$$

最后，我们得到 $h$ 的双共轭表达式：

$$
h^{**}(u) = \sup _{\lambda \geq 0} -\lambda u - \mathbb{E}_{\hat{\mathbb{P}}} \left[ \sup _{z \in \mathcal{Z}} \ell(z) - \lambda c(z, \hat{Z}) \right].
$$
这里，第一个等号成立是因为当 $\lambda<0$ 时，$h^*(-\lambda)=\infty$。第二个等号来源于公式 (4.14)，该公式对任何 $\lambda \geq 0$ 都成立。这验证了 $h^{**}$ 的目标公式。引理 4.2 和我们之前关于 $\operatorname{dom}(h)=\mathbb{R}_{+}$ 的观察最终表明，对于所有 $u \in \mathbb{R}_{++}$，有 $h(u)=h^{**}(u)$。

以下主要定理使用命题 4.17 将最坏情况期望问题 (4.1) 在最优传输不确定性集合下对偶化。



**定理 4.18**（最优传输不确定性集合的对偶理论）

假设 $\mathbb{E}_{\hat{\mathbb{P}}}[\ell(\hat{Z})] > -\infty$ 且 $\ell$ 是上半连续的。如果 $\mathcal{P}$ 是定义于公式 (2.27) 的最优传输不确定性集合，则以下弱对偶关系成立：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] \leq \inf _{\lambda \in \mathbb{R}_{+}} \lambda r + \mathbb{E}_{\hat{P}}\left[\sup _{z \in \mathcal{Z}} \ell(z) - \lambda c(z, \hat{Z})\right] .
$$

如果 $r > 0$，则强对偶性成立，即公式 (4.15) 退化为等式。



**证明**

首先回忆以下等式和不等式：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] = -h(r) \leq -h^{**}(r),
$$

其中，不等式来源于引理 4.2。因此，弱对偶性由命题 4.17 的第一个结论直接得出。如果 $r > 0$，则强对偶性由命题 4.17 的第二个结论直接得出。证明完毕。

Mohajerin Esfahani 和 Kuhn (2018) 以及 Zhao 和 Guan (2018) 使用半无限对偶理论证明了定理 4.18 的特殊情况，即当 $\mathrm{OT}_c$ 为 1-Wasserstein 距离且参考分布 $\hat{\mathbb{P}}$ 为离散分布时。Blanchet 和 Murthy (2019) 以及 Gao 和 Kleywegt (2023) 则通过在 Banach 空间中的 Fenchel 对偶理论，以及分别设计的构造性论证，推广了定理 4.18。他们的证明同时允许任意最优传输差异以及 Polish 空间上的任意参考分布。本文中的证明基于引理 4.16 的可交换性原则以及凸分析的基本工具，由 Zhang et al. (2024b) 提出。




# 5. 最坏情况风险问题的对偶理论

标准的分布鲁棒优化（DRO）问题 (1.2) 假设决策者是风险中性且厌恶不确定性的。风险中性意味着如果 $Z$ 的分布已知，则决策通过其期望损失进行排序。厌恶不确定性意味着如果 $Z$ 的分布不确定，则期望是在不确定性集合 $\mathcal{P}$ 中对决策者最不利的分布下计算的。

如果低概率事件对决策者的负面影响不成比例，那么即使 $Z$ 的分布已知，使用期望损失作为决策标准也是不合适的。相反，更合适的方法是通过基于分布不变风险度量的损失风险来对决策进行排序。一个分布不变的风险度量 $\varrho$ 为 $\mathcal{P}(\mathbb{R})$ 中的每个（单变量）损失分布分配一个风险指数。如果损失可以表示为 $\ell(Z)$，其中 $\ell: \mathbb{R}^d \rightarrow \mathbb{R}$ 是一个 Borel 函数，且 $Z$ 是一个 $d$ 维随机向量，其概率分布为 $\mathbb{P}$，则损失 $\ell(Z)$ 的分布由推前分布 $\mathbb{P} \circ \ell^{-1}$ 给出。本文中，我们用 $\varrho_{\mathbb{P}}[\ell(Z)]$ 表示此类损失分布的风险 $\varrho\left(\mathbb{P} \circ \ell^{-1}\right)$。这些约定形式化为以下定义。本文其余部分中，我们用 $\mathcal{L}\left(\mathbb{R}^d\right)$ 表示所有 Borel 函数 $\ell: \mathbb{R}^d \rightarrow \mathbb{R}$ 的集合。

**定义 5.1**（分布不变风险度量）

一个分布不变风险度量是一个函数 $\varrho: \mathcal{P}(\mathbb{R}) \rightarrow \overline{\mathbb{R}}$。对于任何 Borel 函数 $\ell \in \mathcal{L}\left(\mathbb{R}^d\right)$，Borel 分布 $\mathbb{P} \in \mathcal{P}\left(\mathbb{R}^d\right)$ 和维度 $d \in \mathbb{N}$，我们用 $\varrho_{\mathbb{P}}[\ell(Z)]$ 表示 $\varrho\left(\mathbb{P} \circ \ell^{-1}\right)$。

分布不变风险度量 $\varrho$ 满足以下性质：如果对于两个不同的 Borel 函数 $\ell_1$ 和 $\ell_2$ 以及定义在 $\mathbb{R}^{d_1}$ 和 $\mathbb{R}^{d_2}$ 上的两个不同分布 $\mathbb{P}_1$ 和 $\mathbb{P}_2$，有 $\mathbb{P}_1 \circ \ell_1^{-1}=\mathbb{P}_2 \circ \ell_2^{-1}$，则有 $\varrho_{\mathbb{P}_1}\left[\ell_1\left(Z_1\right)\right]=\varrho_{\mathbb{P}_2}\left[\ell_2\left(Z_2\right)\right]$。实际上，这一性质正是 $\varrho$ 被称为“分布不变”的原因。

符号 $\varrho_{\mathbb{P}}[\ell(Z)]$ 与我们通常对期望值 $\mathbb{E}_{\mathbb{P}}[\ell(Z)]$ 的约定一致，而后者是分布不变风险度量的一个特例。此外，它明确表达了风险对 $\mathbb{P}$ 的依赖性，这是在 $\mathbb{P}$ 不确定时必要的。需要强调的是，与大多数关于风险度量的文献不同，我们对分布不变风险度量 $\varrho$ 的定义并未限定在特定的概率空间上。分布不变风险度量的一个典型例子是价值风险（Value-at-Risk）。

**定义 5.2**（价值风险）

在水平 $\beta \in(0,1)$ 下，不确定损失 $\ell(Z)$ 的价值风险（VaR），其中 $\ell \in \mathcal{L}\left(\mathbb{R}^d\right)$ 且 $Z \sim \mathbb{P} \in \mathcal{P}\left(\mathbb{R}^d\right)$，定义为：

$$
\beta-\operatorname{VaR}_{\mathbb{P}}[\ell(Z)]=\inf \{\tau \in \mathbb{R}: \mathbb{P}(\ell(Z) \leq \tau) \geq 1-\beta\} .
$$

VaR 确实是分布不变的，因为 $\mathbb{P}(\ell(Z) \leq \tau)=F(\tau)$ 仅通过推前分布 $\mathbb{P} \circ \ell^{-1}$ 关联的累积分布函数 $F$ 间接依赖于 $\ell$ 和 $\mathbb{P}$。注意，公式 (5.1) 中的下确界是可以取到的，因为 $F$ 是非减的且右连续的。根据定义，$\beta$ 水平下的 VaR 表示以概率 $1-\beta$ 不小于损失的最小值 $\tau^{\star}$。因此，它与损失分布 $F$ 的左侧 $(1-\beta)$ 分位数一致。

作为后续参考，我们注意到 $\beta$-VaR 可以重写为：

$$
\beta-\operatorname{VaR}_{\mathbb{P}}[\ell(Z)]=\inf \{\tau \in \mathbb{R}: \mathbb{P}(\ell(Z) \geq \tau) \leq \beta\} .
$$

然而，公式 (5.2) 中的下确界可能无法取到。需要注意，VaR 对于任何损失函数 $\ell \in \mathcal{L}\left(\mathbb{R}^d\right)$ 和任何分布 $\mathbb{P} \in \mathcal{P}\left(\mathbb{R}^d\right)$ 都是良定义且有限的。然而，其他分布不变风险度量仅对某些子类的损失函数和分布是有限的。在本文其余部分中，我们将经常研究具有以下某些或全部结构性质的风险度量。

**定义 5.3**（风险度量的性质）

一个分布不变风险度量 $\varrho$ 满足以下性质：

1. **平移不变性**，如果：

$$
\varrho_{\mathbb{P}}[\ell(Z)+c]=\varrho_{\mathbb{P}}[\ell(Z)]+c \quad \forall \ell \in \mathcal{L}\left(\mathbb{R}^d\right), \forall c \in \mathbb{R}, \forall \mathbb{P} \in \mathcal{P}\left(\mathbb{R}^d\right) ;
$$

2. **缩放不变性**，如果：

$$
\varrho_{\mathbb{P}}[c \ell(Z)]=c \varrho_{\mathbb{P}}[\ell(Z)] \quad \forall \ell \in \mathcal{L}\left(\mathbb{R}^d\right), \forall c \in \mathbb{R}_{+}, \forall \mathbb{P} \in \mathcal{P}\left(\mathbb{R}^d\right) ;
$$

3. **单调性**，如果：

$$
\begin{aligned}
& \varrho_{\mathbb{P}}\left[\ell_1(Z)\right] \leq \varrho_{\mathbb{P}}\left[\ell_2(Z)\right] \\
& \quad \forall \ell_1, \ell_2 \in \mathcal{L}\left(\mathbb{R}^d\right) \text { 且 } \ell_1(Z) \leq \ell_2(Z) \mathbb{P} \text {-几乎处处}, \forall \mathbb{P} \in \mathcal{P}\left(\mathbb{R}^d\right) ;
\end{aligned}
$$

4. **凸性**，如果：

$$
\begin{aligned}
\varrho_{\mathbb{P}}\left[\theta \ell_1(Z)+(1-\theta) \ell_2(Z)\right] & \leq \theta \varrho_{\mathbb{P}}\left[\ell_1(Z)\right]+(1-\theta) \varrho_{\mathbb{P}}\left[\ell_2(Z)\right] \\
\forall \ell_1, \ell_2 & \in \mathcal{L}\left(\mathbb{R}^d\right), \forall \theta \in[0,1], \forall \mathbb{P} \in \mathcal{P}\left(\mathbb{R}^d\right) .
\end{aligned}
$$

一个一致风险度量是平移不变、缩放不变、单调且凸的（见 Artzner et al., 1999）。此外，一个凸风险度量是平移不变、单调且凸的（但不一定缩放不变）。

**风险规避 DRO 问题**

任何分布不变的风险度量 $\varrho$ 都会引出一个风险规避 DRO 问题：

$$
\inf _{x \in \mathcal{X}} \sup _{\mathbb{P} \in \mathcal{P}} \varrho_{\mathbb{P}}[\ell(x, Z)] .
$$

该问题寻求一个决策 $x$，使得随机损失 $\ell(x, Z)$ 在 $\mathcal{P}$ 中所有分布上的最坏风险最小化。接下来我们将展示，第四节中开发的最坏期望问题的对偶理论对如下形式的一类广泛的最坏风险问题具有影响：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \varrho_{\mathbb{P}}[\ell(Z)] .
$$

与往常一样，为了避免复杂性，我们省略损失函数对 $x$ 的依赖。


## 5.1 优化的确定性等价

我们现在描述一类分布不变的风险度量，对于这类风险度量，风险规避 DRO 问题 (5.3) 可以转化为形式 (1.2) 的等价风险中性 DRO 问题。这表明，许多风险规避的 DRO 问题可以采用为风险中性问题开发的方法来解决。本节研究的风险度量由以下定义中的不效用函数引导。

**定义 5.4（不效用函数）**

不效用函数 $g: \mathbb{R} \rightarrow \mathbb{R}$ 是一个凸函数（因此是连续的），满足 $g(0)=0$ 且对于所有 $\tau \neq 0$，有 $g(\tau)>\tau$。

Ben-Tal 和 Teboulle (1986) 使用不效用函数构造了一类分布不变的风险度量，称为优化的确定性等价。回想一下，如果最小化（最大化）问题的目标函数可以表示为两个值均为 $\infty$ 的项的差（例如积分的正负部分），则其结果应解释为 $\infty(-\infty)$。

**定义 5.5（优化的确定性等价）**

由不效用函数 $g$ 引导的优化的确定性等价是一种分布不变的风险度量 $\varrho$，其定义为：

$$
\varrho_{\mathbb{P}}[\ell(Z)]=\inf _{\tau \in \mathbb{R}} \tau+\mathbb{E}_{\mathbb{P}}[g(\ell(Z)-\tau)] .
$$

期望不效用 $\mathbb{E}_{\mathbb{P}}[g(\ell(Z))]$ 表示决策者认为与随机未来损失 $\ell(Z)$ 同等（不）可取的确定性当前损失。如果可以将损失 $\ell(Z)$ 的确定部分 $\tau$ 移到当前，则决策者将通过求解 (5.5) 中的最小化问题，在当前损失和未来损失之间进行最佳权衡。因此，将 $\varrho_{\mathbb{P}}[\ell(Z)]$ 解释为“优化的确定性等价”是自然的。

优化的确定性等价与一类 $\phi$-散度之间也有密切关系。为说明这一点，令 $\phi$ 是符合定义 2.4 的熵函数，且满足 $\phi^{\infty}(1)=\infty$。假设 $\phi$ 在 1 的邻域内二阶连续可微，且 $\phi^{\prime}(1)=0$ 和 $\phi^{\prime \prime}(1)>0$。在这些条件下，$\phi^*$ 是符合定义 5.4 的不效用函数。事实上，$\phi^*$ 是实值的，因为 $\phi^{\infty}(1)=\infty$，并且对于所有 $t \in \mathbb{R}$，$\phi^*(t) \geq t$，这是因为 $\phi(1)=0$。最后，由于 $\phi^{\prime}(1)=0$ 和 $\phi^{\prime \prime}(t)>0$，有 $\phi^*(0)=0$ 且 $\phi^*(t)>t$ 对于所有 $t \neq 0$。

如果 $\mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]>-\infty$，则由不效用函数 $g=\phi^*$ 引导的优化的确定性等价满足：

$$
\inf _{\lambda_0 \in \mathbb{R}} \lambda_0+\mathbb{E}_{\mathbb{P}}\left[\phi^*\left(\ell(Z)-\lambda_0\right)\right]=\sup _{\mathbb{P} \in \mathcal{P}(\mathcal{Z})} \mathbb{E}_{\mathbb{P}}[\ell(Z)]-\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})
$$

因此，这与基于惩罚的 $\phi$-散度分布鲁棒优化模型的最优值一致。上述表达式中的等式源自 Ben-Tal 和 Teboulle (2007, Theorem 4.2)，与关于受限 $\phi$-散度不确定性集合的最坏期望问题的强对偶性定理（见定理 4.15）相似。假设 $\phi^{\infty}(1)=\infty$ 确保 $\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$ 仅在 $\mathbb{P} \ll \hat{\mathbb{P}}$ 时是有限的。我们还注意到，如果 $g$ 是符合定义 5.4 的不效用函数且 $g$ 是单调递增的，那么 $g^*$ 构成了符合定义 2.4 的熵函数。

我们将在下文中看到，优化的确定性等价包含了几种广泛使用的风险度量作为特例。其中显著的例子包括均值-方差风险度量、均值-中位绝对偏差（MAD）风险度量、条件 VaR（CVaR）和熵风险度量。更一般地，Rockafellar, Uryasev 和 Zabarankin (2006, 2008) 表明，几乎任何规则风险度量都可以表示为 (5.5) 的形式，只需将期望不效用替换为更一般的遗憾度量；另见 (Rockafellar 和 Royset 2014, 2015) 及综述文章 (Rockafellar 和 Royset 2013, Royset 2022)。

**定义 5.6（均值-方差风险度量）**

风险厌恶系数为 $\beta \in(0, \infty)$ 的均值-方差风险度量是一种分布不变的风险度量 $\varrho$，定义为：

$$
\varrho_{\mathbb{P}}[\ell(Z)]=\mathbb{E}_{\mathbb{P}}[\ell(Z)]+\beta \cdot \mathbb{V}_{\mathbb{P}}[\ell(Z)] ,
$$

其中 $\mathbb{V}_{\mathbb{P}}[\ell(Z)]$ 表示 $\ell(Z)$ 在 $\mathbb{P}$ 下的方差。

**命题 5.7（均值-方差风险度量）**

风险厌恶系数为 $\beta \in(0, \infty)$ 的均值-方差风险度量是由不效用函数 $g(\tau)=\tau+\beta \tau^2$ 引导的优化的确定性等价。问题 (5.5) 的目标函数在 $\tau$ 上是强制性的，并且在 $\tau^{\star}=\mathbb{E}_{\mathbb{P}}[\ell(Z)]$ 处唯一达到最小值。

**定义 5.8（均值-MAD 风险度量）**

风险厌恶系数为 $\beta \in(0, \infty)$ 的均值-中位绝对偏差（MAD）风险度量是一种分布不变的风险度量 $\varrho$，定义为：

$$
\varrho_{\mathbb{P}}[\ell(Z)]=\mathbb{E}_{\mathbb{P}}[\ell(Z)]+\beta \cdot \mathbb{E}_{\mathbb{P}}\left[\left|\ell(Z)-\mathbb{M}_{\mathbb{P}}[\ell(Z)]\right|\right] ,
$$

其中 $\mathbb{M}_{\mathbb{P}}[\ell(Z)]$ 表示 $\ell(Z)$ 在 $\mathbb{P}$ 下的中位数。

**命题 5.9（均值-MAD 风险度量）**

风险厌恶系数为 $\beta \in(0, \infty)$ 的均值-MAD 风险度量是由不效用函数 $g(\tau)=\tau+\beta|\tau|$ 引导的优化的确定性等价。问题 (5.5) 的目标函数在 $\tau$ 上是强制性的，并且在 $\tau^{\star}=\mathbb{M}_{\mathbb{P}}[\ell(Z)]$ 处达到最小值。

**定义 5.10（条件价值风险 CVaR）**

水平 $\beta \in(0,1)$ 的条件 VaR（CVaR）是一种分布不变的风险度量，记为 $\beta$-CVaR，定义为：

$$
\beta-\operatorname{CVaR}_{\mathbb{P}}[\ell(Z)]=\inf _{\tau \in \mathbb{R}} \tau+\frac{1}{\beta} \mathbb{E}_{\mathbb{P}}[\max \{\ell(Z)-\tau, 0\}] .
$$

**命题 5.11（CVaR）**

水平 $\beta \in(0,1)$ 的 CVaR 是由不效用函数 $g(\tau)=\beta^{-1} \max \{\tau, 0\}$ 引导的优化的确定性等价。问题 (5.5) 的目标函数在 $\tau$ 上是强制性的，并且在 $\tau^{\star}=\beta-\mathrm{VaR}_{\mathbb{P}}[\ell(Z)]$ 处达到最小值。

**定义 5.12（熵风险度量）**

风险厌恶参数为 $\beta \in(0, \infty)$ 的熵风险度量是一种分布不变的风险度量，记为 $\beta$-ERM，定义为：

$$
\beta-\mathrm{ERM}_{\mathbb{P}}[\ell(Z)]=\frac{1}{\beta} \log \mathbb{E}_{\mathbb{P}}[\exp (\beta \ell(Z))] .
$$

**命题 5.13（熵风险度量）**

风险厌恶参数为 $\beta \in(0,1)$ 的熵风险度量是由不效用函数 $g(\tau)=\beta^{-1}(\exp (\beta \tau)-1)$ 引导的优化的确定性等价。问题 (5.5) 的目标函数在 $\tau$ 上是强制性的，并且在 $\tau^{\star}=\beta^{-1} \log \left(\mathbb{E}_{\mathbb{P}}[\exp (\beta \ell(Z))]\right)$ 处达到最小值。

**证明**

根据 $g$ 的定义，我们有：

$$
\inf _{\tau \in \mathbb{R}} \tau+\mathbb{E}_{\mathbb{P}}[g(\ell(Z)-\tau)]=\inf _{\tau \in \mathbb{R}} \tau+\frac{1}{\beta} \mathbb{E}_{\mathbb{P}}[\exp (\beta(\ell(Z)-\tau))-1]
$$

$$
=\frac{1}{\beta} \log \mathbb{E}_{\mathbb{P}}[\exp (\beta \ell(Z))]=\beta-\mathrm{ERM}_{\mathbb{P}}[\ell(Z)]。
$$

第二个等号成立是因为该无约束凸最小化问题在 $\tau$ 处唯一解为 $\tau^{\star}=\beta^{-1} \log \left(\mathbb{E}_{\mathbb{P}}[\exp (\beta \ell(Z))]\right)$，这一点可以通过检查问题的一阶最优条件验证。此外，由于 $\beta \in (0,1)$，很明显问题的目标函数在 $\tau$ 上是强制性的。

Kupper 和 Schachermayer (2009) 证明，除了期望值外，熵风险度量是唯一一个满足塔性质的相关分布不变风险度量。即，对于任何随机向量 $Z_1$ 和 $Z_2$，它满足：

$$
\beta-\operatorname{ERM}_{\mathbb{P}}\left[\ell\left(Z_2\right)\right]=\beta-\operatorname{ERM}_{\mathbb{P}}\left[\beta-\operatorname{ERM}_{\mathbb{P}}\left[\ell\left(Z_2\right) \mid Z_1\right]\right],
$$

其中条件熵风险度量 $\beta$-ERM $\mathrm{M}_{\mathbb{P}}\left[Z_1 \mid Z_2\right]$ 通过将 (5.8) 中的无条件期望替换为条件期望定义。熵风险度量经常用于动态优化问题中的风险厌恶建模，其中在不同时间点采取的决策的一致性是一个问题。例如，它在金融学 (Föllmer 和 Schied 2008)、风险敏感控制 (Whittle 1990, Başar 和 Bernhard 1995) 和经济学 (Hansen 和 Sargent 2008) 中占据核心地位。

**命题 5.14（熵风险度量的对偶表示）**

假设 $\mathbb{E}_{\hat{p}}[\ell(Z)]>-\infty$。则熵风险度量具有以下对偶表示：

$$
\beta-\operatorname{ERM}_{\hat{\mathbb{P}}}[\ell(Z)]=\sup _{\mathbb{P} \in \mathcal{P}(\mathcal{Z})} \mathbb{E}_{\mathbb{P}}[\ell(Z)]-\frac{1}{\beta} \cdot \operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}})。
$$

**证明**

令 $\phi$ 是 Kullback-Leibler 散度的熵函数。因此，对于所有 $t \in \mathbb{R}$，有 $\phi^*(t)=\mathrm{e}^t-1$；见表 4.1。根据命题 5.13，熵风险值等价于由以下不效用函数引导的优化确定性等价：

$$
g(t)=\beta^{-1}(\exp (\beta t)-1)=\beta^{-1} \phi^*(\beta t)=\left(\beta^{-1} \phi\right)^*(t),
$$

最后一个等号应用了 (Rockafellar 1970, Theorem 16.1)。由此可得：

$$
\begin{aligned}
\beta-\operatorname{ERM}_{\hat{\mathbb{P}}}[\ell(Z)] & =\inf _{\tau \in \mathbb{R}} \tau+\mathbb{E}_{\mathbb{P}}\left[\left(\beta^{-1} \phi\right)^*(\ell(Z)-\tau)\right] \\
& =\sup _{\mathbb{P} \in \mathcal{P}(\mathcal{Z})} \mathbb{E}_{\mathbb{P}}[\ell(Z)]-\mathrm{D}_{\beta^{-1} \phi}(\mathbb{P}, \hat{\mathbb{P}}) \\
& =\sup _{\mathbb{P} \in \mathcal{P}(\mathcal{Z})} \mathbb{E}_{\mathbb{P}}[\ell(Z)]-\beta^{-1} \operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}})。
\end{aligned}
$$

这里，第二个等号来源于强对偶关系 (5.6)，这一关系适用是因为 $\mathbb{E}_{\hat{p}}[\ell(Z)]>-\infty$，而第三个等号成立是因为假设熵函数 $\phi$ 引导 Kullback-Leibler 散度。

我们注意到，命题 5.14 也可以通过利用命题 2.9 中的 Donsker-Varadhan 公式代替对偶关系 (5.6) 来证明。可以证明，每个优化的确定性等价 $\varrho$ 都是平移不变且凸的。如果基础的不效用函数 $g$ 是单调递增的，则 $\varrho$ 也是单调的；如果 $g$ 是正齐次的，则 $\varrho$ 也是尺度不变的。

接下来我们将展示，如果 $\varrho$ 是任意优化的确定性等价，那么最坏风险问题 (5.4) 可以被简化为形式 (4.1) 的最坏期望问题。这种简化基于下面要推导的不对称极小极大定理，并使我们能够将第 4 节中关于最坏期望问题的对偶理论扩展到一类丰富的最坏风险问题。



## 5.2. 不对称极小极大定理

一个通用的极小极大问题可以表示为：

$$
\inf _{u \in \mathcal{U}} \sup _{v \in \mathcal{V}} H(u, v),
$$

其中 $\mathcal{U}$ 和 $\mathcal{V}$ 是任意的空间，而 $H: \mathcal{U} \times \mathcal{V} \rightarrow \overline{\mathbb{R}}$ 是一个任意函数。极小极大定理提供了在不改变问题最优值的情况下交换极小与极大的条件。以下的极小极大定理受到 (Rockafellar 1974, Example 13) 的启发，对于用优化确定性等价解决最坏风险问题至关重要。回忆第 4.1 节中提到的定义：如果一个凸函数是适当的且下半连续的，或恒等于 $-\infty$，则它是闭的。

**定理 5.15（不对称极小极大定理）**

假设 $\mathcal{U}$ 是一个任意向量空间，$\mathcal{V}$ 是一个局部凸拓扑向量空间。还假设函数 $H: \mathcal{U} \times \mathcal{V} \rightarrow \overline{\mathbb{R}}$ 满足以下条件：

1. $H(u, v)$ 关于 $u$ 是凸的；
2. $-H(u, v)$ 关于 $v$ 是凸的且闭的；
3. $\sup _{v \in \mathcal{V}} \inf _{u \in \mathcal{U}} H(u, v)>-\infty$；
4. 对于每个 $\alpha \in \mathbb{R}$，存在 $u \in \mathcal{U}$ 使得 $\{v \in \mathcal{V}: H(u, v) \geq \alpha\}$ 是紧集。

则有：

$$
\inf _{u \in \mathcal{U}} \sup _{v \in \mathcal{V}} H(u, v)=\sup _{v \in \mathcal{V}} \inf _{u \in \mathcal{U}} H(u, v)。
$$

**证明**

令 $\mathcal{V}^*$ 为 $\mathcal{V}$ 的拓扑对偶，定义双线性形式 $\langle\cdot, \cdot\rangle$ : $\mathcal{V}^* \times \mathcal{V} \rightarrow \mathbb{R}$，通过 $\left\langle v^*, v\right\rangle=v^*(v)$ 给出。如果我们用 $\mathcal{V}$ 引出的弱拓扑装备 $\mathcal{V}^*$，则对于每个 $v \in \mathcal{V}$，$\langle\cdot, v\rangle$ 是 $\mathcal{V}^*$ 上的连续线性泛函，并且每个连续线性泛函都可以这样表示。

定义 $F: \mathcal{U} \times \mathcal{V}^* \rightarrow \overline{\mathbb{R}}$：

$$
F\left(u, v^*\right)=\sup _{v \in \mathcal{V}} H(u, v)-\left\langle v^*, v\right\rangle，
$$

由于引理 4.1，$F(u, v^*)$ 关于 $u$ 和 $v^*$ 联合凸。因此，

$$
F\left(u, v^*\right)=(-H)^*\left(u,-v^*\right)，
$$

这里 $-H(u, v)$ 的共轭是关于其第二个变量 $v$ 求的。由于 $-H(u, v)$ 关于 $v$ 是凸的且闭的，根据引理 4.2，可得：

$$
F^*(u, v)=-H(u,-v)。
$$

这里再次注意，共轭是关于 $v^*$ 求的。此外，定义 $h: \mathcal{V}^* \rightarrow \overline{\mathbb{R}}$：

$$
h\left(v^*\right)=\inf _{u \in \mathcal{U}} F\left(u, v^*\right)，
$$

则 $h(v^*)$ 关于 $v^*$ 是凸的。因此，我们有：

$$
h(0)=\inf _{u \in \mathcal{U}} F(u, 0)=\inf _{u \in \mathcal{U}} \sup _{v \in \mathcal{V}} H(u, v)，
$$

两个等号分别来自 $h$ 和 $F$ 的定义。此外，我们也有：

$$
\begin{aligned}
h^{* *}(0) & =\sup _{v \in \mathcal{V}}-h^*(-v)=\sup _{v \in \mathcal{V}} \inf _{v^* \in \mathcal{V}^*}\left\langle v^*, v\right\rangle+h\left(v^*\right) \\
& =\sup _{v \in \mathcal{V}} \inf _{u \in \mathcal{U}} \inf _{v^* \in \mathcal{V}^*}\left\langle v^*, v\right\rangle+F\left(u, v^*\right) \\
& =\sup _{v \in \mathcal{V}} \inf _{u \in \mathcal{U}}-F^*(u,-v)=\sup _{v \in \mathcal{V}} \inf _{u \in \mathcal{U}} H(u, v)，
\end{aligned}
$$

其中，第一个等号来源于双共轭 $h^{**}$ 和共轭 $h^*$ 的定义，第三个等号利用了 $h$ 的定义。第四个等号来源于 $F^*$ 的定义，最后一个等号成立是因为 $F^*(u, v)=-H(u,-v)$。因此，如果我们能证明 $h(0)=h^{* *}(0)$，则证明成立。

根据 $h^*$ 和 $h$ 的定义以及 $F^*(u, v)=-H(u,-v)$ 的关系，我们有：

$$
\begin{aligned}
\{v \in \mathcal{V}: h^*(v) \leq \alpha\} & =-\bigcap_{u \in \mathcal{U}}\{v \in \mathcal{V}: H(u, v) \geq-\alpha\}。
\end{aligned}
$$

因此，根据 (Rockafellar 1974, Theorem 10 (b))，可以得出 $h(0)=h^{* *}(0)$，从而证明了极小极大结果。

**推论 5.16（逆向不对称极小极大定理）**

假设 $\mathcal{U}$ 是一个局部凸拓扑向量空间，$\mathcal{V}$ 是一个任意向量空间。若 $H(u, v)$ 关于 $u$ 是凸的且闭的，而关于 $v$ 是凸的，并满足：

1. $\inf _{u \in \mathcal{U}} \sup _{v \in \mathcal{V}} H(u, v)<\infty$；
2. 对于每个 $\alpha \in \mathbb{R}$，存在 $v \in \mathcal{V}$ 使得 $\{u \in \mathcal{U}: H(u, v) \leq \alpha\}$ 是紧集。

则有：

$$
\inf _{u \in \mathcal{U}} \sup _{v \in \mathcal{V}} H(u, v)=\sup _{v \in \mathcal{V}} \inf _{u \in \mathcal{U}} H(u, v)。
$$

**引理 5.17（下紧性）**

假设 $\mathcal{U}$ 是一个欧几里得空间，$H: \mathcal{U} \times \mathcal{V} \rightarrow \overline{\mathbb{R}}$ 在其第一个变量上是下半连续的且强制性的。那么，对于任意 $v \in \mathcal{V}$ 和 $\alpha \in \mathbb{R}$，子水平集 $\{u \in \mathcal{U}: H(u, v) \leq \alpha\}$ 是紧集。

**证明**

为了证明子水平集 $\mathcal{U}_\alpha(v)=\{u \in \mathcal{U}: H(u, v) \leq \alpha\}$ 是紧的，首先注意到由于 $H(u, v)$ 关于 $u$ 是下半连续的，$\mathcal{U}_\alpha(v)$ 是闭的。

为了证明 $\mathcal{U}_\alpha(v)$ 是有界的，假设相反的情况：存在一个序列 $\{u_i\}_{i \in \mathbb{N}} \in \mathcal{U}_\alpha(v)$，满足 $\lim_{i \to \infty}\|u_i\|_2 = \infty$。由于 $H(u, v)$ 在 $u$ 上是强制性的，我们有 $\lim_{i \to \infty} H(u_i, v) = \infty$。然而，这与假设 $H(u_i, v) \leq \alpha$ 对所有 $i \in \mathbb{N}$ 相矛盾。因此，$\mathcal{U}_\alpha(v)$ 必须是有界的，并且是紧的。

---

**推论**

如果 $H_0: \mathcal{U}_0 \times \mathcal{V}_0 \rightarrow \overline{\mathbb{R}}$ 定义在凸集 $\mathcal{U}_0 \subseteq \mathcal{U}$ 和 $\mathcal{V}_0 \subseteq \mathcal{V}$ 上，那么它可以扩展为基础向量空间 $\mathcal{U}$ 和 $\mathcal{V}$ 上的函数 $H: \mathcal{U} \times \mathcal{V} \rightarrow \overline{\mathbb{R}}$，定义为：

$$
H(u, v)= 
\begin{cases} 
H_0(u, v) & \text{若 } u \in \mathcal{U}_0 \text{ 且 } v \in \mathcal{V}_0, \\ 
+\infty & \text{若 } u \notin \mathcal{U}_0 \text{ 且 } v \in \mathcal{V}_0, \\ 
-\infty & \text{若 } v \notin \mathcal{V}_0。
\end{cases}
$$

这种构造保证了：

$$
\inf _{u \in \mathcal{U}} \sup _{v \in \mathcal{V}} H(u, v) = \inf _{u \in \mathcal{U}_0} \sup _{v \in \mathcal{V}_0} H_0(u, v)，
$$

以及：

$$
\sup _{v \in \mathcal{V}} \inf _{u \in \mathcal{U}} H(u, v) = \sup _{v \in \mathcal{V}_0} \inf _{u \in \mathcal{U}_0} H_0(u, v)。
$$

同时，如果 $H_0(u, v)$ 关于 $u$ 是凸的且闭的，关于 $v$ 是凹的，那么扩展后的 $H(u, v)$ 也满足这些性质。因此，任何凸-凹极小极大问题的可行集可以扩展到基础向量空间而不会改变问题的最优值。

---

**定理 5.18（优化确定性等价的极小极大定理）**

假设 $\mathcal{P} \subseteq \mathcal{P}(\mathcal{Z})$ 是非空且凸的，$\varrho$ 是由失效函数 $g$ 引导的优化确定性等价，$\sup_{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[g(\ell(Z))] < \infty$，且对所有 $\mathbb{P} \in \mathcal{P}$，有 $\mathbb{E}_{\mathbb{P}}[\ell(Z)] > -\infty$。定义：

$$
G(\tau, \mathbb{P}) = \tau + \mathbb{E}_{\mathbb{P}}[g(\ell(Z) - \tau)]，
$$

其中 $\tau \in \mathbb{R}$，$\mathbb{P} \in \mathcal{P}$。则满足：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \varrho_{\mathbb{P}}[\ell(Z)] = \sup _{\mathbb{P} \in \mathcal{P}} \inf _{\tau \in \mathbb{R}} G(\tau, \mathbb{P}) = \inf _{\tau \in \mathbb{R}} \sup _{\mathbb{P} \in \mathcal{P}} G(\tau, \mathbb{P})。
$$

**证明**

首先，$G(\tau, \mathbb{P})$ 关于 $\tau$ 是凸的，关于 $\mathbb{P}$ 是凹的（事实上是线性的）。此外，$G(\tau, \mathbb{P})$ 关于 $\tau$ 是闭的。这可以通过观察下极限证明：

$$
\liminf_{\tau' \to \tau} G(\tau', \mathbb{P}) = \liminf_{\tau' \to \tau} \mathbb{E}_{\mathbb{P}}[\tau' + g(\ell(Z) - \tau')] \geq \mathbb{E}_{\mathbb{P}}[\tau + g(\ell(Z) - \tau)] = G(\tau, \mathbb{P})。
$$

不等式的第一部分由 Fatou 引理得到，第二部分则利用了 $g$ 的连续性。Fatou 引理适用是因为任何失效函数都满足 $g(\tau) \geq \tau$ 对所有 $\tau \in \mathbb{R}$，这意味着 $\tau + g(\ell(z) - \tau) \geq \ell(z)$ 对所有 $z \in \mathcal{Z}$ 和 $\tau \in \mathbb{R}$。

其次，$G(\tau, \mathbb{P})$ 关于 $\tau$ 是下紧的。这可以通过 $g$ 的定义推导出：$g(\tau)$ 增长速度快于 $\tau$。因此，可以构造 $\varepsilon > 0$，使得：

$$
G(\tau, \mathbb{P}) \geq -\varepsilon \tau + (1+\varepsilon)\mathbb{E}_{\mathbb{P}}[\ell(Z)] - 1，
$$

并且：

$$
G(\tau, \mathbb{P}) \geq \varepsilon \tau + (1-\varepsilon)\mathbb{E}_{\mathbb{P}}[\ell(Z)] - 1。
$$

由此可得，$\{\tau \in \mathbb{R}: G(\tau, \mathbb{P}) \leq \alpha\}$ 是紧集。

最后，结合定理 5.15 和上述性质，得出结论。
**定理 5.18 的推论**

定理 5.18 意味着，对于任意 $\beta \in (0, 1)$，最坏情况下的 $\beta$-CVaR 满足：

$$
\sup_{\mathbb{P} \in \mathcal{P}} \beta-\operatorname{CVaR}_{\mathbb{P}}[\ell(Z)] = \inf_{\tau \in \mathbb{R}} \tau + \frac{1}{\beta} \sup_{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\max\{\ell(Z) - \tau, 0\}]
$$

其中，$\mathcal{P} \subseteq \mathcal{P}(\mathcal{Z})$ 是非空凸的不确定性集合，并假设对所有 $\mathbb{P} \in \mathcal{P}$，有 $\mathbb{E}_{\mathbb{P}}[|\ell(Z)|] < \infty$。

在现有文献中，$\mathbb{P}$ 的 supremum 和 $\tau$ 的 infimum 的交换通常通过 Sion 的极小极大定理（Sion 1958）来证明。然而，许多研究忽视了 Sion 的极小极大定理仅适用于以下情况：$\mathcal{P}$ 是弱紧的，并且 $\mathbb{E}_{\mathbb{P}}[\max\{\ell(Z) - \tau, 0\}]$ 在 $\mathbb{P}$ 上是弱上半连续的。

然而，正如第 3 节所示，许多流行的不确定性集合并不满足弱紧性。此外，除非损失函数 $\ell$ 在 $\mathcal{Z}$ 上是上半连续且有界的，否则 $\mathbb{E}_{\mathbb{P}}[\max\{\ell(Z) - \tau, 0\}]$ 并不是弱上半连续的；详见命题 3.3。所有非平凡的 $\mathbb{R}^d$ 上的凸损失函数都违反了这一条件。

相比之下，定理 5.18 提供了一个更广泛的结果。该定理利用了 $\tau$ 的下紧性，而避免了对 $\mathcal{P}$ 或 $\ell$ 施加任何严格的拓扑条件。


## 5.3 矩不确定性集合

回顾通用的矩不确定性集合 (2.1) 定义如下：

$$
\mathcal{P} = \left\{\mathbb{P} \in \mathcal{P}_f(\mathcal{Z}): \mathbb{E}_{\mathbb{P}}[f(Z)] \in \mathcal{F}\right\},
$$

其中，$\mathcal{Z} \subseteq \mathbb{R}^d$ 是一个非空闭支持集，$f: \mathcal{Z} \rightarrow \mathbb{R}^m$ 是一个 Borel 可测的矩函数，$\mathcal{F} \subseteq \mathbb{R}^m$ 是一个非空闭的矩不确定性集合，$\mathcal{P}_f(\mathcal{Z})$ 表示所有分布的集合 $\mathbb{P} \in \mathcal{P}(\mathcal{Z})$，对于它们，$\mathbb{E}_{\mathbb{P}}[f(Z)]$ 是有限的。同时，$\mathcal{C} = \left\{\mathbb{E}_{\mathbb{P}}[f(Z)]: \mathbb{P} \in \mathcal{P}_f(\mathcal{Z})\right\}$ 表示任意分布在 $\mathcal{Z}$ 上的所有可能矩的集合。接下来的定理为具有矩不确定性集合的最坏情况风险问题 (5.4) 提出了一个对偶性结果。

**定理 5.19（矩不确定性集合的对偶理论 II）**  
如果 $\mathcal{P}$ 是矩不确定性集合 (2.1)，并且 $\varrho$ 是由不满意度函数 $g$ 诱导的优化确定性等价，那么以下弱对偶关系成立：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \varrho_{\mathbb{P}}[\ell(Z)] \leq \begin{cases} 
\inf & \tau + \lambda_0 + \delta_{\mathcal{F}}^*(\lambda) \\ 
\text { s.t. } & \tau, \lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}^m \\ 
& \lambda_0 + f(z)^{\top} \lambda \geq g(\ell(z) - \tau) \forall z \in \mathcal{Z} 
\end{cases}
$$

如果 $\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[g(\ell(Z))] < \infty$, $\mathbb{E}_{\mathbb{P}}[\ell(Z)] > -\infty$ 对所有 $\mathbb{P} \in \mathcal{P}_f(\mathcal{Z})$ 成立，且 $\mathcal{F} \subseteq \mathcal{C}$ 是一个凸且紧的集合，并且 $\operatorname{rint}(\mathcal{F}) \subseteq \operatorname{rint}(\mathcal{C})$，则强对偶性成立，即 (5.10) 中的不等式变为等式。

**证明**  
极大极小不等式表明：

$$
\begin{aligned}
\sup _{\mathbb{P} \in \mathcal{P}} \varrho_{\mathbb{P}}[\ell(Z)] & = \sup _{\mathbb{P} \in \mathcal{P}} \inf _{\tau \in \mathbb{R}} \tau + \mathbb{E}_{\mathbb{P}}[g(\ell(Z) - \tau)] \\ 
& \leq \inf _{\tau \in \mathbb{R}} \sup _{\mathbb{P} \in \mathcal{P}} \tau + \mathbb{E}_{\mathbb{P}}[g(\ell(Z) - \tau)].
\end{aligned}
$$

上界中的内层极大化问题构成一个最坏情况期望问题。因此，其上界由定理 4.5 中推导的对偶问题界定。将该对偶问题代入上述表达式可得 (5.10)。强对偶性则由优化确定性等价的极小极大定理（定理 5.18）和最坏情况期望问题的强对偶性结果（定理 4.5）推导得出，这些结果在给定假设下适用。

**分析**  
(5.10) 中的半无限约束涉及复合函数 $g(\ell(z) - \tau)$，即使 $g$ 是单调递增的并且 $\ell$ 是凹的，该函数也并非对 $z$ 凸。因此，检查给定的 $\left(\tau, \lambda_0, \lambda\right)$ 是否满足 (5.10) 中的半无限约束在一般情况下是困难的。事实上，Chen 和 Sim (2024, Theorem 1) 证明，即使 $\ell$ 是线性的且 $\mathcal{P}$ 是一个马尔可夫不确定性集合，评估最坏情况熵风险仍然是 NP 难的。因此，尽管定理 5.18 提供了理论见解，但它未必能直接实现解决形式 (5.4) 最坏情况风险问题的高效方法。然而，定理 5.18 为 (5.4) 提供了一个简洁的重表述，这一表述有利于采用近似迭代的求解程序。


## 5.4：$\phi$-散度不确定性集合

回顾 $\phi$-散度不确定性集合 (2.10) 定义如下：

$$
\mathcal{P} = \left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\},
$$

其中，$\mathcal{Z}$ 是一个闭支持集，$r \geq 0$ 是一个大小参数，$\phi$ 是定义 2.4 中的熵函数，$\mathrm{D}_\phi$ 是定义 2.5 中对应的 $\phi$-散度，$\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 是一个参考分布。以下定理建立了针对 $\phi$-散度不确定性集合的最坏情况风险问题的对偶性结果。其证明基于定理 4.14 和 5.18，因此略去。

**定理 5.20（$\phi$-散度不确定性集合的对偶理论 II）**  
假设 $\mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)] > -\infty$。如果 $\mathcal{P}$ 是 $\phi$-散度不确定性集合 (2.10)，并且 $\varrho$ 是由不满意度函数 $g$ 诱导的优化确定性等价，则以下弱对偶关系成立：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \varrho_{\mathbb{P}}[\ell(Z)] \leq\left\{\begin{array}{cl}
\inf _{\tau, \lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}_{+}} & \tau+\lambda_0+\lambda r+\mathbb{E}_{\hat{\mathbb{P}}}\left[\left(\phi^*\right)^\pi\left(g(\ell(Z)-\tau)-\lambda_0, \lambda\right)\right] \\ 
\text { s.t. } & \lambda_0+\lambda \phi^{\infty}(1) \geq \sup _{z \in \mathcal{Z}} g(\ell(z)-\tau)
\end{array}\right.
$$

如果 $\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[g(\ell(Z))] < \infty$, $\mathbb{E}_{\mathbb{P}}[\ell(Z)] > -\infty$ 对所有 $\mathbb{P} \in \mathcal{P}$ 成立，且 $r > 0$ 并且 $\phi$ 在 1 处连续，则强对偶性成立，即不等式变为等式。

一个类似于定理 5.20 的对偶性结果也适用于限制性 $\phi$-散度不确定性集合的最坏情况风险问题，该集合形式为：

$$
\mathcal{P} = \left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{P} \ll \hat{\mathbb{P}}, \mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\}.
$$

以下定理的证明直接基于定理 4.15 和 5.18。

**定理 5.21（限制性 $\phi$-散度不确定性集合的对偶理论 II）**  
假设 $\mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)] > -\infty$。如果 $\mathcal{P}$ 是限制性 $\phi$-散度不确定性集合 (2.11)，并且 $\varrho$ 是由不满意度函数 $g$ 诱导的优化确定性等价，则以下弱对偶关系成立：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \varrho_{\mathbb{P}}[\ell(Z)] \leq \inf _{\tau, \lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}_{+}} \tau+\lambda_0+\lambda r+\mathbb{E}_{\hat{\mathbb{P}}}\left[\left(\phi^*\right)^\pi\left(g(\ell(Z)-\tau)-\lambda_0, \lambda\right)\right] .
$$

如果 $\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[g(\ell(Z))] < \infty$, $\mathbb{E}_{\mathbb{P}}[\ell(Z)] > -\infty$ 对所有 $\mathbb{P} \in \mathcal{P}$ 成立，且 $r > 0$ 并且 $\phi$ 在 1 处连续，则强对偶性成立，即不等式变为等式。

最坏情况风险问题与最优运输不确定性集合相关的研究包括：Pflug 和 Wozabal (2007)、Pichler (2013) 和 Wozabal (2014) 在投资组合选择中使用线性损失函数的背景下的研究，以及 Mohajerin Esfahani 等人 (2018) 在逆优化中使用 CVaR 的背景下的研究。Sadana、Delage 和 Georghiou (2024) 研究了 co-Wasserstein 球上的最坏情况熵风险度量，并在标准凸性假设下建立了可求解的重构。Kent、Li、Blanchet 和 Glynn (2021) 以及 Sheriff 和 Mohajerin Esfahani (2023) 开发了分布空间中解决最坏情况风险问题的定制 Frank-Wolfe 算法，这些问题涉及通用损失函数和风险度量。具体来说，Kent 等人 (2021) 研究了 Wasserstein 梯度流，并使用相应的平滑性概念建立了 Frank-Wolfe 算法的收敛性。相比之下，Sheriff 和 Mohajerin Esfahani (2023) 使用 Gateaux 导数，这导致了不同的平滑性定义以及不同的收敛性分析。这两种算法都显示出次线性收敛率。当参考分布 $P$ 是离散的，或者仅使用 $P$ 的样本时，算法的迭代步骤表示具有逐步增加比特大小的离散分布。定理 5.22 提供了一个简洁但可能非凸的最坏情况风险问题重构形式。这种重构适用于有限维度对偶变量空间中的原始-对偶梯度方法，并保证收敛到一个驻点。

最坏情况风险问题是概率分布空间优化问题的特殊实例。解决此类问题的主流方法利用了 Wasserstein 梯度流的工具 (Ambrosio、Gigli 和 Savare 2008)。Wasserstein 梯度流最近被用于分布式鲁棒优化问题 (Lanzetti、Bolognani 和 Dörfler 2022，Lanzetti、Terpin 和 Dörfler 2024，Xu、Lee、Cheng 和 Xie 2024)、非凸优化 (Chizat 和 Bach 2018，Chizat 2022) 或变分推断 (Jiang、Chewi 和 Pooladian 2024，Lambert、Chewi、Bach、Bonnabel 和 Rigollet 2022，Diao、Balasubramanian、Chewi 和 Salim 2023，Zhang 和 Zhou 2020)。本节的结果是这些现有研究的补充并具有新意。



# 6. 分析求解自然的子问题

在分布式鲁棒优化（DRO）中，一个关键挑战是处理嵌套在公式 (1.2) 中的最坏情况期望问题。这个问题由虚构的对手（通常被认为是自然界）解决，当决策者选择了一个 $x \in \mathcal{X}$ 时，它会最大化一个线性函数在无限维度测度空间的凸子集上。因此，这个问题看似难以求解。因此，大量研究致力于识别使该问题能够高效求解的条件。我们现在将展示，在一些有趣的情况下，该问题实际上可以被解析地解决。

第 4 节中推导的对偶理论为解析地求解自然子问题提供了以下简单策略：构造原始最坏情况期望问题和其对偶问题的可行解，并证明它们的目标函数值相等。如果能够找到这样的匹配解，根据弱对偶性原理，这两个解必然分别是各自优化问题的最优解。正如下文所见，这种简单的策略意外地频繁成功。此外，我们还将看到，最坏情况期望问题的解析解有时可以推广为最坏情况风险问题（公式 (5.4)）的解析解。本节回顾的内容涵盖了从 1950 年代到现在的 DRO 研究的多个重要进展。

## 6.1. Jensen 界

考虑以下最坏情况期望问题：

$$
\sup _{\mathbb{P} \in \mathcal{P}(\mathcal{Z})}\left\{\mathbb{E}_{\mathbb{P}}[\ell(Z)]: \mathbb{B}_{\mathbb{P}}[Z]=\mu\right\},
$$

该问题最大化 $\ell(Z)$ 在 Markov 不确定性集合上的期望值，所有分布的支撑集为 $\mathcal{Z}$ 且均值为 $\mu$。Markov 不确定性集合是一个形式为 (2.1) 的矩不确定性集合，其中 $f(z)=z$ 且 $\mathcal{F}=\{\mu\}$。根据定理 4.5，由于 $\mathcal{F}$ 的支撑函数是线性的，问题 (6.1a) 的对偶问题如下：

$$
\inf _{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}^d}\left\{\lambda_0+\lambda^{\top} \mu: \lambda_0+\lambda^{\top} z \geq \ell(z) \forall z \in \mathcal{Z}\right\} .
$$

直观上，这个对偶问题旨在找到一个仿射函数 $a(z)=\lambda_0+\lambda^{\top} z$，它在 $\mathcal{Z}$ 上优于损失函数 $\ell(z)$ 且在原始问题 (6.1a) 的所有可行分布 $\mathbb{P}$ 下期望值 $\mathbb{B}_{\mathbb{P}}[a(Z)]$ 最小。

**命题 6.1 (Jensen 界)** 假设 $\mathcal{Z}$ 是凸集，$\mu \in \mathcal{Z}$，$\ell$ 是凹函数，$\lambda^{\star}$ 是 $\ell$ 在 $\mu$ 处的任意超梯度。则原始问题 (6.1a) 的解为 $\mathbb{P}^{\star}=\delta_\mu$，对偶问题 (6.1b) 的解为 $\left(\lambda_0^{\star}, \lambda^{\star}\right)$，其中 $\lambda_0^{\star}=\ell(\mu)-\mu^{\top} \lambda^{\star}$。此外，(6.1a) 和 (6.1b) 的最优值均为 $\ell(\mu)$。

**证明** 通过构造，$\mathbb{P}^{\star}$ 在原始最坏情况期望问题中是可行的，其目标函数值为 $\ell(\mu)$。此外，$\left(\lambda_0^{\star}, \lambda^{\star}\right)$ 在对偶鲁棒优化问题中是可行的，因为 $\lambda^{\star}$ 是 $\ell$ 在 $\mu$ 处的超梯度，其目标函数值也为 $\ell(\mu)$。因此，根据定理 4.5 中的弱对偶性，$\mathbb{P}^{\star}$ 是原始问题的最优解，而 $\left(\lambda_0^{\star}, \lambda^{\star}\right)$ 是对偶问题的最优解。

命题 6.1 意味着 Jensen 不等式 $\mathbb{E}_{\mathrm{P}}[\ell(Z)] \leq \mathbb{E}_{\mathrm{P} \star}[\ell(Z)]=\ell\left(\mathbb{E}_{\mathrm{P}}[Z]\right)$ 对所有 (6.1a) 的可行分布 $\mathbb{P}$ 都成立（Jensen 1906）。命题 6.1 进一步表明，(6.1b) 的解为任何在 $\mu$ 处与 $\ell$ 相切的仿射函数。

如果 DRO 问题 (1.2) 中的损失函数 $\ell(x, z)$ 对于任意固定 $x \in \mathcal{X}$ 在 $z$ 上是凹的，那么命题 6.1 表明，对于每个 $x \in \mathcal{X}$，相同的分布 $\mathbb{P}^{\star}$ 解决了公式 (1.2) 中的内层最大化问题。因此，DRO 问题 (1.2) 简化为非鲁棒随机规划 $\inf _{\mathcal{X}} \mathbb{E}_{\mathbb{P}^{\star}}[\ell(x, Z)]$。

Jensen 不等式传统上用于近似难以求解的随机优化问题 $\inf _{x \in \mathcal{X}} \mathbb{E}_{\mathbb{P}}[\ell(x, Z)]$，其中 $\mathbb{P}$ 是 $Z$ 的已知连续分布。命题 6.1 表明，如果 $\ell(x, z)$ 对于任意 $x \in \mathcal{X}$ 在 $z$ 上是凹的，那么将 $\mathbb{P}$ 替换为 $\mathbb{P}^{\star}=\delta_{\mathrm{E}_{\mathrm{P}}[Z]}$ 会导致该随机规划的保守近似。由于 $\mathbb{P}^{\star}$ 是离散的（实际上是一个 Dirac 分布），所得的近似问题更易于求解。可以通过将 $\mathcal{Z}$ 划分为有限多个凸单元并为所有单元构造单独的 Jensen 界来提高近似质量（Birge 和 Louveaux 2011，第 10.1 节）。


## 6.2. Edmundson-Madansky 界

当 $\ell$ 是凸函数且 $\mathcal{Z}$ 是一个单纯形时，Markov 不确定性集合上的最坏情况期望问题 (6.1a) 及其对偶问题 (6.1b) 也可以解析求解。

**命题 6.2 (Edmundson-Madansky 界)**  
假设 $\mathcal{Z}$ 是 $\mathbb{R}^d$ 中的概率单纯形，其顶点为 $e_i, i \in[d]$，$\mu \in \operatorname{rint}(\mathcal{Z})$，$\ell$ 是凸的实值函数。那么，原始问题 (6.1a) 的解为 $\mathbb{P}^{\star}=\sum_{i=1}^d \mu_i \delta_{e_i}$，对偶问题 (6.1b) 的解为 $\left(\lambda_0^{\star}, \lambda^{\star}\right)$，其中 $\lambda_0^{\star}=0$ 且 $\lambda_i^{\star}=\ell\left(e_i\right)$ 对所有 $i \in[d]$ 成立。此外，(6.1a) 和 (6.1b) 的最优值均为 $\sum_{i=1}^d \mathbb{B}_{\mathbb{P}}\left[Z_i\right] \ell\left(e_i\right)$。

**证明**  
由于 $\mu$ 属于概率单纯形，$\mathbb{P}^{\star}$ 在原始最坏情况期望问题中是可行的，其目标函数值为 $\sum_{i=1}^d \mu_i \ell\left(e_i\right)$。同时，由于 $\ell$ 是凸函数，Jensen 不等式表明：

$$
\lambda_0^{\star}+z^{\top} \lambda^{\star}=\sum_{i=1}^d z_i \ell\left(e_i\right) \geq \ell\left(\sum_{i=1}^d z_i e_i\right)=\ell(z) \quad \forall z \in \mathcal{Z} .
$$

我们得出 $\left(\lambda_0^{\star}, \lambda^{\star}\right)$ 在对偶鲁棒优化问题中是可行的，其目标函数值也为 $\sum_{i=1}^d \mu_i \ell\left(e_i\right)$。因此，根据定理 4.5 中的弱对偶性，$\mathbb{P}^{\star}$ 是原始问题的最优解，而 $\left(\lambda_0^{\star}, \lambda^{\star}\right)$ 是对偶问题的最优解。

命题 6.2 表明了 Edmundson-Madansky 不等式，即对于所有 (6.1a) 中的可行分布 $\mathbb{P}$，都有：

$$
\mathbb{E}_{\mathbb{P}}[\ell(Z)] \leq \mathbb{E}_{\mathbb{P}^{\star}}[\ell(Z)]=\sum_{i=1}^d \mathbb{E}_{\mathbb{P}}\left[Z_i\right] \ell\left(e_i\right) .
$$

（见 Edmundson 1956, Madansky 1959）。此外，该命题还表明 (6.1b) 的解为一个在顶点 $e_i, i \in[d]$ 处与 $\ell$ 相切的仿射函数。

需要强调的是，如果 $\mathcal{Z}$ 是 $\mathbb{R}^d$ 中的任意正则单纯形，即由 $d+1$ 个仿射独立向量 $v_i \in \mathbb{R}^d, i \in[d+1]$ 的凸包构成，则命题 6.2 仍然成立，仅需少量修改（参见 Birge 和 Wets 1986, Gassmann 和 Ziemba 1986）。

如果 (1.2) 中的损失函数 $\ell(x, z)$ 对于任意固定 $x \in \mathcal{X}$ 是 $z$ 的凸函数，则命题 6.2 表明，DRO 问题 (1.2) 等价于随机规划 $\inf _{x \in \mathcal{X}} \mathbb{E}_{\mathbb{P}^{\star}}[\ell(x, Z)]$，其中 $\mathbb{P}^{\star}$ 与 $x$ 无关。由于 $\mathbb{P}^{\star}$ 是具有 $d$ 个原子点的离散分布，这个随机规划通常易于求解。


## 6.3. 重心逼近

考虑如下最坏情况期望问题：

$$
\sup _{\mathbb{P} \in \mathcal{P}(\mathcal{V} \times \mathcal{W})}\left\{\mathbb{E}_{\mathrm{P}}[\ell(V, W)]: \mathbb{E}_{\mathrm{P}}[V]=\bar{v}, \mathbb{E}_{\mathrm{P}}[W]=\bar{w}, \mathbb{E}_{\mathrm{P}}\left[V W^{\top}\right]=C\right\},
$$

该问题最大化 $\ell(V, W)$ 在 $Z=(V, W)$ 的所有分布上的期望，这些分布的 $V$ 和 $W$ 分别具有均值向量 $\bar{v}$ 和 $\bar{w}$，且具有交叉矩矩阵 $C$。注意，如果 $V$ 和 $W$ 不相关，则 $C=\bar{v} \bar{w}^{\top}$。问题 (6.2a) 在形式上优化了一个 (2.1) 的矩不确定性集合，其中 $f(v, w)=\left(v, w, v w^{\top}\right)$ 且 $\mathcal{F}=\{\bar{v}\} \times\{\bar{w}\} \times\{C\}$。根据定理 4.5 并且由于 $\mathcal{F}$ 的支撑函数是线性的，(6.2a) 的对偶问题为：

$$
\begin{array}{ll}
\text { inf } & \lambda_0+\lambda_v^{\top} \bar{v}+\lambda_w^{\top} \bar{w}+\langle\Lambda, C\rangle \\
\text { s.t. } & \lambda_0 \in \mathbb{R}, \lambda_v \in \mathbb{R}^{d_v}, \lambda_w \in \mathbb{R}^{d_w}, \Lambda \in \mathbb{R}^{d_v \times d_w} \\
& \lambda_0+\lambda_v^{\top} v+\lambda_w^{\top} w+v^{\top} \Lambda w \geq \ell(v, w) \quad \forall v \in \mathcal{V}, \forall w \in \mathcal{W} .
\end{array}
$$

该对偶问题寻找一个双仿射函数 $b(v, w)=\lambda_0+\lambda_v^{\top} v+\lambda_w^{\top} w+v^{\top} \Lambda w$，使其在 $\mathcal{V} \times \mathcal{W}$ 上优于损失函数 $\ell(v, w)$，并且在 (6.2a) 中所有可行分布 $\mathbb{P}$ 下最小化 $\mathbb{E}_{\mathrm{P}}[b(V, W)]$。以下命题表明，当 $\ell$ 是凹-凸鞍函数且 $\mathcal{W}$ 是一个单纯形时，问题 (6.2a) 和 (6.2b) 可解析求解。

**命题 6.3 (重心逼近)**  
假设 $\mathcal{V} \subseteq \mathbb{R}^{d_v}$ 是凸集，$\mathcal{W} \subseteq \mathbb{R}^{d_w}$ 是顶点为 $e_i, i \in\left[d_w\right]$ 的概率单纯形。进一步假设损失函数 $\ell(v, w)$ 对于任意固定的 $w$ 关于 $v$ 凹且可超微分，对任意固定的 $v$ 关于 $w$ 凸。此外，假设 $\bar{v} \in \mathcal{V}, \bar{w} \in \operatorname{rint}(\mathcal{W})$，$C e=\bar{v}$ 且问题 (6.2a) 是可行的。那么 (6.2a) 的解为：

$$
\mathbb{P}^{\star}=\sum_{i=1}^{d_w} \bar{w}_i \delta_{\left(C e_i / \bar{w}_i, e_i\right)} .
$$

若 $\Lambda_i^{\star}$ 是 $\partial_v \ell\left(C e_i / \bar{w}_i, e_i\right)$ 中的任一超梯度，且

$$
\lambda_{w, i}^{\star}=\ell\left(C e_i / \bar{w}_i, e_i\right)-\left(\Lambda_i^{\star}\right)^{\top} C e_i / \bar{w}_i \quad \forall i \in\left[d_w\right],
$$

则对偶问题 (6.2b) 的解为 $\left(\lambda_0^{\star}, \lambda_v^{\star}, \lambda_w^{\star}, \Lambda^{\star}\right)$，其中 $\lambda_0^{\star}=0$ 且 $\lambda_v^{\star}=0$，$\lambda_w^{\star}$ 的元素为 $\lambda_{w, i}^{\star}$，$\Lambda^{\star}$ 的列为 $\Lambda_i^{\star}, i \in\left[d_w\right]$。(6.2a) 和 (6.2b) 的最优值相等，均为：

$$
\sum_{i=1}^{d_w} \mu_{w, i} \ell\left(C e_i / \bar{w}_i, e_i\right) .
$$

**证明**  
由于 $\bar{w}>0$ 且 $e^{\top} \bar{w}=1$，$\bar{w}$ 属于概率单纯形 $\mathcal{W}$ 的相对内部，因此 $\mathbb{P}^{\star}$ 是一个定义良好的概率分布，即 $\mathbb{P}^{\star}$ 的原子具有正概率且概率和为 1。此外，由于：

$$
C e_i / \bar{w}_i \in \mathcal{V}, \quad e_i \in \mathcal{W} \quad \forall i \in\left[d_w\right],
$$

因此 $\mathbb{P}^{\star}$ 支持于 $\mathcal{V} \times \mathcal{W}$。进一步证明和推导过程略。

若 $z=(v, w)$ 且 (1.2) 中的损失函数 $\ell(x, z)=\ell(x, v, w)$ 对任意固定的 $x \in \mathcal{X}$ 关于 $v$ 凹且关于 $w$ 凸，则命题 6.3 表明，DRO 问题 (1.2) 等价于随机规划：

$$
\inf _{x \in \mathcal{X}} \mathbb{E}_{\mathbb{P}^{\star}}[\ell(x, U, V)],
$$

其中 $\mathbb{P}^{\star}$ 与 $x$ 无关。由于 $\mathbb{P}^{\star}$ 是具有 $d_w$ 个原子点的离散分布，该随机规划通常易于求解。传统上，分布 $\mathbb{P}^{\star}$ 被用来近似如下困难的随机优化问题：

$$
\inf _{x \in \mathcal{X}} \mathbb{E}_{\mathbb{P}}[\ell(x, V, W)] ,
$$

其中 $\mathbb{P}$ 是 $(V, W)$ 的已知连续分布。命题 6.3 表明，如果 $\ell(x, v, w)$ 对任意 $x \in \mathcal{X}$ 关于 $v$ 凹且关于 $w$ 凸，那么将 $\mathbb{P}$ 替换为 $\mathbb{P}^{\star}$ 会导致一个保守的近似，称为原始随机规划的**上重心逼近** (Frauendorfer 1992)。更一般的涉及非凸和/或非凹损失函数的随机规划的重心逼近可参见 Kuhn (2005)。


## 6.4. Ben-Tal 和 Hochman 界

考虑如下最坏情况期望问题：

$$
\sup _{\mathbb{P} \in \mathcal{P}(\mathcal{Z})}\left\{\mathbb{E}_P[\ell(Z)]: \mathbb{E}_{\mathbb{P}}[Z]=\mu, \mathbb{B}_P[|Z-\mu|]=\sigma\right\},
$$

该问题在所有支持集为 $\mathcal{Z}$，均值为 $\mu$，且均绝对偏差为 $\sigma$ 的单变量分布上最大化 $\ell(Z)$ 的期望值。注意，问题 (6.3a) 优化的是 (2.1) 的一个矩不确定性集合，其中 $f(z)=(z,|z-\mu|)$ 且 $\mathcal{F}=\{\mu\} \times\{\sigma\}$。根据定理 4.5 并且由于 $\mathcal{F}$ 的支撑函数是线性的，问题 (6.3a) 的对偶问题为：

$$
\inf _{\lambda_0, \lambda_1, \lambda_2 \in \mathbb{R}}\left\{\lambda_0+\lambda_1 \mu+\lambda_2 \sigma: \lambda_0+\lambda_1 z+\lambda_2|z-\mu| \geq \ell(z) \forall z \in \mathcal{Z}\right\} .
$$

直观上，该对偶问题试图通过一个在 $\mu$ 处具有折点的分段线性连续函数从上逼近损失函数。若 $\ell$ 是凸函数，则问题 (6.3a) 和 (6.3b) 可以闭式求解。

**命题 6.4 (Ben-Tal 和 Hochman 界)**  
假设 $\mathcal{Z}=[0,1], \mu \in(0,1)$ 且 $\sigma \in[0,2 \mu(1-\mu)]$。进一步假设 $\ell$ 是实值凸函数。那么，原问题 (6.3a) 的解为：

$$
\mathbb{P}^{\star}=\frac{\sigma}{2 \mu} \delta_0+\left(1-\frac{\sigma}{2 \mu}-\frac{\sigma}{2(1-\mu)}\right) \delta_\mu+\frac{\sigma}{2(1-\mu)} \delta_1,
$$

对偶问题 (6.3b) 的解为：

$$
\begin{aligned}
& \lambda_0^{\star}=\frac{(1-\mu) \ell(0)+\ell(\mu)-\mu \ell(1)}{2(1-\mu)}, \\
& \lambda_1^{\star}=\frac{(\mu-1) \ell(0)+(1-2 \mu) \ell(\mu)+\mu \ell(1)}{2 \mu(1-\mu)}, \\
& \lambda_2^{\star}=\frac{(1-\mu) \ell(0)-\ell(\mu)+\mu \ell(1)}{2 \mu(1-\mu)} .
\end{aligned}
$$

此外，(6.3a) 和 (6.3b) 的最优值相等，均为：

$$
\frac{\sigma}{2 \mu} \ell(0)+\left(1-\frac{\sigma}{2 \mu}-\frac{\sigma}{2(1-\mu)}\right) \ell(\mu)+\frac{\sigma}{2(1-\mu)} \ell(1) .
$$

**证明**  
关于 $\mu$ 和 $\sigma$ 的假设表明，$\mathbb{P}^{\star}$ 支持于 $\mathcal{Z}$，且三个原子的概率均为非负且概率和为 1。此外，我们有：

$$
\mathbb{E}_{\mathbb{P} \star}[Z]=\left(\mu-\frac{\sigma}{2}-\frac{\sigma \mu}{2(1-\mu)}\right)+\frac{\sigma}{2(1-\mu)}=\mu \quad \text { and } \quad \mathbb{E}_{\mathbb{P} *}[|Z-\mu|]=\sigma .
$$

因此，$\mathbb{P}^{\star}$ 在 (6.3a) 中是可行的。此外，可以验证 $\mathbb{P}^{\star}$ 在 (6.3a) 中的目标函数值与命题中的公式相符。

接下来，注意分段线性函数 $\lambda_0^{\star}+\lambda_1^{\star} z+\lambda_2^{\star}|z-\mu|$ 在 $z \in\{0, \mu, 1\}$ 处与损失函数 $\ell(z)$ 相符。由于损失函数是凸的，因此可以得出 $\lambda_0^{\star}+\lambda_1^{\star} z+\lambda_2^{\star}|z-\mu|$ 在 $z \in[0,1]=\mathcal{Z}$ 上优于 $\ell(z)$。这表明 $\left(\lambda_0^{\star}, \lambda_1^{\star}, \lambda_2^{\star}\right)$ 在 (6.3b) 中是可行的。进一步的计算表明 $\left(\lambda_0^{\star}, \lambda_1^{\star}, \lambda_2^{\star}\right)$ 在 (6.3b) 中的目标函数值与命题中的公式相符。根据定理 4.5 中的弱对偶性，得出 $\mathbb{P}^{\star}$ 是原问题的最优解，且 $\left(\lambda_0^{\star}, \lambda_1^{\star}, \lambda_2^{\star}\right)$ 是对偶问题的最优解。

命题 6.4 可直接推广至支持集为 $\mathcal{Z}=[a, b]$ 的情形，其中 $a, b \in \mathbb{R}, a<\mu<b$，通过应用线性坐标变换实现。如果 (1.2) 中的 $\ell(x, z)$ 对任意固定的 $x \in \mathcal{X}$ 关于 $z$ 是凸的，那么命题 6.4 表明，DRO 问题 (1.2) 等价于随机规划：

$$
\inf _{x \in \mathcal{X}} \mathbb{E}_{\mathbb{P}^{\star}}[\ell(x, Z)],
$$

其中三点分布 $\mathbb{P}^{\star}$ 独立于 $x$。传统上，该随机规划被用作以下随机规划的保守近似：

$$
\inf _{x \in \mathcal{X}} \mathbb{E}_{\mathbb{P}}[\ell(x, Z)],
$$

其中 $\mathbb{P}$ 是已知的连续分布 (Ben-Tal 和 Hochman 1972)。与仅利用 $\mathbb{P}$ 的位置信息的 Jensen 和 Edmundson-Madansky 界不同，以及仅利用 $\mathbb{P}$ 的位置和某些交叉矩信息的重心逼近不同，Ben-Tal 和 Hochman 界利用了 $\mathbb{P}$ 的位置和离散度信息，因此提供了更紧的近似。

如果 $Z$ 是一个具有独立分量 $Z_i, i \in[d]$ 的 $d$ 维随机向量，其中每个分量具有已知均值和均绝对偏差，那么可以证明凸损失函数的最坏情况期望值由 $\mathbb{P}^{\star}=\otimes_{i=1}^d \mathbb{P}_i^{\star}$ 达到，其中每个 $\mathbb{P}_i^{\star}$ 是按照命题 6.4 构造的三点分布 (Ben-Tal 和 Hochman 1972)。在这种情况下，$\mathbb{P}^{\star}$ 是具有 $3^d$ 个原子的离散分布。因此，对于提供机会约束安全近似的一类指数损失函数，计算 $\mathbb{P}^{\star}$ 的期望值虽然一般较难，但在这种特定情形下是可行的 (Postek et al. 2018)。


## 6.5 Scarf界限

考虑以下最坏期望问题：

$$
\sup _{\mathbb{P} \in \mathcal{P}(\mathcal{Z})}\left\{\mathbb{E}_{\mathbb{P}}[\ell(Z)]: \mathbb{E}_{\mathbb{P}}[Z]=0, \mathbb{E}_P\left[Z^2\right]=\sigma^2\right\}
$$

此问题在Chebyshev不确定性集合中最大化$\ell(Z)$的期望值，该集合包括支持在$\mathcal{Z}$上的所有单变量分布，其均值为0，方差为$\sigma^2$。这个Chebyshev不确定性集合是形式 (2.1) 的矩不确定性集合，其中$f(z) = \left(z, z^2\right)$且$\mathcal{F} = \{0\} \times \left\{\sigma^2\right\}$。根据定理4.5，由于$\mathcal{F}$的支撑函数是线性的，与(6.4a)问题对偶的问题为：

$$
\inf _{\lambda_0, \lambda_1, \lambda_2 \in \mathbb{R}}\left\{\lambda_0+\lambda_2 \sigma^2: \lambda_0+\lambda_1 z+\lambda_2(z-\mu)^2 \geq \ell(z) \forall z \in \mathcal{Z}\right\}。
$$

该对偶问题寻求一个二次函数$q(z) = \lambda_0 + \lambda_1 z + \lambda_2 z^2$，以使其在$\mathcal{Z}$范围内大于等于损失函数$\ell(z)$，并在均值为0且方差为$\sigma^2$的任何分布$\mathbb{P}$下使期望$\mathbb{E}_{\mathrm{P}}[q(Z)]$最小化。如果$\ell$是斜坡函数，(6.4a)和(6.4b)问题可以得到闭式解。

**命题 6.5** (Scarf界限)
若$\mathcal{Z} = \mathbb{R}, \sigma^2 \in \mathbb{R}_{+}$，且$\ell(z) = \max \{z-a, 0\}$是一个在$a \in \mathbb{R}$处具有折点的斜坡函数，则 (6.4a) 的原问题由以下分布解决：

$$
\mathbb{P}^{\star} = \frac{1}{2}\left(1+\frac{a}{\sqrt{a^2+\sigma^2}}\right) \delta_{a-\sqrt{a^2+\sigma^2}} + \frac{1}{2}\left(1-\frac{a}{\sqrt{a^2+\sigma^2}}\right) \delta_{a+\sqrt{a^2+\sigma^2}},
$$

而 (6.4b) 的对偶问题由以下解解决：

$$
\lambda_0^{\star} = \frac{\left(a-\sqrt{a^2+\sigma^2}\right)^2}{4 \sqrt{a^2+\sigma^2}}, \quad \lambda_1^{\star} = -\frac{a-\sqrt{a^2+\sigma^2}}{2 \sqrt{a^2+\sigma^2}}, \quad \lambda_2^{\star} = \frac{1}{4 \sqrt{a^2+\sigma^2}}。
$$

(6.4a)和(6.4b)的最优值均为$\frac{1}{2}\left(\sqrt{a^2+\sigma^2}-a\right)$。

**证明**
注意，两点分布$\mathbb{P}^{\star}$是定义良好的，即其概率值非负且总和为1。根据$\mathbb{P}^{\star}$的定义，我们有：

$$
\begin{aligned}
\mathbb{E}_{\mathbb{P}^{\star}}[Z] = \frac{1}{2} & \left(1+\frac{a}{\sqrt{a^2+\sigma^2}}\right)\left(a-\sqrt{a^2+\sigma^2}\right) \\
& +\frac{1}{2}\left(1-\frac{a}{\sqrt{a^2+\sigma^2}}\right)\left(a+\sqrt{a^2+\sigma^2}\right) = 0。
\end{aligned}
$$

类似地，很容易验证$\mathbb{E}_{\mathbb{P}^{\star}}\left[Z^2\right]=\sigma^2$。这表明$\mathbb{P}^{\star}$在(6.4a)中是可行的。$\mathbb{P}^{\star}$的目标函数值为：

$$
\mathbb{E}_{\mathbb{P}^{\star}}[\ell(Z)] = \mathbb{E}_{\mathbb{P}^{\star}}[\max \{Z-a, 0\}] = \frac{1}{2}\left(\sqrt{a^2+\sigma^2}-a\right)。
$$

接下来，观察命题中定义的对偶变量$\left(\lambda_0^{\star}, \lambda_1^{\star}, \lambda_2^{\star}\right)$生成了如下二次函数：

$$
q^{\star}(z) = \lambda_0^{\star} + \lambda_1^{\star} z + \lambda_2^{\star} z^2 = \frac{1}{4 \sqrt{a^2+\sigma^2}}\left(z-a+\sqrt{a^2+\sigma^2}\right)^2。
$$

我们将证明$q^{\star}(z) \geq \max \{z-a, 0\} = \ell(z)$对所有$z \in \mathcal{Z}$成立。显然，$q^{\star}$非负并在$a-\sqrt{a^2+\sigma^2}$处取0。此外，$q^{\star}$在$a+\sqrt{a^2+\sigma^2}$处与仿射函数$z-a$相切。具体来说：

$$
q^{\star}\left(a+\sqrt{a^2+\sigma^2}\right) = \sqrt{a^2+\sigma^2}, \quad \frac{\mathrm{d}}{\mathrm{~d} z} q^{\star}\left(a+\sqrt{a^2+\sigma^2}\right) = 1。
$$

因此，$q^{\star}$上界了斜坡函数$\ell(z)$，这意味着$\left(\lambda_0^{\star}, \lambda_1^{\star}, \lambda_2^{\star}\right)$是对偶可行的。此外，$\left(\lambda_0^{\star}, \lambda_1^{\star}, \lambda_2^{\star}\right)$的目标函数值为：

$$
\lambda_0^{\star} + \lambda_2^{\star} \sigma^2 = \frac{1}{4 \sqrt{a^2+\sigma^2}}\left(\sigma^2+\left(a-\sqrt{a^2+\sigma^2}\right)^2\right) = \frac{1}{2}\left(\sqrt{a^2+\sigma^2}-a\right)。
$$

由于$\mathbb{P}^{\star}$和$\left(\lambda_0^{\star}, \lambda_1^{\star}, \lambda_2^{\star}\right)$的目标函数值相等，根据定理4.5中的弱对偶性，$\mathbb{P}^{\star}$是原问题的最优解，$\left(\lambda_0^{\star}, \lambda_1^{\star}, \lambda_2^{\star}\right)$是对偶问题的最优解。至此证明完成。

命题6.5最早由Scarf在1958年的关于分布鲁棒新闻商问题的开创性论文中提出；参见 (Jagannathan 1977, 定理1)。如果已知$Z$的均值为$\mu \neq 0$而非0，则将$a$替换为$a-\mu$后，Scarf界限仍然有效。Gallego和Moon (1993) 将Scarf界限扩展到了更一般的损失函数，如带有间断的斜坡函数或楔形函数。Natarajan等人 (2018) 将Scarf界限扩展到了包含$Z$的均值、方差及其半方差信息的更一般的不确定性集合。此外，Das、Dhara和Natarajan (2021) 讨论了基于$Z$的均值及其$\alpha$次矩（$\alpha > 1$）信息的Scarf界限的变体。

命题6.5通常用于重构形式为 (1.2) 的DRO问题，其目标函数为斜坡函数的期望值。例如，分布鲁棒新闻商问题、支持向量机或均值-CVaR投资组合选择问题。在大多数应用中，斜坡函数的折点$a$是一个决策变量或决策变量的函数。因此，最坏分布$\mathbb{P}^{\star}$依赖于决策变量，这意味着命题6.5无法将DRO问题 (1.2) 简化为具有单一固定最坏分布的随机规划问题。


## 6.6 Marshall和Olkin界限

考虑以下最坏概率问题：

$$
\sup _{\mathbb{P} \in \mathcal{P}(\mathcal{Z})}\left\{\mathbb{P}(Z \in \mathcal{C}): \mathbb{E}_{\mathbb{P}}[Z]=0, \mathbb{E}_{\mathrm{P}}\left[Z Z^{\top}\right]=I_d\right\},
$$

此问题在Chebyshev不确定性集合中最大化事件$Z \in \mathcal{C}$的概率，其中该集合包括定义在$\mathcal{Z}=\mathbb{R}^d$上的所有分布，其均值为0，协方差矩阵为$I_d$。此Chebyshev不确定性集合是形式 (2.1) 的矩不确定性集合，其中$f(z) = \left(z, z z^{\top}\right)$且$\mathcal{F} = \{0\} \times \left\{I_d\right\}$。若令$\ell$为特征函数$\ell(z) = \mathbb{1}_{z \in \mathcal{C}}$，则最坏概率问题 (6.5a) 可重写为最坏期望问题。根据定理4.5，由于$\mathcal{F}$的支撑函数是线性的，相应的对偶问题为：

$$
\inf _{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}^d, \Lambda \in \mathbb{S}^d}\left\{\lambda_0+\left\langle\Lambda, I_d\right\rangle: \lambda_0+\lambda^{\top} z+z^{\top} \Lambda z \geq \ell(z) \forall z \in \mathcal{Z}\right\}。
$$

若$\mathcal{C}$是凸且闭的，则(6.5a)和(6.5b)问题可以解析求解。

**命题 6.6** (Marshall和Olkin界限)
设$\mathcal{Z}=\mathbb{R}^d, \mathcal{C} \subseteq \mathbb{R}^d$为凸且闭的集合，$\ell$是$\mathcal{C}$的特征函数。令$\Delta=\min _{z \in \mathcal{C}}\|z\|_2$，且$z_0 \in \mathbb{R}^d$为该问题的唯一最小化解。则(6.5a)和(6.5b)的最优值均为$\left(1+\Delta^2\right)^{-1}$。如果$\Delta=0$，则(6.5a)的上确界可能无法达到。然而，当$\Delta>0$时，(6.5a)的解为：

$$
\mathbb{P}^{\star}=\frac{1}{1+\Delta^2} \delta_{z_0}+\frac{\Delta^2}{1+\Delta^2} \mathbb{Q},
$$

其中$\mathbb{Q} \in \mathcal{P}(\mathcal{Z})$是一个任意分布，其均值为$-z_0 / \Delta^2$，协方差矩阵为$\frac{1+\Delta^2}{\Delta^2}\left(I_d-z_0 z_0^{\top} / \Delta^2\right)$。

对于任意$\Delta \geq 0$，问题$(6.5b)$的解为：

$$
\lambda_0^{\star}=\frac{1}{\left(1+\Delta^2\right)^2}, \quad \lambda^{\star}=\frac{2 z_0}{\left(1+\Delta^2\right)^2}, \quad \Lambda^{\star}=\frac{z_0 z_0^{\top}}{\left(1+\Delta^2\right)^2}。
$$

**证明**
首先假设$\Delta=0$，即$0 \in \mathcal{C}$。对于任意$j \in \mathbb{N}$，设$\mathbb{Q}_j \in \mathcal{P}(\mathcal{Z})$为均值为0且协方差矩阵为$j I_d$的分布，并令

$$
\mathbb{P}_j=(1-1 / j) \delta_0+(1 / j) \mathbb{Q}_j。
$$

因此，我们有$\mathbb{E}_{\mathbb{P}_j}[Z]=0$且$\mathbb{E}_{\mathbb{P}_j}\left[Z Z^{\top}\right]=I_d$，这表明$\mathbb{P}_j$在(6.5a)中是可行的。此外，$\mathbb{P}_j$在(6.5a)中的目标函数值满足：

$$
\mathbb{P}_j(Z \in \mathcal{C})=1-1 / j+\mathbb{Q}_j(Z \in \mathcal{C}) / j \geq 1-j^{-1}。
$$

令$j \to \infty$，可以看出当$\Delta=0$时，(6.5a)是平凡的，其上确界为1。

现在假设$\Delta>0$，令$\mathbb{Q} \in \mathcal{P}(\mathcal{Z})$为任意分布，其均值为$-z_0 / \Delta^2$，协方差矩阵为$\frac{1+\Delta^2}{\Delta^2}\left(I_d-z_0 z_0^{\top} / \Delta^2\right)$。这样的分布是存在的，因为$I_d \geq z_0 z_0^{\top} / \Delta^2$。此外，按照命题定义构造$\mathbb{P}^{\star}$。通过构造，我们有$\mathbb{E}_{\mathbb{P}^{\star}}[Z]=0$且：

$$
\begin{aligned}
\mathbb{E}_{\mathrm{P}^{\star}}\left[Z Z^{\top}\right] & =\frac{z_0 z_0^{\top}}{1+\Delta^2}+\frac{\Delta^2}{1+\Delta^2} \mathbb{E}_{\mathrm{Q}}\left[Z Z^{\top}\right] \\
& =\frac{z_0 z_0^{\top}}{1+\Delta^2}+I_d-\frac{z_0 z_0^{\top}}{\Delta^2}+\frac{\Delta^2}{1+\Delta^2} \frac{z_0 z_0^{\top}}{\Delta^4}=I_d。
\end{aligned}
$$

此外，$\mathbb{P}^{\star}$在(6.5a)中的目标函数值为$\mathbb{P}^{\star}(Z \in \mathcal{C})=\left(1+\Delta^2\right)^{-1}$。

接下来，利用命题中定义的$\left(\lambda_0^{\star}, \lambda^{\star}, \Lambda^{\star}\right)$构造二次函数：

$$
q^{\star}(z)=\lambda_0^{\star}+\left(\lambda^{\star}\right)^{\top} z+z^{\top} \Lambda^{\star} z=\frac{\left(z_0^{\top} z+1\right)^2}{\left(1+\Delta^2\right)^2}。
$$

注意$q^{\star}$是非负的，且在与$z_0$垂直的任何超平面上是常数。如果$\Delta>0$，我们有$q^{\star}\left(z_0\right)=1$且$q^{\star}\left(-z_0 / \Delta^2\right)=0$。因此，对于所有满足$z_0^{\top} z \geq-1$的$z \in \mathcal{Z}$，二次函数$q^{\star}(z)$在$z_0$的方向上是单调递增的。由于$z_0$最小化了凸闭集合$\mathcal{C}$上的可微凸函数$\|z\|_2^2$，我们有$z_0^{\top}\left(z-z_0\right) \geq 0$对所有$z \in \mathcal{C}$成立。利用$q^{\star}$的单调性，这表明$q^{\star}(z) \geq 1$对每个$z \in \mathcal{C}$成立。因此，二次函数$q^{\star}$上界了$\mathcal{Z}$上的指标函数$\ell$，这意味着$\left(\lambda_0^{\star}, \lambda^{\star}, \Lambda^{\star}\right)$是对偶可行的。

若$\Delta=0$，则$q^{\star}(z)=1$对所有$z \in \mathcal{Z}$成立，且$\left(\lambda_0^{\star}, \lambda^{\star}, \Lambda^{\star}\right)$同样是对偶可行的。在任意情况下，可以验证其目标函数值为：

$$
\lambda_0^{\star}+\left\langle\Lambda^{\star}, I_d\right\rangle=\left(1+\Delta^2\right)^{-1}。
$$

由于当$\Delta>0$时，$\mathbb{P}^{\star}$和$\left(\lambda_0^{\star}, \lambda^{\star}, \Lambda^{\star}\right)$的目标函数值相等，定理4.5中的弱对偶性表明$\mathbb{P}^{\star}$是原问题的最优解，$\left(\lambda_0^{\star}, \lambda^{\star}, \Lambda^{\star}\right)$是对偶问题的最优解。若$\Delta=0$，则原问题的最优值1也与$\left(\lambda_0^{\star}, \lambda^{\star}, \Lambda^{\star}\right)$在(6.5b)中的目标函数值相等。因此，即使原问题的上确界可能无法达到，$\left(\lambda_0^{\star}, \lambda^{\star}, \Lambda^{\star}\right)$仍是对偶最优解。证明至此完成。


## 6.7 Chebyshev风险

通过解析求解最坏期望问题，可以评估随机变量的最坏风险，前提是所用的风险度量具有分布不变性、平移不变性以及尺度不变性；参见定义5.3。例如，很容易验证$\beta$-VaR和$\beta$-CVaR对于任意固定的$\beta \in(0,1)$都构成分布不变、平移不变和尺度不变的风险度量。如果随机变量$Z$的分布未知，仅知道其均值$\mu \in \mathbb{R}^d$和协方差矩阵$\Sigma \in \mathbb{S}_{+}^d$，则在分布不变风险度量$\varrho$下，可以通过相应的Chebyshev风险量化不确定损失$\ell(Z)$的风险性。具体来说，$\ell(Z)$的Chebyshev风险定义为最坏风险：

$$
\sup _{\mathbb{P} \in \mathcal{P}(\mu, \Sigma)} \varrho_{\mathrm{P}}[\ell(Z)],
$$

其中$\mathcal{P}(\mu, \Sigma)$表示Chebyshev不确定性集合，包含所有定义在$\mathbb{R}^d$上的分布，其均值为$\mu \in \mathbb{R}^d$，协方差矩阵为$\Sigma \in \mathbb{S}_{+}^d$。

**命题 6.7** (Chebyshev不确定性集合的推前)
若$\mu \in \mathbb{R}^d, \Sigma \in \mathbb{S}_{+}^d, \theta \in \mathbb{R}^d$，且$\ell: \mathbb{R}^d \rightarrow \mathbb{R}$是通过$\ell(z)=\theta^{\top} z$定义的线性变换，则Chebyshev不确定性集合$\mathcal{P}(\mu, \Sigma)$的推前为：

$$
\mathcal{P}(\mu, \Sigma) \circ \ell^{-1}=\mathcal{P}\left(\theta^{\top} \mu, \theta^{\top} \Sigma \theta\right)。
$$

**证明**
首先，选择任意分布$\mathbb{P} \in \mathcal{P}(\mu, \Sigma)$。如果随机向量$Z$服从分布$\mathbb{P}$，则随机变量$L=\ell(Z)$服从推前分布$\mathbb{P} \circ \ell^{-1}$。因此，有：

$$
\mathbb{E}_{P \circ \ell^{-1}}[L]=\mathbb{E}_{\mathbb{P}}[\ell(Z)]=\mathbb{E}_{\mathbb{P}}\left[\theta^{\top} Z\right]=\theta^{\top} \mu，
$$

其中第一个等式由测度论中的变量变换公式得到。同理，可以证明$\mathbb{E}_{\mathrm{P} \circ \ell^{-1}}\left[\left(L-\theta^{\top} \mu\right)^2\right]=\theta^{\top} \Sigma \theta$。因此，

$$
\mathcal{P}(\mu, \Sigma) \circ \ell^{-1} \subseteq \mathcal{P}\left(\theta^{\top} \mu, \theta^{\top} \Sigma \theta\right)。
$$

接下来，选择任意$\mathbb{Q}_L \in \mathcal{P}\left(\theta^{\top} \mu, \theta^{\top} \Sigma \theta\right)$。如果$\theta^{\top} \Sigma \theta=0$，则$\mathbb{Q}_L=\delta_{\theta^{\top} \mu}$，与任何$\mathbb{P} \in \mathcal{P}(\mu, \Sigma)$的推前分布$\mathbb{P} \circ \ell^{-1}$一致。在证明的剩余部分，我们假设$\theta^{\top} \Sigma \theta \neq 0$。设$L$是服从分布$\mathbb{Q}_L$的随机变量，$M$是服从$\mathbb{R}^d$上分布$\mathbb{Q}_M$的$d$维随机向量，其均值为$\mu$，协方差矩阵为$\Sigma$。例如，可以令$\mathbb{Q}_M$为正态分布$\mathcal{N}(\mu, \Sigma)$。假设$L$和$M$相互独立，则$d$维随机向量

$$
Z=\frac{1}{\theta^{\top} \Sigma \theta} \Sigma \theta L+\left(I_d-\frac{1}{\theta^{\top} \Sigma \theta} \Sigma \theta \theta^{\top}\right) M，
$$

的分布$\mathbb{P}$属于$\mathcal{P}(\mu, \Sigma)$。通过$L$和$M$的构造，我们有：

$$
\mathbb{E}_{\mathbb{P}}[Z]=\frac{1}{\theta^{\top} \Sigma \theta} \Sigma \theta \mathbb{E}_{\mathrm{Q}_L}[L]+\left(I_d-\frac{1}{\theta^{\top} \Sigma \theta} \Sigma \theta \theta^{\top}\right) \mathbb{E}_{\mathrm{Q}_M}[M]=\mu，
$$

且

$$
\mathbb{E}_{\mathbb{P}}\left[(Z-\mu)(Z-\mu)^{\top}\right]=\Sigma。
$$

由此可得，对于每个$\mathbb{Q}_L \in \mathcal{P}\left(\theta^{\top} \mu, \theta^{\top} \Sigma \theta\right)$，存在$\mathbb{P} \in \mathcal{P}(\mu, \Sigma)$，使得$\mathbb{P} \circ \ell^{-1}=\mathbb{Q}_L$，即：

$$
\mathcal{P}(\mu, \Sigma) \circ \ell^{-1} \supseteq \mathcal{P}\left(\theta^{\top} \mu, \theta^{\top} \Sigma \theta\right)。
$$

综上，证明完成。

**命题 6.9** (Chebyshev风险)
若$\varrho$是具有标准风险系数$\alpha$的分布不变、平移不变和尺度不变的风险度量，且存在$\theta \in \mathbb{R}^d$，使得$\ell(z)=\theta^{\top} z$，$\mathcal{P}(\mu, \Sigma)$为Chebyshev不确定性集合，则Chebyshev风险满足：

$$
\sup _{\mathbb{P} \in \mathcal{P}(\mu, \Sigma)} \varrho_{\mathrm{P}}[\ell(Z)]=\theta^{\top} \mu+\alpha \sqrt{\theta^{\top} \Sigma \theta}。
$$

**命题 6.10** (VaR和CVaR的标准风险系数)
对于任意$\beta \in (0,1)$，$\beta$-VaR和$\beta$-CVaR的标准风险系数一致，即：

$$
\sup _{\mathbb{Q} \in \mathcal{P}(0,1)} \beta-\mathrm{CVaR}_{\mathrm{Q}}[L]=\sup _{\mathbb{Q} \in \mathcal{P}(0,1)} \beta-\operatorname{VaR}_{\mathrm{Q}}[L]=\sqrt{\frac{1-\beta}{\beta}}。
$$

**证明**

根据$\beta$-CVaR总是上界$\beta$-VaR的性质，对于所有$\mathbb{Q} \in \mathcal{P}(0,1)$，有：

$$
\sup _{\mathbb{Q} \in \mathcal{P}(0,1)} \beta-\mathrm{CVaR}_{\mathrm{Q}}[L] \geq \sup _{\mathbb{Q} \in \mathcal{P}(0,1)} \beta-\operatorname{VaR}_{\mathrm{Q}}[L]。
$$

以下分两步证明两者相等。

**第一步**  
我们首先将最坏情况下的$\beta$-VaR表示为其最小上界。具体如下：

$$
\begin{aligned}
\sup _{\mathbb{Q} \in \mathcal{P}(0,1)} \beta-\operatorname{VaR}_{\mathbb{Q}}[L] & =\inf _{\tau \in \mathbb{R}}\left\{\tau: \beta-\operatorname{VaR}_{\mathbb{Q}}(L) \leq \tau \forall \mathbb{Q} \in \mathcal{P}(0,1)\right\} \\
& =\inf _{\tau \in \mathbb{R}}\{\tau: \mathbb{Q}(L \geq \tau) \leq \beta \forall \mathbb{Q} \in \mathcal{P}(0,1)\} \\
& =\inf _{\tau \in \mathbb{R}}\left\{\tau: \frac{1}{1+\tau^2} \leq \beta\right\}=\sqrt{\frac{1-\beta}{\beta}}。
\end{aligned}
$$

上述推导中的第二个等式来自于定义(5.2)，第三个等式应用了命题6.6中Marshall和Olkin界限的结果，最后的公式是通过解析求解关于$\tau$的最小化问题得出的。

**第二步**  
根据最大-最小不等式和$\beta$-CVaR的定义，有：

$$
\begin{aligned}
\sup _{\mathbb{Q} \in \mathcal{P}(0,1)} \beta-\mathrm{CVaR}_{\mathrm{Q}}[L] & \leq \inf _{\tau \in \mathbb{R}} \sup _{\mathbb{Q} \in \mathcal{P}(0,1)} \tau+\frac{1}{\beta} \mathbb{E}_{\mathbb{Q}}[\max \{L-\tau, 0\}] \\
& =\inf _{\tau \in \mathbb{R}} \tau+\frac{1}{2 \beta}\left(\sqrt{1+\tau^2}-\tau\right)=\sqrt{\frac{1-\beta}{\beta}}。
\end{aligned}
$$

其中第一个等式是基于命题6.5中Scarf界限的推导，最后的等式通过解析求解凸最小化问题得出。唯一的最优解为：

$$
\tau^{\star}=\frac{1-2 \beta}{2 \sqrt{\beta(1-\beta)}}。
$$

这完成了第二步的证明。结合第一步中关于$\beta$-VaR的最坏情况解析公式和第二步中关于$\beta$-CVaR的解析上界，可以得到两者相等的结论。

因此，对于任意$\beta \in (0,1)$，$\beta$-VaR和$\beta$-CVaR的标准风险系数均为$\sqrt{\frac{1-\beta}{\beta}}$。

**备注**

命题6.9和6.10提供了线性损失函数在VaR和CVaR下Chebyshev风险的解析公式。关于最坏情况下的VaR公式首次由(Lanckriet et al., 2001, 2002; El Ghaoui et al., 2003)提出，见(Calafiore and El Ghaoui, 2006)。最坏情况下VaR和CVaR的等价性由(Zymler et al., 2013a)发现，它不仅适用于线性损失函数，还适用于任意凹或任意二次（不一定凹）损失函数。命题6.9基于(Nguyen et al., 2021)的研究结果。对于VaR和CVaR以外的分布不变、平移不变和尺度不变风险度量，例如所有谱风险度量、具有Kusuoka表示的风险度量，以及所有扭曲风险度量（Cai et al., 2023），也可以解析出标准风险系数。



## 6.8 Gelbrich 风险

记 $\mathcal{G}_r(\hat{\mu}, \hat{\Sigma})$ 为所有分布 $\mathbb{P} \in \mathcal{P}\left(\mathbb{R}^d\right)$ 的 Gelbrich 模糊集，其中均值-协方差对 $(\mu, \Sigma) \in \mathbb{R}^d \times \mathbb{S}_{+}^d$ 关于 $(\hat{\mu}, \hat{\Sigma}) \in \mathbb{R}^d \times \mathbb{S}_{+}^d$ 的 Gelbrich 距离不超过 $r \geq 0$；参见定义 2.1。回顾第 2.1.4 节，Gelbrich 模糊集考虑了矩的不确定性，因此通常比简单的 Chebyshev 模糊集更能真实反映不确定性。如果随机变量 $Z$ 的分布仅已知其均值-协方差对接近 $(\hat{\mu}, \hat{\Sigma})$，则在基于分布不变的风险度量 $\varrho$ 下，可以通过 Gelbrich 风险量化不确定损失 $\ell(Z)$ 的风险性：

$$
\sup _{\mathbb{P} \in \mathcal{G}_r(\hat{\mu}, \hat{\mathcal{L}})} \varrho_{\mathbb{P}}[\ell(Z)] .
$$

通过构造，$\mathcal{G}_r(\hat{\mu}, \hat{\Sigma})$ 是所有 Chebyshev 模糊集 $\mathcal{P}(\mu, \Sigma)$ 的并集，这些集合的均值-协方差对 $(\mu, \Sigma)$ 满足 $\mathrm{G}((\mu, \Sigma),(\hat{\mu}, \hat{\Sigma})) \leq r$。这种 Gelbrich 模糊集对 Chebyshev 模糊集的分解使得我们可以通过命题 6.9 推导 Gelbrich 风险的解析公式。

---

**命题 6.11**（Gelbrich 风险）

假设 $\varrho$ 是具有标准风险系数 $\alpha \in \mathbb{R}_{+}$ 的分布、平移和尺度不变风险度量，存在 $\theta \in \mathbb{R}^d$ 满足 $\ell(z)=\theta^{\top} z, \forall z \in \mathbb{R}^d$，并且 $\mathcal{G}_r(\hat{\mu}, \hat{\Sigma})$ 是所有分布的 Gelbrich 模糊集，其均值-协方差对与 $(\hat{\mu}, \hat{\Sigma}) \in \mathbb{R}^d \times \mathbb{S}_{+}^d$ 的 Gelbrich 距离最多为 $r \geq 0$。则 Gelbrich 风险满足：

$$
\sup _{\mathbb{P} \in \mathcal{G}_r(\hat{\mu}, \hat{\mathcal{L}})} \varrho_{\mathrm{P}}\left[\theta^{\top} Z\right]=\hat{\mu}^{\top} \theta+\alpha \sqrt{\theta^{\top} \hat{\Sigma} \theta}+r \sqrt{1+\alpha^2}\|\theta\|_2
$$

---

**证明**

首先假设 $\hat{\Sigma}>0$。若 $\theta=0$，由于 $\varrho$ 是分布和尺度不变的，结论显然成立。若 $r=0$，结论直接由命题 6.9 得出。因此，我们可以假设 $\theta \neq 0$ 且 $r>0$。此时，

$$
\begin{aligned}
\sup _{\mathbb{P} \in \mathcal{G}_r(\hat{\mu}, \hat{\Sigma})} \varrho_P\left[\theta^{\top} Z\right] & = \begin{cases}\text { sup } & \sup _{P \in \mathcal{P}(\mu, \Sigma)} \varrho_P\left[\theta^{\top} Z\right] \\
\text { s.t. } & \mu \in \mathbb{R}^d, \Sigma \in \mathbb{S}_{+}^d, \mathrm{G}((\mu, \Sigma),(\hat{\mu}, \hat{\Sigma})) \leq r\end{cases} \\
& = \begin{cases}\text { sup } & \mu^{\top} \theta+\alpha \sqrt{\theta^{\top} \Sigma \theta}\end{cases} \\
\text { s.t. } & \mu \in \mathbb{R}^d, \Sigma \in \mathbb{S}_{+}^d \\
& \|\mu-\hat{\mu}\|^2+\operatorname{Tr}\left(\Sigma+\hat{\Sigma}-2\left(\hat{\left.\left.\Sigma^{\frac{1}{2}} \Sigma \hat{\Sigma}^{\frac{1}{2}}\right)^{\frac{1}{2}}\right) \leq r^2.}\right.\right.
\end{aligned}
$$

第一步利用了 Gelbrich 模糊集对 Chebyshev 模糊集的分解，第二步源于命题 6.9 和定义 2.1。通过对得到的凸优化问题进行对偶化，我们可以得到：

$$
\begin{aligned}
\sup _{\mathbb{P} \in \mathcal{G}_r(\hat{\mu}, \hat{\Sigma})} \varrho_{\mathrm{P}}\left[\theta^{\top} Z\right]=\inf _{\gamma \in \mathbb{R}_{+}}\{ & \left\{\gamma\left(r^2-\operatorname{Tr}(\hat{\Sigma})\right)+\sup _{\mu \in \mathbb{R}^d}\left\{\mu^{\top} \theta-\gamma\|\mu-\hat{\mu}\|^2\right\}\right. \\
& \left.+\sup _{\Sigma \in \mathbb{S}_{+}^d}\left\{\alpha \sqrt{\theta^{\top} \Sigma \theta}-\gamma \operatorname{Tr}\left(\Sigma-2\left(\hat{\Sigma}^{\frac{1}{2}} \Sigma \hat{\Sigma}^{\frac{1}{2}}\right)^{\frac{1}{2}}\right)\right\}\right\} .
\end{aligned}
$$

因为 $r>0$，强对偶性成立，$(\hat{\mu}, \hat{\Sigma})$ 构成了原始最大化问题的 Slater 点。对于固定 $\gamma > 0$，$\mu$ 的最大化问题可以闭式求解，其最优值为 $\hat{\mu}^{\top} \theta+\|\theta\|^2 /(4 \gamma)$。

通过引入辅助变量 $t$，对 $\Sigma$ 的最大化问题可以改写为：

$$
\begin{array}{ll}
\text { sup } & \alpha t-\gamma \operatorname{Tr}\left(\Sigma-2\left(\hat{\Sigma}^{\frac{1}{2}} \Sigma \hat{\Sigma}^{\frac{1}{2}}\right)^{\frac{1}{2}}\right) \\
\text { s.t. } & t \in \mathbb{R}_{+}, \Sigma \in \mathbb{S}_{+}^d, t^2-\theta^{\top} \Sigma \theta \leq 0 .
\end{array}
$$

利用变量替换 $B \leftarrow\left(\hat{\Sigma}^{\frac{1}{2}} \Sigma \hat{\Sigma}^{\frac{1}{2}}\right)^{\frac{1}{2}}$，对偶问题可简化为：

$$
\inf _{\lambda \in \mathbb{R}_{+}} \sup _{t \in \mathbb{R}_{+}} \alpha t-\lambda t^2+\sup _{B \in S_{+}^d} \operatorname{Tr}\left(B^2 \Delta_\lambda\right)+2 \gamma \operatorname{Tr}(B),
$$

其中 $\Delta_\lambda=\hat{\Sigma}^{-\frac{1}{2}}\left(\lambda \theta \theta^{\top}-\gamma I_d\right) \hat{\Sigma}^{-\frac{1}{2}}$。

进一步分析得到最终结果：

$$
\sup _{\mathbb{P} \in \mathcal{G}_r(\hat{\mu}, \hat{\Sigma})} \varrho_{\mathrm{P}}\left[\theta^{\top} Z\right]=\hat{\mu}^{\top} \theta+\alpha \sqrt{\theta^{\top} \hat{\Sigma} \theta}+r \sqrt{1+\alpha^2}\|\theta\|_2
$$

这一公式也适用于 $\hat{\Sigma}$ 奇异的情况，利用了 Gelbrich 风险 $J(\hat{\Sigma})$ 的连续性和 Berge 最大值定理。

---


命题 6.11 来自 Nguyen 等人（2021）。对于广泛的风险度量，Gelbrich 模糊集上的最坏情况风险可以简化为一个包含 2-范数正则项的 Markowitz 均值-方差风险函数。风险度量 $\rho$ 仅通过标准风险系数 $\alpha$ 间接影响最终优化模型。


## 6.9 关于 Kullback-Leibler 模糊集的最坏期望

考虑最坏期望问题：

$$
\sup _{\mathbb{P} \in \mathcal{P}(\mathcal{Z})}\left\{\mathbb{E}_{\mathbb{P}}[\ell(Z)]: \operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\},
$$

它在 Kullback-Leibler 模糊集上最大化 $\ell(Z)$ 的期望，其中模糊集包括所有在 $\mathcal{Z}$ 上的分布，这些分布相对于 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 的 Kullback-Leibler 散度不超过 $r \geq 0$。Kullback-Leibler 模糊集是一个形式如 (2.10) 的 $\phi$ 散度模糊集，其中 $\phi$ 满足 $\phi(s)=s \log (s)-s+1, \forall s \geq 0$。由于 $\phi^{\infty}(1)=+\infty$，如果 $\mathbb{P} \nless \hat{\mathbb{P}}$，则 $\operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}})=\infty$。因此，问题 (6.11a) 仅在 $\mathbb{P}$ 对 $\hat{\mathbb{P}}$ 绝对连续的分布上最大化。注意到 $\phi^*(t)=e^t-1, \forall t \in \mathbb{R}$。根据定理 4.14 和透视函数的定义，(6.11a) 的对偶问题为：

$$
\inf _{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}_{+}} \lambda_0+\lambda(r-1)+\mathbb{E}_{\hat{\mathrm{p}}}\left[\lambda \exp \left(\frac{\ell(Z)-\lambda_0}{\lambda}\right)\right] .
$$

如果损失函数 $\ell$ 是线性的，并且标称分布 $\hat{\mathbb{P}}$ 是高斯分布，则问题 (6.11a) 和 (6.11b) 可以闭式求解。

---

**命题 6.12**（关于 KL 模糊集的最坏期望）

假设 $\mathcal{Z}=\mathbb{R}^d, \hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 是均值为 $\hat{\mu} \in \mathbb{R}^d$、协方差矩阵为 $\hat{\Sigma} \in \mathbb{S}_{+}^d$ 的正态分布，且 $r>0$。还假设 $\ell$ 是线性的，即存在 $\theta \in \mathbb{R}^d$ 满足 $\ell(z)=\theta^{\top} z, \forall z \in \mathcal{Z}$。则原问题 (6.11a) 的解是均值为 $\hat{\mu}+(2 r)^{\frac{1}{2}} \hat{\Sigma} \theta /\left(\theta^{\top} \hat{\Sigma} \theta\right)^{\frac{1}{2}}$，协方差矩阵为 $\hat{\Sigma}$ 的正态分布 $\mathbb{P}^{\star}$。对偶问题 (6.11b) 的解为 $\left(\lambda_0^{\star}, \lambda^{\star}\right)$，其中：

$$
\lambda^{\star}=\left(\theta^{\top} \hat{\Sigma} \theta\right)^{\frac{1}{2}} /(2 r)^{\frac{1}{2}}
$$

$$
\lambda_0^{\star}=\lambda^{\star} \log \mathbb{E}_{\hat{p}}\left[\exp \left(\ell(Z) / \lambda^{\star}\right)\right] .
$$

将该参数最小值代入 (6.11b)，可以证明对偶问题的最优值为：

$$
\begin{aligned}
\inf _{\lambda \in \mathbb{R}} \lambda r+\lambda \log \mathrm{B}_p\left[\exp \left(\frac{\ell(Z)}{\lambda}\right)\right] & =\inf _{\lambda \in \mathcal{R}} \lambda r+\hat{\mu}^{\top} \theta+\frac{1}{2 \lambda} \theta^{\top} \hat{\Sigma} \theta \\
& =\hat{\mu}^{\top} \theta+(2 r)^{\frac{1}{2}}\left(\theta^{\top} \hat{\Sigma} \theta\right)^{\frac{1}{2}}
\end{aligned}
$$

其中，第一步利用了 $\ell$ 的线性、高斯分布的性质以及对数正态分布期望值的公式。第二步成立是因为 $\lambda \geq 0$ 的最小化问题由 $\lambda^*=\left(\theta^{\top} \hat{\Sigma} \theta\right)^{1/2}/(2 r)^{1/2}$ 解出。接下来，定义 $\mathrm{P}^* \in \mathcal{P}(\mathcal{Z})$ 为均值 $\mu^*=\hat{\mu}+\hat{\Sigma} \theta / \lambda^*$ 和协方差矩阵 $\Sigma^*=\hat{\Sigma}$ 的正态分布。比较 $\hat{\mathrm{P}}$ 和 $\mathrm{P}^*$ 的密度函数可以得到：

$$
\frac{\mathrm{dP}}{\mathrm{dP}}(z)=\exp \left(\frac{\theta^{\top}(z-\hat{\mu})}{\lambda^*}-\frac{\theta^{\top} \hat{\Sigma} \theta}{2\left(\lambda^*\right)^2}\right) \quad \forall z \in \mathcal{Z} .
$$

根据定义 2.8，可以计算得到：

$$
\mathrm{KL}\left(\mathrm{P}^*, \hat{\mathrm{P}}\right)=\int_Z \log \left(\frac{\mathrm{dP}^*}{\mathrm{~d} \hat{\mathrm{P}}}(z)\right) \mathrm{dP}^*(z)=\frac{\theta^{\top} \hat{\Sigma} \theta}{2\left(\lambda^*\right)^2}=r,
$$

其中第二步和第三步分别由 Radon-Nikodym 导数公式和基本代数计算得出。因此，$\mathrm{P}^*$ 在 (6.11b) 中是可行的。此外，其目标函数值为：

$$
\mathrm{Bp}[\ell(Z)]=\theta^{\top} \mu^*=\hat{\mu}^{\top} \theta+(2 r)^{\frac{1}{2}}\left(\theta^{\top} \hat{\Sigma} \theta\right)^{\frac{1}{2}}
$$

由于 $\mathrm{P}^*$ 和 $\left(\lambda_0^*, \lambda^*\right)$ 的目标函数值相等，并且根据定理 4.14 的弱对偶性，$\mathrm{P}^*$ 是原问题的最优解，而 $\left(\lambda_0^*, \lambda^*\right)$ 是对偶问题的最优解。至此，证明完成。

---

命题 6.12 由 Hu 和 Hong（2013）提出。它与风险敏感控制理论（Hansen 和 Sargent 2008）相似。在这类文献中，一个虚构的对手可以任意扰动最优控制问题的外部噪声分布，但会因与高斯基线模型的 Kullback-Leibler 散度而受到惩罚。


## 6.10 关于全变差球的最坏期望

考虑最坏期望问题：

$$
\sup _{\mathrm{P} \in \mathcal{P}(Z)}\left\{\mathbb{E}_{\mathrm{P}}[\ell(Z)]: \mathrm{TV}(\mathrm{P}, \hat{\mathrm{P}}) \leq r\right\},
$$

它在以 $\hat{\mathrm{P}} \in \mathcal{P}(\mathcal{Z})$ 为中心、半径为 $r \in [0,1]$ 的全变差球内最大化 $l(Z)$ 的期望。回顾第 2.2.3 节，全变差距离是一个 $\phi$-散度，其对应的熵函数满足 $\phi(s) = \frac{1}{2}|s - 1|, \forall s \geq 0$，并且对所有 $s < 0$，$\phi(s) = \infty$。同时回顾，两分布间的全变差距离上界为 1，当两分布互为正交时达到此界。简单计算显示，共轭熵函数满足：

$$
\phi^*(t) = 
\begin{cases} 
\max \left\{t + \frac{1}{2}, 0 \right\} - \frac{1}{2}, & t \leq \frac{1}{2}, \\
+\infty, & t > \frac{1}{2}.
\end{cases}
$$

根据定理 4.14，问题 (6.12a) 的对偶形式为：

$$
\begin{aligned}
\inf_{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}_+} \quad & \lambda_0 + \lambda \left(r - \frac{1}{2}\right) + \mathbb{E}_{\hat{\mathrm{P}}}\left[\max \left\{\ell(Z) - \lambda_0 + \frac{\lambda}{2}, 0\right\}\right], \\
\text{s.t.} \quad & \lambda_0 + \frac{\lambda}{2} \geq \sup_{z \in \mathcal{Z}} \ell(z).
\end{aligned}
$$

当 $\mathcal{Z}$ 是紧集时，(6.12a) 和 (6.12b) 问题均可闭式求解。

---

**命题 6.13**（关于全变差球的最坏期望）

假设 $\mathcal{Z} \subseteq \mathbb{R}^d$ 是紧集，$\hat{\mathrm{P}} \in \mathcal{P}(\mathcal{Z})$ 且 $r \in (0,1)$，定义 $\beta_r = 1 - r$。此外，假设 $\mathbb{E}_{\hat{\mathrm{P}}}[\ell(Z)] > -\infty$ 且 $\ell$ 是上半连续的。那么，(6.12a) 和 (6.12b) 的最优值均为：

$$
\left(1 - \beta_r\right) \cdot \sup_{z \in \mathcal{Z}} \ell(z) + \beta_r \cdot \mathrm{CVaR}_{\hat{\mathrm{P}}}[\ell(Z)].
$$

命题 6.13 的证明将表明 (6.12a) 和 (6.12b) 均是可解的。事实上，我们将分别为 (6.12a) 和 (6.12b) 构造最优解 $\mathrm{P}^*$ 和 $\left(\lambda_0^*, \lambda^*\right)$。由于这些最优解的具体描述较为繁琐，此处省略不表。然而，如果 $\ell(Z)$ 在 $\hat{\mathrm{P}}$ 下具有连续分布，则 $\mathrm{P}^*$ 可以有更简单直观的描述。在这种情况下，$\mathrm{P}^*$ 是通过将 $\hat{\mathrm{P}}$ 中与高损失 $\ell(z) \geq \beta_r - \mathrm{VaR}_{\hat{\mathrm{P}}}[\ell(Z)]$ 相关的所有结果的概率质量移到与最高可能损失 $\ell(z) = \max_{z' \in \mathcal{Z}} \ell(z')$ 相关的某一结果 $z \in \mathcal{Z}$ 获得的。

---

**命题 6.13 的证明**

为方便表示，设 $\bar{\ell} = \sup_{z \in \mathcal{Z}} \ell(z)$。首先关注对偶问题 (6.12b)，固定任意 $\lambda \geq 0$。注意到对偶目标函数关于 $\lambda_0$ 是非减的，因此 $\lambda_0$ 的部分最小化问题由 $\lambda_0^*(\lambda) = \bar{\ell} - \frac{\lambda}{2}$ 解出。将该参数化的最优解代入 (6.12b)，显示对偶问题的最优值为：

$$
\begin{aligned}
\bar{\ell} + \inf_{\lambda \in \mathbb{R}_+} \left\{\lambda(r - 1) + \mathbb{E}_{\hat{\mathrm{P}}}\left[\max\{\ell(Z) - \bar{\ell} + \lambda, 0\}\right]\right\}.
\end{aligned}
$$

通过代换 $\tau = \bar{\ell} - \lambda$，该表达式等价于：

$$
r \cdot \bar{\ell} + (1 - r) \inf_{\tau \leq \bar{\ell}} \left\{\tau + (1 - r)^{-1} \cdot \mathbb{E}_{\hat{\mathrm{P}}}\left[\max\{\ell(Z) - \tau, 0\}\right]\right\}.
$$

根据定义 5.10，对 $\tau$ 的最小化问题的值为：

$$
\beta_r \cdot \mathrm{CVaR}_{\hat{\mathrm{P}}}[\ell(Z)],
$$

其中 $\beta_r = 1 - r$。该最小值在 $\tau^* = \beta_r - \mathrm{VaR}_{\hat{\mathrm{P}}}[\ell(Z)]$ 处取得，且 $\tau^* \leq \bar{\ell}$。综上，我们证明了问题 (6.12b) 的最优值为：

$$
\left(1 - \beta_r\right) \cdot \bar{\ell} + \beta_r \cdot \mathrm{CVaR}_{\hat{\mathrm{P}}}[\ell(Z)].
$$

为了构造原问题的最优解，首先假设 $\hat{\mathrm{P}}(\ell(Z) < \bar{\ell}) \leq r$，此时 $\beta_r \cdot \mathrm{CVaR}_{\hat{\mathrm{P}}}[\ell(Z)] = \bar{\ell}$。因此，对偶问题 (6.12b) 的最优值简化为 $\bar{\ell}$，该值可以通过任意将概率质量从 $\{z \in \mathcal{Z}: \ell(z) < \bar{\ell}\}$ 移到 $\{z \in \mathcal{Z}: \ell(z) = \bar{\ell}\}$ 的分布 $P^*$ 达到。

接下来，假设 $\hat{\mathrm{P}}(\ell(Z) < \bar{\ell}) > r$，这意味着 $\beta_r - \operatorname{VaR}_p[\ell(Z)] < \bar{\ell}$。在这种情况下，我们将 $\mathcal{Z}$ 划分为以下四个子集：

$$
\begin{aligned}
& Z_1 = \left\{z \in \mathcal{Z} : \beta_r - \operatorname{VaR}_p[\ell(Z)] > \ell(z)\right\}, \\
& Z_2 = \left\{z \in \mathcal{Z} : \bar{\ell} > \ell(z) = \beta_r - \operatorname{VaR}_p[\ell(Z)]\right\}, \\
& Z_3 = \left\{z \in \mathcal{Z} : \bar{\ell} > \ell(z) > \beta_r - \operatorname{VaR}_p[\ell(Z)]\right\}, \\
& Z_4 = \{z \in \mathcal{Z} : \bar{\ell} = \ell(z)\}.
\end{aligned}
$$

注意，$Z_1$ 和 $Z_3$ 可能为空，而 $Z_2$ 和 $Z_4$ 必须非空。我们还定义 $\hat{P}_i$ 为 $\hat{P}$ 在事件 $Z \in Z_i$ 上的条件分布 $(i \in [4])$，并定义 $\mathrm{U}_{\mathcal{Z}_4}$ 为 $\mathcal{Z}_4$ 上的均匀分布。接下来，我们设置：

$$
\begin{aligned}
\mathrm{P}^* = \left(\beta_r - \hat{\mathrm{P}}(Z \in Z_3) - \hat{\mathrm{P}}(Z \in Z_4)\right) \cdot \hat{\mathrm{P}}_2 
+ \hat{\mathrm{P}}(Z \in Z_3) \cdot \hat{\mathrm{P}}_3 
+ \hat{\mathrm{P}}(Z \in Z_4) \cdot \hat{\mathrm{P}}_4 
+ \left(1 - \beta_r\right) \cdot \mathrm{U}_{\mathcal{Z}_4}.
\end{aligned}
$$

因此，$\mathrm{P}^*$ 是四个概率分布的混合分布。由于非负的混合概率之和为 1，$\mathrm{P}^*$ 是一个概率分布。使用 $\rho = \hat{\mathrm{P}} + \mathrm{U}_{\mathcal{Z}_4}$ 作为 $\hat{\mathrm{P}}$ 和 $\mathrm{P}^*$ 的支配测度，并回顾 $\phi(s) = \frac{1}{2} |s - 1|$（当 $s \geq 0$ 时），我们有：

$$
\begin{aligned}
\mathrm{TV}(\mathrm{P}^*, \hat{\mathrm{P}}) 
& = \mathrm{D}_\phi(\mathrm{P}^*, \hat{\mathrm{P}}) \\
& = \frac{1}{2} \sum_{i=1}^4 \int_{Z_i} \left|\frac{\mathrm{d}\mathrm{P}^*}{\mathrm{d}\rho}(z) - \frac{\mathrm{d}\hat{\mathrm{P}}}{\mathrm{d}\rho}(z)\right| \mathrm{d}\rho(z) \\
& = \hat{\mathrm{P}}(Z \in Z_1) 
+ \left(\hat{\mathrm{P}}(Z \in Z_2) + \hat{\mathrm{P}}(Z \in Z_3) + \hat{\mathrm{P}}(Z \in Z_4) - \beta_r\right) 
+ 0 + \left(1 - \beta_r\right) \\
& = r,
\end{aligned}
$$

其中第三步利用了 $\mathrm{P}^*$ 的定义以及以下关系：

$$
\hat{\mathrm{P}}(Z \in Z_2) + \hat{\mathrm{P}}(Z \in Z_3) + \hat{\mathrm{P}}(Z \in Z_4) = \hat{\mathrm{P}}(\ell(Z) \geq \beta_r - \operatorname{VaR}_p[\ell(Z)]) \geq \beta_r,
$$

最后一步利用了 $\beta_r = 1 - r$ 的定义。因此，$\mathrm{P}^*$ 在 (6.12a) 中是可行的。此外，在 (6.12a) 中，$\mathrm{P}^*$ 的目标函数值为：

$$
\begin{aligned}
\mathbb{E}_{\mathrm{P}^*}[\ell(Z)] 
& = \left(1 - \beta_r\right) \cdot \bar{\ell} 
+ \mathbb{E}_{\hat{\mathrm{P}}}\left[\ell(Z) \mid \ell(Z) > \beta_r - \operatorname{VaR}_p[\ell(Z)]\right] 
\cdot \hat{\mathrm{P}}(\ell(Z) > \beta_r - \operatorname{VaR}_p[\ell(Z)]) \\
& \quad + \mathbb{E}_{\hat{\mathrm{P}}}\left[\ell(Z) \mid \ell(Z) = \beta_r - \operatorname{VaR}_p[\ell(Z)]\right] 
\cdot \left(\beta_r - \hat{\mathrm{P}}(\ell(Z) > \beta_r - \operatorname{VaR}_p[\ell(Z)])\right) \\
& = \left(1 - \beta_r\right) \cdot \bar{\ell} + \beta_r \cdot \mathrm{CVaR}_p[\ell(Z)].
\end{aligned}
$$

这里，第二步来源于 (Follmer and Schied 2008, 定理 4.47 和备注 4.48)。注意，如果 $\ell(Z)$ 在 $\hat{\mathrm{P}}$ 下的边际分布是连续的，那么上述推导将更加简化。在这种情况下，我们有：

$$
\hat{\mathrm{P}}(\ell(Z) > \beta_r - \operatorname{VaR}_p[\ell(Z)]) = \beta_r,
$$

并且：

$$
\mathbb{E}_{\hat{\mathrm{P}}}\left[\ell(Z) \mid \ell(Z) > \beta_r - \operatorname{VaR}_p[\ell(Z)]\right] = \beta_r - \operatorname{CVaR}_p[\ell(Z)].
$$

无论 $\hat{\mathrm{P}}$ 的形式如何，$\mathrm{P}^*$ 在 (6.12a) 中的目标函数值与 (6.12b) 的最优值相等。因此，根据定理 4.14 的弱对偶性，$\mathrm{P}^*$ 是原问题 (6.12a) 的最优解。至此，证明完成。


Jiang 和 Guan (2018) 以及 Shapiro (2017) 研究了问题 (6.12a) 的一种变体，该问题在受限的全变差球上取最大值。因此，他们在 (6.12a) 中额外引入了约束 $\mathrm{P} \ll \hat{\mathrm{P}}$。结果受限问题的上确界为：

$$
\left(1 - \beta_r\right) \cdot \operatorname{ess} \sup_\ell[\ell(Z)] + \beta_r \cdot \mathrm{CVaR}_F[\ell(Z)],
$$

其值可能严格小于 (6.13)。此外，如果 $\ell(Z)$ 在 $\hat{\mathrm{P}}$ 下具有连续边际分布，则此问题的上确界不再能够取到。


## 6.11 关于 Lévy-Prokhorov 球的最坏期望

考虑最坏期望问题：

$$
\sup _{\mathrm{P} \in \mathcal{P}(Z)}\left\{\mathbb{E}_{\mathrm{P}}[\ell(Z)] : \operatorname{LP}(\mathrm{P}, \hat{\mathrm{P}}) \leq r\right\},
$$

它在以 $\hat{\mathrm{P}} \in \mathcal{P}(\mathcal{Z})$ 为中心、半径为 $r \in [0,1]$ 的 Lévy-Prokhorov 球上最大化 $\ell(Z)$ 的期望。假设 Lévy-Prokhorov 距离由 $\mathbb{R}^d$ 上的某种范数 $\|\cdot\|$ 所诱导。根据命题 2.22，半径为 $r \in (0,1)$ 的 Lévy-Prokhorov 球等价于以下形式的最优传输不确定集：

$$
\mathcal{P} = \left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}) : \mathrm{OT}_c(\mathrm{P}, \hat{\mathrm{P}}) \leq r\right\},
$$

其中传输成本函数 $c_r$ 定义为：

$$
c_r(z, z') = \mathbf{1}_{\|z - z'\| > r}.
$$

根据定理 4.18，问题 (6.14a) 的对偶形式为：

$$
\inf_{\lambda \geq 0} \left\{\lambda r + \mathbb{E}_{\hat{\mathrm{P}}}\left[\sup_{z \in \mathcal{Z}} \ell(z) - \lambda c_r(z, \hat{\mathrm{P}})\right]\right\},
$$

只要 $\ell$ 是上半连续的。如果 $\mathcal{Z}$ 是紧集，则可以利用命题 6.13 闭式求解 (6.14a) 和 (6.14b)。

---

**命题 6.14**（关于 Lévy-Prokhorov 球的最坏期望）

假设 $\mathcal{Z} \subseteq \mathbb{R}^d$ 是紧集，$\hat{\mathrm{P}} \in \mathcal{P}(\mathcal{Z})$ 且 $r \in (0,1)$，定义 $\beta_r = 1 - r$。此外，假设 $\mathbb{E}_{\hat{\mathrm{P}}}[\ell(Z)] > -\infty$ 且 $\ell$ 是上半连续的。那么，(6.14a) 和 (6.14b) 的最优值均为：

$$
\left(1 - \beta_r\right) \cdot \sup_{z \in \mathcal{Z}} \ell(z) + \beta_r \cdot \mathrm{CVaR}_{\hat{\mathrm{P}}}[\ell_r(Z)],
$$

其中：

$$
\ell_r(Z) = \sup_{z \in \mathcal{Z}} \{\ell(z) : \|z - z'\| \leq r\},
$$

是一个对抗性损失函数，为 $\mathcal{Z} \in \mathcal{Z}$ 分配其 $r$ 邻域内的最坏损失。

命题 6.14 的证明将表明 (6.14a) 和 (6.14b) 是可解的。然而，最优解的具体描述较为繁琐，此处省略不表。注意，由于 (Berge 1963, 定理 2, 第 116 页)，对抗性损失函数 $\ell_r$ 继承了 $\ell$ 的上半连续性。

---

**命题 6.14 **的证明需要的引理**
假设 $\mathcal{Z} \subseteq \mathbb{R}^d$ 是紧集，$\ell$ 是上半连续的，$z \in \mathcal{Z}$ 且 $r, \lambda \geq 0$。则以下等式成立：

$$
\sup_{z \in \mathcal{Z}}\{\ell(z) - \lambda \cdot \mathbf{1}_{\|z - z'\| > r}\} 
= \sup_{z \in \mathcal{Z}}\{\ell_r(z) - \lambda \cdot \mathbf{1}_{z \neq z'}\}.
$$

**引理 6.15**

假设 $\mathcal{Z} \subseteq \mathbb{R}^d$ 是紧集，$\ell$ 是上半连续的，$\ell \in \mathcal{Z}$ 且 $r, \lambda \geq 0$。则以下等式成立：

$$
\sup _{z \in \mathcal{Z}}\left\{\ell(z) - \lambda \cdot 1_{\|z - \hat{z}\| > r}\right\} 
= \sup _{z \in \mathcal{Z}}\left\{\ell_r(z) - \lambda \cdot 1_{z \neq \hat{z}}\right\}.
$$

**证明**

为了简化符号，我们引入两个辅助函数 $f$ 和 $g$，它们从 $\mathcal{Z}$ 映射到扩展实数集 $\overline{\mathbb{R}}$，定义为：

$$
f(z) = \ell(z) - \lambda \cdot 1_{\|z - \hat{z}\| > r}, \quad 
g(z) = \ell_r(z) - \lambda \cdot 1_{z \neq \hat{z}}, \quad \forall z \in \mathcal{Z}.
$$

注意，$f$ 和 $g$ 均为上半连续的。

首先，选择 $z^* \in \arg \max_{z \in \mathcal{Z}} f(z)$，该点的存在性是因为 $\mathcal{Z}$ 是紧集且 $f$ 是上半连续的。如果 $\|z^* - \hat{z}\| > r$，则根据 $\ell_r$ 的定义，有：

$$
\sup _{z \in \mathcal{Z}} f(z) = f(z^*) = \ell(z^*) - \lambda \leq \ell_r(z^*) - \lambda = g(z^*) \leq \sup _{z \in \mathcal{Z}} g(z)。
$$

另一方面，如果 $\|z^* - \hat{z}\| \leq r$，则有：

$$
\sup _{z \in \mathcal{Z}} f(z) = f(z^*) = \ell(z^*) \leq \ell_r(\hat{z}) = g(\hat{z}) \leq \sup _{z \in \mathcal{Z}} g(z)。
$$

接下来，选择 $z \in \arg \max_{z \in \mathcal{Z}} g(z)$。如果 $z \neq \hat{z}$，则令 $z^* \in \arg \max_{z \in \mathcal{Z}} \ell(z)$，有：

$$
\sup _{z \in \mathcal{Z}} g(z) = g(z) = \ell_r(z) - \lambda \leq \ell(z^*) - \lambda \leq f(z^*) = \sup _{z \in \mathcal{Z}} f(z)，
$$

其中不等式来源于 $z^*$ 的定义及 $\lambda$ 的非负性。反之，如果 $z = \hat{z}$，则令 $z_r^* \in \arg \max_{z' \in \mathcal{Z}}\{\ell(z') : \|z' - \hat{z}\| \leq r\}$，有：

$$
\sup _{z \in \mathcal{Z}} g(z) = g(\hat{z}) = \ell_r(\hat{z}) = \ell(z_r^*) = f(z_r^*) = \sup _{z \in \mathcal{Z}} f(z)。
$$

因此，命题得证。

---
**命题 6.14 的证明**

引理 6.15 允许我们用对抗性损失函数 $\ell_r$ 将对偶问题 (6.14b) 重写为：

$$
\inf_{\lambda \geq 0} \left\{\lambda r + \mathbb{E}_{\hat{\mathrm{P}}}\left[\sup_{z \in \mathcal{Z}} \ell_r(z) - \lambda \cdot 1_{z \neq \hat{z}}\right]\right\}.
$$

由于 $r > 0$，$\mathcal{Z}$ 是紧集，且 $\ell_r$ 是上半连续的，根据定理 4.18，(6.16) 是一个强对偶问题，其最大化形式为：

$$
\sup_{\mathrm{P} \in \mathcal{P}(\mathcal{Z})} \left\{\mathbb{E}_{\mathrm{P}}[\ell_r(Z)] : \mathrm{OT}_c(\mathrm{P}, \hat{\mathrm{P}}) \leq r\right\} = 
\sup_{\mathrm{P} \in \mathcal{P}(\mathcal{Z})} \left\{\mathbb{E}_{\mathrm{P}}[\ell_r(Z)] : \mathrm{TV}(\mathrm{P}, \hat{\mathrm{P}}) \leq r\right\},
$$

其中等式成立是因为 $\mathrm{TV} = \mathrm{OT}_c$（见命题 2.24）。由于 $\sup_{z \in \mathcal{Z}} \ell_r(z) = \sup_{z \in \mathcal{Z}} \ell(z) = \bar{\ell}$，命题 6.13 直接表明：

$$
\sup_{\mathrm{P} \in \mathcal{P}(\mathcal{Z})} \left\{\mathbb{E}_{\mathrm{P}}[\ell_r(Z)]\right\} = \left(1 - \beta_r\right) \cdot \bar{\ell} + \beta_r \cdot \mathrm{CVaR}_{\hat{\mathrm{P}}}[\ell_r(Z)]。
$$

假设 $\psi : \mathcal{Z} \rightarrow \mathcal{Z}$ 是一个 Borel 可测函数，满足：

$$
\psi(\hat{z}) \in \arg \max_{z \in \mathcal{Z}}\{\ell(z) : \|z - \hat{z}\| \leq r\}, \quad \forall \hat{z} \in \mathcal{Z},
$$

其存在性由 (Rockafellar 和 Wets 2009, 推论 14.6 和定理 14.37) 保证。令 $\hat{\mathrm{P}}_\psi = \hat{\mathrm{P}} \circ \psi^{-1}$ 为 $\hat{\mathrm{P}}$ 在 $\psi$ 下的推前分布。接下来，在假设 $\hat{\mathrm{P}}_\psi(\ell(Z) < \bar{\ell}) > r$ 下构造原问题的最优解。

将 $\mathcal{Z}$ 划分为以下四个子集：

$$
\begin{aligned}
\mathcal{Z}_1 &= \{z \in \mathcal{Z} : \beta_r - \mathrm{VaR}_{\hat{\mathrm{P}}}[\ell(Z)] > \ell(z)\}, \\
\mathcal{Z}_2 &= \{z \in \mathcal{Z} : \bar{\ell} > \ell(z) = \beta_r - \mathrm{VaR}_{\hat{\mathrm{P}}}[\ell(Z)]\}, \\
\mathcal{Z}_3 &= \{z \in \mathcal{Z} : \bar{\ell} > \ell(z) > \beta_r - \mathrm{VaR}_{\hat{\mathrm{P}}}[\ell(Z)]\}, \\
\mathcal{Z}_4 &= \{z \in \mathcal{Z} : \bar{\ell} = \ell(z)\}。
\end{aligned}
$$

构造混合分布：

$$
\mathbb{P}^* = \left(\beta_r - \hat{\mathrm{P}}_\psi(\mathcal{Z}_3) - \hat{\mathrm{P}}_\psi(\mathcal{Z}_4)\right) \cdot \hat{\mathrm{P}}_2 + 
\hat{\mathrm{P}}_\psi(\mathcal{Z}_3) \cdot \hat{\mathrm{P}}_3 + 
\hat{\mathrm{P}}_\psi(\mathcal{Z}_4) \cdot \hat{\mathrm{P}}_4 + 
\left(1 - \beta_r\right) \cdot \mathrm{U}_{\mathcal{Z}_4}。
$$

如命题 6.13 的证明所示，$\mathbb{P}^*$ 满足对偶问题的约束条件，并且其目标函数值与对偶问题的最优值匹配。因此，$\mathbb{P}^*$ 也是原问题的最优解。

如果 $\hat{\mathrm{P}}_\psi(\ell(Z) < \bar{\ell}) \leq r$，则原问题的最优解构造更为简单，此处省略。


本节结果最初由 Bennouna 和 Van Parys (2023) 在假设名义分布 $\hat{\mathrm{P}}$ 为离散分布的条件下得到。


## 6.12 关于 $\infty$-Wasserstein 球的最坏期望

考虑最坏期望问题：

$$
\sup _{\mathbb{P} \in \mathcal{P}(\mathcal{Z})}\left\{\mathbb{E}_{\mathbb{P}}[\ell(Z)] : \mathrm{W}_{\infty}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\},
$$

它在以 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 为中心、半径为 $r \in \mathbb{R}_{+}$ 的 $\infty$-Wasserstein 球上最大化 $\ell(Z)$ 的期望。假设 $\infty$-Wasserstein 距离由 $\mathbb{R}^d$ 上的某种范数 $\|\cdot\|$ 所诱导。根据命题 2.27，$\infty$-Wasserstein 不确定集等价于最优传输不确定集：

$$
\mathcal{P} = \left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}) : \mathrm{OT}_{c_r}(\mathbb{P}, \hat{\mathbb{P}}) \leq 0\right\},
$$

其中传输成本函数 $c_r$ 定义为：

$$
c_r(z, \hat{z}) = \mathbb{1}_{\|z - \hat{z}\| > r}。
$$

我们强调，尽管 $\infty$-Wasserstein 球的半径为 $r$，但对应的最优传输不确定集 $\mathcal{P}$ 的半径为 0。根据定理 4.18，问题 (6.17a) 的对偶形式为：

$$
\inf _{\lambda \in \mathbb{R}_{+}} \mathbb{E}_{\hat{\mathbb{P}}}\left[\sup _{z \in \mathcal{Z}} \ell(z) - \lambda c_r(z, \hat{Z})\right],
$$

只要 $\ell$ 是上半连续的。如果 $\mathcal{Z}$ 是紧集，则可以闭式求解问题 (6.17a) 和 (6.17b)。

---
**命题 6.16**  （关于 $\infty$-Wasserstein 球的最坏期望）

假设 $\mathcal{Z} \subseteq \mathbb{R}^d$ 是紧集，$\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$, $r \in \mathbb{R}_{+}$, $\mathbb{E}_{\hat{\mathbb{P}}}[\ell(\hat{Z})] > -\infty$ 且 $\ell$ 是上半连续的。定义对抗性损失函数为：

$$
\ell_r(\hat{z}) = \sup_{z \in \mathcal{Z}} \{\ell(z) : \|z - \hat{z}\| \leq r\},
$$

并令 $\psi : \mathcal{Z} \to \mathcal{Z}$ 为满足以下条件的 Borel 函数：

$$
\psi(\hat{z}) \in \arg \max_{z \in \mathcal{Z}} \{\ell(z) : \|z - \hat{z}\| \leq r\}, \quad \forall \hat{z} \in \mathcal{Z}。
$$

那么，原问题 (6.17a) 的解为 $\mathbb{P}^\star = \hat{\mathbb{P}} \circ \psi^{-1}$。此外，(6.17a) 和 (6.17b) 的最优值均为：

$$
\mathbb{E}_{\hat{\mathbb{P}}}[\ell_r(\hat{Z})]。
$$

---

**证明**

Borel 函数 $\psi$ 的存在性由 (Rockafellar 和 Wets 2009, 推论 14.6 和定理 14.37) 保证。这确保了推前分布 $\mathbb{P}^\star = \hat{\mathbb{P}} \circ \psi^{-1}$ 的良定义性。此外，有：

$$
\mathrm{W}_{\infty}(\mathbb{P}^\star, \hat{\mathbb{P}}) = \inf \left\{r' \geq 0 : \mathrm{OT}_{c_{r'}}(\mathbb{P}^\star, \hat{\mathbb{P}}) \leq 0\right\} \leq r，
$$

其中等式来源于命题 2.27 和 $d(z, \hat{z}) = \|z - \hat{z}\|$，不等式成立是因为 $\mathrm{OT}_{c_r}(\mathbb{P}^\star, \hat{\mathbb{P}}) = 0$。此外，还有：

$$
\mathbb{E}_{\mathbb{P}^\star}[\ell(Z)] = \mathbb{E}_{\hat{\mathbb{P}}}[\ell(\psi(Z))] = \mathbb{E}_{\hat{\mathbb{P}}}[\ell_r(Z)]。
$$

接下来，注意到对于任意固定的 $\hat{z} \in \mathcal{Z}$，$\sup_{z \in \mathcal{Z}} \ell(z) - \lambda c_r(z, \hat{z})$ 关于 $\lambda$ 是单调递减的。此外，由于 $\mathcal{Z}$ 是紧集且 $\ell$ 是上半连续的，其上界为有限常数 $\sup_{z \in \mathcal{Z}} \ell(z)$。根据单调收敛定理，对偶问题 (6.17b) 的最优值为：

$$
\inf_{\lambda \in \mathbb{R}_{+}} \mathbb{E}_{\hat{\mathbb{P}}}\left[\sup_{z \in \mathcal{Z}} \ell(z) - \lambda c_r(z, \hat{Z})\right] 
= \mathbb{E}_{\hat{\mathbb{P}}}\left[\inf_{\lambda \in \mathbb{R}_{+}} \sup_{z \in \mathcal{Z}} \ell(z) - \lambda c_r(z, \hat{Z})\right] 
= \mathbb{E}_{\hat{\mathbb{P}}}[\ell_r(\hat{Z})]，
$$

其中第二个等式成立是因为 $\mathcal{Z}$ 是紧集。根据定理 4.18 的弱对偶性，可知 $P^\star$ 是原问题 (6.17a) 的最优解。

---

命题 6.16 表明，关于 $\infty$-Wasserstein 球的原始损失 $\ell(Z)$ 的最坏期望，与名义分布 $\hat{\mathbb{P}}$ 下对抗性损失 $\ell_r(\hat{Z})$ 的期望一致。该结果最早由 Gao 等人 (2017) 针对离散名义分布提出，随后 Gao 等人 (2024) 将其推广至一般名义分布。

损失函数 $\ell_r$ 在机器学习中被广泛用于神经网络的对抗性训练（Szegedy 等人，2014；Goodfellow 等人，2015）。命题 6.16 揭示了对抗性训练与基于 $\infty$-Wasserstein 不确定集的分布鲁棒优化之间的紧密联系。这一联系在对抗性分类的背景下得到了进一步探讨（García Trillos 和 García Trillos, 2022；Bungert 等人，2023, 2024；Pydi 和 Jog, 2024；Frank 和 Niles-Weed, 2024）。


## 6.13 关于 1-Wasserstein 球的最坏期望

考虑最坏期望问题：

$$
\sup _{\mathcal{P} \in \mathcal{P}(\mathcal{Z})}\left\{\mathbb{E}_{\mathrm{P}}[\ell(Z)]: \mathrm{W}_1(\mathrm{P}, \hat{\mathrm{P}}) \leq r\right\},
$$

它在以 $\hat{P} \in \mathcal{P}(\mathcal{Z})$ 为中心、半径为 $r \in \mathbb{R}_{+}$ 的 1-Wasserstein 球上最大化 $\ell(Z)$ 的期望。假设 1-Wasserstein 距离由 $\mathbb{R}^d$ 上的某种范数 $\|\cdot\|$ 所诱导。因此，1-Wasserstein 不确定集等价于最优传输不确定集：

$$
\mathcal{P} = \left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}) : \mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\}，
$$

其中传输成本函数 $c$ 定义为：

$$
c(z, \hat{z}) = \|z - \hat{z}\|。
$$

根据定理 4.18，问题 (6.18a) 的对偶形式为：

$$
\inf _{\lambda \geq 0} \lambda r + \mathrm{E}_{\hat{\mathrm{P}}}\left[\sup _{z \in \mathcal{Z}} \ell(z) - \lambda\|z - \hat{Z}\|\right],
$$

只要 $\ell$ 是上半连续的。如果 $\mathcal{Z} = \mathbb{R}^d$ 且 $\ell$ 是凸函数且 Lipschitz 连续，则可以闭式求解问题 (6.18a) 和 (6.18b)。

---

**命题 6.17** （关于 1-Wasserstein 球的最坏期望）

假设 $\mathcal{Z} = \mathbb{R}^d, \hat{\mathbf{P}} \in \mathcal{P}(\mathcal{Z})$ 且 $r \in \mathbb{R}_{+}$。如果 $\mathbb{E}_{\hat{p}}[\ell(\hat{Z})] > -\infty$ 且 $\ell$ 是凸函数且 Lipschitz 连续，则问题 (6.18a) 和 (6.18b) 的最优值为：

$$
\mathbb{E}_{\hat{p}}[\ell(\hat{Z})] + r \operatorname{lip}(\ell)。
$$

---

**证明**

由于凸函数 $\ell$ 是 Lipschitz 连续的，因此它特别是适当且闭合的。根据 Fenchel-Moreau 定理（引理 4.2），$\ell$ 具有如下对偶表示：

$$
\ell(z) = \sup _{y \in \operatorname{dom}(\ell^*)} z^\top y - \ell^*(y)，
$$

其中 $\ell^*$ 表示 $\ell$ 的凸共轭函数。换言之，$\ell$ 是由仿射函数 $f_y(z) = y^\top z - \ell^*(y)$ 的逐点上确界构成，$f_y$ 的参数 $y \in \operatorname{dom}(\ell^*)$。根据 Hölder 不等式，有：

$$
\left|f_y(z) - f_y(\hat{z})\right| = \left|y^\top(z - \hat{z})\right| \leq \|y\|_* \|z - \hat{z}\|，
$$

其中 $\|\cdot\|_*$ 表示 $\|\cdot\|$ 的对偶范数。由于 Hölder 不等式是紧的，$f_y$ 是 Lipschitz 连续的，其 Lipschitz 模量为 $\operatorname{lip}(f_y) = \|y\|_*$。此外，由仿射函数逐点上确界的 Lipschitz 模量等于对应仿射函数 Lipschitz 模量的上确界可知，$\ell$ 的 Lipschitz 模量为：

$$
\operatorname{lip}(\ell) = \sup _{y \in \operatorname{dom}(\ell^*)} \|y\|_* = \max _{y \in \mathrm{cl}(\operatorname{dom}(\ell^*))} \|y\|_*。
$$

由于假设 $\operatorname{lip}(\ell) < \infty$，上述最大值在某些 $y^\star \in \mathbb{R}^d$ 处达到。定义 $z^\star$ 为 $\max_{\|z\| \leq 1}(y^\star)^\top z$ 的任意最优解。因此有：

$$
(y^\star)^\top z^\star = \|y^\star\|_* = \operatorname{lip}(\ell)。
$$


问题 (6.18b) 的对偶解为 $\lambda^\star = \operatorname{lip}(\ell)$，其值为：

$$
\lambda^\star r + \mathbb{E}_{\hat{\mathrm{P}}}\left[\sup _{z \in \mathcal{Z}} \ell(z) - \lambda^\star \|z - \hat{Z}\|\right] = \mathbb{E}_{\hat{\mathrm{P}}}[\ell(\hat{Z})] + r \operatorname{lip}(\ell)。
$$

这表明对偶问题的值与原问题的值一致。

---

命题 6.17 表明，对 1-Wasserstein 球的分布鲁棒最坏期望等价于 Lipschitz 正则化的名义损失：

$$
\mathbb{E}_{\hat{\mathrm{P}}}[\ell(Z)] + r \operatorname{lip}(\ell)。
$$

这种鲁棒化与 Lipschitz 正则化的联系最早由 Mohajerin Esfahani 和 Kuhn (2018) 提出，并为统计学和机器学习中常用的正则化技术提供了概率解释（Shafieezadeh-Abadeh 等人，2015, 2019）。对于非凸损失函数的进一步扩展参见 (Blanchet 等人，2019；Ho-Nguyen 和 Wright，2023；Shafiee 等人，2023；Gao 等人，2024；Zhang 等人，2024)。


## 6.14 1-Wasserstein 风险

考虑一个可以表示为不同风险水平 $\beta \in [0,1]$ 的 CVaR 叠加的与法律无关的风险测度 $\varrho$。具体而言，假设

$$
\varrho_{\mathrm{P}}[\ell(Z)] = \int_0^1 \beta - \mathrm{CVaR}_{\mathrm{P}}[\ell(Z)] \, \mathrm{d} \sigma(\beta),
$$

对于所有 $\mathcal{P} \in \mathcal{P}(\mathcal{Z})$，其中 $\sigma$ 是定义在 $[0,1]$ 上的概率分布，并满足 $\int_0^1 \beta^{-1} \, \mathrm{d} \sigma(\beta) < \infty$。具有这些性质的任何 $\varrho$ 称为**谱风险测度**（Acerbi 2002），而 (6.19) 称为 $\varrho$ 的**Kusuoka 表示**（Kusuoka 2001，Shapiro 2013）。

如果只知道 $Z$ 的分布接近 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$，那么自然会通过 1-Wasserstein 风险来量化不确定损失 $\ell(Z)$ 的风险性，即在 $\hat{\mathbb{P}}$ 周围的 1-Wasserstein 球中对所有分布 $\mathrm{P}$ 的 $\varrho_{\mathrm{P}}[\ell(Z)]$ 的上确界。当 $\mathcal{Z} = \mathbb{R}^d$ 且 $\ell$ 是凸的且 Lipschitz 连续时，可以用闭式形式表示 1-Wasserstein 风险。

---

**命题 6.18** （1-Wasserstein 风险）

令 $\varrho$ 为满足 (6.19) 的谱风险测度，且 $\int_0^1 \beta^{-1} \, \mathrm{d} \sigma(\beta) < \infty$。假设 $\hat{\mathbb{P}} \in \mathcal{P}(\mathbb{R}^d)$ 且 $\mathrm{B}_{\hat{\mathrm{P}}}[\|Z\|] < \infty$，其中 $\|\cdot\|$ 是 $\mathbb{R}^d$ 上的某种范数。定义

$$
\mathcal{P} = \left\{\mathbb{P} \in \mathcal{P}(\mathbb{R}^d) : \mathrm{W}_1(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\},
$$

其中 $r \geq 0$，$\mathrm{W}_1$ 是具有传输成本函数 $c(z, \hat{z}) = \|z - \hat{z}\|$ 的 1-Wasserstein 距离。如果 $\ell$ 是凸的且 Lipschitz 连续的，且 $\operatorname{lip}(\ell) < \infty$，则有

$$
\sup _{\mathcal{P} \in \mathcal{P}} \varrho_{\mathrm{P}}[\ell(Z)] = \varrho_{\mathrm{P}}[\ell(Z)] + r \operatorname{lip}(\ell) \int_0^1 \beta^{-1} \, \mathrm{d} \sigma(\beta)。
$$

---
**证明**

假设 $\int_0^1 \beta^{-1} \, \mathrm{d} \sigma(\beta) < \infty$，这保证了 $\sigma(\{0\}) = 0$，且假设 $\mathrm{B}_{\mathrm{P}}[\|Z\|] < \infty$ 确保了 $\ell$ 的 Lipschitz 连续性，从而 $\mathrm{B}_{\mathrm{P}}[\ell(Z)]$ 有界。我们首先从上界限制最坏风险：

$$
\begin{aligned}
\sup _{\mathbb{P} \in \mathcal{P}} \varrho_{\mathrm{P}}[\ell(Z)] & \leq \int_0^1 \sup _{\mathcal{P} \in \mathcal{P}} \beta - \mathrm{CVaR}_{\mathrm{P}}[\ell(Z)] \, \mathrm{d} \sigma(\beta) \\
& \leq \int_0^1 \inf _{\tau \in \mathbb{R}} \tau + \frac{1}{\beta} \sup _{\mathcal{P} \in \mathcal{P}} \mathbb{B}_{\mathrm{P}}[\max\{\ell(Z) - \tau, 0\}] \, \mathrm{d} \sigma(\beta) \\
& = \int_0^1 \inf _{\tau \in \mathbb{R}} \tau + \frac{1}{\beta} \left(\mathbb{E}_{\hat{\mathrm{P}}}[\max\{\ell(Z) - \tau, 0\}] + r \operatorname{lip}(\ell)\right) \, \mathrm{d} \sigma(\beta) \\
& = \varrho_{\mathrm{P}}[\ell(Z)] + r \operatorname{lip}(\ell) \int_0^1 \beta^{-1} \, \mathrm{d} \sigma(\beta) < +\infty。
\end{aligned}
$$

---

为了证明下界，假设 $\sigma(\{1\}) = 0$。推广情况将在后续讨论。设 $\mu = \inf _{\mathcal{P} \in \mathcal{P}} \mathbb{E}_{\mathrm{P}}[\ell(Z)]$，因为 $\ell$ 是 Lipschitz 连续的且 $\mathrm{B}_{\mathrm{P}}[\|Z\|] < \infty$，所以 $\mu$ 有界。假设 $\mu \geq 0$，否则我们可以将 $\ell(z)$ 替换为 $\ell(z) - \mu$，此时最坏风险仅增加 $-\mu$。

因此，对于任意 $\varepsilon > 0$，存在 $\delta > 0$ 使得：

$$
\left|\sup _{\mathbb{P} \in \mathcal{P}} \varrho_{\mathrm{P}}[\ell(Z)] - \sup _{\mathcal{P} \in \mathcal{P}} \int_\delta^{1 - \delta} \beta - \mathrm{CVaR}_{\mathrm{P}}[\ell(Z)] \, \mathrm{d} \sigma(\beta)\right| \leq \varepsilon。
$$

结合之前结果可得：

$$
\sup _{\mathcal{P} \in \mathcal{P}} \varrho_{\mathrm{P}}[\ell(Z)] = \varrho_{\mathrm{P}}[\ell(Z)] + r \operatorname{lip}(\ell) \int_0^1 \beta^{-1} \, \mathrm{d} \sigma(\beta)。
$$

---

命题 6.18 表明，Lipschitz 连续的凸损失函数的 1-Wasserstein 风险等于名义风险与 Lipschitz 正则化项之和。它通过分布 $\mathbb{P}_i$ 渐近地达到，该分布将总概率质量的一部分 $q_i$ 沿方向 $z^{\star}$ 移动距离 $r / q_i$。当 $\sigma = \delta_1$ 时，命题 6.17 成为命题 6.18 的特例。$p$-Wasserstein 球 ($p \geq 1$) 的最坏风险最早由 Pflug 等人 (2012) 研究，线性损失函数的类似结果由 Pichler (2013) 和 Wozabal (2014) 获得。对于凸损失函数的扩展是新的贡献。


## 6.15 $p$-Wasserstein 风险

我们现在证明，如果损失函数 $\ell(z)$ 是线性的，则即使 $p \in (1, \infty)$，在 $p$-Wasserstein 球中的最坏风险也可以用闭式形式表示。本节的结果依赖于以下引理，该引理刻画了范数幂的共轭；参见 (Zhen et al. 2023, Lemma C.9)。

---

**引理 6.19** （范数幂的共轭）

假设 $\|\cdot\|$ 和 $\|\cdot\|_*$ 是 $\mathbb{R}^d$ 上的一对对偶范数，且 $p, q \in (1, \infty)$ 是共轭指数，满足 $\frac{1}{p} + \frac{1}{q} = 1$。定义 $\varphi(q) = \frac{(q-1)^{q-1}}{q^q}$。则以下结论成立：
(i) 如果 $f(z) = \frac{1}{p}\|z\|^p$，则 $f^*(y) = \frac{1}{q}\|y\|_*^q$。
(ii) 如果 $g(z) = \|z - \hat{z}\|^p$，则 $g^*(y) = y^{\top} \hat{z} + \varphi(q)\|y\|_*^q$。

---

**证明**

**关于结论 (i)**，固定任意 $z, y \in \mathbb{R}^d$。有

$$
z^{\top} y - \frac{1}{p}\|z\|^p \leq \|z\|\|y\|_* - \frac{1}{p}\|z\|^p \leq \max_{t \geq 0} t\|y\|_* - \frac{1}{p}t^p = \frac{1}{q}\|y\|_*^q,
$$

其中第一个不等式来自对偶范数的定义，第二个不等式通过对 $t = \|z\|$ 最大化获得。等号成立是因为最大化问题由 $\tau = \|y\|_*^{1/(p-1)}$ 解出。如果 $z \in \arg \max_{\|z\| = \tau} z^{\top} y$，两个不等式均退化为等式。这允许我们得出

$$
f^*(y) = \sup_{z \in \mathbb{R}^d} z^{\top} y - \frac{1}{p}\|z\|^p = \frac{1}{q}\|y\|_*^q。
$$

**关于结论 (ii)**，注意到

$$
\begin{aligned}
g^*(y) & = \sup_{z \in \mathbb{R}^d} y^{\top} z - \|z - \hat{z}\|^p = y^{\top} \hat{z} + \sup_{z \in \mathbb{R}^d} (y / p)^{\top} z - \frac{1}{p}\|z\|^p \\
& = y^{\top} \hat{z} + \frac{p}{q}\|y / p\|_*^q = y^{\top} \hat{z} + \varphi(q)\|y\|_*^q,
\end{aligned}
$$

其中最后两个等式利用了结论 (i) 和 $\varphi(q)$ 的定义。

---

我们现在证明，对于线性损失函数 $\ell(z) = \theta^{\top} z$，在半径为 $r$ 的 $p$-Wasserstein 球 $\hat{\mathbb{P}}$ 中的最坏 CVaR 等于在 $\hat{\mathbb{P}}$ 下的名义 CVaR 加上一个正则化项，该正则化项与 $\theta$ 的范数和 $r$ 成比例。

---
**命题 6.20** （$p$-Wasserstein 风险）

假设 $\hat{\mathbb{P}} \in \mathcal{P}(\mathbb{R}^d)$ 且 $\mathbb{E}_{\hat{\mathrm{P}}}[\|Z\|^p] < \infty$，其中 $p \in (1, \infty)$ 且 $\|\cdot\|$ 是 $\mathbb{R}^d$ 上的某种范数。定义

$$
\mathcal{P} = \left\{\mathbb{P} \in \mathcal{P}(\mathbb{R}^d) : \mathrm{W}_p(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\},
$$

其中 $r \geq 0$ 且 $\mathrm{W}_p$ 是带有传输成本函数 $c(z, \hat{z}) = \|z - \hat{z}\|^p$ 的 $p$-Wasserstein 距离。如果 $\theta \in \mathbb{R}^d$ 且 $\beta \in (0, 1)$，则有

$$
\sup_{\mathcal{P} \in \mathcal{P}} \beta - \operatorname{CVaR}_P\left[\theta^{\top} Z\right] = \beta - \operatorname{CVaR}_{\mathbb{P}}\left[\theta^{\top} Z\right] + r \beta^{-1 / p}\|\theta\|_*。
$$

---

**证明**

利用 Rockafellar 和 Uryasev（2000）的 CVaR 定义，有

$$
\sup_{\mathcal{P} \in \mathcal{P}} \beta - \mathrm{CVaR}_{\mathcal{P}}\left[\theta^{\top} Z\right] \leq \inf_{\tau \in \mathbb{R}} \tau + \frac{1}{\beta} \sup_{\mathcal{P} \in \mathcal{P}} \mathbb{E}_{\mathcal{P}}\left[\max\{\theta^{\top} Z - \tau, 0\}\right],
$$

其中不等式通过交换 $\mathbb{P}$ 上的上确界和 $\tau$ 上的下确界得到。最坏期望问题满足

$$
\sup_{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}\left[\max\{\theta^{\top} Z - \tau, 0\}\right]
$$

通过引入对偶并利用引理 6.19 (ii)，最终得到：

$$
\sup_{\mathbb{P} \in \mathcal{P}} \beta - \mathrm{CVaR}_{\mathbb{P}}\left[\theta^{\top} Z\right] \leq \beta - \mathrm{CVaR}_{\hat{\mathbb{P}}}\left[\theta^{\top} \hat{Z}\right] + r \beta^{-1 / p}\|\theta\|_*。
$$

类似地，利用构造 $\mathbb{P}^{\star}$ 的方法可得反向不等式，最终证明成立。


# 7. 自然子问题的有限凸重构

尽管自然子问题在某些重要特例中有解析解（参见第 6 节），但通常只能通过数值方法求解。有时，自然子问题可以被重构为等价的凸优化问题。在这种情况下，可以使用现成的求解器来解决。但在其他情况下，开发定制的解决算法可能是必要或更优的选择。

本节聚焦于有限凸重构，即描述将第 4 节中推导出的对偶最坏期望问题重构为有限凸最小化问题的条件。这些有限重构至关重要，因为它们可以与外层关于 $x \in \mathcal{X}$ 的最小化问题相结合，将整体 DRO 问题 (1.2) 转化为经典的、适合标准优化软件的最小化问题。随后，我们对自然子问题的有限凸重构进行对偶化，从而得到等价的有限凸最大化问题。这些有限双对偶最大化问题至关重要，因为它们的最优解能够构造出（渐近地）达到自然子问题 (4.1) 上确界的最坏分布。

尽管本节仅讨论最坏期望问题，但由于定理 5.18 的结果，所有结论都可以轻松扩展到最坏优化确定性等价问题。然而，为了简洁起见，我们将不对这些扩展进行详细讨论。为简化记号，我们将始终省略损失函数 $\ell$ 对决策变量 $x$ 的依赖性。

本节的其余部分结构如下：在第 7.1 节中，我们首先概述一种用于推导自然子问题 (4.1) 的有限凸对偶和双对偶重构的通用策略。随后，我们将这一策略应用于 Chebyshev 模糊集合（第 7.2 节）、$\phi$-散度模糊集合（第 7.3 节）以及最优传输模糊集合（第 7.4 节）的最坏期望问题。

---

## 7.1 通用证明策略

最坏期望问题 (4.1) 是一个半无限规划问题，它包含无限多的决策变量（因为它在无限维测度空间的子集上优化），但只有有限个约束（例如，矩条件和/或相对于参考分布的散度或差异的界限）。第 4 节中的对偶性结果使我们能够将这个半无限最大化问题重铸为具有有限变量和无限约束的半无限最小化问题。然后，我们利用鲁棒优化中的重构技术，将对偶半无限规划问题重铸为有限维凸最小化问题。这些技术利用了凸分析的标准结果以及 $\mathcal{S}$-引理，后者将在下面回顾。在整个讨论中，我们采用 $0 \cdot \infty = \infty$ 的惯例。

我们首先展示缩放与透视化构成对偶操作。

---

**引理 7.1** （缩放与透视化的对偶性）

如果 $f: \mathbb{R}^d \to \overline{\mathbb{R}}$ 是一个适当的、闭的、凸函数，且 $\alpha \in \mathbb{R}_{+}$ 是一个固定常数，那么有以下结论：

(i) 如果 $g(z) = \alpha f(z)$，则 $g^*(y) = \left(f^*\right)^\pi(y, \alpha)$ 对于所有 $y \in \mathbb{R}^d$ 成立。

(ii) 如果 $g(z) = f^\pi(z, \alpha)$，则 $g^*(y) = \operatorname{cl}\left(\alpha f^*\right)(y)$ 对于所有 $y \in \mathbb{R}^d$ 成立。

---

**证明**

**关于结论 (i)**，通过区分情况证明。首先，如果 $\alpha > 0$，则有

$$
\begin{aligned}
g^*(y) & = \sup_{z \in \mathbb{R}^d} y^{\top} z - \alpha f(z) = \alpha \sup_{z \in \mathbb{R}^d} (y / \alpha)^{\top} z - f(z) \\
& = \alpha f^*(y / \alpha) = \left(f^*\right)^\pi(y, \alpha)。
\end{aligned}
$$

另一方面，如果 $\alpha = 0$，则类似推导表明

$$
\begin{aligned}
g^*(y) & = \sup_{z \in \mathbb{R}^d} y^{\top} z - \delta_{\operatorname{dom}(f)}(z) = \delta_{\operatorname{dom}(f)}^*(y) = \delta_{\operatorname{dom}\left(f^{**}\right)}^*(y) \\
& = \left(f^*\right)^{\infty}(y) = \left(f^*\right)^\pi(y, \alpha)。
\end{aligned}
$$

其中第一个等式利用了 $0 \cdot \infty = \infty$ 的约定，导致 $0 f(z) = \delta_{\operatorname{dom}(f)}(z)$。第二个等式来自支持函数的定义；第三个等式成立是因为 $f$ 是凸的和闭的，因此由引理 4.2 推导出 $f = f^{**}$。最后一个等式使用了（Rockafellar 1970，第 13.3 条）定理，而最后一项利用了 $\alpha = 0$ 时透视函数的定义。至此，完成了结论 (i) 的证明。

**关于结论 (ii)**，首先假设 $\alpha > 0$，注意到

$$
g^*(y) = \sup_{z \in \mathbb{R}^d} y^{\top} z - f^\pi(z, \alpha) = \alpha \sup_{z \in \mathbb{R}^d} y^{\top}(z / \alpha) - f(z / \alpha) = \alpha f^*(y) = \operatorname{cl}\left(\alpha f^*\right)(y),
$$

最后的等式成立是因为 $f^*$ 是闭的。如果 $\alpha = 0$，则有

$$
g^*(y) = \sup_{z \in \mathbb{R}^d} y^{\top} z - f^{\infty}(z) = \sup_{z \in \mathbb{R}^d} y^{\top}(z) - \delta_{\operatorname{dom}\left(f^*\right)}^* = \delta_{\mathrm{cl}\left(\operatorname{dom}\left(f^*\right)\right)}(y) = \operatorname{cl}\left(\alpha f^*\right)(y)。
$$

其中，第一个等式利用了透视函数的定义，第二个等式和第三个等式分别来自（Rockafellar 1970，第 13.3 条）和（Rockafellar 1970，第 13.2 条）定理。最后一个等式是由于 $0 f^* = \delta_{\operatorname{dom}\left(f^*\right)}$ 的扩展运算约定。至此，完成了结论 (ii) 的证明。

以下引理推导了函数和的共轭公式。

---

**引理 7.2** （和的共轭）

如果 $f_k: \mathbb{R}^d \rightarrow \overline{\mathbb{R}}, k \in [K]$ 是适当的、凸的且闭的函数，那么 $f = \sum_{k \in [K]} f_k$ 的共轭满足：

$$
f^*(y) \leq \inf_{y_1, \ldots, y_K \in \mathbb{R}^d}\left\{\sum_{k \in [K]} f_k^*(y_k): \sum_{k \in [K]} y_k = y\right\}, \quad \forall y \in \mathbb{R}^d。
$$

如果存在 $\bar{z} \in \cap_{k \in [K]} \operatorname{rint}\left(\operatorname{dom}\left(f_k\right)\right)$，则上述不等式成为等式，且对于每个 $y \in \mathbb{R}^d$，该下确界能够达到。

---

**证明**

通过标准的变量分裂技巧和最大-最小不等式，可以表明函数 $f$ 的共轭有如下上界：

$$
\begin{aligned}
f^*(y) &= \sup_{z, z_1, \ldots, z_K \in \mathbb{R}^d} \left\{y^{\top} z - \sum_{k \in [K]} f_k(z_k): z_k = z \, \forall k \in [K] \right\} \\
&= \sup_{z, z_1, \ldots, z_K \in \mathbb{R}^d} \inf_{y_1, \ldots, y_K \in \mathbb{R}^d} \left\{y^{\top} z - \sum_{k \in [K]} f_k(z_k) - y_k^{\top}(z - z_k) \right\} \\
&\leq \inf_{y_1, \ldots, y_K \in \mathbb{R}^d} \sup_{z, z_1, \ldots, z_K \in \mathbb{R}^d} \left\{y^{\top} z - \sum_{k \in [K]} f_k(z_k) - y_k^{\top}(z - z_k) \right\} \\
&= \inf_{y_1, \ldots, y_K \in \mathbb{R}^d} \sup_{z \in \mathbb{R}^d} \left\{y^{\top} z - \sum_{k \in [K]} y_k^{\top} z \right\} + \sum_{k \in [K]} \sup_{z_k \in \mathbb{R}^d} \left\{y_k^{\top} z_k - f_k(z_k) \right\}。
\end{aligned}
$$

在上述表达式中，对 $z$ 的上确界在 $\sum_{k \in [K]} y_k = y$ 时取值为 0，否则取值为无穷大。而对 $z_k$ 的上确界则取值为 $f_k^*(y_k)$，对每个 $k \in [K]$ 均成立。将这些解析公式代入上式可得：

$$
f^*(y) \leq \inf_{y_1, \ldots, y_K \in \mathbb{R}^d} \left\{\sum_{k \in [K]} f_k^*(y_k): \sum_{k \in [K]} y_k = y \right\}。
$$

如果 $\cap_{k \in [K]} \operatorname{rint}\left(\operatorname{dom}\left(f_k\right)\right)$ 非空，则上述不等式成为等式，且下确界能够达到。这个结论可由（Rockafellar 1970，第 16.4 条）定理直接推导出。

---

**定义 7.3** （Slater 点）

集合 $\mathcal{Z} = \{z \in \mathbb{R}^d: g_k(z) \leq 0 \, \forall k \in [K]\}$ 的 Slater 点是满足以下条件的任意向量 $\bar{z} \in \mathcal{Z}$：
- 对所有 $k \in [K]$，$\bar{z} \in \operatorname{rint}\left(\operatorname{dom}\left(g_k\right)\right)$ 且 $g_k(\bar{z}) < 0$；
- 若 $g_k$ 是非线性的，则上述条件成立。

如果 $\bar{z} \in \operatorname{rint}(\operatorname{dom}(f))$，则 $\bar{z}$ 也是最小化问题 $\inf \{f(z): z \in \mathcal{Z}\}$ 的 Slater 点。

最大化问题的 Slater 点定义类似，只需将 $\bar{z} \in \operatorname{rint}(\operatorname{dom}(f))$ 替换为 $\bar{z} \in \operatorname{rint}(\operatorname{dom}(-f))$。

---

**定理 7.4** （凸对偶性）

假设函数 $f$ 和 $g_k, k \in [K]$ 是适当的、闭的且凸的。那么：
- 问题 (P) 的下确界大于或等于问题 (D) 的上确界；
- 如果问题 (P) 或 (D) 存在 Slater 点，则它们的目标值相等，且 (D) 或 (P) 可解。

**证明**

最大-最小不等式（max-min inequality）直接表明，问题 $(\mathrm{P})$ 的下确界被其拉格朗日对偶问题的最优值下界。具体而言，我们有：

$$
\begin{aligned}
\inf (\mathrm{P}) & = \inf _{z \in \mathbb{R}^d} \sup _{\alpha \in \mathbb{R}_{+}^K} f(z) + \sum_{k \in [K]} \alpha_k g_k(z) \\
& \geq \sup _{\alpha \in \mathbb{R}_{+}^K} \inf _{z \in \mathbb{R}^d} f(z) + \sum_{k \in [K]} \alpha_k g_k(z) \\
& = \sup _{\alpha \in \mathbb{R}_{+}^K} -\sup _{z \in \mathbb{R}^d} 0^{\top} z - f(z) - \sum_{k \in [K]} \alpha_k g_k(z) \\
& = \sup _{\alpha \in \mathbb{R}_{+}^K} -\left(f + \sum_{k \in [K]} \alpha_k g_k\right)^*(0) .
\end{aligned}
$$

得到的下界涉及多个函数和的共轭函数。根据引理 7.2，多个函数和的共轭函数的下界由所有函数共轭函数的下卷积（infimal convolution）给出。因此，我们得到：

$$
\inf (\mathrm{P}) \geq \sup _{\substack{\alpha_1, \ldots, \alpha_K \in \mathbb{R}_{+} \\ \beta_0, \ldots, \beta_K \in \mathbb{R}^d}} \left\{-f^*\left(\beta_0\right) - \sum_{k=1}^K \left(\alpha_k g_k\right)^*\left(\beta_k\right): \sum_{k=0}^K \beta_k = 0 \right\}
$$

根据引理 7.1(i)，我们进一步有 $\left(\alpha_k g_k\right)^*\left(\beta_k\right) = \left(g_k^*\right)^\pi\left(\beta_k, \alpha_k\right)$，对于所有 $\beta_k \in \mathbb{R}^d$ 和 $\alpha_k \in \mathbb{R}_{+}$。因此，上述不等式中的下界与 (D) 的上确界相符。这证明了弱对偶性。

关于强对偶性和可解性的证明，在满足条件 (i) 和 (ii) 时，可以参考 (Zhen et al. 2023, Theorem 2)。

**引理 7.6** ($\mathcal{S}$-引理，Yakubovich 1971)
假设 $f: \mathbb{R}^d \rightarrow \mathbb{R}$ 和 $g: \mathbb{R}^d \rightarrow \mathbb{R}$ 是二次函数。如果存在一个满足 Slater 条件的点 $\bar{z} \in \mathbb{R}^d$，使得 $g(\bar{z})<0$，那么以下两种说法是等价的：

1. (i) 不存在 $z \in \mathbb{R}^d$ 满足 $f(z)<0$ 且 $g(z) \leq 0$。
2. (ii) 存在 $\alpha \in \mathbb{R}_{+}$，使得 $f(z)+\alpha g(z) \geq 0$ 对所有 $z \in \mathbb{R}^d$ 成立。

$\mathcal{S}$-引理允许我们将半无限约束进行有限凸重构，其中该约束要求一个（可能是不定的）二次函数在单一二次约束的可行集合上为非负。需要特别注意的是，涉及的函数 $f$ 和 $g$ 不要求在 $z$ 上是凸的。

---

**命题 7.7**（半无限约束 II）
假设 $Q_0, Q_1 \in \mathbb{S}^d, q_0, q_1 \in \mathbb{R}^d, r_0, r_1 \in \mathbb{R}$。另外，假设存在一个满足 Slater 条件的点 $\bar{z} \in \mathbb{R}^d$，使得：

$$
\bar{z}^{\top} Q_0 \bar{z}+2 q_0^{\top} \bar{z}+r_0<0。
$$

那么，以下半无限约束：

$$
z^{\top} Q_1 z+2 q_1^{\top} z+r_1 \geq 0 \quad \forall z \in \mathbb{R}^d: z^{\top} Q_0 z+2 q_0^{\top} z+r_0 \leq 0
$$

当且仅当存在 $\alpha \in \mathbb{R}_{+}$ 满足：

$$
\left[\begin{array}{ll}
Q_1+\alpha Q_0 & q_1+\alpha q_0 \\
q_1^{\top}+\alpha q_0^{\top} & r_1+\alpha r_0
\end{array}\right] \geq 0
$$

成立。

**证明.** 我们观察到：

$$
\begin{aligned}
& z^{\top} Q_1 z+2 q_1^{\top} z+r_1 \geq 0 \quad \forall z \in \mathbb{R}^d: z^{\top} Q_0 z+2 q_0^{\top} z+r_0 \leq 0 \\  
\Longleftrightarrow & \exists \alpha \in \mathbb{R}_{+} \text { 使得 } z^{\top}\left(Q_1+\alpha Q_0\right) z+2\left(q_1+\alpha q_0\right)^{\top} z+r_1+\alpha r_0 \geq 0 \quad \forall z \in \mathbb{R}^d \\  
\Longleftrightarrow & \exists \alpha \in \mathbb{R}_{+} \text { 使得 }\left[\begin{array}{l}  
z \\  
1  
\end{array}\right]^{\top}\left[\begin{array}{cc}  
Q_1+\alpha Q_0 & q_1+\alpha q_0 \\  
q_1^{\top}+\alpha q_0^{\top} & r_1+\alpha r_0  
\end{array}\right]\left[\begin{array}{l}  
z \\  
1  
\end{array}\right] \geq 0 \quad \forall z \in \mathbb{R}^d,
\end{aligned}
$$

其中，第一个等价关系使用了引理 7.6，将其应用于 $f(z)=z^{\top} Q_1 z+2 q_1^{\top} z+r_1$ 和 $g(z)=z^{\top} Q_0 z+2 q_0^{\top} z+r_0$。由于二次形式是二次齐次的且连续的，最后一个陈述等价于所需的半正定性条件。这一观察完成了证明。

---


**命题 7.7 在推导 Chebyshev 或 Gelbrich 模糊集合下对偶最坏期望问题的有限凸重构方面特别有用；参见 (4.5) 和 (4.6)。** 正如我们将看到的，相应的半无限约束在 $z$ 上并不是凸的，这意味着命题 7.5 不适用。

有限的凸重构对解决分布鲁棒优化（DRO）问题 (1.2) 至关重要。它们允许我们将外层关于 $x \in \mathcal{X}$ 的最小化与对偶最坏期望问题的辅助决策变量的内层最小化相结合，从而对 (1.2) 进行有限凸重构。然而，(4.1) 的有限对偶重构并不直接支持识别（渐近）达到 (4.1) 上确界的最坏分布。这样的最坏分布使决策者能够评估在最具挑战性的条件下某个候选决策的表现，这是压力测试和污染实验的核心；参见例如 (Dupačová 2006)。它们还在不确定性最优量化中起关键作用，用于在给定关于潜在概率分布的有限信息的情况下，确定感兴趣量的最紧 probabilistic bounds。

有关更多详细信息，请参阅 (Owhadi, Scovel, Sullivan, McKerns 和 Ortiz 2013, Ghanem, Higdon 和 Owhadi 2017)。

为识别达到 (4.1) 上确界的最坏分布，或识别渐近达到该确界的一系列分布，我们考虑通过对 (4.1) 的有限凸对偶进行对偶化得到的最坏期望问题 (4.1) 的双对偶重构。双对偶通常可以被解释为将最坏期望问题 (4.1) 限制到一个由有限多个决策变量参数化的分布子集 $\mathbb{P} \in \mathcal{P}$ 上。通过 (4.1) 问题、其对偶问题和双对偶问题之间的强对偶性，我们可以得出结论：任何双对偶问题的最优解都表示一个（序列）分布，这些分布（渐近）达到 (4.1) 的上确界。

**从 (4.1) 的有限双对偶中提取最坏分布的思想由 Delage 和 Ye (2010, § 4.2) 首次在 Chebyshev 模糊集合的背景下形式化，后来由 Mohajerin Esfahani 和 Kuhn (2018) 扩展到最优传输模糊集合中。**

在第 7.2 节中，我们将看到，对于具有不确定矩的 Chebyshev 模糊集合 (2.4)，最坏分布由具有一阶和二阶矩的分布的混合物组成，这些矩由有限双对偶问题的最优解确定。对于以离散分布 $\hat{\mathbb{P}}$ 为中心的 $\phi$-散度模糊集合，第 7.3 节将表明，最坏分布支持于 $\hat{\mathbb{P}}$ 的原子上，并且（如果 $\phi$ 的增长最多是线性的）支持于 $\arg \max _{z \in \mathcal{Z}} \ell(z)$，其概率权重由有限双对偶问题的最优解决定。同样地，对于以离散分布 $\hat{\mathbb{P}}$ 为中心的最优传输模糊集合 (2.27)，第 7.4 节将表明，最坏分布由离散分布的混合物组成，其原子的位置和概率权重由有限双对偶问题的最优解确定。



## 7.2. 带有不确定矩的 Chebyshev 模糊集合

回顾具有不确定矩的 Chebyshev 模糊集合 (2.4) 定义为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}_2(\mathcal{Z}): \mathbb{E}_{\mathbb{P}}[Z]=\mu, \mathbb{E}_{\mathbb{P}}\left[Z Z^{\top}\right]=M \quad \forall(\mu, M) \in \mathcal{F}\right\},
$$

其中，$\mathcal{F} \subseteq \mathbb{R}^d \times \mathbb{S}_{+}^d$ 表示一个封闭的矩不确定集合，$\mathcal{P}_2(\mathcal{Z})$ 代表所有在 $\mathcal{Z}$ 上具有有限二阶矩的概率分布的集合。本节结合 Chebyshev 模糊集合的对偶结果（参见定理 4.6）与随后的半无限规划的有限对偶重构（参见命题 7.7），将自然问题 (4.1) 的子问题等价地重构为一个有限维最小化问题。我们还展示了相应的双对偶如何帮助提取最坏分布 $\mathbb{P}^{\star} \in \mathcal{P}$，以达到 (4.1) 的最优值。

由于支持集模糊集合（参见第 2.1.1 节）、Markov 模糊集合（参见第 2.1.2 节）、具有已知矩的 Chebyshev 模糊集合（参见第 2.1.3 节）和均值-离散模糊集合（参见第 2.1.5 节）都可以看作具有不确定矩的 Chebyshev 模糊集合的特例，我们的结果可直接扩展到这些模糊集合，因此为了简洁起见，我们不再重新推导相关结论。然而，考虑到其在统计学 (Nguyen, Kuhn 和 Mohajerin Esfahani 2022b)、信号处理 (Nguyen et al. 2023) 和控制 (Taşkesen et al. 2024) 中的最新应用，我们将报告 Gelbrich 模糊集合（带有矩不确定集合 (2.8)）的有限对偶和双对偶重构。本节中推导的所有重构均依赖于引理 7.6。因此，它们需要损失函数 $\ell$ 和支持集 $\mathcal{Z}$ 的二次表示形式，如以下假设所述。

**假设 7.8**（Chebyshev 模糊集合的正则性条件）
(i) 损失函数 $\ell$ 是二次函数的逐点最大值，

$$
\ell(z)=\max _{j \in[J]} \ell_j(z) \quad \text { 其中 } \quad \ell_j(z)=z^{\top} Q_j z+2 q_j^{\top} z+q_j^0,
$$

其中，$J \in \mathbb{N}, Q_j \in \mathbb{S}^d, q_j \in \mathbb{R}^d, q_j^0 \in \mathbb{R}$ 对所有 $j \in[J]$ 成立。

(ii) 支持集 $\mathcal{Z}$ 是以下形式的椭球体：

$$
\mathcal{Z}=\left\{z \in \mathbb{R}^d:\left(z-z_0\right)^{\top} Q_0\left(z-z_0\right) \leq 1\right\},
$$

其中 $Q_0 \in \mathbb{S}_{+}^d$ 且 $z_0 \in \mathbb{R}^d$。

**注意：** 假设 7.8 不对构成损失函数 $\ell$ 的二次函数 $z^{\top} Q_j z+2 q_j^{\top} z+q_j^0$ 施加任何凸性条件。

**定理 7.9**（Chebyshev 模糊集合的有限对偶重构）

如果 $\mathcal{P}$ 是 Chebyshev 模糊集合 (2.4) 且 $\mathcal{F} \subseteq \mathbb{R}^d \times \mathbb{S}_{+}^d$ 且假设 7.8 成立，则最坏期望问题 (4.1) 满足以下弱对偶关系：

$$
\begin{aligned}
& \sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] \\  
& \leq\left\{\begin{array}{cc}  
\inf & \lambda_0+\delta_{\mathcal{F}}^*(\lambda, \Lambda) \\  
\text { s.t. } & \lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}^d, \Lambda \in \mathbb{S}^d, \alpha \in \mathbb{R}_{+}^J \\  
& {\left[\begin{array}{cc}  
\Lambda-Q_j+\alpha_j Q_0 & \frac{1}{2} \lambda-q_j-\alpha_j Q_0 z_0 \\  
\left(\frac{1}{2} \lambda-q_j-\alpha_j Q_0 z_0\right)^{\top} & \lambda_0-q_j^0+\alpha_j\left(z_0^{\top} Q_0 z_0-1\right)  
\end{array}\right] \geq 0 \quad \forall j \in[J] .}  
\end{array}\right.
\end{aligned}
$$

如果 $\mathcal{F}$ 是一个凸紧集，且对于所有 $(\mu, M) \in \operatorname{rint}(\mathcal{F})$ 有 $M>\mu \mu^{\top}$，则强对偶性成立，即上述不等式变为等式。

**证明.** 弱对偶性来自于定理 4.6，以及对偶问题 (4.5) 中半无限约束的以下等价重构：

$$
\begin{aligned}
& \lambda_0+\lambda^{\top} z+z^{\top} \Lambda z \geq \ell(z) \quad \forall z \in \mathcal{Z} \\  
\Longleftrightarrow & \lambda_0+\lambda^{\top} z+z^{\top} \Lambda z \geq z^{\top} Q_j z+2 q_j^{\top} z+q_j^0 \quad \forall z \in \mathcal{Z}, \forall j \in[J] \\  
\Longleftrightarrow & \exists \alpha \in \mathbb{R}_{+}^J \text { 使得 } \\  
& {\left[\begin{array}{cc}  
\Lambda-Q_j+\alpha_j Q_0 & \frac{1}{2} \lambda-q_j-\alpha_j Q_0 z_0 \\  
\left(\frac{1}{2} \lambda-q_j-\alpha_j Q_0 z_0\right)^{\top} & \lambda_0-q_j^0+\alpha_j\left(z_0^{\top} Q_0 z_0-1\right)  
\end{array}\right] \geq 0 \quad \forall j \in[J] }
\end{aligned}
$$

其中，第一个等价关系基于假设 7.8 (i)，第二个等价关系基于命题 7.7。由于 $z_0 \in \operatorname{rint}(\mathcal{Z})$ 是一个 Slater 点（由假设 7.8 (ii) 保证），加之损失函数是二次的，因此根据定理 4.6 可以直接得出强对偶性。

**回顾 Gelbrich 模糊集合 (2.8)**

Gelbrich 模糊集合 (2.8) 被定义为 Chebyshev 模糊集合 (2.4) 的一种形式，其中矩不确定集合为：

$$
\mathcal{F}=\left\{(\mu, M) \in \mathbb{R}^d \times \mathbb{S}_{+}^d: \begin{array}{l}
\exists \Sigma \in \mathbb{S}_{+}^d \text { 使得 } M=\Sigma+\mu \mu^{\top}, \\  
\mathrm{G}((\mu, \Sigma),(\hat{\mu}, \hat{\Sigma})) \leq r
\end{array}\right\} .
$$

其中，$(\hat{\mu}, \hat{\Sigma})$ 是一个标称的均值-协方差对，$r \geq 0$ 是一个大小参数。以下结果直接来源于定理 4.9 和 7.9，因此省略证明。

**定理 7.10**（Gelbrich 模糊集合的有限对偶重构）

如果 $\mathcal{P}$ 是 Chebyshev 模糊集合 (2.4)，其 $\mathcal{F}$ 由 (2.8) 给出，且假设 7.8 成立，则最坏期望问题 (4.1) 满足以下弱对偶关系：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)]
$$

$$
\leq \begin{cases}\inf & \lambda_0+\gamma\left(r^2-\|\hat{\mu}\|^2-\operatorname{Tr}(\hat{\Sigma})\right)+\operatorname{Tr}\left(A_0\right)+\alpha_0 \\  
\text { s.t. } & \lambda_0 \in \mathbb{R}, \alpha_0, \gamma \in \mathbb{R}_{+}, \alpha \in \mathbb{R}_{+}^J, \lambda \in \mathbb{R}^d, \Lambda \in \mathbb{S}^d, A_0 \in \mathbb{S}_{+}^d \\  
& {\left[\begin{array}{cc}  
\Lambda-Q_j+\alpha_j Q_0 & \frac{1}{2} \lambda-q_j-\alpha_j Q_0 z_0 \\  
\left(\frac{1}{2} \lambda-q_j-\alpha_j Q_0 z_0\right)^{\top} & \lambda_0-q_j^0+\alpha_j\left(z_0^{\top} Q_0 z_0-1\right)
\end{array}\right] \geq 0 \quad \forall j \in[J]} \\  
& {\left[\begin{array}{cc}  
\gamma I_d-\Lambda & \gamma \hat{\Sigma}^{\frac{1}{2}} \\  
\gamma \hat{\Sigma}^{\frac{1}{2}} & A_0
\end{array}\right] \geq 0,\left[\begin{array}{cc}  
\gamma I_d-\Lambda & \gamma \hat{\mu}+\frac{\lambda}{2} \\  
\left(\gamma \hat{\mu}+\frac{\lambda}{2}\right)^{\top} & \alpha_0
\end{array}\right] \geq 0 .}
\end{cases}
$$

如果 $r>0$，则强对偶性成立，即上述不等式变为等式。

---

为了刻画在 Chebyshev 和 Gelbrich 模糊集合下的最坏期望问题 (4.1) 中达到上确界的极值分布，我们首先推导 (4.1) 的相应双对偶。

**定理 7.11**（Chebyshev 模糊集合的有限双对偶重构）

如果 $\mathcal{P}$ 是 Chebyshev 模糊集合 (2.4)，其 $\mathcal{F} \subseteq \mathbb{R}^d \times \mathbb{S}_{+}^d$ 且假设 7.8 成立，则最坏期望问题 (4.1) 满足以下弱对偶关系：

$$
\begin{aligned}
& \sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] \\  
& \leq\left\{\begin{aligned}  
& \sup \sum_{j \in[J]} \operatorname{Tr}\left(Q_j \Theta_j\right)+2 q_j^{\top} \theta_j+q_j^0 p_j \\  
& \text { s.t. } \mu \in \mathbb{R}^d, M \in \mathbb{S}_{+}^d, p_j \in \mathbb{R}_{+}, \theta_j \in \mathbb{R}^d, \Theta_j \in \mathbb{S}_{+}^d \quad \forall j \in[J] \\  
& {\left[\begin{array}{cc}  
\Theta_j & \theta_j \\  
\theta_j^{\top} & p_j  
\end{array}\right] \geq 0, \operatorname{Tr}\left(Q_0 \Theta_j\right)-2 z_0^{\top} Q_0 \theta_j+z_0^{\top} Q_0 z_0 p_j \leq p_j \quad \forall j \in[J] } \\  
& \sum_{j \in[J]} p_j=1, \mu=\sum_{j \in[J]} \theta_j, M=\sum_{j \in[J]} \Theta_j,(\mu, M) \in \mathcal{F} .
\end{aligned}\right.
\end{aligned}
$$

如果 $\mathcal{F}$ 是一个凸紧集，且对于所有 $(\mu, M) \in \operatorname{rint}(\mathcal{F})$ 有 $M>\mu \mu^{\top}$，则强对偶性成立，即 (7.5) 成为等式。

**证明.** 通过将 Gelbrich 模糊集合分解为形式为 $\mathcal{P}(\mu, M)=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_{\mathbb{P}}[Z]=\mu, \mathbb{E}_{\mathbb{P}}\left[Z Z^{\top}\right]=M\right\}$ 的 Chebyshev 模糊集合，我们得到：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)]=\sup _{(\mu, M) \in \mathcal{F}} \sup _{\mathbb{P} \in \mathcal{P}(\mu, M)} \mathbb{E}_{\mathbb{P}}[\ell(Z)] .
$$

右侧 (7.6) 的内层最大化问题表示一个关于矩不确定集合为单例 $\{(\mu, M)\}$ 的模糊集合 (2.4) 的最坏期望问题。该单例的支持函数为 $\delta_{\{(\mu, M)\}}^*(\lambda, \Lambda)=\lambda^{\top} \mu+\operatorname{Tr}(\Lambda M)$。因此，定理 7.9 表明 (7.6) 的右侧内层确界可以被如下上界：

$$
\begin{array}{ll}  
\inf & \lambda_0+\lambda^{\top} \mu+\operatorname{Tr}(\Lambda M) \\  
\text { s.t. } & \lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}^d, \Lambda \in \mathbb{S}^d, \alpha \in \mathbb{R}_{+}^J \\  
& {\left[\begin{array}{cc}  
\Lambda-Q_j+\alpha_j Q_0 & \frac{1}{2} \lambda-q_j-\alpha_j Q_0 z_0 \\  
\left(\frac{1}{2} \lambda-q_j-\alpha_j Q_0 z_0\right)^{\top} & \lambda_0-q_j^0+\alpha_j\left(z_0^{\top} Q_0 z_0-1\right)
\end{array}\right] \geq 0 \quad \forall j \in[J] .}
\end{array}
$$

此半定规划问题的对偶形式为：

$$
\begin{array}{lll}  
\sup & \sum_{j \in[J]} \operatorname{Tr}\left(Q_j \Theta_j\right)+2 q_j^{\top} \theta_j+q_j^0 p_j & \\  
\text { s.t. } & p_j \in \mathbb{R}_{+}, \theta_j \in \mathbb{R}^d, \Theta_j \in \mathbb{S}_{+}^d & \forall j \in[J] \\  
& {\left[\begin{array}{cc}  
\Theta_j & \theta_j \\  
\theta_j^{\top} & p_j  
\end{array}\right] \geq 0, \operatorname{Tr}\left(Q_0 \Theta_j\right)-2 z_0^{\top} Q_0 \theta_j+z_0^{\top} Q_0 z_0 p_j \leq p_j} & \forall j \in[J] \\  
& \sum_{j \in[J]} p_j=1, \sum_{j \in[J]} \theta_j=\mu, \sum_{j \in[J]} \Theta_j=M . &  
\end{array}
$$

由于原始最小化问题存在一个 Slater 点，因此强对偶性成立。通过定义 $\Lambda=\lambda_0 I_d$ 并将 $\lambda_0$ 设为一个较大的值，可以确保原始问题中的线性矩阵不等式严格成立。将 (7.6) 右侧内层确界替换为上述对偶半定规划，可得 (7.5) 的上界。如果 $\mathcal{F}$ 是凸紧集且满足 $M>\mu \mu^{\top}$，则 (7.5) 成为等式，依据定理 7.9。

**注意：**关于 (7.5) 的双对偶重构

当 $\mathcal{F}$ 是紧集时，双对偶重构 (7.5) 可解。事实上，其目标函数显然是连续的。此外，很容易验证，如果 $\mathcal{F}$ 是紧集，其可行区域也是紧的。

**定理 7.12**（Gelbrich 模糊集合的有限双对偶重构）

如果 $\mathcal{P}$ 是 Chebyshev 模糊集合 (2.4)，其 $\mathcal{F}$ 由 (2.8) 给出，且假设 7.8 成立，则最坏期望问题 (4.1) 满足以下弱对偶关系：

$$
\begin{aligned}
& \sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] \\  
& \leq\left\{\begin{array}{lll}  
\max & \sum_{j \in[J]} \operatorname{Tr}\left(Q_j \Theta_j\right)+2 q_j^{\top} \theta_j+q_j^0 p_j \\  
\text { s.t. } & \mu \in \mathbb{R}^d, M, U \in \mathbb{S}_{+}^d, C \in \mathbb{R}^{d \times d} \\  
& p_j \in \mathbb{R}_{+}, \theta_j \in \mathbb{R}^d, \Theta_j \in \mathbb{S}_{+}^d \\  
& {\left[\begin{array}{cc}  
M-U & C \\  
C^{\top} & \hat{\Sigma}  
\end{array}\right] \geq 0,\left[\begin{array}{cc}  
U & \mu \\  
\mu^{\top} & 1  
\end{array}\right] \geq 0,\left[\begin{array}{cc}  
\Theta_j & \theta_j \\  
\theta_j^{\top} & p_j  
\end{array}\right] \geq 0 \quad \forall j \in[J]} \\  
& \operatorname{Tr}\left(Q_0 \Theta_j\right)-2 z_0^{\top} Q_0 \theta_j+z_0^{\top} Q_0 z_0 p_j \leq p_j \\  
& \sum_{j \in[J]} p_j=1, \mu=\sum_{j \in[J]} \theta_j, M=\sum_{j \in[J]} \Theta_j \\  
& \|\hat{\mu}\|_2^2-2 \mu^{\top} \hat{\mu}+\operatorname{Tr}(M+\hat{\Sigma}-2 C) \leq r^2 .
\end{array} \quad \forall j \in[J]\right. \\  
\end{aligned}
$$

如果 $r>0$，则强对偶性成立，即上述不等式变为等式。

**证明.** 定理 7.12 的证明基于命题 2.3 和定理 7.11，因此省略。

---

**定理 7.14**（Chebyshev 模糊集合的极值分布）

如果定理 7.11 中关于弱对偶性和强对偶性的所有条件都满足，且 $\left(\mu^{\star}, M^{\star}, p^{\star}, \theta^{\star}, \Theta^{\star}\right)$ 是 (7.5) 的解，则以下结论成立：

1. **情况 (i)：如果 $\mathcal{J}^{\infty}=\emptyset$，则** 对于所有 $j \in \mathcal{J}^{+}$，存在离散分布 $\mathbb{P}_j^{\star} \sim\left(\theta_j^{\star} / p_j^{\star}, \Theta_j^{\star} / p_j^{\star}\right)$，其支持集为 $\mathcal{Z}$，并且 (4.1) 被 $\mathbb{P}^{\star}=\sum_{j \in \mathcal{J}^{+}} p_j^{\star} \mathbb{P}_j^{\star}$ 解出。此外，有 $\mathbb{P}^{\star} \sim\left(\mu^{\star}, M^{\star}\right)$ 且 $\mathbb{P}^{\star}$ 的支持集为 $\mathcal{Z}$。

2. **情况 (ii)：如果 $\mathcal{J}^{\infty} \neq \emptyset$，则** 对于所有 $j \in \mathcal{J}$，存在离散分布 $\mathbb{P}_j^m \sim\left(\theta_j^{\star} / p_j^m, \Theta_j^{\star} / p_j^m\right)$，其支持集为 $\mathcal{Z}$，其中 $p_j^m=\left(1-\left|\mathcal{J}^{\infty}\right| / m\right) p_j^{\star}$（对于 $j \in \mathcal{J}^{+}$）和 $p_j^m=1 / m$（对于 $j \in \mathcal{J}^{\infty}$），且 $m$ 为任意满足 $m \geq\left|\mathcal{J}^{\infty}\right|$ 的整数。当 $m$ 增大时，(4.1) 被 $\mathbb{P}^m=\sum_{j \in \mathcal{J}} p_j^m \mathbb{P}_j^m$ 渐近解出。

**证明：关于结论 (i)**，问题 (7.7) 的约束表明：

$$
\operatorname{Tr}\left(Q_0 \Theta_j^{\star} / p_j^{\star}\right)-2 z_0^{\top} Q_0 \theta_j^{\star} / p_j^{\star}+z_0^{\top} Q_0 z_0 \leq 1
$$

且

$$
\left[\begin{array}{cc}
\Theta_j^{\star} & \theta_j^{\star} \\  
\left(\theta_j^{\star}\right)^{\top} & p_j^{\star}
\end{array}\right] \geq 0 \quad \Longleftrightarrow \quad \Theta_j^{\star} / p_j^{\star} \geq\left(\theta_j^{\star} / p_j^{\star}\right)\left(\theta_j^{\star} / p_j^{\star}\right)^{\top}
$$

对所有 $j \in \mathcal{J}^{+}$ 成立。因此，Lemma 7.13 保证存在离散分布 $\mathbb{P}_j^{\star} \sim\left(\theta_j^{\star} / p_j^{\star}, \Theta_j^{\star} / p_j^{\star}\right)$，其中 $j \in \mathcal{J}^{+}$，且这些分布均支持于 $\mathcal{Z}$。由此可得：

$$
\mathbb{P}^{\star}=\sum_{j \in \mathcal{J}^{+}} p_j^{\star} \mathbb{P}_j^{\star}
$$

也支持于 $\mathcal{Z}$。此外，我们有：

$$
\mathbb{E}_{\mathbb{P}^{\star}}[Z]=\sum_{j \in \mathcal{J}^{+}} p_j^{\star} \cdot \mathbb{E}_{\mathbb{P}_j^{\star}}[Z]=\sum_{j \in \mathcal{J}^{+}} p_j^{\star} \cdot \theta_j^{\star} / p_j^{\star}=\sum_{j \in \mathcal{J}^{+}} \theta_j^{\star}=\mu^{\star}
$$

以及

$$
\mathbb{E}_{\mathbb{P}^{\star}}\left[Z Z^{\top}\right]=\sum_{j \in \mathcal{J}^{+}} p_j^{\star} \cdot \mathbb{E}_{\mathbb{P}_j^{\star}}\left[Z Z^{\top}\right]=\sum_{j \in \mathcal{J}^{+}} p_j^{\star} \cdot \Theta_j^{\star} / p_j^{\star}=\sum_{j \in \mathcal{J}^{+}} \Theta_j^{\star}=M^{\star},
$$

即 $\mathbb{P}^{\star} \sim\left(\mu^{\star}, M^{\star}\right)$。由于 $\left(\mu^{\star}, M^{\star}\right) \in \mathcal{F}$，因此 $\mathbb{P}^{\star} \in \mathcal{P}$ 且满足：

$$
\mathbb{E}_{\mathbb{P}^{\star}}[\ell(Z)] \leq \sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)]=\sum_{j \in[J]} \operatorname{Tr}\left(Q_j \Theta_j^{\star}\right)+2 q_j^{\top} \theta_j^{\star}+q_j^0 p_j^{\star},
$$

其中等号来自于定理 7.11 所确立的强对偶性。同时，由于 $\mathbb{P}^{\star}$ 是混合分布，且 $\ell$ 在 (7.3) 中定义为二次分量函数的逐点最大值，因此有：

$$
\mathbb{E}_{\mathbb{P}^{\star}}[\ell(Z)] \geq \sum_{j \in \mathcal{J}^{+}} p_j^{\star} \cdot \mathbb{E}_{\mathbb{P}_j^{\star}}\left[\ell_j(Z)\right]=\sum_{j \in[J]} \operatorname{Tr}\left(Q_j \Theta_j^{\star}\right)+2 q_j^{\top} \theta_j^{\star}+q_j^0 p_j^{\star}.
$$

具体而言，不等式成立是因为 $\ell \geq \ell_j$ 对每个 $j \in[J]$ 成立，而等号成立是因为当 $p_j^{\star}=0$ 时，$\theta_j^{\star}=0$ 且 $\Theta_j^{\star}=0$。实际上，若 $p_j^{\star}=0$，则 $\Theta_j^{\star}=0$（因为 $\mathcal{J}^{\infty}$ 是空集），而 (7.5) 中的线性矩阵不等式表明，当 $\Theta_j^{\star}=0$ 时，$\theta_j^{\star}=0$。上述不等式保证了 $\mathbb{P}^{\star}$ 解决了最坏期望问题 (4.1)。这完成了对结论 (i) 的证明。

---

**接下来讨论结论 (ii)。** 类似于结论 (i) 的证明，可以证明 $\mathbb{P}^m \in \mathcal{P}$ 对于每个 $m \geq\left|\mathcal{J}^{\infty}\right|$ 成立。这意味着：

$$
\mathbb{E}_{\mathbb{P}^m}[\ell(Z)] \leq \sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)]
$$

对于 $m \geq\left|\mathcal{J}^{\infty}\right|$ 成立。此外，我们可以观察到：

$$
\begin{aligned}
\lim _{m \rightarrow \infty} \mathbb{E}_{\mathbb{P}^m}[\ell(Z)] & \geq \lim _{m \rightarrow \infty} \sum_{j \in \mathcal{J}} p_j^m \cdot \mathbb{E}_{\mathbb{P}^m}\left[\ell_j(Z)\right] \\  
& =\sum_{j \in \mathcal{J}} \lim _{m \rightarrow \infty} p_j^m \cdot \mathbb{E}_{\mathbb{P}^m}\left[\ell_j(Z)\right] \\  
& =\sum_{j \in[J]} \operatorname{Tr}\left(Q_j \Theta_j^{\star}\right)+2 q_j^{\top} \theta_j^{\star}+q_j^0 p_j^{\star}=\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)],
\end{aligned}
$$

其中第二个等号利用了 $\mathbb{P}^m$ 的定义，第三个等号基于定理 7.11 所确立的强对偶性。这完成了证明。

**定理 7.14 的应用于 Gelbrich 模糊集合**

定理 7.14 同样适用于 Gelbrich 模糊集合，它是形式 (2.4) 的 Chebyshev 模糊集合，其中 $\mathcal{F}$ 由 (2.8) 给出。定理 7.14 (i) 中识别的极值分布 $\mathbb{P}^{\star}$ 构成了不同分布 $\mathbb{P}_j^{\star}$ 的混合，每个分布对应于损失函数 $\ell$ 的一个分量 $\ell_j$（参见假设 7.8 (i)）。这些混合分量 $\mathbb{P}_j^{\star}$ 可以设置为任何满足给定矩条件并支持在 $\mathcal{Z}$ 上的分布。根据引理 7.13，可以保证存在与这些条件一致的离散分布。然而，如果 $\mathcal{Z}=\mathbb{R}^d$，则可以选择将 $\mathbb{P}_j^{\star}$ 设置为具有给定一阶和二阶矩的高斯分布。

从定理 7.14 的证明可以看出，$\mathbb{P}_j^{\star}$ 必须支持在集合 $\left\{z \in \mathcal{Z}: \ell_j(z) \geq \ell_{j^{\prime}}(z) \forall j^{\prime} \neq j\right\}$ 上，该集合通常是非凸的。因此，Kuhn 等（2019，第 2.2 节）推测构造 $\mathbb{P}_j^{\star}$ 是 NP 难的。然而，根据 (Hanasusanto et al. 2015a，第 6 节) 中引理 7.13 的证明，可以明确地高效构造 $\mathbb{P}_j^{\star}$。对于定理 7.14 (ii) 中出现的分布 $\mathbb{P}_j^m$，类似的评论也适用。

如果 $\mathcal{J}^{\infty} \neq \emptyset$，则定理 7.14 中构造的极值分布包含发散的混合分量，其协方差矩阵沿支持集 $\mathcal{Z}$ 的某些递归方向（即 $\Theta_j^{\star}, j \in \mathcal{J}^{\infty}$ 的非零特征值对应的特征向量方向）发散。然而，这些发散的混合分量被分配了随着其方差衰减的权重，因此整个混合分布的协方差矩阵仍然有界。

以下引理确立了 $\mathcal{J}^{\infty}$ 为空的一个充分条件，从而通过定理 7.14 (i) 保证问题 (4.1) 可解。

**证明.** 如果对某些 $j \in[J]$ 有 $p_j^{\star}=0$，则 (7.5) 中的线性矩阵不等式表明 $\theta_j^{\star}=0$。因此，第 $j$ 个迹不等式简化为 $\operatorname{Tr}\left(Q_0 \Theta_j^{\star}\right) \leq 0$。由于 $Q_0>0$（因为 $\mathcal{Z}$ 是紧集），我们可以得出 $\Theta_j^{\star}=0$。总结起来，我们证明了 $p_j^{\star}=0$ 蕴含 $\Theta_j^{\star}=0$，因此 $\mathcal{J}^{\infty}$ 是空集。

---


根据我们的术语，Richter (1957) 和 Rogosinski (1958) 表明，如果 $\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_{\mathbb{P}}[f(Z)]=\mu\right\}$，其中 $f: \mathcal{Z} \rightarrow \mathbb{R}^m$ 且 $\mu \in \mathbb{R}^m$，并且 (4.1) 可解，那么 (4.1) 中的上确界由一个至多包含 $m+2$ 个原子的离散分布达到。参见 (Shapiro et al. 2009，第 7.32 定理) 对这一结果的现代证明。

此外，在给定假设下，最坏期望问题 (4.1) 可重写为：

$$
\sup _{\varrho \in \mathcal{M}_{+}(\mathcal{Z})}\left\{\int_{\mathcal{Z}} \ell(z) \mathrm{d} \rho(z): \int_{\mathcal{Z}} \mathrm{d} \rho(z)=1, \int_{\mathcal{Z}} f(z) \mathrm{d} \rho(z)=\mu\right\}.
$$

问题 (7.8) 是一个具有 $m+1$ 个线性等式约束的无限维线性规划，定义在 $\mathcal{Z}$ 上的非负 Borel 测度上。众所周知，任何具有非负变量和 $m+1$ 个等式约束的有限维线性规划都有一个至多包含 $m+1$ 个非零变量的最优基可行解。其无限维对应物是一个至多包含 $m+1$ 个原子的离散测度。因此，可以证明如果 (7.8) 可解，则其上确界由一个至多包含 $m+1$ 个原子的测度达到（Pinelis 2016，第 5 推论和第 6(v) 命题）。这一结果强化了 Richter-Rogosinski 定理。然而，在没有额外假设的情况下，所需的原子最小数量不能少于 $m+1$。

上述推理表明，最坏期望问题 (4.1) 及其作为半无限线性规划的重构 (7.8) 可以归约为一个关于离散测度中 $m+1$ 个原子的位置和概率的有限维优化问题。这类有限归约在最优不确定性量化（Owhadi et al. 2013）中被广泛研究。然而，它们通常表示非凸优化问题。实际上，即使是相对于离散测度积分的线性函数，其计算也涉及概率和测度原子坐标的乘积。

如果 (7.8) 可解且 $\ell$ 可表示为 $J$ 个凹函数的逐点最大值，则极值测度的 $m+1$ 个原子可以进一步缩减。通过归纳论证和 Jensen 不等式的迭代应用，可以证明 (7.8) 由一个包含至多 $J$ 个原子的离散测度解出（Han, Tao, Topcu, Owhadi 和 Murray 2015，第 3.1 引理）。尽管 $J$ 不一定比 $m+1$ 小，这一结果仍具有重要意义。它表明 (7.8) 允许通过优化至多 $J$ 个原子的离散测度进行有限归约。这一（非凸）有限归约与定理 4.5 中推导的对偶问题 (4.4) 通过 "主最坏等于对偶最优" 的对偶性方案密切相关（Beck 和 Ben-Tal 2009）。具体而言，(4.4) 可被视为 "主最坏" 的鲁棒优化问题，而与 $J$ 个原子离散测度相关的有限归约可视为对应的 "对偶最优" 优化问题（Zhen et al. 2023）。在温和的正则性条件下，这些问题具有相同的最优值。

此外，通过应用变量变换和透视化技巧，可以使（对偶最优）有限归约凸化（Han et al. 2015，第 1.1 定理）。通过对 (4.4) 的有限对偶重构进行对偶化，也可以得到相同的凸重构，详见第 7.1 节。有关进一步的详细信息，请参见（Zhen et al. 2023）。



## 7.3. $\phi$-散度模糊集合

回顾 $\phi$-散度模糊集合 (2.10) 定义为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\},
$$

其中，$r$ 是大小参数，$\phi$ 是定义 2.4 中的熵函数，$\mathrm{D}_\phi$ 是定义 2.5 中相应的 $\phi$-散度，$\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 是参考分布。在以下内容中，我们首先证明，对于离散的 $\hat{\mathbb{P}}$ 和实值的 $\ell$，在 $\phi$-散度模糊集合上的最坏期望问题 (4.1) 可以重构为有限凸规划。

**假设 7.16（离散参考分布）**

假设 $\hat{\mathbb{P}}=\sum_{i \in[N]} \hat{p}_i \delta_{\hat{z}_i}$，其中 $N \in \mathbb{N}$，概率 $\hat{p}_i, i \in[N]$ 严格为正并且和为 1，且对每个 $i \in[N]$，$\hat{z}_i \in \mathcal{Z}$。此外，对于所有 $z \in \mathcal{Z}$，$\ell(z) \in \mathbb{R}$。

**注：** 要求所有 $\hat{p}_i$ 为正并无限制性，因为概率为零的原子可以被直接去除，而不改变 $\hat{\mathbb{P}}$。

**定理 7.17（$\phi$-散度模糊集合的有限对偶重构）**

如果 $\mathcal{P}$ 是 $\phi$-散度模糊集合 (2.10)，且假设 7.16 成立，则最坏期望问题 (4.1) 满足以下弱对偶关系：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] \leq\left\{\begin{array}{cl}
\inf _{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}_{+}} & \lambda_0+\lambda r+\sum_{i \in[N]} \hat{p}_i \cdot\left(\phi^*\right)^\pi\left(\ell\left(\hat{z}_i\right)-\lambda_0, \lambda\right) \\  
\text { s.t. } & \lambda_0+\lambda \phi^{\infty}(1) \geq \sup _{z \in \mathcal{Z}} \ell(z),
\end{array}\right.
$$

其中，当 $\lambda=0$ 且 $\phi^{\infty}(1)=\infty$ 时，假设乘积 $\lambda \phi^{\infty}(1)$ 取值为 $\infty$。如果 $r>0$ 且 $\phi$ 在 1 处连续，则强对偶性成立，即上述不等式变为等式。

**证明.** 定理 7.17 是定理 4.11 的直接推论。事实上，通过将离散参考分布 $\hat{\mathbb{P}}$ 的积分重新表达为加权和，可以从 (4.11) 推导出问题 (7.9)。因此，无需额外证明。

---

接下来，我们推导一个 (7.9) 的有限凸对偶规划，用于构造极值分布。

**定理 7.18（$\phi$-散度模糊集合的有限双对偶重构）**

如果 $\mathcal{P}$ 是 $\phi$-散度模糊集合 (2.10)，假设 7.16 成立，$r>0$ 且 $\phi$ 在 1 处连续，则问题 (4.1) 满足以下强对偶关系：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)]=\left\{\begin{array}{cl}
\max _{p_0, \ldots, p_N \in \mathbb{R}_{+}} & p_0 \bar{\ell}+\sum_{i \in[N]} p_i \ell\left(\hat{z}_i\right) \\  
\text { s.t. } & p_0+\sum_{i \in[N]} p_i=1 \\  
& p_0 \phi^{\infty}(1)+\sum_{i \in[N]} \hat{p}_i \phi\left(\frac{p_i}{\hat{p}_i}\right) \leq r,
\end{array}\right.
$$

其中，$\bar{\ell}$ 是 $\sup _{z \in \mathcal{Z}} \ell(z)$ 的简写。假设当 $p_0=0$ 且 $\phi^{\infty}(1)=\infty$ 时，$p_0 \phi^{\infty}(1)$ 的值为 0。同样，假设当 $p_0=0$ 且 $\bar{\ell}=\infty$ 时，$p_0 \bar{\ell}$ 的值为 0。

**证明.** 首先假设 $\phi^{\infty}(1)<\infty$。根据定理 7.17 的假设，最坏期望问题 (4.1) 和其对偶 (7.9) 具有相同的最优值。通过对 (4.11) 中的单一显式约束进行对偶化并应用引理 7.1 (i)，可以得到：

$$
\begin{aligned}
& \sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] =\inf _{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}_{+}} \lambda_0+\lambda r+\sum_{i \in[N]} \hat{p}_i\left(\sup _{y_i \in \mathbb{R}_{+}} y_i\left(\ell\left(\hat{z}_i\right)-\lambda_0\right)-\lambda \phi\left(y_i\right)\right) \\  
&+\sup _{p_0 \in \mathbb{R}_{+}}\left(\bar{\ell}-\lambda_0-\lambda \phi^{\infty}(1)\right) p_0 .
\end{aligned}
$$

通过交换最小值和最大值，并重新排列项，进一步得到：

$$
\begin{aligned}
& \sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] \\  
& =\sup _{p_0, y_1, \ldots, y_N \in \mathbb{R}_{+}} p_0 \bar{\ell}+\sum_{i \in[N]} \hat{p}_i y_i \ell\left(\hat{z}_i\right)+\inf _{\lambda_0 \in \mathbb{R}}\left(1-p_0-\sum_{i \in[N]} \hat{p}_i y_i\right) \lambda_0 \\  
&+\inf _{\lambda \in \mathbb{R}_{+}}\left(r-p_0 \phi^{\infty}(1)-\sum_{i \in[N]} \phi\left(y_i\right)\right) \lambda \\  
& =\left\{\begin{array}{cc}  
\sup _{p_0, y_0, \ldots, y_N \in \mathbb{R}_{+}} & p_0 \bar{\ell}+\sum_{i \in[N]} \hat{p}_i y_i \ell\left(\hat{z}_i\right) \\  
\text { s.t. } & p_0+\sum_{i \in[N]} \hat{p}_i y_i=1, \quad p_0 \phi^{\infty}(1)+\sum_{i \in[N]} \hat{p}_i \phi\left(y_i\right) \leq r .
\end{array}\right.
\end{aligned}
$$

通过代换 $p_i \leftarrow \hat{p}_i y_i, i \in[N]$，可以最终验证问题等价于 (7.10)。这证明了 $\phi^{\infty}(1)<\infty$ 的情况。

接下来，假设 $\phi^{\infty}(1)=\infty$，此时 $0 \phi^{\infty}(1)$ 取值为 $\infty$。因此，(4.11) 中的约束对于任何 $\left(\lambda_0, \lambda\right) \in \mathbb{R} \times \mathbb{R}_{+}$ 都成立并冗余。重复上述证明的步骤，并进行适当调整，表明当 $p_0=0$ 时，(7.10) 中的约束自然满足，问题可以通过去除 $p_0$ 进一步简化。证明完毕。

**有限双对偶重构的可解性**

(7.10) 右侧的有限双对偶重构具有线性目标函数和紧凸可行区域。因此，根据 Weierstrass 最大值定理，该问题是可解的。特别地，可行区域是 $\mathbb{R}^{N+1}$ 中概率单纯形的一个子集。如果存在最坏情形 $\hat{z}_0 \in \arg \max _{z \in \mathcal{Z}} \ell(\hat{z})$（满足 $\ell\left(z_0\right)=\bar{\ell}$），则双对偶的任意最优解 $p^{\star}$ 可用于构造最坏期望问题 (4.1) 的极值分布 $\mathbb{P}^{\star}=\sum_{i=0}^N p_i^{\star} \delta_{\hat{z}_i}$。事实上，(7.10) 的约束确保 $p_0^{\star}, \ldots, p_N^{\star}$ 是非负的概率且和为 1。因此，$\mathbb{P}^{\star}$ 是支持在 $\mathcal{Z}$ 上的有效分布。令 $\rho=\sum_{i=0}^N \delta_{\hat{z}_i}$，我们还可以得到：

$$
\begin{aligned}
\mathrm{D}_\phi\left(\mathbb{P}^{\star}, \hat{\mathbb{P}}\right) & =\int_{\mathcal{Z}} \phi^\pi\left(\frac{\mathrm{d} \mathbb{P}}{\mathrm{~d} \rho}(z), \frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z)\right) \mathrm{d} \rho(z) \\  
& =\phi^\pi\left(p_0^{\star}, 0\right)+\sum_{i \in[N]} \phi^\pi\left(p_i^{\star}, \hat{p}_i\right) \leq r,
\end{aligned}
$$

其中，第一个等号利用了 $\mathrm{D}_\phi$ 的定义，第二个等号利用了我们选择的参考分布 $\rho$。此外，不等式来自 (7.10) 的约束以及以下观察：

$$
\phi^\pi\left(p_0^{\star}, 0\right)=\phi^{\infty}\left(p_0^{\star}\right)=p_0^{\star} \phi^{\infty}(1)
$$

这证实了 $\mathbb{P}^{\star}$ 在 (4.1) 中是可行的。此外，其目标函数值为：

$$
\mathbb{E}_{\mathbb{P}^{\star}}[\ell(Z)]=\sum_{i=0}^N p_i^{\star} \ell\left(\hat{z}_i\right)
$$

由于 $\ell\left(\hat{z}_0\right)=\bar{\ell}$，我们可以得出 $\mathbb{E}_{\mathbb{P}^{\star}}[\ell(Z)]$ 与 (7.10) 中双对偶重构的最大值一致，而根据定理 7.18，该值又与 (4.1) 的上确界一致。因此，$\mathbb{P}^{\star}$ 确实是问题 (4.1) 的最优解。

**$\phi^{\infty}(1)$ 的影响**

回顾，如果 $\phi^{\infty}(1)=\infty$，则 $\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})=\infty$，除非 $\mathbb{P} \ll \hat{\mathbb{P}}$。因此，$\phi$-散度模糊集合中任何 $\mathbb{P}$ 都必须相对于 $\hat{\mathbb{P}}$ 绝对连续。如果 $\phi^{\infty}(1)<\infty$，则 $\mathbb{P}$ 可以对 $\mathcal{Z}$ 中 $\hat{\mathbb{P}}$ 赋零概率的点分配正概率。需要注意的是，$\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$ 仅依赖于 $\mathbb{P}$ 从 $\hat{\mathbb{P}}$ 支持集中移除的概率质量，但不依赖于这些概率质量的移动位置。由于自然的目标是最大化期望损失，它将把所有的概率质量移到 $\mathcal{Z}$ 中损失最大的点（即某点 $\hat{z}_0 \in \arg \max _{z \in \mathcal{Z}} \ell(\hat{z})$）。

**关于限制性 $\phi$-散度模糊集合的扩展**

如果 $\mathcal{P}$ 是限制性 $\phi$-散度模糊集合 (2.11)，假设 7.16 成立，$r>0$ 且 $\phi$ 在 1 处连续，则定理 7.18 仍然适用，只需对 (7.10) 右侧的有限双对偶重构附加约束 $p_0=0$。具体细节从略。


## 7.4. 最优传输模糊集合

回顾，最优传输模糊集合 (2.27) 定义为

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{OT}_c(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\},
$$

其中 $r \geq 0$ 是一个尺度参数，$c$ 是定义 2.14 中的传输成本函数，$\mathrm{OT}_c$ 是定义 2.15 中的对应最优传输差异，$\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 是一个参考分布。我们首先展示如何将最优传输模糊集合上的最坏期望问题 (4.1) 通常重构为有限凸最小化问题。为此，我们仅关注假设 7.16 中的离散参考分布，并对传输成本函数、损失函数以及支持集合 $\mathcal{Z}$ 施加凸性条件。此外，我们对离散参考分布 $\hat{\mathbb{P}}$ 的支持点施加了一个温和的技术条件。

**假设 7.19（最优传输模糊集合的正则性条件）**
1. 损失函数 $\ell$ 是 $J \in \mathbb{N}$ 个凹函数的逐点最大值，即 $\ell(z)=\max _{j \in[J]} \ell_j(z)$，其中 $-\ell_j: \mathcal{Z} \rightarrow \mathbb{R}$ 是适当、凸且闭合的。
2. 支持集合表示为 $\mathcal{Z}=\left\{z \in \mathbb{R}^d: g_k(z) \leq 0 \forall k \in[K]\right\}$，其中 $K \in \mathbb{N}$，$g_k: \mathcal{Z} \rightarrow \overline{\mathbb{R}}$ 是适当、凸且闭合的。
3. 对于每个固定的 $\hat{z} \in \mathcal{Z}$，传输成本函数 $c(z, \hat{z})$ 关于 $z$ 是凸的。
4. 支持点 $\hat{z}_i$ 属于 $\operatorname{rint}\left(\operatorname{dom}\left(c\left(\cdot, \hat{z}_i\right)\right)\right)$，并对 $\mathcal{Z}$ 构成定义 7.3 意义下的 Slater 点，对于每个 $i \in[N]$。

假设 7.19 (1) 是非限制性的，因为任何紧集 $\mathcal{Z}$ 上的连续函数 $\ell$ 都可以通过有限多凹函数 $\ell_j, j \in[J]$ 的逐点最大值来一致逼近，但可能需要大量的片段 $J$。假设 7.19 (2) 和 (3) 是限制性的，但通常满足于应用中常见的支持集合和传输成本函数。最后，假设 7.19 (4) 纯属技术性质，可通过稍微调整问题数据来始终实现。

**定理 7.20（最优传输模糊集合的有限对偶重构）**  
如果 $\mathcal{P}$ 是最优传输模糊集合 (2.27) 且假设 7.16 和 7.19 成立，则最坏期望问题 (4.1) 满足如下弱对偶关系：

$$
\begin{aligned}
& \sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] \\ 
& \leq \left\{\begin{array}{ll}
\inf & \lambda r+\sum_{i \in[N]} \hat{p}_i s_i \\
\text { s.t. } & \lambda \in \mathbb{R}_{+}, \alpha_{i j k} \in \mathbb{R}_{+}, s_i \in \mathbb{R} \quad \forall i \in[N], j \in[J], k \in[K] \\
& \zeta_{i j}^{\ell}, \zeta_{i j}^c, \zeta_{i j k}^g \in \mathbb{R}^d \quad \forall i \in[N], j \in[J], k \in[K] \\
& \left(-\ell_j\right)^*\left(\zeta_{i j}^{\ell}\right)+\left(c_i^*\right)^\pi\left(\zeta_{i j}^c, \lambda\right) \\
& +\sum_{k \in[K]}\left(g_k^*\right)^\pi\left(\zeta_{i j k}^g, \alpha_{i j k}\right) \leq s_i \quad \forall i \in[N], j \in[J] \\
& \zeta_{i j}^{\ell}+\zeta_{i j}^c+\sum_{k \in[K]} \zeta_{i j k}^g=0 \quad \forall i \in[N], j \in[J],
\end{array}\right.
\end{aligned}
$$

其中 $c_i: \mathcal{Z} \rightarrow \overline{\mathbb{R}}$ 定义为 $c_i(z)=c\left(z, \hat{z}_i\right)$，对所有 $i \in[N]$ 成立。如果 $r>0$，则强对偶成立，即上述不等式变为等式。

**定理 7.20 的证明**  
根据定理 4.18，我们有：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] \leq\left\{\begin{array}{lll}
\text { inf } & \lambda r+\sum_{i \in[N]} \hat{p}_i s_i & \\ 
\text { s.t. } & \lambda \in \mathbb{R}_{+}, s_i \in \mathbb{R} & \forall i \in[N] \\
& \sup _{z \in \mathcal{Z}} \ell(z)-\lambda c\left(z, \hat{z}_i\right) \leq s_i & \forall i \in[N],
\end{array}\right.
$$

这里 $s_i$ 表示对每个 $i \in[N]$ 的辅助凸包决策变量。假设 7.19 (1) 和定义 $c_i$ 表明上述最小化问题等价于以下鲁棒凸规划：

$$
\begin{array}{ll}
\text { inf } & \lambda r+\sum_{i \in[N]} \hat{p}_i s_i \\ 
\text { s.t. } & \lambda \in \mathbb{R}_{+}, s_i \in \mathbb{R} \\
& \sup _{z \in \mathcal{Z}} \ell_j(z)-\lambda c_i(z) \leq s_i \quad \forall i \in[N], j \in[J]
\end{array}
$$

对任何固定的 $i \in[N]$ 和 $j \in[J]$，假设 7.19 (1) 和 (2) 表明嵌入的最大化问题是一个凸规划。此外，由假设 7.19 (1) 和 (4) 可知，该问题存在 Slater 点 $\hat{z}_i$。利用引理 7.2，我们得出：

$$
f^*(\zeta)=\min _{\zeta_{i j}^{\ell}, \zeta_{i j}^c \in \mathbb{R}^d}\left\{\left(-\ell_j\right)^*\left(\zeta_{i j}^{\ell}\right)+\left(c_i^*\right)^\pi\left(\zeta_{i j}^c, \lambda\right): \zeta_{i j}^{\ell}+\zeta_{i j}^c=\zeta\right\} .
$$

通过进一步变换并应用假设，我们可以得到目标问题的有限凸对偶重构，从而证明了弱对偶关系的成立。如果 $r>0$，则根据定理 4.18，可知强对偶也成立。证明完毕。

我们强调，引理 7.21 和 7.22 与引理 4.11 是互补的。确实，虽然引理 4.11 仅针对透视函数的第一个参数求共轭，但引理 7.21 和 7.22 针对两个参数进行了求共轭。现在我们准备推导一个关于最优传输模糊集合上的最坏期望问题的有限双对偶重构。

**定理 7.23（最优传输模糊集合的有限双对偶重构）**  
如果 $\mathcal{P}$ 是最优传输模糊集合 (2.27) 并且假设 7.16 和 7.19 成立，则最坏期望问题 (4.1) 满足如下弱对偶关系：

$$
\begin{aligned}
& \sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] \\ 
& \leq\left\{\begin{array}{lll}
\sup & \sum_{i \in[N]} \sum_{j \in[J]}-\left(-\ell_j\right)^\pi\left(p_{i j} \hat{z}_i+z_{i j}, p_{i j}\right) \\ 
\text { s.t. } & p_{i j} \in \mathbb{R}_{+}, z_{i j} \in \mathbb{R}^d & \forall i \in[N], j \in[J] \\ 
& g_k^\pi\left(p_{i j} \hat{z}_i+z_{i j}, p_{i j}\right) \leq 0 & \forall i \in[N], j \in[J], k \in[K] \\ 
& \sum_{j \in[J]} p_{i j}=\hat{p}_i & \forall i \in[N] \\ 
& \sum_{i \in[N]} \sum_{j \in[J]} c_i^\pi\left(p_{i j} \hat{z}_i+z_{i j}, p_{i j}\right) \leq r,
\end{array}\right.
\end{aligned}
$$

其中 $c_i: \mathcal{Z} \rightarrow \overline{\mathbb{R}}$ 通过 $c_i(z)=c\left(z, \hat{z}_i\right)$ 定义，对于所有 $i \in[N]$。如果 $r>0$，则强对偶成立，即上述不等式变为等式。

**证明**  
我们将展示如何通过对问题 (4.1) 的有限对偶重构 (7.11) 进行对偶化来得到 (7.13)。为此，我们为 (7.11) 中的第一和第二约束组分别分配拉格朗日乘数 $p_{i j} \in \mathbb{R}_{+}$ 和 $z_{i j} \in \mathbb{R}^d, i \in[N], j \in[J]$。问题 (7.11) 的拉格朗日对偶可以简洁地表示为：

$$
\sup _{p \geq 0, z} \inf _{\substack{\lambda \geq 0, \alpha \geq 0 \\ s, \zeta^{\ell}, \zeta^c, \zeta^g}} L_1(s ; p, z)+L_2\left(\zeta^{\ell} ; p, z\right)+L_3\left(\lambda, \zeta^c ; p, z, \lambda\right)+L_4\left(\alpha, \zeta^g ; p, z\right),
$$

其中拉格朗日函数在四组不相交的原始决策变量上可加分离，分别为 $s, \zeta^{\ell},\left(\lambda, \zeta^c\right)$ 和 $\left(\alpha, \zeta^g\right)$。对应的部分拉格朗日函数定义如下：

$$
\begin{aligned}
L_1(s ; p, z) & =\sum_{i \in[N]} \hat{p}_i s_i-\sum_{i \in[N]} \sum_{j \in[J]} p_{i j} s_i, \\ 
L_2\left(\zeta^{\ell} ; p, z\right) & =\sum_{i \in[N]} \sum_{j \in[J]} p_{i j} \cdot\left(-\ell_j\right)^*\left(\zeta_{i j}^{\ell}\right)-z_{i j}^{\top} \zeta_{i j}^{\ell}, \\ 
L_3\left(\lambda, \zeta^c ; p, z\right) & =\lambda r+\sum_{i \in[N]} \sum_{j \in[J]} p_{i j} \cdot\left(c_i^*\right)^\pi\left(\zeta_{i j}^c, \lambda\right)-z_{i j}^{\top} \zeta_{i j}^c, \\ 
L_4\left(\alpha, \zeta^g ; p, z\right) & =\sum_{i \in[N]} \sum_{j \in[J]} \sum_{k \in[K]} p_{i j} \cdot\left(g_k^*\right)^\pi\left(\zeta_{i j k}^g, \alpha_{i j k}\right)-z_{i j}^{\top} \zeta_{i j k}^g.
\end{aligned}
$$

通过分别对每个部分拉格朗日函数的原始决策变量求最小值，可以得到以下结果：

$$
\inf _s L_1(s ; p, z)= \begin{cases}0 & \text { 如果 } \sum_{j \in[J]} p_{i j}=\hat{p}_i \forall i \in[N], \\ -\infty & \text { 否则。 }\end{cases}
$$

类似地，对 $\zeta^{\ell}$ 求最小值可以得到：

$$
\inf _{\zeta^{\ell}} L_2\left(\zeta^{\ell} ; p, z\right)=\sum_{i \in[N]} \sum_{j \in[J]}-\left(-\ell_j\right)^\pi\left(z_{i j}, p_{i j}\right).
$$

对 $\lambda$ 和 $\zeta^c$ 求最小值可以得到：

$$
\inf _{\lambda \geq 0, \zeta^c} L_3\left(\lambda, \zeta^c ; p, z\right)= \begin{cases}0 & \text { 如果 } c_i^\pi\left(z_{i j}, p_{i j}\right) \leq \beta_{i j} \forall i, j, \\ -\infty & \text { 否则。 }\end{cases}
$$

对 $\alpha$ 和 $\zeta^g$ 求最小值可以得到：

$$
\inf _{\alpha \geq 0, \zeta^g} L_4\left(\alpha, \zeta^g ; p, z\right)= \begin{cases}0 & \text { 如果 } g_k^\pi\left(z_{i j}, p_{i j}\right) \leq 0 \forall i, j, k, \\ -\infty & \text { 否则。 }\end{cases}
$$
**将部分拉格朗日函数的极小值代入对偶目标函数，得到与问题 (7.11) 对偶问题等价的重构形式：**

$$
\begin{array}{lll}
\text { sup } & \sum_{i \in[N]} \sum_{j \in[J]}-\left(-\ell_j\right)^\pi\left(z_{i j}, p_{i j}\right) & \\
\text { s.t. } & p_{i j} \in \mathbb{R}_{+}, \beta_{i j} \in \mathbb{R}, z_{i j} \in \mathbb{R}^d & \forall i \in[N], j \in[J] \\
& g_k^\pi\left(z_{i j}, p_{i j}\right) \leq 0 & \forall i \in[N], j \in[J], k \in[K] \\
& \sum_{j \in[J]} p_{i j}=\hat{p}_i & \forall i \in[N] \\
& c_i^\pi\left(z_{i j}, p_{i j}\right) \leq \beta_{i j} & \forall i \in[N], j \in[J] \\
& \sum_{i \in[N]} \sum_{j \in[J]} \beta_{i j}=r & 
\end{array}
$$

**说明：**  
如果将最坏期望问题的有限对偶重构形式 (7.11) 视为原始凸规划问题 (P) 的一个实例，那么问题 (7.14) 就是相应的对偶凸规划问题 (D) 的实例。根据假设 7.16 和 7.19，问题 (7.14) 存在一个 Slater 点，其中 $p_{i j}=\hat{p}_i / J$ 且 $z_{i j}=\hat{z}_i$ 对所有 $i \in[N]$ 和 $j \in[J]$ 成立。因此，根据定理 7.4 (i)，强对偶性成立。

要证明 (7.14) 等价于 (7.13)，首先注意到 (7.14) 的最后一个约束可以放宽为小于等于的不等式，而不会增加问题的上确界，从而在最优情况下有 $\beta_{i j}=c_i^\pi\left(z_{i j}, p_{i j}\right)$。这使得我们可以从 (7.14) 中消除 $\beta_{i j}$ 变量。通过将 $z_{i j} \leftarrow z_{i j}-p_{i j} \hat{z}_i$ 应用于问题 (7.14)，即可得到问题 (7.13)。

---

**假设 7.24（不可区分性标识假设）：**  
传输成本函数是实值的，并且满足 $c(z, \hat{z})=0$ 当且仅当 $z=\hat{z}$。

---

**引理 7.25（有限双对偶重构形式的可解性）：**  
假设 7.16、7.19 和 7.24 成立，则问题 (7.13) 是可解的。

**证明：**  
在所述假设下，问题 (7.13) 在紧致的可行区域上最大化一个上半连续函数，因此由魏尔施特拉斯最大值定理得证。  
为了证明 (7.13) 的目标函数是上半连续的，注意到 $-\ell_j$ 对所有 $j \in[J]$ 都是适当的、凸的和闭合的（假设 7.19 (i)）。根据 (Rockafellar 1970, 第 35 和 67 页)，它们的透视函数也是适当的、凸的和闭合的；参见 (Zhen et al. 2023, 命题 C.2)。因此，问题 (7.13) 中目标函数的负透视部分确实是上半连续的。类似地，可以证明问题 (7.13) 的可行区域是闭合的。实际上，$g_k$ 和 $c_i$ 对所有 $k \in[K]$ 和 $i \in[N]$ 都是适当的、凸的和闭合的（假设 7.19 和定义 2.14）。这进一步意味着它们的透视函数是下半连续的，因此问题 (7.13) 的可行区域确实是闭合的。  

为了证明可行区域是有界的，首先注意到对于所有 $i \in[N]$ 和 $j \in[J]$，$p_{i j} \in [0,1]$。实际上，这些变量必须是非负的，并与离散参考分布中 $\hat{p}_i, i \in[N]$ 的概率兼容。接下来，我们证明对于 $i \in[N]$ 和 $j \in[J]$ 的变量 $z_{i j}$ 也限制在一个有界集合内。确实，根据 (Zhen et al. 2023, 引理 C.10)，假设 7.24 和定义 2.14 成立，则存在 $\delta > 0$，使得 $c_i\left(\hat{z}_i+z\right) \geq \delta\|z\|_2 - 1$ 对所有 $z \in \mathbb{R}^d$ 和 $i \in[N]$ 成立。问题 (7.13) 的最后一个约束条件因此暗示：

$$
\sum_{i \in[N]} \sum_{j \in[J]} c_i^\pi\left(p_{i j} \hat{z}_i+z_{i j}, p_{i j}\right) \leq r \quad \Longrightarrow \quad \sum_{i \in[N]} \sum_{j \in[J]} \|z_{i j}\|_2 \leq \frac{1+r}{\delta},
$$

其中我们使用了等式 $\sum_{i \in[N]} \sum_{j \in[J]} p_{i j} = \sum_{i \in[N]} \hat{p}_i = 1$ 和透视函数的定义。因此，问题 (7.13) 的可行区域确实是有界的。

**构造极值分布：**  
我们现在准备构造极值分布 $\mathbb{P}^\star \in \mathcal{P}(\mathcal{Z})$，该分布达到最优传输模糊集合 (2.27) 上的最坏期望问题 (4.1) 的上确界。为此，固定问题 (7.13) 的任意最优解 $(p^\star, z^\star)$，其存在性由引理 7.25 保证。接下来定义索引集合：

$$
\mathcal{J}_i^\infty = \left\{j \in [J]: p_{ij}^\star = 0, z_{ij}^\star \neq 0\right\} \quad \text{和} \quad \mathcal{J}_i^+ = \left\{j \in [J]: p_{ij}^\star > 0\right\},
$$

并为每个 $i \in [N]$ 定义 $\mathcal{J}_i = \mathcal{J}_i^+ \cup \mathcal{J}_i^\infty$。

以下定理使用 $(p^\star, z^\star)$ 及其相应的索引集合来构造 $\mathbb{P}^\star$。

**定理 7.26（最优运输模糊集的极值分布）**  
假设定理 7.23 的所有关于弱对偶和强对偶的条件都成立，假设 7.24 成立，并且 $\left(p^{\star}, z^{\star}\right)$ 是问题 (7.13) 的解。那么，有以下结论：  

**(i)** 如果对所有 $i \in[N]$ 有 $\mathcal{J}_i^{\infty}=\emptyset$，则问题 (4.1) 的解为：

$$
\mathbb{P}^{\star}=\sum_{i \in[N]} \sum_{j \in \mathcal{J}_i^{+}} p_{i j}^{\star} \delta_{\hat{z}_i+z_{i j}^{\star} / p_{i j}^{\star}}
$$

**(ii)** 如果对某些 $i \in[N]$ 有 $\mathcal{J}_i^{\infty} \neq \emptyset$，则问题 (4.1) 由分布序列 $\mathbb{P}^m=\sum_{i \in[N]} \sum_{j \in \mathcal{J}_i} p_{i j}^m \delta_{z_{i j}^m}$ 随 $m \in \mathbb{N}, m \geq \max _{i \in[N]}\left|\mathcal{J}_i^{\infty}\right|$ 收敛时渐近求解，其中：

$$
p_{i j}^m=\left\{\begin{array}{ll}
\left(1-\frac{\left|\mathcal{J}_i^{\infty}\right|}{m}\right) p_{i j}^{\star} & \text { 如果 } j \in \mathcal{J}_i^{+}, \\[5pt]
\frac{\hat{p}_i}{m} & \text { 如果 } j \in \mathcal{J}_i^{\infty},
\end{array} \quad z_{i j}^m=\begin{cases}
\hat{z}_i+\frac{z_{i j}^{\star}}{p_{i j}^{\star}} & \text { 如果 } j \in \mathcal{J}_i^{+}, \\[5pt]
\hat{z}_i+\frac{z_{i j}^{\star}}{p_{i j}^m} & \text { 如果 } j \in \mathcal{J}_i^{\infty}.
\end{cases}\right.
$$

---

**证明**  
对于断言 **(i)**，我们首先证明定理中定义的 $\mathbb{P}^{\star}$ 在最坏期望问题 (4.1) 中是可行的。为此，首先注意到 $\left(p^{\star}, z^{\star}\right)$ 在问题 (7.13) 中的可行性意味着 $p_{i j}^{\star} \geq 0$ 对所有 $i \in[N]$ 和 $j \in[J]$ 成立，并且 $\sum_{i \in[N]} \sum_{j \in \mathcal{J}_i^{+}} p_{i j}^{\star}=1$。还需注意，对于所有 $i \in[N]$ 和 $j \in \mathcal{J}_i^{+}$，有 $\hat{z}_i+z_{i j}^{\star} / p_{i j}^{\star} \in \mathcal{Z}$，这是由于问题 (7.13) 的第二组约束。因此，可以确认 $\mathbb{P}^{\star} \in \mathcal{P}(\mathcal{Z})$。问题 (7.13) 的倒数第二组约束还意味着：

$$
\sum_{i \in[N]} \sum_{j \in \mathcal{J}_i^{+}} p_{i j}^{\star} \delta_{\left(\hat{z}_i+z_{i j}^{\star} / p_{i j}^{\star}, \hat{z}_i\right)} \in \Gamma\left(\mathbb{P}^{\star}, \hat{\mathbb{P}}\right)
$$

构成了一个有效的运输计划，用于将 $\hat{\mathbb{P}}$ 转化为 $\mathbb{P}^{\star}$。因此，我们有：

$$
\begin{aligned}
\mathrm{OT}_c\left(\mathbb{P}^{\star}, \hat{\mathbb{P}}\right) & \leq \sum_{i \in[N]} \sum_{j \in \mathcal{J}_i^{+}} p_{i j}^{\star} \cdot c\left(\hat{z}_i+z_{i j}^{\star} / p_{i j}^{\star}, \hat{z}_i\right) \\
& =\sum_{i \in[N]} \sum_{j \in[J]} c_i^\pi\left(p_{i j}^{\star} \hat{z}_i+z_{i j}^{\star}, p_{i j}^{\star}\right) \leq r。
\end{aligned}
$$

等式成立是因为所有与 $j \notin \mathcal{J}_i^{+}$ 对应的项都为零。实际上，如果 $j \notin \mathcal{J}_i^{+}$，则 $p_{i j}^{\star}=0$。由于 $\mathcal{J}_i^{\infty}=\emptyset$，这进一步意味着 $z_{i j}^{\star}=0$。因此，根据透视函数和衰退函数的定义，有 $c_i^\pi\left(p_{i j}^{\star} \hat{z}_i+z_{i j}^{\star}, p_{i j}^{\star}\right)=c_i^\pi(0,0)=c_i^{\infty}(0)=0$。上述表达式中的第二个不等式来源于问题 (7.13) 的最后一个约束。

综上所述，我们证明了 $\mathbb{P}^{\star}$ 在 (4.1) 中是可行的。

关于 $\mathbb{P}^{\star}$ 的目标函数值，注意到：

$$
\mathbb{E}_{\mathbb{P}^{\star}}[\ell(Z)] \leq \sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] \leq \sum_{i \in[N]} \sum_{j \in[J]}-\left(-\ell_j\right)^\pi\left(p_{i j}^{\star} \hat{z}_i+z_{i j}^{\star}, p_{i j}^{\star}\right),
$$

其中第二个不等式来自定理 7.23 中建立的弱对偶关系。同时，$\mathbb{P}^{\star}$ 下的期望损失满足：

$$
\begin{aligned}
\mathbb{E}_{\mathbb{P}^{\star}}[\ell(Z)] & =\sum_{i \in[N]} \sum_{j \in \mathcal{J}^{+}} \max _{j^{\prime} \in[J]} p_{i j}^{\star} \ell_{j^{\prime}}\left(\hat{z}_i+\frac{z_{i j}^{\star}}{p_{i j}^{\star}}\right) \\
& \geq \sum_{i \in[N]} \sum_{j \in \mathcal{J}^{+}}-\left(-\ell_j\right)^\pi\left(p_{i j}^{\star} \hat{z}_i+z_{i j}^{\star}, p_{i j}^{\star}\right) \\
& =\sum_{i \in[N]} \sum_{j \in[J]}-\left(-\ell_j\right)^\pi\left(p_{i j}^{\star} \hat{z}_i+z_{i j}^{\star}, p_{i j}^{\star}\right)。
\end{aligned}
$$

以上不等式利用了透视函数的定义以及显然的事实 $j \in \mathcal{J}^{+}$ 是 $j^{\prime} \in[J]$ 的一个可行选择。最后一个等式再次成立，因为 $p_{i j}^{\star}=0$ 导致 $z_{i j}^{\star}=0$ 且 $\left(-\ell_j\right)^\pi(0,0)=\left(-\ell_j\right)^{\infty}(0)=0$。综上，上述不等式表明 $\mathbb{P}^{\star}$ 在 (4.1) 中是最优的，因此断言 (i) 得证。
**(ii)** 接下来我们证明，对于任意固定的 $m \geq \max _{i \in[N]}\left|\mathcal{J}_i^{\infty}\right|$，$\mathbb{P}^m \in \mathcal{P}$。问题 (7.13) 的约束条件表明，对于所有 $i \in[N]$ 和 $j \in \mathcal{J}_i$，都有 $p_{i j}^m \geq 0$，并且 $\sum_{i \in[N]} \sum_{j \in \mathcal{J}} p_{i j}^m=1$。这些约束还表明，对于每个 $j \in \mathcal{J}_i$ 和 $i \in[N]$，$z_{i j}^m \in \mathcal{Z}$。

当 $j \in \mathcal{J}_i^{+}$ 时，这是显而易见的。如果 $j \in \mathcal{J}_i^{\infty}$，则有 $p_{i j}^{\star}=0$，$z_{i j}^{\star} \neq 0$，并且 $g_k^\pi\left(z_{i j}^{\star}, 0\right) \leq 0$ 对所有 $k \in[K]$ 成立。这通过 (Rockafellar 1970, 定理 8.6) 表明，$z_{i j}^{\star}$ 是 $\mathcal{Z}$ 的衰退方向。从几何上讲，这意味着从 $\mathcal{Z}$ 中任何一点沿方向 $z_{i j}^{\star}$ 发出的射线永远不会离开 $\mathcal{Z}$。因此，$z_{i j}^m=\hat{z}_i+m z_{i j}^{\star} / \hat{p}_i \in \mathcal{Z}$ 对所有 $i \in[N]$ 和 $j \in \mathcal{J}_i^{\infty}$ 都成立。此外可以验证：

$$
\sum_{i \in[N]} \sum_{j \in \mathcal{J}_i} p_{i j}^m \delta_{\left(z_{i j}^m, \hat{z}_i\right)} \in \Gamma\left(\mathbb{P}^{\star}, \hat{\mathbb{P}}\right)
$$

构成了一个有效的运输计划，用于将 $\hat{\mathbb{P}}$ 转换为 $\mathbb{P}^m$。因此我们有：

$$
\begin{aligned}
& \mathrm{OT}_c\left(\mathbb{P}^m, \hat{\mathbb{P}}\right) \\
& \leq \sum_{i \in[N]} \sum_{j \in \mathcal{J}_i} p_{i j}^m c\left(z_{i j}^m, \hat{z}_i\right) \\
& =\sum_{i \in[N]} \sum_{j \in \mathcal{J}_i^{+}} p_{i j}^{\star}\left(1-\frac{\left|\mathcal{J}_i^{\infty}\right|}{m}\right) c\left(\hat{z}_i+\frac{z_{i j}^{\star}}{p_{i j}^{\star}}, \hat{z}_i\right)+\sum_{i \in[N]} \sum_{j \in \mathcal{J}_i^{\infty}} \frac{\hat{p}_i}{m} c\left(\hat{z}_i+m \frac{z_{i j}^{\star}}{\hat{p}_i}, \hat{z}_i\right) \\
& \leq \sum_{i \in[N]} \sum_{j \in \mathcal{J}_i^{+}} p_{i j}^{\star} c\left(\hat{z}_i+\frac{z_{i j}^{\star}}{p_{i j}^{\star}}, \hat{z}_i\right)+\sum_{i \in[N]} \sum_{j \in \mathcal{J}_i^{\infty}} \lim _{m \rightarrow \infty} \frac{\hat{p}_i}{m} c\left(\hat{z}_i+m \frac{z_{i j}^{\star}}{\hat{p}_i}, \hat{z}_i\right) \\
& =\sum_{i \in[N]} \sum_{j \in \mathcal{J}_i^{+}} p_{i j}^{\star} c\left(\hat{z}_i+\frac{z_{i j}^{\star}}{p_{i j}^{\star}}, \hat{z}_i\right)+\sum_{i \in[N]} \sum_{j \in \mathcal{J}_i^{\infty}} \lim _{m \rightarrow \infty} \frac{\hat{p}_i}{m} c\left(m \frac{z_{i j}^{\star}}{\hat{p}_i}, \hat{z}_i\right) \\
& =\sum_{i \in[N]} \sum_{j \in[J]} c_i^\pi\left(p_{i j}^{\star} \hat{z}_i+z_{i j}^{\star}, p_{i j}^{\star}\right) \leq r,
\end{aligned}
$$

其中第一等式源自 $p_{i j}^m$ 和 $z_{i j}^m$ 的定义。第二个不等式成立是因为运输成本函数 $c(z, \hat{z})$ 在 $z$ 上非负且凸，这意味着第三行中的两项在 $m$ 上单调不减。第二等式依据假设 7.24，该假设确保 $c(z, \hat{z})$ 是实值函数，从而可以自由选择定义 $c\left(\cdot, \hat{z}_i\right)$ 衰退函数的参考点。第三等式利用了透视函数 $c_i^\pi$ 的定义以及 $c_i^\pi(0,0)=c_i^{\infty}(0)=0$ 的事实。最后一个不等式来自问题 (7.13) 的最后一个约束条件。

因此，我们已经证明 $\mathbb{P}^m$ 在问题 (4.1) 中是可行的。类比对 $\mathbb{P}^{\star}$ 的分析，可以证明渐近期望损失 $\lim _{m \rightarrow \infty} \mathbb{E}_{\mathbb{P}^m}[\ell(Z)]$ 至少等于有限双对偶重构 (7.13) 的最优值 $\sum_{i \in[N]} \sum_{j \in[J]}-\left(-\ell_j\right)^\pi\left(p_{i j}^{\star} \hat{z}_i+z_{i j}^{\star}, p_{i j}^{\star}\right)$。然而，由于 (4.1) 和 (7.13) 的上确界相同，显然分布 $\mathbb{P}^m, m \in \mathbb{N}$ 必须在问题 (4.1) 中渐近最优。

如果对于某些 $i \in [N]$，$\mathcal{J}_i^{\infty} \neq \emptyset$，那么由定理 7.26 构造的极端分布会将具有递减概率的原子沿着支撑集 $\mathcal{Z}$ 的特定衰退方向 $z_{i j}^{\star}, j \in \mathcal{J}_i^{\infty}$ 移向无穷远。即使只有有限的运输预算 $r$ 可用，当被运输的概率质量与运输成本成反比缩放时，这种将原子移向无穷远的方法也是可行的。以下引理确立了充分条件，使得对于每个 $i \in [N]$，$\mathcal{J}_i^{\infty}$ 为空，从而根据定理 7.26 (i) 确保问题 (4.1) 是可解的。

****引理 7.27****  
如果定理 7.26 的所有假设成立并且满足以下任一条件，则对于每个 $i \in [N]$，$\mathcal{J}_i^{\infty} = \emptyset$，并且 (4.1) 是可解的。  
(i) 运输成本函数在其第一个参数上超线性增长。这意味着对于任意 $z \neq 0$ 和 $i \in [N]$，$c_i^{\infty}(z) = \infty$。  
(ii) 支撑集 $\mathcal{Z}$ 是有界的。  

****证明****  
如往常一样，设 $(p^{\star}, z^{\star})$ 是问题 (7.13) 的一个最优解，根据引理 7.25，这样的最优解是存在的。  

对于断言 (i)，假设运输成本函数超线性增长。为了便于讨论，还假设存在 $i \in [N]$ 使得 $\mathcal{J}_i^{\infty} \neq \emptyset$。对于每个 $j \in \mathcal{J}_i^{\infty}$，我们有 $p_{i j}^{\star} = 0$ 且 $z_{i j}^{\star} \neq 0$。因此，我们得到

$$
c_i^\pi\left(p_{i j}^{\star} \hat{z}_i + z_{i j}^{\star}, p_{i j}^{\star}\right) = c_i^{\infty}\left(z_{i j}^{\star}\right) = \infty,
$$

其中，第一个等式使用了透视函数的定义，第二个等式因为运输成本函数超线性增长而成立。因此，$(p^{\star}, z^{\star})$ 违反了问题 (7.13) 的最后一个约束条件，这与其假定的可行性相矛盾。因此我们可以得出结论 $\mathcal{J}_i^{\infty} = \emptyset$ 并且问题 (4.1) 是可解的。  

对于断言 (ii)，假设现在 $\mathcal{Z}$ 是有界的。没有损失一般性，可以假设对于某些 $i \in [N]$ 和 $j \in [J]$，$p_{i j}^{\star} = 0$，否则 $\mathcal{J}_i^{\infty}$ 显然为空。问题 (7.13) 的约束条件确保了 $g_k^\pi\left(z_{i j}^{\star}, 0\right) \leq 0$ 对所有 $k \in [K]$ 成立，这通过 (Rockafellar 1970, 定理 8.6) 表明 $z_{i j}^{\star}$ 是 $\mathcal{Z}$ 的衰退方向。然而，由于 $\mathcal{Z}$ 是紧的，这意味着 $z_{i j}^{\star} = 0$。因此，我们再次得出结论 $\mathcal{J}_i^{\infty} = \emptyset$ 并且问题 (4.1) 是可解的。  

引理 7.27 的条件 (i) 在 $\mathcal{P}$ 是一个 $p$-Wasserstein 球且运输成本函数的形式为 $c(z, \hat{z}) = \|z - \hat{z}\|^p$，其中 $p > 1$，时是成立的。  

对于在最优传输模糊集上求解最坏情况下期望问题 (4.1) 的分布的结构性质，以及这些分布存在的必要和充分条件，已有研究由 Wozabal (2012)、Owhadi 和 Scovel (2017)、Yue 等人 (2022) 以及 Gao 和 Kleywegt (2023) 进行探讨。特别是，对于以离散参考分布（具有 $N$ 个原子）为中心的 Wasserstein 球的极端分布的表征，已投入了大量精力。该领域的最早结果由 Wozabal (2012, 定理 3.3) 提出，他证明了一个连续有界损失函数的最坏情况期望由最多包含 $N+3$ 个原子的离散分布达到。后来，Owhadi 和 Scovel (2017, 定理 2.3) 以及 Gao 和 Kleywegt (2023, 推论 1) 进一步改进了这一结果，表明最坏情况期望实际上是由最多包含 $N+2$ 或甚至仅 $N+1$ 个原子的离散分布达到的；参见 Yue 等人 (2022, 定理 4)。定理 7.26 (i) 和引理 7.27 表明，如果 $\mathcal{Z}$ 是有界的并且损失函数 $\ell$ 是凹的，因此满足假设 7.19 (i) 且 $J = 1$，则最坏情况下的期望损失由一个 $N$ 点分布达到。然而，对于更一般的损失函数，即使问题 (4.1) 可解，每个 $N$ 点分布也可能是严格次优的；参见 Kuhn 等人 (2019, 示例 5)。本节的结果基于 Zhen 等人 (2023, § 6) 的研究。





## 7.5. 纳什均衡与对抗攻击

分布鲁棒优化（DRO）问题 (1.2) 可以被看作一个零和博弈，其中决策者首先选择一个决策 $x \in \mathcal{X}$，随后自然选择一个分布 $\mathbb{P} \in \mathcal{P}$ 来适应 $x$。在本节中，我们将 (1.2) 称为原始 DRO 问题。此外，还可以研究对偶 DRO 问题

$$
\sup _{\mathbb{P} \in \mathcal{P}} \inf _{x \in \mathcal{X}} \mathbb{E}_{\mathbb{P}}[\ell(x, Z)],
$$

其中自然首先选择一个分布 $\mathbb{P} \in \mathcal{P}$，然后决策者选择一个决策 $x \in \mathcal{X}$ 来适应 $\mathbb{P}$。与目标函数在 $\mathbb{P}$ 上线性的原始 DRO 问题 (1.2) 不同，对偶 DRO 问题 (7.15) 的目标函数在 $\mathbb{P}$ 上是凹的。这一差异使得对偶 DRO 问题更难求解。自然地，我们希望寻找某些条件以确保强对偶性，从而保证原始 DRO 问题 (1.2) 的下确界与对偶 DRO 问题 (7.15) 的上确界一致。可以验证，强对偶性可以通过存在一个满足鞍点条件的纳什均衡 $\left(x^{\star}, \mathbb{P}^{\star}\right) \in \mathcal{X} \times \mathcal{P}$ 来保证：

$$
\mathbb{E}_{\mathbb{P}}\left[\ell\left(x^{\star}, Z\right)\right] \leq \mathbb{E}_{\mathbb{P}^{\star}}\left[\ell\left(x^{\star}, Z\right)\right] \leq \mathbb{E}_{\mathbb{P}^{\star}}[\ell(x, Z)] \quad \forall x \in \mathcal{X}, \mathbb{P} \in \mathcal{P} .
$$

需要强调的是，反过来并不成立，即强对偶性并不必然意味着纳什均衡的存在。

原始 DRO 问题在许多应用中自然出现。而对偶 DRO 问题的实用性较不明显，因为这一问题假设决策者能够观察支配 $Z$ 的分布，这是不太现实的。然而，对偶 DRO 问题在稳健统计、机器学习以及其他多个领域具有深远的联系，下面将对此进行解释。

从稳健统计的角度来看，原始 DRO 问题 (1.2) 的最优解 $x^{\star}$ 可以被视为 $\min _{x \in \mathcal{X}} \mathbb{E}_{\mathbb{P}_0}[\ell(x, Z)]$ 对应的未知分布 $\mathbb{P}_0$ 的稳健估计器。当 $x^{\star}$ 和 $\mathbb{P}^{\star}$ 满足鞍点条件 (7.16) 时，稳健估计器 $x^{\star}$ 构成了 $\mathbb{P}^{\star}$ 的最佳响应。因此，它解决了对应于 $\mathbb{P}^{\star}$ 的随机规划问题；参见 (Lehmann and Casella 2006, 第 5 章)。因此，$\mathbb{P}^{\star}$ 通常被称为最不利分布。$\mathbb{P}^{\star}$ 的存在使得 $x^{\star}$ 成为一个合理的估计器，因为这保证了 $x^{\star}$ 是某个模糊集中的分布对应随机规划的最优解。

用于计算 DRO 问题纳什均衡的算法在机器学习中也具有相关性。为说明这一点，回顾对抗训练旨在使机器学习模型免受输入数据的对抗扰动 (Szegedy et al. 2014; Goodfellow et al. 2015; Mądry et al. 2018; Wang et al. 2019; Kurakin et al. 2022)。在这种背景下，生成对抗样本，即恶意设计的输入来误导由参数 $x \in \mathcal{X}$ 表示的预测模型，是一个重要问题。作为构造对抗样本的一个简单方法，可以求解最坏情况期望问题

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(\hat{x}, Z)],
$$

该问题寻找一个测试分布以最大化某特定模型 $\hat{x}$ 的期望预测损失。因此，(7.17) 的任何解 $\mathbb{P}^{\star}$ 都可以被视为一种对抗攻击，并且从 $\mathbb{P}^{\star}$ 生成的样本自然可以被解释为对抗样本。然而，为了开发高效的攻击和防御策略，希望构造能够迷惑多个模型的对抗攻击。这类攻击在机器学习文献中被称为可迁移攻击 (Tramèr et al. 2017; Demontis et al. 2019; Kurakin et al. 2022)。对偶 DRO 问题 (7.15) 可以系统地用于构造可迁移攻击。实际上，(7.15) 的解并不针对特定模型 $\hat{x} \in \mathcal{X}$，而是旨在同时攻击所有模型 $x \in \mathcal{X}$。如果原始 DRO 问题 (1.2) 存在唯一的最优解 $x^{\star}$，那么可以通过求解对应于对抗者纳什策略 $\mathbb{P}^{\star}$ 的随机规划来恢复该最优解。

目前，对偶 DRO 问题仅在特定应用背景下得到了研究。例如，已知在具有以高斯参考分布为中心的 2-Wasserstein 模糊集的分布鲁棒估计和卡尔曼滤波问题中，最不利分布本身是高斯分布，并且可以通过半定规划高效计算 (ShafieezadehAbadeh et al. 2018; Nguyen et al. 2023)。最近的一些研究描述了类似结果，这些结果涉及具有 2-Wasserstein 模糊集的分布鲁棒最优控制问题 (Taha et al. 2023; Hajar et al. 2023; Kargin et al. 2024a,b,c,d; Taşkesen et al. 2024)。当 Wasserstein 模糊集被 Kullback-Leibler 模糊集替代时，最不利分布仍然是高斯分布，可以通过准闭形式确定 (Levy and Nikoukhah 2004, 2012)。事实上，这些结果甚至可以推广到广义 $\tau$-散度模糊集 (Zorzi 2016, 2017b)。此外，高斯分布还解决了一些类似于 DRO 问题的极小极大博弈，这些博弈在统计学、控制论和信息论中具有重要应用 (Başar 和 Mintz 1972, 1973; Başar 和 Max 1973; Başar 1977; Başar 和 Başar 1982, 1983, 1984; Başar 和 Wu 1985, 1986)。此外，分布鲁棒定价和拍卖设计问题的纳什均衡可以通过仅基于支持集和 Markov 模糊集的方式进行闭式表述 (Bergemann 和 Schlag 2008; Koçyigit et al. 2020, 2022; Anunrojwong et al. 2024; Chen et al. 2024a)。有关更一般的最优传输模糊集的原始和对偶 DRO 问题的强对偶性极小极大定理，详见 (Blanchet et al. 2022b; Shafiee et al. 2023; Frank 和 Niles-Weed 2024b; Pydi 和 Jog 2024)。


# 8. 通过鲁棒化进行正则化

经典的随机优化寻求在从训练数据中估计得到的概率分布 $\hat{\mathbb{P}}$ 下表现良好的决策。然而，由于忽略了关于 $\hat{\mathbb{P}}$ 估计误差的信息，随机优化往往输出过拟合的决策，这些决策在 $\hat{\mathbb{P}}$ 下期望损失较低，但在未知的总体分布 $\mathbb{P}$ 下可能表现较差。如果训练数据稀少，这一问题会更加严重。相较于随机优化，分布鲁棒优化（DRO）的一个关键优势在于它能够利用关于估计误差的信息。DRO 使用这一信息来防止过拟合。因此，将一个随机优化问题针对分布不确定性进行鲁棒化可以视为一种隐式正则化形式。

我们现在展示，针对分布不确定性进行鲁棒化所实现的隐式正则化与通过向问题的目标函数添加惩罚项实现的显式正则化之间通常存在深刻联系。这一讨论补充并扩展了第6节中的若干结果。例如，在6.9节中我们已经看到，针对以高斯分布为中心的Kullback-Leibler模糊集的线性损失函数的最坏期望值与标称期望损失和方差正则化项一致。同样，在6.13节中我们看到，针对1-Wasserstein模糊集的凸损失函数的最坏期望值与标称期望损失和Lipschitz正则化项一致。有关这些结果的一些变体和推广，请参见6.14节和6.15节。在8.1节中，我们将广泛地展示，针对 $\phi$-散度模糊集的最坏期望损失与标称期望损失和方差正则化项密切相关。同样，在8.2节中，我们将展示，针对Wasserstein模糊集的最坏期望损失与标称期望损失和变异性及Lipschitz正则化项密切相关。在8.3节中，我们还将进一步展示，许多流行的风险度量在与Wasserstein距离相关的相关风险因子的分布中是Lipschitz连续的。这表明，针对Wasserstein模糊集的最坏风险与标称风险及Lipschitz正则化项密切相关。需要指出的是，针对矩模糊集的鲁棒化和正则化之间的关系尚未被很好地理解。从6.8节中我们知道，针对Gelbrich模糊集的线性损失函数的最坏风险通常与标称风险和 $2$ 范数正则化项一致。然而，对于非线性损失函数或其他矩模糊集是否可以得到类似结果尚不清楚。因此，本节将不涉及矩模糊集。我们强调，鲁棒化与正则化之间的联系通常使得DRO问题的统计分析成为可能；参见第10节。

## 8.1. $\phi$-散度模糊集

作为一个激励示例，我们展示了针对Pearson $\chi^2$-散度模糊集进行鲁棒化与方差正则化密切相关。为了说明这一点，首先回忆Pearson $\chi^2$-散度模糊集 (2.17) 定义为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathbb{P}(\mathcal{Z}): \chi^2(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\} .
$$

如果 $\ell$ 是一个有界Borel函数，命题2.13 直接意味着：

$$
\mathcal{P} \subseteq\left\{\mathbb{P} \in \mathbb{P}(\mathcal{Z}): \mathbb{E}_{\mathrm{P}}[\ell(Z)] \leq \mathbb{B}_{\mathrm{p}}[\ell(Z)]+r^{\frac{1}{2}} \mathrm{~V}_{\hat{\mathrm{P}}}[\ell(Z)]^{\frac{1}{2}}\right\},
$$

因此我们可以得出结论：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathrm{P}}[\ell(\mathrm{Z})] \leq \mathbb{B}_{\hat{\mathrm{p}}}[\ell(Z)]+r^{\frac{1}{2}} \mathrm{~V}_{\hat{\mathrm{p}}}[\ell(Z)]^{\frac{1}{2}}
$$

因此，关于以 $\hat{\mathbb{P}}$ 为中心的Pearson $\chi^2$-散度模糊集半径 $r$ 的最坏期望损失的上界是一个风险厌恶系数为 $r^{\frac{1}{2}}$ 的均值-标准差风险度量。在术语上稍作滥用，缩放的标准差 $r^{\frac{1}{2}} \mathrm{~V}_{\hat{\mathrm{p}}}[\ell(\mathrm{Z})]^{\frac{1}{2}}$ 通常被称为方差正则化项。通过利用定理4.15，上述上界可以扩展到任意（可能是无界的）Borel损失函数。这一扩展关键依赖于以下引理。

**引理 8.1（方差公式）** 对于任何参考分布 $\hat{\mathrm{P}} \in \mathcal{P}(\mathcal{Z})$、参数 $r>0$ 和 $\mathbb{E}_{\hat{\mathrm{p}}}[|\ell(Z)|]<\infty$ 的Borel函数 $\ell \in \mathcal{L}\left(\mathbb{R}^d\right)$，我们有：

**证明** 如果 $\mathbb{E}_{\hat{p}}\left[\ell(Z)^2\right]=\infty$，那么 (8.1) 两边都为 $\infty$，因此结论成立。在证明的剩余部分中，我们可以假设 $\mathbb{E}_{\hat{p}}\left[\ell(Z)^2\right]<\infty$。在这种情况下，可以验证，通过 $\lambda_0$ 的部分最小化问题由 $\lambda_0^{\star}=\mathbb{E}_{\hat{p}}[\ell(Z)]$ 解决。将 $\lambda_0^{\star}$ 代入目标函数表明 (8.1) 左侧的下确界等于 $\inf _{\lambda \in \mathbb{R}_{+}} \lambda r+\mathbb{V}_{\hat{\mathbb{P}}}[\ell(Z)] / 4 \lambda$。为了证明 (8.1)，只需意识到这个关于 $\lambda$ 的最小化问题由 $\lambda^{\star}=\sqrt{\mathbb{V}_{\hat{\mathbb{P}}}[\ell(Z)] /(4 r)}$ 解决即可。此观察完成了证明。

**定理 8.2 （方差正则化）** 如果 $\mathcal{P}$ 是 Pearson $\chi^2$-散度模糊集 (2.17)，且 $\mathbb{E}_{\hat{\mathrm{P}}}[|\ell(Z)|]<\infty$，则我们有：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] \leq \mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]+r^{\frac{1}{2}} \mathbb{V}_{\hat{\mathbb{P}}}[\ell(Z)]^{\frac{1}{2}}
$$

**证明** 该命题在 $r=0$ 时显然成立。因此我们可以假设 $r>0$。回忆，诱导 Pearson $\chi^2$-散度的熵函数 $\phi$ 满足 $\phi(s)=(s-1)^2$ （如果 $s \geq 0$）以及 $\phi(s)=\infty$ （如果 $s<0$）。因此，其共轭熵函数 $\phi^*$ 满足 $\phi^*(t)=\frac{1}{4} t^2+t$ （如果 $t \geq-2$）以及 $\phi^*(t)=-1$ （如果 $t<-2$），其定义域为 $\operatorname{dom}\left(\phi^*\right)=\mathbb{R}$。由于 $\phi^{\infty}(1)=\infty$，所有分布 $\mathbb{P} \in \mathcal{P}$ 都绝对连续于 $\hat{\mathbb{P}}$。因此，定理 4.15 适用，并且我们有：

$$
\begin{aligned}
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] & =\inf _{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}_{+}} \lambda_0+\lambda r+\mathbb{E}_{\hat{\mathbb{P}}}\left[\left(\phi^*\right)^\pi\left(\ell(Z)-\lambda_0, \lambda\right)\right] \\
& \leq \inf _{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}_{+}} \mathbb{E}_{\hat{\mathrm{P}}}[\ell(Z)]+\lambda r+\frac{\mathbb{E}_{\hat{\mathbb{P}}}\left[\left(\ell(Z)-\lambda_0\right)^2\right]}{4 \lambda} \\
& =\mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]+r^{\frac{1}{2}} \mathbb{V}_{\hat{\mathbb{P}}}[\ell(Z)]^{\frac{1}{2}},
\end{aligned}
$$

其中不等式是因为 $\phi^*(t) \leq \frac{1}{4} t^2+t$，第二个等式依据引理 8.1。于是，命题得证。

大多数 $\phi$-散度是平滑且非负的，因此在 1 附近局部上与 Pearson $\chi^2$-散度类似（见 Polyanskiy 和 Wu，2024，第 7.10 节）。因此，可以通过泰勒展开表明，针对足够小尺寸 $r$ 的 $\phi$-散度模糊集进行鲁棒化通常等价于方差正则化。为了形式化这一结果，我们现在假设 $\phi$ 是可微的。

**假设 8.3 （可微性）** 熵函数 $\phi$ 在 1 的某邻域内二阶连续可微，且 $\phi(1)=\phi^{\prime}(1)=0$ 且 $\phi^{\prime \prime}(1)=2$。

假设 $\phi^{\prime}(1)=0$ 并不损失一般性。实际上，任何熵函数 $\phi$ 都等价于一个通过 $\tilde{\phi}(t)=\phi(t)-\phi^{\prime}(1) \cdot t+\phi^{\prime}(1)$ 定义的变换熵函数 $\tilde{\phi}$，其中 $\tilde{\phi}^{\prime}(1)=0$。即 $\phi$ 和 $\tilde{\phi}$ 诱导相同的散度。注意，表 2.1 中列出的所有熵函数（除了与全变差相关的熵函数）都满足 $\phi^{\prime}(1)=0$。假设 $\phi^{\prime \prime}(1)=2$ 是一个任意的归一化，但将简化计算。

回忆，受限 $\phi$-散度模糊集 (2.11) 定义为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{P} \ll \hat{\mathbb{P}}, \mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\} .
$$

这里 $\mathcal{Z}$ 是闭支集，$r \in \mathbb{R}_{+}$ 是尺寸参数，$\phi$ 是定义 2.4 意义下的熵函数，$\mathrm{D}_\phi$ 是定义 2.5 意义下的对应 $\phi$-散度，$\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 是参考分布。以下定理提供了关于 $\mathcal{P}$ 的最坏期望的泰勒展开的主导项。

**定理 8.4 （最坏期望的泰勒展开）** 如果 $\mathcal{P}$ 是受限 $\phi$-散度模糊集 (2.11)，熵函数 $\phi$ 满足假设 8.3，且损失 $\ell(Z)$ 在 $\hat{\mathbb{P}}$ 几乎处处有界，则有：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)]=\mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]+r^{\frac{1}{2}} \mathbb{V}_{\hat{\mathbb{P}}}[\ell(Z)]^{\frac{1}{2}}+o\left(r^{\frac{1}{2}}\right) .
$$

**证明** 如果 $r=0$，则 (8.2) 显然成立。同样，如果 $\mathbb{V}_{\hat{\mathbb{P}}}[\ell(Z)]=0$，则 $\ell(Z)$ 在 $\hat{\mathbb{P}}$ 几乎处处与 $\mathbb{E}_{\hat{\mathbb{p}}}[\ell(Z)]$ 一致。由于 $\mathcal{P}$ 是受限 $\phi$-散度模糊集，这直接意味着对于所有 $\mathbb{P} \in \mathcal{P}$，都有 $\mathbb{E}_{\mathbb{P}}[\ell(Z)]=\mathbb{E}_{\hat{p}}[\ell(Z)]$。事实上，任何 $\mathbb{P} \in \mathcal{P}$ 都满足 $\mathbb{P} \ll \hat{\mathbb{P}}$。因此 (8.2) 再次显然成立。在证明的其余部分中，我们可以假设 $r>0$ 且 $\mathbb{V}_{\hat{\mathbb{P}}}[\ell(Z)]>0$。

假设 8.3 表明 $\phi(s)=(s-1)^2+o\left(s^2\right)$。通过带有 Peano 余项的泰勒定理，$\phi$ 可以在 1 附近局部上由二次函数进行上下界，从而其二阶导数略小（或略大于） $\phi^{\prime \prime}(1)=2$。因此存在函数 $\kappa: \mathbb{R}_{+} \rightarrow \mathbb{R}_{+}$ 满足 $\lim _{\varepsilon \downarrow 0} \kappa(\varepsilon)=0$ 且

$$
\frac{1}{1+\kappa(\varepsilon)} \cdot s^2 \leq \phi(1+s) \leq(1+\kappa(\varepsilon)) \cdot s^2 \quad \forall s \in[-\varepsilon,+\varepsilon]
$$

对于所有充分小的 $\varepsilon$ 都成立。证明的其余部分分两步完成，这两步都利用了 (8.3)。首先，我们证明 (8.2) 右侧为 $\mathcal{P}$ 上最坏期望损失的下界（步骤1）。接着，我们证明 (8.2) 右侧也为 $\mathcal{P}$ 上最坏期望损失的上界（步骤2）。步骤1和2结合将推出命题。

**步骤 1** 受限的 $\phi$-散度模糊集 $\mathcal{P}$ 中的每个分布 $\mathbb{P}$ 都满足 $\mathbb{P} \ll \hat{\mathbb{P}}$，因此相对于 $\hat{\mathbb{P}}$ 有一个密度函数 $f \in \mathcal{L}^1(\hat{\mathbb{P}})$。这里，$\mathcal{L}^1(\hat{\mathbb{P}})$ 通常指所有从 $\mathcal{Z}$ 到 $\mathbb{R}$ 的 Borel 函数，它们关于 $\hat{\mathbb{P}}$ 是可积的。由于 $\mathbb{P} \ll \hat{\mathbb{P}}$，我们有 $\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})=\mathbb{E}_{\hat{\mathbb{P}}}[\phi(f(Z))]$ （参见第 2.2 节）。因此，关于 $\mathcal{P}$ 的最坏期望问题可以重写为：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)]=\left\{\begin{array}{cl}
\sup _{f \in \mathcal{L}^1(\hat{\mathbb{P}})} & \mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z) f(Z)] \\
\text { s.t. } & \hat{\mathbb{P}}(f(Z) \geq 0)=1 \\
& \mathbb{E}_{\hat{\mathbb{P}}}[f(Z)]=1 \\
& \mathbb{E}_{\hat{\mathbb{P}}}[\phi(f(Z))] \leq r .
\end{array}\right.
$$

将 $f(z)+1$ 重命名为 $f(z)$，进一步得到：

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)]=\mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]+\left\{\begin{array}{cl}
\sup _{f \in \mathcal{L}^1(\hat{\mathbb{P}})} & \mathbb{E}_{\hat{\mathrm{p}}}[\ell(Z) f(Z)] \\
\text { s.t. } & \hat{\mathbb{P}}(f(Z) \geq-1)=1 \\
& \mathbb{E}_{\hat{\mathrm{p}}}[f(Z)]=0 \\
& \mathbb{E}_{\hat{\mathrm{p}}}[\phi(1+f(Z))] \leq r .
\end{array}\right.
$$

接下来，引入一个辅助函数 $\varepsilon: \mathbb{R}_{+} \rightarrow \mathbb{R}_{+}$，其满足：

$$
\varepsilon(r)=2 r^{\frac{1}{2}} \cdot \frac{\operatorname{ess} \sup _{\hat{\mathbb{P}}}\left[\left|\ell(Z)-\mathbb{E}_{\hat{\mathbb{p}}}[\ell(Z)]\right|\right]}{\mathbb{V}_{\hat{\mathbb{P}}}[\ell(Z)]^{\frac{1}{2}}}
$$

此外，对于每个 $r \in \mathbb{R}_{+}$，定义函数 $f_r^{\star} \in \mathcal{L}^1(\widehat{\mathbb{P}})$ 为：

$$
f_r^{\star}(z)=\frac{r^{\frac{1}{2}}}{(1+\kappa(\varepsilon(r)))^{\frac{1}{2}}} \cdot \frac{\ell(z)-\mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]}{\mathbb{V}_{\hat{\mathbb{P}}}[\ell(Z)]^{\frac{1}{2}}} .
$$

根据构造，我们可以得出：

$$
\left|f_r^{\star}(Z)\right| \leq r^{\frac{1}{2}} \cdot \frac{\left|\ell(Z)-\mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]\right|}{\mathbb{V}_{\hat{\mathbb{P}}}[\ell(Z)]^{\frac{1}{2}}} \leq \varepsilon(r) \quad \hat{\mathbb{P}} \text {-几乎处处成立。}
$$

对任意 $r \in \mathbb{R}_{+}$，上述不等式分别由 $f_r^{\star}(z)$ 和 $\varepsilon(r)$ 的定义得到。此外，我们有 $\mathbb{E}_{\hat{\mathrm{p}}}\left[f_r^{\star}(Z)\right]=0$ 并且：

$$
\mathbb{E}_{\hat{\mathbb{P}}}\left[\phi\left(1+f_r^{\star}(Z)\right)\right] \leq(1+\kappa(\varepsilon(r))) \cdot \mathbb{E}_{\hat{\mathbb{P}}}\left[f_r^{\star}(Z)^2\right]=r,
$$

对所有足够小的 $r$ 都成立。上述表达式中的不等式来自 (8.5) 和 (8.3) 中对 $\phi$ 的上界，它们在所有足够小的 $\varepsilon$ 上成立。等式利用了 $f_r^{\star}$ 的定义。这表明，当 $r$ 足够小时，$f_r^{\star}$ 构成 (8.4) 中最大化问题的一个可行解。将 $f_r^{\star}$ 代入 (8.4) 则得到所需的下界。具体来说：

$$
\begin{aligned}
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] & \geq \mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]+\mathbb{E}_{\hat{\mathbb{P}}}\left[\ell(Z) f_r^{\star}(Z)\right] \\
& =\mathbb{E}_{\hat{\mathrm{P}}}[\ell(Z)]+\frac{r^{\frac{1}{2}}}{(1+\kappa(\varepsilon(r)))^{\frac{1}{2}}} \cdot \frac{\mathbb{E}_{\hat{\mathbb{P}}}\left[\ell(Z)\left(\ell(Z)-\mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]\right)\right]}{\mathbb{V}_{\hat{\mathbb{P}}}[\ell(Z)]^{\frac{1}{2}}} \\
& =\mathbb{E}_{\hat{\mathrm{P}}}[\ell(Z)]+r^{\frac{1}{2}} \mathbb{V}_{\hat{\mathrm{P}}}[\ell(Z)]^{\frac{1}{2}}+o\left(r^{\frac{1}{2}}\right),
\end{aligned}
$$

对所有足够小的 $r$ 成立，其中第一个等式来自 $f_r^{\star}$ 的定义，第二个等式利用了逆平方根函数在 1 附近的泰勒展开，以及 $\lim _{r \downarrow 0} \kappa(\varepsilon(r))=0$ 的基本观察。

**步骤 2** 调和损失函数 $h_{\varepsilon}: \mathbb{R} \rightarrow \mathbb{R}$ 具有调节参数 $\varepsilon>0$，定义为：

$$
h_{\varepsilon}(s)= \begin{cases}\frac{1}{2} s^2 & \text { 如果 }|s| \leq \varepsilon \\ \varepsilon|s|-\frac{1}{2} \varepsilon^2 & \text { 否则 }\end{cases}
$$

通过构造，$h_{\varepsilon}$ 是连续可微的，当 $|s| \leq \varepsilon$ 时关于 $s$ 是二次的，当 $|s|>\varepsilon$ 时关于 $s$ 是线性的。其共轭函数 $h_{\varepsilon}^*: \mathbb{R} \rightarrow \overline{\mathbb{R}}$ 满足：

$$
h_{\varepsilon}^*(t)= \begin{cases}\frac{1}{2} t^2 & \text { 如果 }|t| \leq \varepsilon, \\ \infty & \text { 否则。 }\end{cases}
$$

(8.3) 中对 $\phi$ 的下界及其凸性意味着：

$$
\phi(s) \geq \frac{2}{1+\kappa(\varepsilon)} h_{\varepsilon}(s-1) \quad \forall s \in \mathbb{R},
$$

当 $\varepsilon$ 足够小时。这一基于 $h_{\varepsilon}$ 的 $\phi$ 的统一下界导出了基于 $h_{\varepsilon}^*$ 的 $\phi^*$ 的统一上界。具体来说：

$$
\begin{aligned}
\phi^*(t) & \leq \sup _{s \in \mathbb{R}} s t-\frac{2}{1+\kappa(\varepsilon)} h_{\varepsilon}(s-1) \\
& =t+\frac{2}{1+\kappa(\varepsilon)} h_{\varepsilon}^*\left(\frac{1}{2} t(1+\kappa(\varepsilon))\right)=t+ \begin{cases}\frac{(1+\kappa(\varepsilon)) t^2}{4} & \text { 如果 }|t| \leq \frac{2 \varepsilon}{1+\kappa(\varepsilon)}, \\
\infty & \text { 否则， }\end{cases}
\end{aligned}
$$

对所有足够小的 $\varepsilon$ 成立。(8.6) 中的第一个等式通过变量替换 $s \leftarrow s-1$ 并提取常数 $2 /(1+\kappa(\varepsilon))$ 得到。第二个等式来自 $h_{\varepsilon}^*$ 的定义。利用定理 4.15 中建立的弱对偶性，我们有：

$$
\begin{aligned}
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] & \leq \inf _{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}_{+}} \lambda_0+\lambda r+\mathbb{E}_{\hat{\mathbb{P}}}\left[\left(\phi^*\right)^\pi\left(\ell(Z)-\lambda_0, \lambda\right)\right] \\
& \leq\left\{\begin{array}{cl}
\inf _{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}_{+}} & \mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]+\lambda r+\frac{1+\kappa(\varepsilon(r))}{4 \lambda} \mathbb{E}_{\hat{\mathbb{P}}}\left[\left(\ell(Z)-\lambda_0\right)^2\right] \\
\text { s.t. } & \hat{\mathbb{P}}\left(\left|\ell(Z)-\lambda_0\right| \leq \frac{2 \varepsilon(r) \lambda}{1+\kappa(\varepsilon(r))}\right)=1
\end{array}\right.
\end{aligned}
$$

第二个不等式来源于透视函数的定义以及 (8.6)，它对所有足够小的 $\varepsilon$ 都成立。这里我们重新使用了步骤 1 中引入的函数 $\varepsilon(r)$。接着我们设定 $\lambda_0^{\star}=\mathbb{E}_{\hat{p}}[\ell(Z)]$ 并定义：

$$
\lambda_r^{\star}=\frac{(1+\kappa(\varepsilon(r)))^{\frac{1}{2}}}{2 r^{\frac{1}{2}}} \cdot \mathbb{V}_{\hat{\mathbb{P}}}[\ell(Z)]^{\frac{1}{2}}。
$$

**对于任意 $r>0$**，注意，只要 $r$ 足够小，$\left(\lambda_0^{\star}, \lambda_r^{\star}\right)$ 在 (8.7) 中是可行的；特别地，$r$ 必须小到满足 $\kappa(\varepsilon(r)) \leq 3$。确实，我们有：

$$
\begin{aligned}
& \hat{\mathbb{P}}\left(\left|\ell(Z)-\lambda_0^{\star}\right| \leq \frac{2 \varepsilon(r) \lambda_r^{\star}}{1+\kappa(\varepsilon(r))}\right) \\
& =\hat{\mathbb{P}}\left(\left|\ell(Z)-\mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]\right| \leq \frac{\varepsilon(r) \mathbb{V}_{\hat{\mathbb{P}}}[\ell(Z)]^{\frac{1}{2}}}{r^{\frac{1}{2}}(1+\kappa(\varepsilon(r)))^{\frac{1}{2}}}\right) \\
& =\hat{\mathbb{P}}\left(\left|\ell(Z)-\mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]\right| \leq \frac{2}{(1+\kappa(\varepsilon(r)))^{\frac{1}{2}}} \operatorname{ess}^2 \sup _{\hat{\mathbb{P}}}\left[\left|\ell(Z)-\mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]\right|\right]\right)=1,
\end{aligned}
$$

其中，第一个等式由 $\lambda_0^{\star}$ 和 $\lambda_r^{\star}$ 的定义得出，第二个等式由 $\varepsilon(r)$ 的定义得出，最后一个等式成立是因为 $\kappa(\varepsilon(r)) \leq 3$。将 $\left(\lambda_0^{\star}, \lambda_r^{\star}\right)$ 代入 (8.7) 则得出所需的上界：

$$
\begin{aligned}
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(Z)] \leq & \mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]+\lambda_r^{\star} r+\frac{1+\kappa(\varepsilon(r))}{4 \lambda_r^{\star}} \mathbb{E}_{\hat{\mathbb{P}}}\left[\left(\ell(Z)-\lambda_0^{\star}\right)^2\right] \\
= & \mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]+\frac{(1+\kappa(\varepsilon(r)))^{\frac{1}{2}}}{2} \cdot r^{\frac{1}{2}} \mathbb{V}_{\hat{\mathbb{P}}}[\ell(Z)]^{\frac{1}{2}} \\
& +\frac{(1+\kappa(\varepsilon(r)))^{\frac{1}{2}}}{2 \mathbb{V}_{\hat{\mathbb{P}}}[\ell(Z)]^{\frac{1}{2}}} r^{\frac{1}{2}} \mathbb{E}_{\hat{\mathbb{P}}}\left[\left(\ell(Z)-\mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]\right)^2\right] \\
= & \mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]+r^{\frac{1}{2}} \mathbb{V}_{\hat{\mathbb{P}}}[\ell(Z)]^{\frac{1}{2}}+o\left(r^{\frac{1}{2}}\right),
\end{aligned}
$$

这里，第一个等式来自 $\lambda_0^{\star}$ 和 $\lambda_r^{\star}$ 的定义，第二个等式成立是因为 $\lim _{r \downarrow 0} \kappa(\varepsilon(r))=0$。因此，命题得证。

**定理 8.4** 揭示，针对受限散度模糊集的鲁棒化在 $r$ 的主导阶下等价于方差正则化。要求损失几乎处处有界虽然较为严格但却必要。然而，如果熵函数 $\phi$ 超线性增长，这一要求可以放宽。例如，从命题 6.12 可以回忆到，针对以高斯分布为中心的 Kullback-Leibler 模糊集的线性损失函数的最坏期望值恰好等于 $\mathbb{E}_{\hat{\mathbb{P}}}[\ell(Z)]+(2 r)^{\frac{1}{2}} \mathbb{V}_{\hat{\mathrm{p}}}[\ell(Z)]^{\frac{1}{2}}$，且没有任何高阶误差项。此公式与定理 8.4 一致，因为 Kullback-Leibler 散度的熵函数满足 $\phi^{\prime \prime}(1)=1$。因此，必须将其按 2 缩放以满足假设 8.3。注意，对于任何（非常数）线性损失函数，相对于任何（非退化）高斯分布 $\hat{\mathbb{P}}$，都无法几乎处处有界。然而，定理 8.4 的结论依然成立，因为底层熵函数增长快于线性。

类似于 (8.2) 的泰勒展开，对于经验参考分布和 Kullback-Leibler 模糊集 (2.13)，由 Lam (2019) 提出。Duchi 等人 (2021) 将这一结果推广到其他 $\phi$-散度。类似结果在不同背景下由 Lam $(2016,2018)$、Duchi 和 Namkoong (2019) 以及 Blanchet 和 Shapiro (2023) 针对经验参考分布提出。在平行研究中，Gotoh、Kim 和 $\operatorname{Lim}(2018,2021)$ 推导了基于惩罚的最坏期望损失 $\sup _{\mathbb{P} \in \mathcal{P}(\mathcal{Z})} \mathbb{E}_{\mathbb{P}}[\ell(Z)]-\frac{1}{r} \mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$ 的泰勒展开。他们再次关注离散经验参考分布，并提供了相应泰勒展开的一阶和高阶项。

Maurer 和 Pontil (2009) 显示，与标准经验风险最小化相比，方差正则化的经验风险最小化可以更快地收敛到总体分布下的期望损失。这种改进的收敛性突出了在学习过程中引入方差正则化的潜在益处。不幸的是，即使底层损失函数在决策变量上是凸的，具有均值-方差目标的简单随机优化问题仍然是 NP 难的 (Ahmed 2006)。相比之下，相对于任何模糊集的最坏期望保留了底层损失函数的凸性。因此，定理 8.4 表明，相对于受限 $\phi$-散度模糊集的最坏期望损失为非凸但统计上吸引人的方差正则化经验损失提供了一个凸替代。

